In [2]:
"""
Exact diagonalization script for a two-site Bose-Hubbard current-measurement analysis

- Truncation: up to `nmax` bosons per site (default: 3)
- Operators: H_swap = J(a1^\dag a2 + a2^\dag a1), H_int = U/2 sum_i n_i(n_i-1)
- Current: j = i J (a1^\dag a2 - a2^\dag a1)
- Measurement operator (naive): D = J (n2 - n1) measured after evolution time t

Features:
- Build full Hamiltonian and operators in the truncated Fock basis
- Compute Heisenberg-evolved current j(t) = U^\dag j U at arbitrary times
- Compute operator-level diagnostics (off-diagonal norm, operator difference)
- Evaluate expectation values for a list of initial states (including user-supplied qutip.Qobj)
- Helpers to enumerate Fock basis states and total-particle sectors

Requires: qutip

Author: ChatGPT (generated)
"""

import numpy as np
import itertools
from math import pi
from typing import List, Tuple, Dict, Optional
import pickle

import qutip as qt

import sys
sys.path.append(r"C:\Users\mattm\OneDrive\Desktop\Research\Projects\Triangle Lattice\Jupyter Notebooks\8Q_Triangle_Lattice_v1")

from current_measurements.src.src_current_measurement_simulations import CurrentMeasurementSimulation



In [3]:
# ----------------------------- builder helpers -----------------------------

def build_operators(nmax: int = 3, J: float = 1.0, U: float = 0.0):
    """Build single-site operators, two-site operators, and Hamiltonians.

    Arguments:
        nmax: maximum number of bosons per site (Hilbert space dimension per site = nmax+1)
        J: tunneling amplitude in H_swap
        U: on-site interaction strength in H_int

    Returns:
        dict with keys: a1,a2,n1,n2,H_swap,H_int,H_total,j_op,D_op, dims
    """
    d = nmax + 1
    a = qt.destroy(d)
    I = qt.qeye(d)

    a1 = qt.tensor(a, I)
    a2 = qt.tensor(I, a)
    n1 = a1.dag() * a1
    n2 = a2.dag() * a2

    H_swap = J * (a1.dag() * a2 + a2.dag() * a1)
    # On-site interaction U/2 n(n-1) per site

    H_int = (U / 2.0) * (n1 * (n1 - qt.qeye([d]*2)) + n2 * (n2 - qt.qeye([d]*2)))

    H_total = H_swap + H_int

    # Current operator j = i J (a1^dag a2 - a2^dag a1)
    j_op = 1j * J * (a1.dag() * a2 - a2.dag() * a1)

    # Density difference times J (measurement operator we would read out)
    D_op = J * (n2 - n1)

    return {
        'a1': a1, 'a2': a2, 'n1': n1, 'n2': n2,
        'H_swap': H_swap, 'H_int': H_int, 'H': H_total,
        'j': j_op, 'D': D_op,
        'dims': (d, d)
    }



In [4]:
# ----------------------------- basis helpers -----------------------------

def fock_state(n1: int, n2: int, nmax: int = 3) -> qt.Qobj:
    """Return qutip.Fock state |n1, n2> with truncation nmax per site."""
    d = nmax + 1
    if not (0 <= n1 <= nmax and 0 <= n2 <= nmax):
        raise ValueError("Occupations must satisfy 0 <= n <= nmax")
    return qt.tensor(qt.basis(d, n1), qt.basis(d, n2))


def all_fock_states(nmax: int = 3) -> List[qt.Qobj]:
    """Return a list of all two-site Fock states (ordered lexicographically by (n1,n2))."""
    states = []
    for n1 in range(nmax + 1):
        for n2 in range(nmax + 1):
            states.append(fock_state(n1, n2, nmax))
    return states


def total_number_sector(N: int, nmax: int = 3) -> List[qt.Qobj]:
    """Return all |n1,n2> with n1+n2 == N and 0<=n_i<=nmax."""
    states = []
    for n1 in range(nmax + 1):
        n2 = N - n1
        if 0 <= n2 <= nmax:
            states.append(fock_state(n1, n2, nmax))
    return states


In [5]:
# ----------------------------- evolution & diagnostics -----------------------------

def heisenberg_evolve_op(H: qt.Qobj, op: qt.Qobj, t: float) -> qt.Qobj:
    """Return the Heisenberg-evolved operator op(t) = U^\dag op U with U = exp(-i H t).

    Uses exact matrix exponential via qutip Qobj.expm().
    """
    U = (-1j * H * t).expm()
    return U.dag() * op * U


def off_diagonal_norm_in_fock_basis(op: qt.Qobj) -> float:
    """Compute Frobenius norm of off-diagonal elements of `op` in the Fock basis.

    The returned value is sqrt(sum_{m!=n} |op_mn|^2).
    """
    M = op.full()
    diag = np.diag(np.diag(M))
    off = M - diag
    return np.linalg.norm(off)


def operator_difference_norm(opA: qt.Qobj, opB: qt.Qobj) -> float:
    """Frobenius norm ||opA - opB||_F."""
    return np.linalg.norm((opA - opB).full())


In [6]:
# ----------------------------- analysis utilities -----------------------------

def analyze_time(H: qt.Qobj, j0: qt.Qobj, D: qt.Qobj, times: List[float],
                 init_states: Optional[List[qt.Qobj]] = None,
                 state_labels: Optional[List[str]] = None) -> Dict:
    """Analyze mapping between j(0) and D(t) across a set of times and initial states.

    Returns a dictionary with per-time diagnostics:
      - j_t: Heisenberg-evolved j operator at time t
      - op_offdiag_norm: off-diagonal norm of j_t in Fock basis
      - op_diff_norm: operator Frobenius norm between j0 and j_t
      - expectations: list of (exp_j0, exp_Dt, bias = exp_j0 - exp_Dt) for each initial state

    If init_states is None, default to all Fock states in the truncated basis.
    """
    # Determine Hilbert space dimension from H
    dim = H.shape[0]

    if init_states is None:
        # default to full Fock basis inferred from local dims (we assume square local dims)
        d_local = int(np.sqrt(dim))
        nmax = d_local - 1
        init_states = all_fock_states(nmax)
        state_labels = [f"|{n1},{n2}>" for n1 in range(nmax + 1) for n2 in range(nmax + 1)]

    if state_labels is None:
        state_labels = [f"state_{i}" for i in range(len(init_states))]

    results = {}
    for t in times:
        j_t = heisenberg_evolve_op(H, j0, t)
        offdiag = off_diagonal_norm_in_fock_basis(j_t)
        diffnorm = operator_difference_norm(j0, j_t)

        expectations = []
        for psi, lab in zip(init_states, state_labels):
            exp_j0 = qt.expect(j0, psi)
            exp_Dt = qt.expect(D, psi)  # note: D is in Schrödinger picture (we measure D after evolving state by U); equivalently use expect(U.dag D U, psi)
            # The above line assumes we apply the swap evolution and then measure D; but since we're analyzing operator mapping
            # we should compare exp_j0 (on initial state) to exp_{psi}[ D_meas ] where D_meas = measurement of D on the state after evolution.
            # This is equivalent to: exp_Dt_correct = expect( heisenberg_evolve_op(H, D, -t) ??? )
            # Simpler: to compare j0 with D measured at t we compute expectation of D in the state U psi.
            # To be explicit below, we compute psi_evolved = U * psi and then expect(D, psi_evolved).
            expectations.append({'label': lab, 'exp_j0': exp_j0, 'exp_Dt_naive': exp_Dt})

        # compute expectations using evolved states (correct method)
        U = (-1j * H * times[0]).expm() if False else None  # placeholder to avoid linter complaints
        # Instead compute per-time evolved state expectations properly below.
        psi_evolved_exps = []
        U_t = (-1j * H * t).expm()
        for psi in init_states:
            if psi.type == 'ket':
                psi_evolved = U_t * psi
            elif psi.type == 'oper':  # density matrix
                psi_evolved = U_t * psi * U_t.dag()
            else:
                raise ValueError("init_states must contain kets or density matrices (Qobj)")
            exp_D_evolved = qt.expect(D, psi_evolved)
            psi_evolved_exps.append(exp_D_evolved)

        # rewrite expectations list to include the evolved-state expectation
        for i in range(len(expectations)):
            expectations[i]['exp_Dt'] = psi_evolved_exps[i]
            expectations[i]['bias'] = expectations[i]['exp_j0'] - expectations[i]['exp_Dt']

        results[t] = {
            'j_t': j_t,
            'offdiag_norm': offdiag,
            'op_diff_norm': diffnorm,
            'expectations': expectations,
        }

    return results


def compute_mapping_matrix(H: qt.Qobj, op_meas: qt.Qobj, basis_states: List[qt.Qobj]) -> np.ndarray:
    """Compute mapping matrix M where M_{i,j} = <phi_i | op_meas | phi_j> in given basis.

    Useful if you want to see how `op_meas` looks in the chosen basis (e.g. Fock basis).
    Returns a complex ndarray of shape (dim, dim) in the order of basis_states.
    """
    dim = len(basis_states)
    M = np.zeros((dim, dim), dtype=complex)
    for i, psi_i in enumerate(basis_states):
        for j, psi_j in enumerate(basis_states):
            M[i, j] = (psi_i.dag() * (op_meas * psi_j)).data[0, 0]
    return M


In [7]:
# ----------------------------- utility: pretty prints & I/O -----------------------------

def summarize_results(results: Dict, max_entries: int = 10):
    """Print a compact summary for a results dict returned from analyze_time."""
    for t, info in results.items():
        print(f"Time t = {t:.6f}")
        print(f"  off-diagonal norm of j(t): {info['offdiag_norm']:.6e}")
        print(f"  operator Frobenius norm between j(0) and j(t): {info['op_diff_norm']:.6e}")
        print("  sample state biases (first few):")
        for e in info['expectations'][:max_entries]:
            print(f"    {e['label']}: <j(0)> = {e['exp_j0']:.6e}, <D(t)> = {e['exp_Dt']:.6e}, bias = {e['bias']:.6e}")
        print('')



In [8]:
"""
Streamlined usage section for the Bose-Hubbard current–density mapping analysis.

We fix U/J = 20 (strongly interacting regime) and perform:
  1. A sanity check for the two-level system (nmax = 1), verifying that the density operator at time 0 equals the current operator at time -t_star.
  2. Analysis for a truncated 3-boson-per-site Hilbert space (nmax = 3) using three representative initial states.

Interpretation guidance:
  - If the operator-difference norms and off-diagonal norms are small at t_star, the mapping between current and density is accurate.
  - Deviations quantify how interactions and higher Fock states affect the mapping.
  - The biases printed for each initial state indicate how much the naive density measurement deviates from the true current expectation.
"""


# -----------------------------------------------------------------------------
# Parameters
J = 1.0
U = 20.0 * J  # fixed U/J = 20

# Nominal swap time (pi / 4J)
t_star = pi / (4.0 * J)

# -----------------------------------------------------------------------------
# 1. Sanity check: two-level (nmax = 1)

ops_2lvl = build_operators(nmax=1, J=J, U=U)
H_2lvl = ops_2lvl['H']
D_2lvl = ops_2lvl['D']
j_2lvl = ops_2lvl['j']

# In the ideal two-level limit, D(0) = j(-t_star)
D_t0 = D_2lvl
j_minus_t = heisenberg_evolve_op(H_2lvl, j_2lvl, -t_star)

print("===== Sanity check: two-level system (nmax=1) =====")
print(f"Frobenius norm ||D(0) - j(-t*)|| = {operator_difference_norm(D_t0, j_minus_t):.6e}")
print("If this is numerically ~0, the ideal mapping holds exactly in the two-level limit.\n")


===== Sanity check: two-level system (nmax=1) =====
Frobenius norm ||D(0) - j(-t*)|| = 4.509747e-16
If this is numerically ~0, the ideal mapping holds exactly in the two-level limit.



In [22]:

# -----------------------------------------------------------------------------
# 2. Full analysis: nmax = 3 with representative initial states

nmax = 3
ops = build_operators(nmax=nmax, J=J, U=U)
H = ops['H']
j0 = ops['j']
D = ops['D']

# define some useful initial states
current_state = qt.basis([nmax+1]*2, [1,0]) + 1j*qt.basis([nmax+1]*2, [0,1])  # |1,0> + i|0,1>
current_state = current_state.unit()  # normalize

current_2_state = qt.basis([nmax+1]*2, [2,1]) + 1j*qt.basis([nmax+1]*2, [1,2])  # |2,1> + i|1,2>
current_2_state = current_2_state.unit()  # normalize

# Representative initial states (|1,0>, |1,1>, |2,1>)
init_states = [
    fock_state(1, 0, nmax),
    fock_state(1, 1, nmax),
    fock_state(2, 1, nmax),
    current_state,
    current_2_state,
]
state_labels = ["|1,0>", "|1,1>", "|2,1>", "|1,0> + i|0,1>", "|2,1> + i|1,2>"]

# Analyze mapping at t_star
print("===== Strongly interacting case (U/J = 20, nmax=3) =====")
results = analyze_time(H=H, j0=j0, D=D, times=[t_star], init_states=init_states, state_labels=state_labels)
summarize_results(results)

# -----------------------------------------------------------------------------
# Output interpretation (printed automatically)
# - off-diagonal norm: measures how non-diagonal j(t*) is in Fock basis; smaller is better.
# - operator Frobenius norm: ||j(0) - j(t*)||, global measure of mapping accuracy.
# - bias per state: difference between expected <j(0)> and measured <D(t*)>.
#   Small biases indicate accurate mapping for that initial state.

print("\nInterpretation:")
print("- The smaller the off-diagonal and difference norms, the closer the density measurement at t* reproduces the current operator.")
print("- If biases for all representative states are small, the mapping remains robust even with higher Fock states.")


===== Strongly interacting case (U/J = 20, nmax=3) =====
Time t = 0.785398
  off-diagonal norm of j(t): 7.211053e+00
  operator Frobenius norm between j(0) and j(t): 1.469560e+01
  sample state biases (first few):
    |1,0>: <j(0)> = 0.000000e+00, <D(t)> = -1.665335e-16, bias = 1.665335e-16
    |1,1>: <j(0)> = 0.000000e+00, <D(t)> = 4.163336e-17, bias = -4.163336e-17
    |2,1>: <j(0)> = 0.000000e+00, <D(t)> = 9.962699e-01, bias = -9.962699e-01
    |1,0> + i|0,1>: <j(0)> = -1.000000e+00, <D(t)> = -1.000000e+00, bias = -6.217249e-15
    |2,1> + i|1,2>: <j(0)> = -2.000000e+00, <D(t)> = 1.476463e-03, bias = -2.001476e+00


Interpretation:
- The smaller the off-diagonal and difference norms, the closer the density measurement at t* reproduces the current operator.
- If biases for all representative states are small, the mapping remains robust even with higher Fock states.


In [19]:
#### import ramp initial state with dephasing

ramp_times_filename = r"C:\Users\mattm\OneDrive\Desktop\Research\Projects\Triangle Lattice\Jupyter Notebooks\8Q_Triangle_Lattice_v1\current_measurements\ramp_with_dephasing_simulation_files\Tlist.pkl"

with open(ramp_times_filename, "rb") as f:
    ramp_times = pickle.load(f)

T2_to_filename = {}
T2s = [2, 5, 10, 40]
for T2 in T2s:
    T2_to_filename[T2] = r"C:\Users\mattm\OneDrive\Desktop\Research\Projects\Triangle Lattice\Jupyter Notebooks\8Q_Triangle_Lattice_v1\current_measurements\ramp_with_dephasing_simulation_files\hemistates_{}us.pkl".format(T2)



T2_to_states = {}

for T2 in T2s:
    with open(T2_to_filename[T2], "rb") as f:
        T2_to_states[T2] = pickle.load(f)


target_ramp_time = 500
ramp_time_index = np.argmin(np.abs(ramp_times - target_ramp_time))

print(ramp_time_index)

8


In [ ]:
# -----------------------------------------------------------------------------
# 3. Full analysis: nmax = 3 with ramp initial states

nmax = 3
ops = build_operators(nmax=nmax, J=J, U=U)
H = ops['H']
j0 = ops['j']
D = ops['D']



In [10]:
def partial_trace_from_restricted_basis(rho_restricted, basis_restricted, keep=(0,1), total_sites=8):
    """
    Compute reduced density matrix over selected sites directly from
    a restricted-basis density matrix, without embedding in the full Hilbert space.

    Parameters
    ----------
    rho_restricted : np.ndarray
        Density matrix in restricted basis (square, Hermitian).
    basis_restricted : list of tuple
        Basis states (tuples of occupations, e.g. (n1,...,n8)).
    keep : tuple of int
        Indices of sites to keep (0-based).
    total_sites : int
        Total number of sites.

    Returns
    -------
    rho_reduced : np.ndarray
        Reduced density matrix over the kept sites.
        Basis order corresponds to all possible occupation tuples for those sites
        appearing in the restricted basis.
    """

    # Build mapping from (kept, traced) occupations to restricted-basis indices
    env_map = {}
    for idx, occ in enumerate(basis_restricted):
        kept_occ = tuple(occ[i] for i in keep)
        env_occ = tuple(occ[i] for i in range(total_sites) if i not in keep)
        env_map.setdefault(env_occ, []).append((kept_occ, idx))

    # Find all unique local (kept) configurations appearing in the restricted basis
    kept_basis = sorted({tuple(occ[i] for i in keep) for occ in basis_restricted})
    kept_index = {b: i for i, b in enumerate(kept_basis)}
    dim = len(kept_basis)
    rho_reduced = np.zeros((dim, dim), dtype=complex)

    # Sum over environmental configurations (trace)
    for env_occ, members in env_map.items():
        for kept_L, idx_L in members:
            for kept_R, idx_R in members:
                rho_reduced[kept_index[kept_L], kept_index[kept_R]] += rho_restricted[idx_L, idx_R]

    return rho_reduced, kept_basis


In [9]:
def convert_density_matrix_to_basis(rho, local_basis, basis):
    
    rho_qobj_data = np.zeros((len(basis), len(basis)), dtype=complex)

    local_state_to_index = {}
    for i in range(len(local_basis)):
        local_state_to_index[local_basis[i]] = basis.index(local_basis[i])

    for i in range(len(local_basis)):
        left_state = local_basis[i]
        for j in range(len(local_basis)):
            right_state = local_basis[j]

            rho_qobj_data[local_state_to_index[left_state], local_state_to_index[right_state]] = rho[i,j]


    return qt.Qobj(rho_qobj_data, dims=[[nmax+1]*len(basis[0])]*2)


In [129]:
# define some useful initial states

# this state is defined on only the basis states with 4 total particle on 8 qubits with up to 3 particles per site
# let's convert this to 8 qubits with up to nmax particles per site, then trace out the 6 rightmost qubits
ramp_state_5mus = T2_to_states[5][ramp_time_index]

reduced_basis = []
basis = []
reduced_state_to_index = {}

for i, state in enumerate(itertools.product(range(nmax+1), repeat=8)):
    basis.append(state)
    if sum(state) == 4 and not any(n > 2 for n in state):
        reduced_basis.append(state)
        reduced_state_to_index[state] = i

print(f'len(basis) = {len(basis)}')
print(f'len(reduced_basis) = {len(reduced_basis)}')

# full_state = np.zeros((len(basis), len(basis)), dtype=complex)

# for i in range(len(basis)):
#     left_state = basis[i]
#     for j in range(len(basis)):
#         right_state = basis[j]
#         if left_state in reduced_state_to_index and right_state in reduced_state_to_index:
#             full_state[i,j] = ramp_state_5mus[reduced_state_to_index[left_state], reduced_state_to_index[right_state]]


len(basis) = 65536
len(reduced_basis) = 266


In [97]:
### import eigenstate from exact diagonalization of triangle lattice Hamiltonian

eigenstate_data = [0j, (-4.7707597104803106e-26+0j), (5.990912875824446e-17+0j), (4.50251895018005e-21+0j), (-2.7541178502900948e-15+0j), (-1.129547904746919e-16+0j), (6.097224294605576e-17+0j), (7.560019496445803e-17+0j), (-8.516242594213299e-06+0j), (2.5921134049814503e-20+0j), (1.6190513310890595e-16+0j), (-8.77280601592725e-18+0j), (1.7338547543538917e-15+0j), (1.8444169160939788e-15+0j), (-0.00011414095426704862+0j), (-2.1203235785113026e-16+0j), (0.0003595705075394941+0j), (-1.7299059760892983e-17+0j), (-6.467091258330663e-17+0j), (2.0765309163470712e-16+0j), (-4.1993815931164236e-05+0j), (1.5382118322138094e-18+0j), (0.0017700848882049119+0j), (-1.1166435343653441e-17+0j), (-0.00012075215555091485+0j), (2.3035908033527952e-17+0j), (1.389811659854856e-17+0j), (7.453889935837843e-20+0j), (2.1720398103806243e-15+0j), (-9.676334982843676e-17+0j), (-1.1541230482154524e-15+0j), (-1.0891861143471487e-15+0j), (0.00037976628403070653+0j), (2.6291902682773483e-17+0j), (0.000451703090542162+0j), (5.958521338608044e-17+0j), (2.1575352181918417e-15+0j), (-1.4238013979622766e-15+0j), (0.0004831295243123984+0j), (5.034120093439648e-16+0j), (-0.015800458579631818+0j), (-5.974066326977633e-17+0j), (9.546343673103816e-05+0j), (-2.032819031785159e-17+0j), (-6.944357806819536e-18+0j), (-6.560778396252909e-17+0j), (-0.0018870490158483823+0j), (2.777243262555117e-17+0j), (0.003115152581536211+0j), (-3.1036562256561977e-16+0j), (-1.6920489417216303e-16+0j), (1.3157564065537485e-16+0j), (6.075302754562469e-17+0j), (9.194034422677078e-17+0j), (5.8695995112934e-17+0j), (5.1892626480587456e-17+0j), (3.2078220269032696e-05+0j), (2.120563924110086e-16+0j), (-0.0018012998829168576+0j), (1.1999569743760738e-16+0j), (-1.0851816391042627e-05+0j), (2.5232245069030977e-16+0j), (-1.1181098400777665e-16+0j), (1.8973538018496328e-17+0j), (0.0004848532867848321+0j), (-2.768702586460544e-16+0j), (0.0023271105766127374+0j), (1.3818191795777397e-16+0j), (1.567985896428369e-16+0j), (2.3467377804609117e-16+0j), (-1.076919316999306e-17+0j), (-2.5800587687389212e-17+0j), (-0.0002395676307727389+0j), (1.012299820535689e-16+0j), (-1.21495538760233e-17+0j), (4.581266535096497e-17+0j), (4.095078087838301e-18+0j), (-5.415623963167765e-18+0j), (-9.541483778205892e-18+0j), (-5.366923498300802e-18+0j), (-9.540979117872439e-18+0j), (5.421010862427522e-20+0j), (-7.635493799729165e-16+0j), (8.470329472543003e-17+0j), (4.769405356763734e-16+0j), (2.0486000049113606e-16+0j), (0.0013768091280972985+0j), (-8.120674271916428e-17+0j), (-0.0003496833951489645+0j), (-1.3552099890547926e-16+0j), (-1.4554872064531654e-15+0j), (6.074242671350039e-17+0j), (0.0012191621225350828+0j), (-2.1580773192780844e-15+0j), (-0.05077826222986999+0j), (6.085742268390284e-18+0j), (0.0036328384526220538+0j), (-1.972814276856924e-18+0j), (1.5689874983424405e-18+0j), (1.8111597291370352e-16+0j), (-0.0006911216763212008+0j), (4.930115936257911e-17+0j), (0.0036564280393827876+0j), (1.2267642358949383e-17+0j), (1.0916282726230137e-17+0j), (1.9006779533876246e-16+0j), (6.111809660785611e-18+0j), (-3.298831369816817e-19+0j), (-9.690056916589196e-18+0j), (1.717782817031721e-16+0j), (-0.0033533488109832865+0j), (6.632064689093831e-16+0j), (0.054613627836702226+0j), (-1.568981975143456e-18+0j), (0.0008748480313964163+0j), (4.518912622818934e-18+0j), (4.819085492742687e-18+0j), (4.721700461174372e-16+0j), (0.0761515523598111+0j), (3.94655857505858e-17+0j), (-0.09761422902904494+0j), (1.810743021569383e-16+0j), (1.4099112972468937e-17+0j), (-1.5622083527472978e-17+0j), (4.0336680307805738e-19+0j), (-1.5199998359406696e-18+0j), (-0.0002486038701790287+0j), (3.111043804869405e-18+0j), (7.417589360589077e-20+0j), (5.020939400521336e-18+0j), (3.310883615874474e-19+0j), (1.2317881083706107e-19+0j), (3.143739686379961e-18+0j), (5.447960235913283e-20+0j), (5.292090724993298e-20+0j), (3.359671481989457e-17+0j), (-0.0076022617169812035+0j), (1.418907101864592e-16+0j), (0.0035566499030110963+0j), (5.780579462083735e-18+0j), (3.479702280168585e-18+0j), (1.0470164335757563e-16+0j), (5.298327633666453e-18+0j), (3.3268117903787714e-19+0j), (0.006163860789212469+0j), (1.3218952214750103e-18+0j), (1.1662133155902168e-18+0j), (5.732518563444422e-18+0j), (5.34036760228253e-19+0j), (1.7066397617533527e-19+0j), (8.673617379884035e-19+0j), (2.3580297291397395e-20+0j), (6.89463679915147e-20+0j), (4.699182161732462e-17+0j), (3.848560003302231e-18+0j), (-1.1035176501499268e-20+0j), (3.645629804982509e-18+0j), (-4.353343722759796e-21+0j), (-1.9773608149980035e-21+0j), (-2.8917546623720815e-21+0j), (-8.942804648188538e-22+0j), (1.6155871338926322e-26+0j), (-3.8977068100853884e-17+0j), (8.330738442835495e-17+0j), (-0.00016128177881297702+0j), (-2.211772431870429e-17+0j), (0.005759307895171371+0j), (1.867822959921987e-17+0j), (-0.00022351528917908024+0j), (-1.7375902591749894e-17+0j), (-1.942429497436105e-17+0j), (-4.1226787608761306e-17+0j), (0.0011773692583648022+0j), (-3.6403177033998347e-17+0j), (-0.0046752070933520126+0j), (-5.2642575655816884e-21+0j), (5.914711018188333e-18+0j), (1.6056667623426525e-16+0j), (5.065383311717458e-18+0j), (6.1544063376733175e-21+0j), (-1.7020036731041165e-05+0j), (2.909528032927062e-17+0j), (3.816356731324327e-17+0j), (1.8467573950216374e-17+0j), (3.1435461613419944e-18+0j), (1.4949456914301223e-20+0j), (-3.6429322831111644e-17+0j), (-3.3686380317109287e-20+0j), (5.7856861178305255e-21+0j), (-2.927345865710862e-17+0j), (-0.009910731766192064+0j), (4.098781241041518e-17+0j), (0.0074541952321461715+0j), (-5.26929443010718e-18+0j), (6.0435166904993706e-18+0j), (1.0185390262274807e-17+0j), (3.970320385572163e-18+0j), (2.5187213132902345e-20+0j), (0.00451340620541854+0j), (7.498092492964578e-18+0j), (7.371912623788085e-18+0j), (-1.1750633367695908e-17+0j), (5.957739948487473e-19+0j), (-1.139447032922514e-20+0j), 0j, (-5.088059880785569e-22+0j), (-1.549763847898071e-21+0j), (4.771202665354741e-17+0j), (4.269046054161674e-18+0j), (-6.678391575112097e-21+0j), (1.5178830414797062e-18+0j), (-1.7569231890763263e-21+0j), (-3.2978462671396476e-23+0j), (-8.696692249345341e-21+0j), (1.7003013287825062e-24+0j), (-4.091520612713623e-29+0j), (-0.00048126663407117593+0j), (1.2310579905319685e-17+0j), (-3.816419430690415e-17+0j), (8.803874170612075e-17+0j), (3.0357660829594124e-18+0j), (4.552204453428297e-21+0j), (-1.9949319973733282e-17+0j), (-2.6309071098432404e-21+0j), (-1.2371759345340428e-21+0j), (4.8742464618021474e-17+0j), (-2.168404344971009e-19+0j), (-8.66603521952118e-21+0j), (-8.673617379884035e-19+0j), (-1.1199812583121385e-21+0j), (1.10718167904256e-22+0j), (-1.8011691500487557e-21+0j), (-2.543729320539466e-24+0j), (-5.480933061786687e-29+0j), (-4.0766001685454967e-17+0j), (-5.791461355270332e-21+0j), (9.617699477131126e-22+0j), (-2.783068089403824e-21+0j), (-5.2376088480568886e-24+0j), (9.364259537845382e-31+0j), (4.665850250009834e-22+0j), (5.2945130724878035e-31+0j), (2.311115933264683e-33+0j), (5.421010862427522e-20+0j), (-4.673927802949229e-16+0j), (-1.2841019480375193e-17+0j), (-1.2251755599629321e-15+0j), (1.37506206338608e-15+0j), (-0.0014293813534402996+0j), (-9.546400128734867e-17+0j), (-0.0021744516669558052+0j), (-3.214413664095908e-17+0j), (-6.179410282081133e-16+0j), (7.155327762589647e-16+0j), (-0.001520409835160273+0j), (-1.040183564282593e-15+0j), (0.05854781936234524+0j), (3.608063445721387e-18+0j), (-0.0007718788464247998+0j), (4.267835411926908e-18+0j), (5.403691092074746e-18+0j), (-4.3286771736483765e-17+0j), (0.002683450988615863+0j), (3.908976973860626e-17+0j), (-0.006926343226710627+0j), (1.633739395967887e-17+0j), (1.101313924433376e-17+0j), (-3.130507078248741e-17+0j), (4.422001197966677e-18+0j), (-5.94960390139217e-19+0j), (-8.94046663958703e-16+0j), (-5.440390976260701e-16+0j), (0.0007807757959740328+0j), (6.593439986699035e-16+0j), (0.033069046504850416+0j), (-2.0353612216662626e-18+0j), (-0.0005463218150649548+0j), (6.116749361049717e-19+0j), (2.0398282567818975e-18+0j), (-4.868287983026201e-16+0j), (-0.06005122997216667+0j), (1.0394185283138886e-17+0j), (-0.01122424693040089+0j), (-9.8332031956714e-17+0j), (3.882316118921653e-18+0j), (9.025503800329472e-18+0j), (8.935035965080263e-19+0j), (1.5531889565243898e-18+0j), (0.005659695728227896+0j), (2.1801275924742267e-18+0j), (1.069345372232435e-18+0j), (1.8510996889994037e-17+0j), (5.750088046984103e-19+0j), (2.596137837403686e-19+0j), (4.9060148304969076e-18+0j), (4.4048937996555784e-20+0j), (6.372337713992835e-20+0j), (-9.269928574751063e-17+0j), (-0.003543533017215219+0j), (5.041338630810685e-17+0j), (0.0017279323649542462+0j), (3.743370356447583e-18+0j), (2.710064837414934e-18+0j), (7.693003130185753e-18+0j), (9.791700870259712e-19+0j), (-2.312156615945512e-20+0j), (0.003103849814581976+0j), (4.480000493732166e-19+0j), (4.248344033474839e-18+0j), (5.755762606596894e-18+0j), (4.514685608865421e-19+0j), (-3.610809433175756e-20+0j), (2.137911158869854e-18+0j), (-6.644314014629561e-22+0j), (-1.0112509927965977e-21+0j), (-6.3211516927527755e-18+0j), (2.2463313761184045e-18+0j), (-3.637173613273161e-21+0j), (3.6253010142484055e-19+0j), (-3.4219926444293882e-22+0j), (-8.842425947640382e-22+0j), (-2.5290418912930726e-21+0j), (2.067608061344436e-23+0j), (-2.0278300972331116e-29+0j), (1.9607796289400348e-16+0j), (2.5221253037444047e-16+0j), (0.005749608991475586+0j), (-2.505862271157122e-17+0j), (-0.14589414690001493+0j), (2.59053388922304e-18+0j), (0.0004430856743303535+0j), (1.7087137197790883e-18+0j), (3.2273671834674486e-18+0j), (-3.9889153178457315e-16+0j), (-0.08989306564400971+0j), (1.3014342743539057e-17+0j), (0.19635143639091385+0j), (-5.266940795235675e-17+0j), (3.9539997528050423e-20+0j), (9.190616734868007e-18+0j), (3.1432833293447747e-19+0j), (1.0888907489188923e-20+0j), (0.0005942194104370989+0j), (-1.1942914000282431e-18+0j), (1.300804281046905e-18+0j), (3.1220996624901393e-18+0j), (2.7418434283532217e-19+0j), (-4.966265674265095e-21+0j), (1.0842021724855044e-18+0j), (-4.903747820332606e-22+0j), (-1.2228734571820534e-21+0j), (-5.698024517497569e-16+0j), (0.1765535749904091+0j), (1.6003305155461815e-17+0j), (-0.0656951158644541+0j), (4.544945452576602e-17+0j), (6.498506694146931e-19+0j), (-2.6831247484605e-18+0j), (2.0921713797181218e-19+0j), (-2.045108120542637e-20+0j), (-0.13698002571951642+0j), (2.040370956254598e-17+0j), (6.5703449057311e-19+0j), (-3.9165788434112955e-17+0j), (3.477070248478903e-19+0j), (-1.0426102933224823e-20+0j), (-9.67311625764411e-19+0j), (-1.3742631050593537e-21+0j), (-1.7624416314698695e-21+0j), (1.2744536290768509e-17+0j), (1.7279472123987727e-19+0j), (6.685616646386351e-22+0j), (5.21772295508649e-19+0j), (5.058420397167249e-22+0j), (7.32782881838858e-23+0j), (7.106216729266869e-22+0j), (4.623048100907998e-23+0j), (-4.2696697981931184e-29+0j), (0.004573751141417035+0j), (-6.061010338899102e-19+0j), (-8.673617379884035e-19+0j), (3.878177459294517e-19+0j), (4.404571325722362e-20+0j), (-4.515562435121646e-23+0j), (-1.2197274440461925e-18+0j), (-4.2740682551232725e-22+0j), (4.1447353861182644e-22+0j), (5.568093458460377e-18+0j), (2.913793338554793e-19+0j), (-2.3327708916662195e-22+0j), (8.639736061993863e-20+0j), (-4.39899281399047e-23+0j), (4.6315707174697345e-23+0j), (-2.585802893822532e-22+0j), (-7.349917633709583e-25+0j), (1.0595274970088822e-30+0j), (-9.75781955236954e-18+0j), (2.4615504772981327e-23+0j), (7.932527118032022e-23+0j), (1.2857004987771968e-22+0j), (-1.6967419390743424e-25+0j), (5.373839626326756e-31+0j), (-2.784762133974011e-24+0j), (-2.4495392092816385e-32+0j), (-1.1199632499436611e-35+0j), (-4.4655576979246714e-17+0j), (0.0008555207857561557+0j), (-8.299290611985063e-18+0j), (-0.008165580285269208+0j), (6.072306502571693e-18+0j), (3.1151058286359655e-19+0j), (1.6789511822089478e-17+0j), (1.8041801776516597e-18+0j), (-9.384077640643469e-22+0j), (0.0032077066823793365+0j), (-1.1976171382833676e-20+0j), (2.236166980751353e-18+0j), (5.1320157797969525e-18+0j), (2.363221922839498e-19+0j), (-1.0222714515398112e-22+0j), (7.352245982167327e-19+0j), (-2.9955144869873532e-22+0j), (8.976533595259838e-23+0j), (2.3871047700630916e-17+0j), (7.775762455794477e-19+0j), (-3.1639159463104508e-21+0j), (1.170599533105443e-18+0j), (-2.7881079580347913e-22+0j), (-8.676823683134528e-24+0j), (-2.3828868260994114e-22+0j), (1.1833205456850109e-24+0j), (4.466828574614783e-31+0j), (0.006459908851319755+0j), (8.306402776369358e-19+0j), (-2.2768245622195593e-18+0j), (2.608323938291896e-18+0j), (2.0837010502455788e-19+0j), (-6.750754242994307e-22+0j), (-2.7647155398380363e-18+0j), (4.065983924878549e-23+0j), (4.1114644832429716e-22+0j), (2.3270294286877614e-18+0j), (2.862971361719535e-19+0j), (-1.0153261572582019e-22+0j), (1.6517142471458857e-19+0j), (4.059433912094882e-23+0j), (2.3769528839078445e-24+0j), (-2.9376233035246996e-22+0j), (-1.1482084454012368e-24+0j), (1.0768250389099222e-30+0j), (-4.336808689942018e-19+0j), (3.0164976969652096e-22+0j), (5.497685044775166e-23+0j), (-5.39807403040855e-22+0j), (-4.57072664498942e-25+0j), (-3.264631694119216e-32+0j), (-1.0116936603432774e-23+0j), (-1.9488099133684347e-32+0j), (-1.382174179907699e-35+0j), (1.0610675962351968e-18+0j), (5.233604822847508e-19+0j), (1.0894261171877983e-21+0j), (8.639736061993863e-19+0j), (-9.806507230582002e-23+0j), (-5.220271657512394e-24+0j), (-1.566863378392406e-22+0j), (-7.303281717736872e-25+0j), (-1.4720989057784073e-32+0j), (1.4230153513872246e-19+0j), (2.1685366371492057e-23+0j), (2.539157772986097e-24+0j), (-1.7038238685906112e-23+0j), (1.6685215495257087e-25+0j), (-2.2532231487273723e-32+0j), (1.780461893815656e-24+0j), (2.806393580252361e-34+0j), (2.294794610163783e-37+0j), (2.7021926563540336e-22+0j), (1.6451628812321889e-25+0j), (1.5433322471335055e-33+0j), (1.1291854567911688e-26+0j), (-2.7813748360340392e-34+0j), (1.3331935722487784e-37+0j), (2.9242439143377512e-33+0j), (-3.188047753336653e-39+0j), (-1.5930919111324523e-58+0j), (7.318364664277155e-17+0j), (3.3152869555533315e-17+0j), (-9.048448720377939e-05+0j), (-1.3234042767901188e-17+0j), (0.00023307555696059363+0j), (1.4302660866888143e-17+0j), (2.452039536160076e-05+0j), (1.5990386354782157e-17+0j), (2.081649906843929e-17+0j), (2.879234394306818e-17+0j), (0.0032594452950725963+0j), (4.518187267276205e-17+0j), (-0.0015590203089898835+0j), (4.300007782126574e-18+0j), (-3.255623032965937e-19+0j), (2.917486624221039e-17+0j), (-2.4936649967166602e-18+0j), (-2.2643419685465463e-20+0j), (-0.00021912423707534832+0j), (1.7339772240566177e-17+0j), (-5.898066045851281e-17+0j), (-6.715741850685041e-18+0j), (-3.415236843329339e-18+0j), (-4.5897302517369444e-21+0j), (-9.71445146547012e-17+0j), (-9.828705474230676e-21+0j), (3.08707591137135e-21+0j), (6.033585089881832e-17+0j), (-0.0009802997805908608+0j), (6.085779468915284e-17+0j), (0.00023823487297677646+0j), (7.487719764849986e-19+0j), (3.0899761915836876e-18+0j), (6.417916147647053e-17+0j), (1.1079190950086248e-18+0j), (-9.75145715629818e-21+0j), (0.0006614991696787994+0j), (4.365531119227469e-18+0j), (-5.421010862427522e-19+0j), (3.538207982219344e-18+0j), (2.303929616531697e-19+0j), (1.0809182569241104e-21+0j), (-1.8973538018496328e-19+0j), (-6.320139698264497e-22+0j), (1.9687545760248702e-22+0j), (2.2445578092440568e-17+0j), (1.0062751413381088e-18+0j), (-9.500984477631077e-21+0j), (-5.421010862427522e-20+0j), (-3.1892297789215333e-22+0j), (1.439314350240915e-22+0j), (-2.7525269570310983e-21+0j), (-9.020109719461613e-25+0j), (4.326031740404647e-31+0j), (-2.103863599666516e-05+0j), (-3.58793374793479e-20+0j), (-1.33939625883428e-16+0j), (4.741229683768552e-17+0j), (-4.851804721872632e-18+0j), (-1.839449240963297e-21+0j), (8.673617379884035e-17+0j), (1.4438517184606798e-21+0j), (1.420231275223521e-21+0j), (1.5588403528087987e-17+0j), (3.0899761915836876e-18+0j), (-7.418204642706994e-21+0j), (2.0057740190981832e-18+0j), (-7.042329521540151e-22+0j), (4.710566262442435e-23+0j), (-4.1425345283610724e-22+0j), (-2.5767401985901416e-23+0j), (-3.521077541694606e-32+0j), (-5.551115123125783e-17+0j), (-2.664151836290316e-22+0j), (4.856058853197959e-22+0j), (-4.0285501914206727e-22+0j), (2.2000572649891715e-23+0j), (-1.6174990346410808e-32+0j), (-7.342102795786111e-23+0j), (-3.923652030756592e-32+0j), (1.908810693673632e-37+0j), (-4.0427188506553247e-17+0j), (-0.00681392054785557+0j), (7.498612021415495e-18+0j), (0.0019422209211010148+0j), (-9.049346832033824e-19+0j), (2.608861477543245e-18+0j), (5.5198147650014914e-17+0j), (9.84252284709497e-19+0j), (-5.55434300373844e-21+0j), (0.005404823403939068+0j), (7.063560674694571e-18+0j), (-3.3068166260807885e-18+0j), (6.94349068029108e-18+0j), (1.0028870095490916e-18+0j), (2.3552169967590795e-22+0j), (-5.095750210681871e-18+0j), (1.8031830505781162e-22+0j), (-7.673124596200852e-23+0j), (1.6715862767697184e-17+0j), (2.0853951161400874e-18+0j), (-2.871750152558806e-21+0j), (1.4907779871675686e-19+0j), (1.3205974258095295e-22+0j), (1.6245231234294655e-23+0j), (-8.803189047212564e-22+0j), (4.574051890920153e-24+0j), (2.770056493168231e-31+0j), (-0.0002279113790832428+0j), (5.191016256635871e-19+0j), (3.017978391067072e-18+0j), (-3.0929463641587664e-19+0j), (-7.962109704190423e-20+0j), (-4.9157658834788665e-22+0j), (-4.933119884809045e-18+0j), (-1.0874457707168337e-22+0j), (5.544632960166273e-23+0j), (-4.1194903228290766e-19+0j), (-6.352747104407253e-19+0j), (-9.897299929376775e-22+0j), (6.369687763352339e-19+0j), (7.146676732071732e-23+0j), (5.482443268475355e-25+0j), (-1.4543686844397247e-22+0j), (-2.351680837340571e-24+0j), (-3.104624872453225e-32+0j), (5.800481622797449e-18+0j), (1.4810686299569591e-22+0j), (-1.94611425966338e-23+0j), (-6.982292136523267e-23+0j), (1.3952174780014859e-24+0j), (-1.067730722539662e-32+0j), (8.40419438812934e-24+0j), (5.691824461918135e-34+0j), (2.274326992853066e-37+0j), (6.304007792754061e-18+0j), (4.1843427594362437e-19+0j), (-4.961197388725384e-22+0j), (-2.1006417091906648e-19+0j), (6.249351306023603e-23+0j), (-9.69987349109542e-24+0j), (4.1009924875446393e-22+0j), (1.668614761453529e-24+0j), (-2.0271335427741053e-32+0j), (7.589415207398531e-18+0j), (-1.1190253350697893e-22+0j), (4.168606351622917e-23+0j), (1.5439882879207446e-23+0j), (1.787409996164754e-24+0j), (1.6180740350496252e-33+0j), (9.506448710335985e-24+0j), (-5.077695703525685e-34+0j), (-6.697986499099298e-39+0j), (-5.283824254030196e-22+0j), (5.613650521943571e-25+0j), (-4.68815358972348e-33+0j), (1.830551026397222e-25+0j), (7.071859421558309e-34+0j), (-3.556005360575041e-39+0j), (-1.7055890384553675e-33+0j), (-1.2920539216043558e-40+0j), (1.1433085963291198e-60+0j), (-0.0002403182613346544+0j), (4.925137494274489e-19+0j), (-3.8163916471489756e-17+0j), (7.362089591505478e-18+0j), (-1.1275702593849246e-17+0j), (-5.804845857928296e-22+0j), (-3.122502256758253e-17+0j), (7.166343542643656e-23+0j), (5.289518774962735e-23+0j), (1.3279768257038288e-17+0j), (-8.429671891074797e-18+0j), (-7.299185280770823e-22+0j), (-3.2526065174565133e-19+0j), (-4.153882250780275e-23+0j), (-4.625597344366265e-24+0j), (-5.179591241814206e-22+0j), (-1.1981735528418017e-23+0j), (-1.5521559485411592e-32+0j), (-1.5092094240998222e-16+0j), (-1.241739620109529e-21+0j), (1.8933182883950202e-22+0j), (-7.4116726270153605e-22+0j), (1.0834171688687678e-23+0j), (-1.4352697861463819e-33+0j), (1.5155932713987015e-22+0j), (-7.75684330599848e-33+0j), (-3.6197669173452035e-38+0j), (6.2991828049897595e-18+0j), (2.439454888092385e-19+0j), (-1.4767573923714763e-21+0j), (-8.348356728138384e-18+0j), (-3.3469933640313465e-23+0j), (5.1208256146400097e-23+0j), (-1.4774769788506626e-22+0j), (-1.1395666744812548e-24+0j), (-9.17064562656297e-33+0j), (-1.111307226797642e-18+0j), (6.037074714823324e-23+0j), (1.5997814339806364e-23+0j), (-6.437834992303035e-23+0j), (2.26938212914988e-25+0j), (9.704032800404306e-34+0j), (8.101942782094848e-24+0j), (-2.425313923750122e-34+0j), (-9.377208660855334e-39+0j), (5.875114754006139e-24+0j), (9.16433833983971e-25+0j), (3.795703584899814e-33+0j), (3.1147255361163248e-24+0j), (-9.027603759773236e-35+0j), (-4.915856126154405e-39+0j), (9.171821074996044e-33+0j), (3.900778624662959e-40+0j), (1.4084062305472516e-60+0j), (5.963111948670274e-19+0j), (-5.898145909722424e-23+0j), (-1.85066443618394e-23+0j), (-1.2743751312145083e-23+0j), (1.0272127258831352e-23+0j), (5.790219951237684e-33+0j), (2.5611956287791523e-23+0j), (-1.951384448034827e-33+0j), (1.1649300279525977e-38+0j), (-5.2991257991678335e-24+0j), (1.9244891300031925e-24+0j), (7.062295856659408e-33+0j), (2.7944747584168752e-24+0j), (1.358534111855428e-33+0j), (-5.169492129754501e-40+0j), (-1.5572095089579396e-34+0j), (4.693686787985901e-40+0j), (-1.8829833409023993e-62+0j), (-3.0516580806835964e-23+0j), (-3.793404069993766e-33+0j), (9.662842398462928e-39+0j), (-2.4755220769769377e-33+0j), (2.1560579952326474e-40+0j), (-1.1066230799354697e-62+0j), (4.3600336730633796e-39+0j), (2.1064596678699095e-64+0j), (-3.770183781348654e-87+0j), (2.202285662861181e-20+0j), (-2.4328277023130673e-15+0j), (-7.868597266813548e-17+0j), (3.1933819737844926e-16+0j), (-6.750717064345213e-16+0j), (-0.0018629334018180677+0j), (-6.561794835789614e-17+0j), (-6.007811382591565e-05+0j), (4.659315174401848e-18+0j), (2.7206156164721884e-15+0j), (-1.995125120885302e-15+0j), (-0.0017455336911048511+0j), (2.3089440515794424e-15+0j), (0.07079445397362016+0j), (1.1998269182192548e-19+0j), (-0.00508497954370973+0j), (1.5323944023332021e-18+0j), (4.215053728941119e-18+0j), (-2.4157379655692646e-17+0j), (0.0019914895617923543+0j), (4.6042453780375e-17+0j), (-0.007103431925281722+0j), (-2.8873956230132197e-18+0j), (2.069900035997384e-18+0j), (3.006989030384988e-17+0j), (3.722328238620556e-18+0j), (-4.278693208541436e-21+0j), (-7.058291668152195e-16+0j), (1.0924150039420821e-15+0j), (0.004426967829194563+0j), (-6.526795434409066e-16+0j), (-0.0538199699002521+0j), (2.5444199684943557e-18+0j), (-0.001925081327300399+0j), (1.216810895451938e-18+0j), (1.8109572153252267e-18+0j), (2.730224358225841e-16+0j), (-0.11921759725101264+0j), (-4.795992328092781e-19+0j), (0.13958038725418812+0j), (-4.096460693570336e-17+0j), (-4.752297241467088e-20+0j), (3.982359036263185e-18+0j), (2.600075917462916e-19+0j), (1.5211332645330919e-21+0j), (0.0019759103803974956+0j), (1.6726492923417225e-18+0j), (1.9783495853238197e-18+0j), (2.5341738271318427e-18+0j), (9.19856669101132e-20+0j), (-4.180883213734762e-21+0j), (2.625802136488331e-18+0j), (-9.167272515368398e-22+0j), (-8.58063595602893e-22+0j), (-3.458604930228759e-17+0j), (0.006603136737591641+0j), (4.094456712633315e-18+0j), (-0.002494097301093418+0j), (2.2625938700722874e-18+0j), (4.63126321826129e-18+0j), (2.088796606595999e-17+0j), (1.2976544751935881e-18+0j), (-2.898285502399893e-21+0j), (-0.007783703036708023+0j), (5.576482977778459e-18+0j), (2.8051861022530186e-18+0j), (-1.655472038024654e-19+0j), (4.2859867131067597e-19+0j), (-1.70821857042874e-21+0j), (1.849919956803392e-18+0j), (-3.6676785174285627e-22+0j), (-4.306689807914805e-22+0j), (1.4874752395852965e-17+0j), (1.650020181251377e-18+0j), (2.6237429691745775e-21+0j), (2.6495190590114515e-18+0j), (-5.597077346190234e-23+0j), (9.211419927460834e-24+0j), (-3.6369425829725667e-22+0j), (8.03722334700561e-24+0j), (-2.4260677197881624e-29+0j), (2.9889039851525682e-16+0j), (9.455869247332327e-16+0j), (0.002672924009922194+0j), (-3.7604197099944114e-16+0j), (-0.08369188500853364+0j), (5.943526864626415e-18+0j), (0.003231993172189091+0j), (1.2843903647357292e-18+0j), (-1.2486272205862441e-19+0j), (9.802271841441446e-16+0j), (-0.0297488210523301+0j), (1.7433828699388048e-18+0j), (0.08317196669159756+0j), (6.480541010797514e-18+0j), (4.365374695417407e-19+0j), (1.367825528559158e-18+0j), (3.11413972563912e-19+0j), (-5.1885511374867965e-22+0j), (0.00015811527666930247+0j), (2.5924445512654194e-18+0j), (1.5178830414797062e-18+0j), (2.6911525055833534e-18+0j), (-2.371692252312041e-20+0j), (-9.323320953367114e-22+0j), (5.421010862427522e-19+0j), (-7.759206145884458e-23+0j), (6.569033768848256e-23+0j), (-6.042800808347959e-16+0j), (0.1500721888363539+0j), (9.516140729172574e-19+0j), (-0.12025511636442017+0j), (2.0101979497941914e-17+0j), (-1.019248150210149e-19+0j), (1.6561573851562559e-18+0j), (1.2281977735187355e-19+0j), (-1.1710001676637027e-21+0j), (-0.06353023806753615+0j), (-4.6279750427118455e-18+0j), (4.336808689942018e-19+0j), (2.151276416515289e-18+0j), (2.879912020664621e-19+0j), (4.819995581042746e-22+0j), (2.1429933565533799e-19+0j), (3.153315905096411e-23+0j), (5.677260436514827e-23+0j), (2.1950952527091047e-18+0j), (1.5500702934753696e-19+0j), (-1.722894371360026e-23+0j), (-6.378158092824882e-19+0j), (-2.5575717186839127e-23+0j), (2.785524950143408e-24+0j), (-1.2812824143412516e-22+0j), (-9.746070800404828e-25+0j), (8.015132593202188e-31+0j), (0.007313226359099404+0j), (-8.38536104418344e-21+0j), (-1.4094628242311558e-18+0j), (2.068325152617741e-18+0j), (-5.421010862427522e-20+0j), (-3.731858322882528e-23+0j), (-3.308510691975297e-18+0j), (-2.0513193225792441e-22+0j), (-1.158825278618195e-22+0j), (9.945967697878052e-19+0j), (-5.421010862427522e-20+0j), (-4.1165633296921076e-22+0j), (-2.2404021454876244e-19+0j), (-2.5857401644382396e-23+0j), (5.543704727260622e-24+0j), (1.3601994717041134e-22+0j), (8.092959130972642e-24+0j), (7.321771482149325e-31+0j), (4.607859233063394e-18+0j), (-2.0227725734586968e-23+0j), (1.4733318037132964e-23+0j), (-7.049960291247082e-23+0j), (-8.665758947514575e-25+0j), (-3.2005521175096345e-32+0j), (9.853819074288742e-24+0j), (-1.3913599293553846e-32+0j), (-1.4024408028567458e-35+0j), (-1.6738726290460582e-16+0j), (-0.0010901001894210057+0j), (-1.2340569361539712e-17+0j), (-0.0026840646327200104+0j), (1.976491030944563e-18+0j), (6.352747104407253e-19+0j), (-1.6027926124975608e-18+0j), (5.836057006582129e-19+0j), (-5.3024089630300805e-21+0j), (0.0017441313648045448+0j), (1.5516827212934088e-18+0j), (1.6296913905172739e-18+0j), (2.6182000314442156e-19+0j), (2.7316812548951186e-19+0j), (-3.184235318599533e-23+0j), (5.539595475043124e-19+0j), (-6.601308295703273e-23+0j), (-9.822703706966391e-24+0j), (-2.658792720077459e-18+0j), (4.688327363052552e-19+0j), (-1.744933646522693e-21+0j), (4.764560328305439e-19+0j), (-1.6483420080810066e-23+0j), (4.402089753868545e-25+0j), (-3.674408016953434e-22+0j), (5.11407856743634e-25+0j), (-1.2123217072127867e-32+0j), (0.0036336639190927035+0j), (-2.730464222336998e-19+0j), (9.147955830346444e-20+0j), (8.338678823773211e-19+0j), (2.50801161726078e-20+0j), (-1.8083127371052297e-22+0j), (3.9979955110402976e-19+0j), (-2.761946150637991e-23+0j), (2.0153953400408816e-23+0j), (4.036754314258586e-19+0j), (6.961552035246281e-20+0j), (-6.285208953588011e-23+0j), (1.98840984367947e-19+0j), (-7.50887816114809e-24+0j), (-1.078488425559738e-24+0j), (-4.917233010897266e-23+0j), (-2.2475198159785117e-25+0j), (-2.4818798323047436e-32+0j), (-1.6263032587282567e-19+0j), (4.1427711830732794e-24+0j), (1.671762901948064e-24+0j), (-2.79786729905994e-23+0j), (2.527469294909147e-25+0j), (9.398758754205681e-34+0j), (2.954953059892942e-25+0j), (4.489246445746414e-34+0j), (1.722148779140068e-37+0j), (1.0630344960938958e-17+0j), (1.6977716636528382e-19+0j), (-4.619429258616823e-22+0j), (1.7406527066075872e-19+0j), (-1.7605120504207754e-24+0j), (1.5720809619316276e-25+0j), (-3.3923703579489045e-22+0j), (2.127038322519271e-26+0j), (-1.365967341199303e-32+0j), (-3.9302328752599536e-19+0j), (-5.483939107066406e-23+0j), (2.0527341826537263e-24+0j), (3.130015766089399e-23+0j), (-1.498506944419116e-25+0j), (7.114279155588646e-34+0j), (8.477109184858977e-25+0j), (2.766332640940841e-34+0j), (1.4522343147348636e-37+0j), (-1.7061587260313478e-24+0j), (4.4325781108159696e-26+0j), (2.908956472552063e-33+0j), (3.346821587046868e-26+0j), (1.2368083198321703e-35+0j), (-3.202096921003907e-39+0j), (1.2539303363657326e-33+0j), (-1.2497223255370932e-39+0j), (5.9493881195928396e-61+0j), (-1.07073773675595e-15+0j), (-9.719059324703183e-16+0j), (-0.007782021217223236+0j), (3.43312617917535e-16+0j), (0.1619927573380449+0j), (1.441252768803222e-18+0j), (0.000716143405581117+0j), (9.072321340475158e-19+0j), (1.3008815851488102e-18+0j), (-1.3805959413887292e-15+0j), (0.15404912289628286+0j), (9.421091208891243e-19+0j), (-0.28309083296897886+0j), (3.959053108645922e-17+0j), (2.5397691312705534e-19+0j), (-1.3001047590875458e-18+0j), (1.8634724839594607e-19+0j), (-3.0551201675930905e-22+0j), (-0.00138708450541603+0j), (-6.983602293792528e-19+0j), (-1.463672932855431e-18+0j), (1.5059286007534352e-19+0j), (-1.6940658945086007e-20+0j), (-1.880693336450632e-22+0j), (-6.776263578034403e-19+0j), (3.162514105773534e-22+0j), (7.199668582460202e-23+0j), (3.4193026014761596e-17+0j), (-0.2150912451367855+0j), (3.33928881561685e-18+0j), (0.08288772541958181+0j), (-2.2223956676538398e-17+0j), (-2.574980159653073e-19+0j), (-8.028006561852e-19+0j), (1.411368648362478e-19+0j), (-3.605778830234889e-22+0j), (0.19644755753314094+0j), (-2.383063238406085e-17+0j), (2.270048298641525e-19+0j), (2.802685712865268e-17+0j), (-1.4584848560534984e-19+0j), (8.14716810516435e-23+0j), (3.1509625637859973e-19+0j), (2.6394021949020802e-23+0j), (2.752270577848384e-23+0j), (1.3339931094852604e-18+0j), (2.3656042030036507e-19+0j), (2.3173017473186705e-22+0j), (-4.0657581468206416e-19+0j), (-6.334222157432795e-23+0j), (-6.683302262843306e-24+0j), (-1.5386369848852148e-22+0j), (-3.7228374969020162e-25+0j), (4.968932287910327e-31+0j), (-0.0046220038286997845+0j), (1.5382233345557371e-18+0j), (1.6940658945086007e-19+0j), (-7.082048076025873e-19+0j), (9.147955830346444e-20+0j), (-1.8288481610197414e-23+0j), (-3.74049749507499e-18+0j), (-5.120675452877067e-23+0j), (-5.415985089869332e-24+0j), (-5.308521301200585e-19+0j), (-3.9471735342050396e-19+0j), (5.405984816095569e-23+0j), (-5.793705359219414e-19+0j), (4.679250246764764e-23+0j), (-2.274125308646545e-24+0j), (2.351398684085607e-23+0j), (3.790214162283725e-24+0j), (-1.6787487555378515e-32+0j), (2.927345865710862e-18+0j), (-1.372707513310762e-22+0j), (2.471430671554683e-23+0j), (-6.039915483328934e-23+0j), (-1.1764017486711327e-24+0j), (-1.0387170056687732e-32+0j), (5.630932197942605e-24+0j), (4.261833747432899e-34+0j), (1.186594374717233e-37+0j), (1.0635345685724995e-15+0j), (-0.07092731906694419+0j), (3.902986834521796e-18+0j), (0.14369337284750888+0j), (-1.9190414961637185e-17+0j), (1.6347735882007997e-19+0j), (7.617841697399901e-19+0j), (2.4034559878340772e-20+0j), (-3.033993141656643e-22+0j), (-0.01433306387532545+0j), (4.451965916759397e-18+0j), (4.0657581468206416e-20+0j), (8.96951528392631e-19+0j), (1.402898318889935e-21+0j), (2.334871378717932e-23+0j), (-2.0328790734103208e-19+0j), (-4.082003465722651e-23+0j), (-1.319324504480092e-25+0j), (1.6564495026967416e-18+0j), (1.2003217868756985e-19+0j), (-3.935089115861065e-22+0j), (1.7110065534536867e-19+0j), (3.0690147392201754e-24+0j), (-5.797343907739148e-25+0j), (-4.202330972830228e-25+0j), (-2.7722060568127377e-25+0j), (-1.744257512872842e-32+0j), (-0.10028976995894616+0j), (1.4999669912619312e-17+0j), (3.6845933205562065e-19+0j), (-1.2140112855510351e-17+0j), (-1.521482931505537e-19+0j), (-1.683063468768228e-23+0j), (-1.8973538018496328e-19+0j), (4.496300213916863e-24+0j), (-8.412079399544414e-24+0j), (-2.015212244940385e-18+0j), (-6.541211935171334e-19+0j), (-2.1804131791435517e-23+0j), (1.2959604092990795e-19+0j), (-2.5605239884113365e-23+0j), (1.3628834315141333e-25+0j), (-5.2418807091085714e-24+0j), (-9.914758536076235e-26+0j), (-3.0915841677139427e-32+0j), (-2.236166980751353e-19+0j), (-1.3980836886763146e-23+0j), (-1.3382426539380794e-24+0j), (1.3812264789522834e-23+0j), (-1.5048864987671214e-25+0j), (1.8868364001922444e-33+0j), (1.4002499737758845e-24+0j), (8.779398586774591e-34+0j), (2.5378472824720955e-37+0j), (6.979111890410099e-19+0j), (4.0234064994579266e-20+0j), (-8.644125459929911e-23+0j), (-2.0328790734103208e-20+0j), (1.7945642372657206e-24+0j), (8.0149066383972545e-25+0j), (1.957710466268547e-23+0j), (-6.054466848968038e-26+0j), (-1.3621004055954793e-34+0j), (-3.5575383784680614e-19+0j), (-7.56860274843014e-24+0j), (-9.118664446577967e-25+0j), (-6.191855977984077e-24+0j), (3.431855339840538e-26+0j), (8.929840971104468e-34+0j), (3.695985834468497e-25+0j), (-9.022393243309972e-35+0j), (-6.45519960541199e-39+0j), (4.401688320978624e-23+0j), (1.39254320452901e-25+0j), (-2.1907828265406613e-34+0j), (6.918611849556683e-26+0j), (-1.729928773973971e-35+0j), (-4.4411480957291056e-39+0j), (7.828221762360776e-34+0j), (1.7830554723591249e-40+0j), (1.2188127578653691e-60+0j), (0.0024973295180318635+0j), (-2.0040846308362157e-20+0j), (-3.0899761915836876e-18+0j), (4.483156247198654e-19+0j), (1.6263032587282567e-19+0j), (-2.5761731349899238e-23+0j), (-1.4094628242311558e-18+0j), (-2.188645920118239e-23+0j), (9.55412898599146e-25+0j), (4.807982576718945e-19+0j), (1.6940658945086007e-20+0j), (-8.304717203833414e-24+0j), (-3.2695471764015993e-19+0j), (1.0198636225138283e-24+0j), (1.4791516866713226e-24+0j), (-1.741811391261756e-23+0j), (1.4909838853444424e-24+0j), (-2.9399076979866947e-34+0j), (2.8731357570865868e-18+0j), (2.4147305332965017e-23+0j), (1.747466805337925e-24+0j), (1.256311439355905e-23+0j), (3.5077818400948487e-25+0j), (-2.5849469749467503e-34+0j), (3.088220554770972e-24+0j), (2.9278065346579562e-34+0j), (-3.0694608862279495e-39+0j), (-5.241201798856168e-22+0j), (3.441071348220595e-20+0j), (-3.855163563341608e-23+0j), (6.606856988583543e-20+0j), (-6.453201970632419e-24+0j), (7.198634904427872e-26+0j), (3.950932421843386e-23+0j), (1.6147101819147199e-25+0j), (8.551542323014634e-35+0j), (-9.994988777600744e-20+0j), (-2.979017021759196e-24+0j), (7.402238488025424e-25+0j), (-1.5157439896010981e-24+0j), (1.8751267580157934e-26+0j), (-1.3832275058697358e-34+0j), (1.756416632845113e-25+0j), (-5.933886299401292e-35+0j), (-4.4795105387233125e-39+0j), (1.4453042499803487e-23+0j), (9.102719029668895e-27+0j), (-8.72219505977837e-34+0j), (3.6039962516822903e-25+0j), (7.068463344910386e-36+0j), (2.3804365679507077e-40+0j), (1.5003342651618905e-34+0j), (5.798166758560767e-41+0j), (-2.258004403942516e-62+0j), (-2.710505431213761e-20+0j), (-3.7139520463263345e-24+0j), (8.180396842522924e-24+0j), (-1.5696236799333868e-23+0j), (2.9194863895929673e-25+0j), (-5.638113094457692e-34+0j), (-1.46992251954845e-24+0j), (2.5650658527666173e-35+0j), (7.715780909798792e-40+0j), (8.384897224902761e-25+0j), (1.2944205421689067e-25+0j), (3.2878978772795073e-35+0j), (1.0391496596070074e-25+0j), (8.773289603474384e-36+0j), (1.5961241515002999e-40+0j), (2.5680233575232003e-34+0j), (1.1521413869320452e-41+0j), (-1.2439867997011314e-62+0j), (6.18526931759628e-25+0j), (3.0985863774880936e-34+0j), (-1.9939357060116e-40+0j), (2.7121479260695363e-34+0j), (-1.476494090412994e-41+0j), (3.903082624448163e-64+0j), (6.788450280575144e-41+0j), (1.1160286707223006e-64+0j), (1.4598600732052665e-89+0j), (-7.533849846058649e-17+0j), (0.009729035625242006+0j), (1.3558651148967665e-17+0j), (-0.007055767402917388+0j), (1.861478241383535e-18+0j), (7.254837193233082e-19+0j), (1.9452195479219176e-17+0j), (6.496742705440484e-19+0j), (1.1250531575015526e-21+0j), (-0.006505761122047564+0j), (1.6564361791169986e-18+0j), (3.9302328752599536e-19+0j), (-1.3055081691227902e-18+0j), (4.5263323118901674e-20+0j), (-1.1807252817243516e-22+0j), (2.778268066994105e-19+0j), (-1.7643061892213368e-22+0j), (1.0687693633845757e-24+0j), (4.581066737978144e-18+0j), (4.516273795641523e-19+0j), (-6.104378574424184e-22+0j), (4.290221877843031e-19+0j), (-5.398052530526024e-23+0j), (8.605276488873214e-25+0j), (1.1555063539796607e-22+0j), (8.153785693206051e-25+0j), (-1.6308720461232105e-33+0j), (0.00368085834011289+0j), (-5.799769196072926e-19+0j), (1.8295911660692887e-19+0j), (2.4196104427476247e-19+0j), (5.1880768019325896e-20+0j), (-1.662915008814896e-22+0j), (-3.3881317890172014e-20+0j), (2.722407491180728e-24+0j), (4.222435742278918e-24+0j), (4.245665553348701e-19+0j), (4.298692207315574e-20+0j), (5.912253792374406e-23+0j), (1.088437337221776e-19+0j), (1.3918878317662014e-24+0j), (4.099964809409444e-26+0j), (-8.307513008785826e-24+0j), (-1.5685993351116424e-26+0j), (6.57417715393923e-34+0j), (3.7269449679189215e-19+0j), (-1.5262772712667707e-24+0j), (1.8504926693870294e-24+0j), (1.9420163033258023e-23+0j), (6.563748593068121e-26+0j), (3.662330710158571e-35+0j), (1.1372852105987482e-25+0j), (-1.2518906929549376e-34+0j), (-4.0438791082135026e-39+0j), (2.0199435492576442e-18+0j), (7.39036246479377e-20+0j), (-1.4346736076712723e-22+0j), (1.359487880343152e-19+0j), (5.402450808467704e-24+0j), (-2.0146080867479037e-25+0j), (2.0028110581440183e-22+0j), (2.182019922238621e-26+0j), (-2.4982606950583376e-33+0j), (1.8634724839594607e-20+0j), (3.3761456112303347e-23+0j), (-9.804847341560858e-26+0j), (-1.3150879269886026e-24+0j), (-1.8748769697168312e-26+0j), (-1.522631148767052e-34+0j), (-3.346300196132595e-26+0j), (3.885012273298766e-35+0j), (1.4978560806249806e-40+0j), (-4.1998803132672866e-23+0j), (7.312139743385848e-26+0j), (2.3133496021199858e-33+0j), (7.200525589402887e-26+0j), (-7.470314526125195e-35+0j), (1.2149219068919265e-40+0j), (3.117900363821984e-33+0j), (-1.7462527550188102e-41+0j), (-1.0083001500180521e-62+0j), (0.003042980832073251+0j), (-2.0466465395094005e-19+0j), (-5.421010862427522e-20+0j), (6.002166305578206e-19+0j), (8.300922883092143e-20+0j), (8.556682853363861e-24+0j), (-4.391018798566293e-18+0j), (-1.5130584959498813e-23+0j), (-8.756547913716523e-24+0j), (8.048732086748225e-19+0j), (8.809142651444724e-20+0j), (-5.231355394802549e-24+0j), (-2.1514636860259229e-19+0j), (-2.1857364818353685e-24+0j), (1.684820445429723e-24+0j), (-2.326962195426573e-23+0j), (3.4437283243172763e-25+0j), (-5.501305296653651e-34+0j), (1.6263032587282567e-18+0j), (8.828825728963583e-24+0j), (9.160846411701934e-25+0j), (1.0838223643935116e-24+0j), (3.231305465262712e-25+0j), (1.611266637061418e-34+0j), (9.195776065295773e-24+0j), (-3.1756356412998345e-34+0j), (1.730131053401786e-40+0j), (5.964457706714078e-20+0j), (-1.5246593050577406e-20+0j), (6.99751411537081e-23+0j), (1.7279472123987727e-19+0j), (-2.0283869457831513e-24+0j), (-3.0536341299395775e-25+0j), (-4.452558141008094e-24+0j), (1.1868679676645002e-25+0j), (-4.677387530837886e-34+0j), (-1.1011428314305904e-20+0j), (-4.3385432159925345e-24+0j), (4.157708541743848e-25+0j), (4.75386514147907e-24+0j), (1.747312171580096e-25+0j), (-1.6074706168277274e-34+0j), (4.975712690353384e-26+0j), (3.116628981733736e-35+0j), (1.2514959723417032e-40+0j), (3.3666412489621616e-23+0j), (4.2747017079223634e-26+0j), (1.975347772813553e-34+0j), (4.6023532484263844e-26+0j), (4.404435848469843e-35+0j), (7.374709207226744e-41+0j), (4.05820661481586e-34+0j), (-1.3037285033150022e-42+0j), (-2.172991307166599e-62+0j), (-4.0826988057657276e-19+0j), (-4.2332421875821695e-24+0j), (8.275350337815106e-24+0j), (2.143641788605439e-23+0j), (-1.7145909052225389e-25+0j), (-8.739045151407808e-35+0j), (2.571991615668953e-25+0j), (3.480274013673491e-35+0j), (-9.369786912555586e-42+0j), (-6.501122626783952e-24+0j), (5.58029409700323e-26+0j), (3.3630829666150186e-34+0j), (9.138658254435254e-26+0j), (1.1181485362042064e-35+0j), (-1.3674495726105282e-41+0j), (3.231137348932526e-34+0j), (-8.508684275380289e-42+0j), (4.7855991203874025e-64+0j), (7.682367744483157e-25+0j), (-7.3283004018524024e-37+0j), (1.5127389203901933e-42+0j), (2.3817154667773002e-34+0j), (3.570508487099634e-42+0j), (2.8147234988077685e-64+0j), (-4.2857312232910205e-41+0j), (-1.102715917229092e-65+0j), (1.699848203716493e-89+0j), (1.4739327795691914e-18+0j), (3.366955965335844e-20+0j), (-1.4864779795688713e-22+0j), (1.291725244562808e-20+0j), (3.672705265907198e-24+0j), (1.3970966431408907e-25+0j), (1.1285845482520371e-22+0j), (1.4835969397074737e-25+0j), (1.8892161334041864e-33+0j), (-7.623296525288703e-20+0j), (4.2029499288216826e-24+0j), (-3.1771142362271964e-26+0j), (-1.0996089930056728e-24+0j), (3.0486588554035527e-26+0j), (-1.574534504896885e-35+0j), (1.3380957517372386e-25+0j), (8.02899605398826e-35+0j), (1.1065611179927878e-40+0j), (-1.7311016139659554e-24+0j), (2.903078336898726e-26+0j), (1.790661989655447e-33+0j), (2.4544210856193154e-26+0j), (2.3920671854281734e-35+0j), (5.4890256474022847e-42+0j), (1.7939510578613306e-33+0j), (-8.942611238187481e-42+0j), (2.272762394302746e-64+0j), (-1.6940658945086007e-21+0j), (1.4029354773940145e-24+0j), (3.776001304784554e-25+0j), (-1.2171429569963618e-24+0j), (8.656804822791245e-27+0j), (1.4879301881887934e-35+0j), (9.822632289905849e-26+0j), (9.232274444961852e-36+0j), (9.405764281330733e-42+0j), (9.89446145313369e-25+0j), (2.5726709496839703e-26+0j), (1.1924001464198643e-34+0j), (-1.76774778744548e-26+0j), (7.560954331297857e-36+0j), (-4.503053117231817e-43+0j), (2.6043317513814177e-35+0j), (-6.0255833965967134e-43+0j), (3.442877736057334e-64+0j), (-8.182216220616564e-26+0j), (-3.264053436420417e-36+0j), (1.9663020051405833e-41+0j), (4.6967796621837256e-35+0j), (5.261875733539688e-43+0j), (-1.1050674248268343e-65+0j), (1.9127724038033753e-43+0j), (-4.3125287367133267e-66+0j), (-2.3046501095570932e-91+0j), (1.2378022774710138e-23+0j), (1.4610027656152666e-26+0j), (1.4328542327861494e-33+0j), (4.059017689570474e-27+0j), (4.829664526506343e-36+0j), (5.582773081870071e-42+0j), (-6.791420549030763e-35+0j), (5.482580241670847e-43+0j), (-6.901199188298897e-66+0j), (3.0673669883360375e-26+0j), (3.5156812968369264e-35+0j), (3.609744844100729e-42+0j), (-1.2999621488841608e-35+0j), (-5.083210179338274e-43+0j), (-6.581028017054045e-66+0j), (5.5950344434329134e-42+0j), (2.026224052656518e-67+0j), (-1.4144271144575407e-91+0j), (5.417661501865892e-35+0j), (1.1911036946760945e-44+0j), (-6.319036367505127e-67+0j), (-4.241117383433079e-43+0j), (6.3098762049501e-68+0j), (2.8497065722739e-93+0j), (-1.158091586055297e-67+0j), (1.0302199730833535e-93+0j), (-1.984832206659219e-263+0j), (4.9873299934333204e-18+0j), (2.636813564802637e-17+0j), (6.625273749604092e-06+0j), (8.822758706071836e-17+0j), (0.0012294559147695632+0j), (4.805592686458596e-18+0j), (-0.00013013413458707728+0j), (6.413901346960018e-18+0j), (-2.4286199643627506e-17+0j), (-3.7499842640842385e-17+0j), (-0.0012676677328911838+0j), (2.315256039554532e-17+0j), (0.0005608103490019227+0j), (-2.848458698501605e-18+0j), (-1.1384122811097797e-18+0j), (6.332958420612159e-18+0j), (-2.2632720350634905e-18+0j), (-1.3953826620854024e-21+0j), (2.47666633410617e-05+0j), (9.466350975066597e-18+0j), (-2.3418766925686896e-17+0j), (-1.5452111965428939e-18+0j), (-8.300922883092143e-19+0j), (4.0626496410208106e-22+0j), (-4.423544863740858e-17+0j), (-2.1030734823727635e-21+0j), (6.551538293363129e-23+0j), (4.2824291747282917e-17+0j), (-0.0032783620663029435+0j), (1.940655981048135e-17+0j), (0.0032601050429366843+0j), (-1.2076786866547774e-18+0j), (-1.4501204056993622e-18+0j), (9.055741454187317e-18+0j), (-3.3711911300721153e-19+0j), (-2.196089904597953e-21+0j), (0.00041648155321934735+0j), (1.0357124882048203e-18+0j), (-5.305814381600937e-18+0j), (1.2965982841669334e-18+0j), (4.353749348887104e-19+0j), (1.7815430672101956e-22+0j), (-4.2825985813177425e-18+0j), (4.47103374396641e-23+0j), (1.9197756657566768e-23+0j), (3.9031873310624e-18+0j), (-1.6771252355635147e-19+0j), (-6.170651291891093e-22+0j), (2.2768245622195593e-18+0j), (-2.9076921071131265e-23+0j), (-3.4259254808900837e-23+0j), (1.1517954062330575e-22+0j), (6.0040497963355986e-24+0j), (-1.7417589261617724e-32+0j), (-0.00021872373807622998+0j), (5.0826605777280904e-18+0j), (1.6479873021779667e-17+0j), (-1.0173383239608602e-18+0j), (-5.637851296924623e-18+0j), (-2.006643345065613e-21+0j), (-4.2500725161431774e-17+0j), (-5.178911238522772e-22+0j), (4.229879487521733e-22+0j), (2.3973650469527124e-18+0j), (-1.0299920638612292e-18+0j), (2.7883270266003974e-22+0j), (4.933119884809045e-18+0j), (-1.4535834279766123e-22+0j), (1.1537592750215858e-23+0j), (-5.5954628875707755e-22+0j), (1.544575155254715e-23+0j), (-6.766045566313267e-33+0j), (-7.719519468096792e-17+0j), (-3.570943296792835e-22+0j), (3.3211168462170725e-22+0j), (-7.824968208924353e-23+0j), (-4.454284361261712e-24+0j), (-1.9053823031348623e-33+0j), (1.2600312300029323e-22+0j), (-2.33521356745654e-33+0j), (1.0656176771108427e-37+0j), (4.66613509983449e-17+0j), (0.0018889753438960275+0j), (-1.3767132854210081e-18+0j), (-0.0018568863471047773+0j), (9.194102088765964e-19+0j), (6.306160292308266e-19+0j), (7.736278272548322e-18+0j), (4.0816400145816598e-19+0j), (-7.6401689986720375e-22+0j), (-0.00047379842965344937+0j), (5.311682687484635e-19+0j), (-4.851804721872632e-18+0j), (3.1333131738299366e-19+0j), (-9.486769009248164e-20+0j), (-2.41293285026761e-22+0j), (4.174178364069192e-18+0j), (3.8340609055464445e-23+0j), (2.5850537016844505e-23+0j), (9.852119583853436e-18+0j), (3.1170812458958252e-19+0j), (-1.0490037276068778e-21+0j), (-4.946672411965114e-19+0j), (2.353851667170781e-23+0j), (2.1429062295832845e-24+0j), (4.264149722247327e-22+0j), (9.918456198309942e-25+0j), (-9.719945320494593e-33+0j), (0.0008370691754092189+0j), (-2.3421178957404366e-19+0j), (-7.697835424647081e-18+0j), (1.0278860903076954e-18+0j), (6.776263578034403e-20+0j), (3.5844404951938405e-23+0j), (5.421010862427522e-20+0j), (-3.4433655300746564e-24+0j), (9.550649936836113e-24+0j), (3.7716070422630606e-19+0j), (4.0657581468206416e-20+0j), (-2.4529419043524185e-23+0j), (-7.962109704190423e-19+0j), (-1.02844106341859e-23+0j), (2.510483270726992e-24+0j), (7.243986302148023e-23+0j), (6.116614890343904e-25+0j), (5.552356648967879e-34+0j), (-1.0842021724855044e-18+0j), (-8.232746455323668e-24+0j), (5.9828689219082775e-24+0j), (-3.2168077723792422e-24+0j), (2.7638080455495095e-25+0j), (1.1441573244682962e-34+0j), (3.2071493737766617e-24+0j), (-5.070303634851188e-34+0j), (-2.982188708218197e-39+0j), (4.041773899779524e-18+0j), (4.743384504624082e-20+0j), (1.4741867067784165e-22+0j), (-1.4907779871675686e-19+0j), (-3.554180495955976e-23+0j), (1.3468694904269006e-24+0j), (2.5921213521798686e-23+0j), (1.4081056918928009e-24+0j), (-1.7875992385504076e-33+0j), (7.318364664277155e-19+0j), (-1.5663341621255385e-23+0j), (-2.6038165003079273e-24+0j), (6.013099802892687e-24+0j), (5.951526647682855e-25+0j), (3.353692270558198e-34+0j), (2.3556904444094116e-24+0j), (-3.573727823465341e-35+0j), (-2.9971177956788345e-39+0j), (6.212255647243949e-23+0j), (1.151932789781357e-25+0j), (-1.5017388488975974e-33+0j), (5.761991539034145e-25+0j), (3.2935319849688e-35+0j), (-1.0705941875917397e-40+0j), (1.7845114307042134e-34+0j), (2.7656430716038368e-40+0j), (-1.0572495298902982e-62+0j), (-1.1297141411216477e-05+0j), (2.213052291067895e-19+0j), (-3.469446951953614e-17+0j), (3.138052915466696e-19+0j), (-1.4094628242311558e-18+0j), (-8.14908188191963e-23+0j), (-3.252606517456513e-17+0j), (2.812407436448844e-22+0j), (1.762325032567978e-22+0j), (4.202090953756539e-18+0j), (5.149960319306146e-19+0j), (5.595125951132715e-22+0j), (2.3445871979999033e-18+0j), (-1.9514312305426133e-23+0j), (2.3797143588178222e-24+0j), (8.242985740091668e-23+0j), (6.591410603044286e-25+0j), (-2.7724328137731356e-34+0j), (9.920449878242366e-18+0j), (3.445724239166206e-23+0j), (6.54251047073251e-23+0j), (-3.395317920588756e-23+0j), (2.1087158252281885e-24+0j), (-4.7422001567449e-33+0j), (2.0702616302781215e-23+0j), (-3.190626867692665e-33+0j), (1.512476363861133e-39+0j), (3.625602664713874e-18+0j), (6.437450399132683e-20+0j), (1.4222121033951525e-22+0j), (-1.1926223897340549e-18+0j), (2.1514759934369065e-24+0j), (6.665706528866766e-24+0j), (-2.3235240883375587e-23+0j), (2.2655820907396016e-25+0j), (6.430726435520601e-33+0j), (-2.574980159653073e-19+0j), (-1.99104958380928e-23+0j), (2.760634144842198e-24+0j), (1.4159005641435028e-23+0j), (-4.0258619446546016e-27+0j), (9.611330836340064e-35+0j), (3.835911341649957e-25+0j), (-1.5128378114567637e-34+0j), (2.2251007809238714e-40+0j), (1.8482316811731712e-23+0j), (2.289968418807691e-25+0j), (-2.6032960285033535e-33+0j), (6.462057015411851e-25+0j), (-2.8816879538283e-35+0j), (2.9338540402062955e-40+0j), (1.9976388076455528e-33+0j), (-4.0422859575123e-41+0j), (1.6145086044036304e-64+0j), (-3.713392440762853e-18+0j), (9.128067306493372e-24+0j), (4.5735654235736556e-23+0j), (4.1352568279115813e-23+0j), (8.076016969504924e-26+0j), (9.725721589531939e-34+0j), (-1.237214467073012e-23+0j), (9.61925107391184e-34+0j), (4.362690404565716e-39+0j), (-1.1684249269738294e-22+0j), (2.5997504768197532e-24+0j), (1.710163341560123e-33+0j), (-2.696512842320244e-25+0j), (6.888927819780768e-35+0j), (-1.0421833495904497e-40+0j), (1.7764693876960804e-33+0j), (4.271287715385971e-41+0j), (1.3568127175742236e-64+0j), (-2.084137892914945e-23+0j), (2.6814912357251686e-33+0j), (5.6338924498486405e-40+0j), (3.4539379681839124e-33+0j), (1.3647526003752258e-40+0j), (-1.8674390529600556e-65+0j), (-3.5828399135856923e-40+0j), (-8.327490599256324e-66+0j), (-6.774846949231237e-92+0j), (7.235694248625135e-17+0j), (0.0035964622282034925+0j), (6.512966206548106e-18+0j), (-0.005213825486052969+0j), (1.5822380807035587e-18+0j), (2.8672065264558066e-19+0j), (9.042419836154582e-18+0j), (4.183912625517716e-19+0j), (-1.0480778159780746e-21+0j), (-0.0006080811329916221+0j), (2.885399981187269e-19+0j), (3.5067164016328034e-19+0j), (1.4192682912806718e-19+0j), (-3.8963515573697816e-20+0j), (-5.994175027486294e-23+0j), (1.4568966692773966e-19+0j), (-3.9998884509894964e-23+0j), (2.5332333474692014e-24+0j), (5.047747865848652e-18+0j), (4.0149361699853836e-19+0j), (-1.1671387146420718e-21+0j), (-1.7787691892340307e-19+0j), (2.6219944336200615e-23+0j), (1.3454596218852037e-24+0j), (-1.3164433919941986e-22+0j), (2.0139991154032083e-25+0j), (-3.1539461174882324e-33+0j), (0.0037051924586153635+0j), (-5.244562449220343e-19+0j), (-3.7947076036992655e-18+0j), (1.1377394890138961e-18+0j), (2.879912020664621e-20+0j), (-8.334995230855734e-23+0j), (-9.75781955236954e-19+0j), (1.6966965762485584e-23+0j), (1.1555576902228759e-23+0j), (7.63639180577823e-19+0j), (3.3881317890172014e-19+0j), (-6.35565393975256e-23+0j), (-4.0826988057657276e-19+0j), (1.3493943870239047e-23+0j), (1.1319299493323794e-24+0j), (2.411371583151362e-23+0j), (8.398655338498014e-25+0j), (-3.996579773750004e-35+0j), (-6.478107980600889e-18+0j), (-9.257307386783215e-25+0j), (1.3417413820289317e-23+0j), (-1.1663003425268836e-23+0j), (1.7376525316381777e-25+0j), (-1.8329047193765179e-34+0j), (3.3190773958163614e-24+0j), (-1.8468439701608586e-35+0j), (1.0247330391500853e-39+0j), (3.7296004760238666e-19+0j), (4.086933970501999e-20+0j), (-2.275540709211757e-22+0j), (2.371692252312041e-20+0j), (4.9485651068010855e-24+0j), (1.3370139306632317e-24+0j), (7.560301551763962e-23+0j), (1.2839279740510014e-25+0j), (-1.965077947031449e-33+0j), (9.656175598699024e-20+0j), (4.220839488058522e-24+0j), (1.2107027115522798e-24+0j), (2.070374912083408e-24+0j), (7.331907350510677e-26+0j), (-1.477045156417927e-34+0j), (1.522810249993811e-25+0j), (-6.454504596980199e-35+0j), (-1.3192826271099542e-40+0j), (7.085965169253085e-24+0j), (1.0427912067722412e-26+0j), (8.42014455428926e-34+0j), (1.5995496096999433e-25+0j), (3.277488840364665e-34+0j), (3.9043993510068056e-41+0j), (7.233473192215509e-34+0j), (2.1148460226833537e-41+0j), (-6.794636845630608e-63+0j), (8.516367566742e-05+0j), (1.1328208136981004e-19+0j), (-4.716279450311944e-18+0j), (1.491540439748569e-19+0j), (3.8624702394796095e-19+0j), (-1.3256140691029491e-24+0j), (-4.8382521947165635e-18+0j), (-1.2382983615454473e-23+0j), (4.8837034843873444e-24+0j), (1.8944718267120838e-20+0j), (1.8973538018496328e-19+0j), (4.822334480627972e-23+0j), (2.473336205982557e-19+0j), (6.06247722484224e-25+0j), (3.7906212772961355e-25+0j), (9.800967931370618e-24+0j), (5.541191501109357e-25+0j), (2.39939978344426e-35+0j), (1.0706496453294356e-18+0j), (-2.226279070504047e-24+0j), (1.3101017375425352e-24+0j), (1.4112153614552142e-24+0j), (3.367870622764385e-25+0j), (-1.7020245584663653e-35+0j), (1.3571414526901821e-24+0j), (1.5667970773596487e-34+0j), (-4.9157759305817e-40+0j), (2.4913340211336653e-19+0j), (-3.5045988192646677e-20+0j), (5.205866213991064e-24+0j), (9.571472303973594e-20+0j), (5.174510654818528e-24+0j), (1.4250543996415394e-25+0j), (-9.288010432748742e-24+0j), (7.083539875670477e-26+0j), (7.129756650933633e-35+0j), (8.978549240895584e-20+0j), (-1.8510589586574833e-24+0j), (1.4933798428672295e-25+0j), (3.999991795126421e-24+0j), (-3.173977813722389e-27+0j), (7.005495762997808e-36+0j), (2.790932888991969e-25+0j), (-3.414403138645656e-35+0j), (8.80349523300926e-41+0j), (-3.9392053292137104e-24+0j), (2.606701567874335e-26+0j), (2.5637983507280818e-34+0j), (-4.62925238583519e-26+0j), (4.6858850262494964e-36+0j), (5.570647916001268e-41+0j), (4.792433071117367e-34+0j), (3.5774767585622486e-42+0j), (5.156156617758926e-64+0j), (-5.065257024580716e-19+0j), (-1.1208135741380136e-25+0j), (2.680154194731886e-24+0j), (4.422467830639344e-24+0j), (6.354721781579749e-28+0j), (5.298281925026666e-35+0j), (-1.180181596523851e-24+0j), (1.130605188998854e-34+0j), (-2.2565163272562005e-41+0j), (-1.1888701821532407e-24+0j), (-7.818238313224505e-26+0j), (5.5083074088525876e-37+0j), (7.418375585767887e-26+0j), (1.2010327978530856e-35+0j), (-1.2021303751881318e-41+0j), (1.6433380837163541e-34+0j), (1.0442476156148537e-41+0j), (1.6399582567538906e-64+0j), (-3.512501694005919e-25+0j), (5.413464282258802e-35+0j), (1.1849379814330653e-41+0j), (5.757100260867487e-35+0j), (8.404988189020253e-42+0j), (-1.3243440045451786e-65+0j), (-6.038054952929204e-41+0j), (-2.896975095565474e-66+0j), (-2.3091042159791937e-91+0j), (5.35339141275863e-19+0j), (3.705769144237564e-20+0j), (6.470103353813213e-23+0j), (3.9387032047324966e-20+0j), (-2.8014280901698242e-24+0j), (2.024512023575706e-25+0j), (2.4976977089980093e-24+0j), (7.34586184007817e-27+0j), (2.414966083714032e-33+0j), (1.249373597200093e-19+0j), (2.762250102172928e-24+0j), (2.557407811389159e-25+0j), (1.2761118873834428e-24+0j), (-9.553619669476802e-27+0j), (-3.721277332269056e-35+0j), (7.111469096655965e-26+0j), (1.0343810753696202e-34+0j), (-5.713154988753405e-41+0j), (1.024443801601318e-23+0j), (-1.0897050437319714e-26+0j), (-2.523647757013438e-34+0j), (-8.792679127736474e-27+0j), (1.6802507300280226e-34+0j), (-5.773349673018246e-42+0j), (1.3611681260767945e-33+0j), (4.74759919713248e-42+0j), (-7.040189092514335e-66+0j), (-1.1468826105823227e-18+0j), (2.1354022942225866e-24+0j), (1.46470272622236e-24+0j), (-2.259398606748846e-24+0j), (3.9775302856220865e-26+0j), (3.177342949269189e-34+0j), (-9.209873301843662e-25+0j), (3.7259417503893463e-35+0j), (-8.938571240844437e-41+0j), (2.6190686924316933e-24+0j), (-1.6331856155500475e-26+0j), (5.806082053640581e-34+0j), (4.0808062480618774e-26+0j), (4.0317094608086953e-35+0j), (-2.2420775429197073e-43+0j), (-6.086227271407189e-35+0j), (1.700615816304598e-41+0j), (-1.1133192864789865e-65+0j), (-4.213024050555766e-25+0j), (9.091627037349363e-35+0j), (1.0922841269719084e-40+0j), (1.2232972125881015e-34+0j), (-5.809783433090692e-42+0j), (1.7468057202548156e-67+0j), (-9.604499674482296e-42+0j), (-1.8419424556964097e-67+0j), (5.596767276172434e-93+0j), (5.7810756360471794e-24+0j), (-4.778902193976532e-28+0j), (-3.5218285712342426e-35+0j), (-5.39727971411412e-26+0j), (1.4532037253237306e-35+0j), (2.4405014054681014e-41+0j), (2.3280463553038926e-34+0j), (-1.7810503481568425e-42+0j), (-1.8616918608160733e-66+0j), (-1.64653706715706e-25+0j), (3.156895398483754e-35+0j), (-3.0203587100057107e-41+0j), (7.168990262510777e-35+0j), (-2.3436716815832566e-43+0j), (1.4743552607476632e-67+0j), (4.560175527529036e-42+0j), (-1.533496051151444e-67+0j), (2.89562922679032e-93+0j), (-7.987414107768763e-35+0j), (5.574540453392163e-43+0j), (-1.3946034364578915e-67+0j), (1.3649522854063921e-43+0j), (-1.720688478050109e-68+0j), (-6.754573373968438e-95+0j), (-5.662036340491552e-68+0j), (-3.399702857476568e-95+0j), (1.0253314378498982e-265+0j), (-0.00011720859516435001+0j), (5.3163967375236877e-20+0j), (1.3010426069826053e-17+0j), (5.371981320354699e-19+0j), (-4.607859233063394e-19+0j), (-1.1439033693236367e-22+0j), (-2.927345865710862e-18+0j), (-2.0043721389232394e-24+0j), (1.4245792066764176e-22+0j), (7.505834616025947e-19+0j), (-3.3881317890172014e-18+0j), (-5.75423477870472e-23+0j), (3.178067618098135e-18+0j), (-4.8281496430924995e-27+0j), (8.320525706891305e-24+0j), (6.721488711846907e-23+0j), (6.2442293312388484e-24+0j), (-1.5239139988128829e-33+0j), (-2.3798237686056822e-17+0j), (3.9652970614260764e-23+0j), (2.74456754298554e-23+0j), (5.389275561239042e-23+0j), (3.840826399824113e-24+0j), (1.2946332976308928e-33+0j), (2.9078263679152356e-24+0j), (-1.9997330816971247e-34+0j), (-1.5710957250980826e-39+0j), (1.3484950102696173e-18+0j), (1.1096131609031334e-19+0j), (1.8783203098864531e-22+0j), (1.0842021724855044e-19+0j), (2.4015702745313977e-24+0j), (8.601400200946433e-24+0j), (1.4420730757125635e-23+0j), (7.039095605020644e-26+0j), (-2.564078735977289e-33+0j), (-9.554531645028508e-19+0j), (8.004426454871046e-24+0j), (3.5996378692013533e-25+0j), (-5.7999578106745495e-25+0j), (2.2399493618525753e-25+0j), (2.495282096836231e-34+0j), (1.860537777537844e-26+0j), (2.346476565431e-35+0j), (8.203305475492966e-41+0j), (-3.462203227931911e-24+0j), (1.898912151674938e-25+0j), (1.794262615521765e-33+0j), (1.1552287492619627e-24+0j), (1.5154437154067281e-34+0j), (-4.236339506925959e-40+0j), (3.698570269007629e-33+0j), (5.806980836162042e-42+0j), (1.6877679708438918e-64+0j), (-8.080694316806025e-18+0j), (1.9181058247140276e-24+0j), (-5.585100383457667e-24+0j), (-2.4923662714561637e-23+0j), (7.075108531022088e-25+0j), (1.3914282664439632e-33+0j), (4.706266851414828e-24+0j), (1.1847360710502546e-33+0j), (3.6483085778389477e-40+0j), (3.9056011168287437e-23+0j), (1.5737692026665743e-24+0j), (2.2568754957711548e-33+0j), (-4.581211647866425e-25+0j), (1.5348722284964408e-34+0j), (1.0916675556476055e-40+0j), (3.6064331036479987e-34+0j), (7.532259505438757e-41+0j), (-1.499184442966732e-65+0j), (8.443758312161234e-24+0j), (4.484351954596979e-34+0j), (2.3381281448583876e-39+0j), (1.85499697125483e-34+0j), (-6.841139102833757e-41+0j), (-7.140663028070985e-66+0j), (5.10739659083252e-40+0j), (-2.9760429669226954e-66+0j), (-6.899238198445243e-92+0j), (7.741938160422696e-19+0j), (3.134021904840911e-20+0j), (1.290660249524146e-22+0j), (5.69629657028517e-20+0j), (-8.27867237084932e-24+0j), (1.6224027999926004e-25+0j), (9.96817261611754e-24+0j), (1.5360989957778398e-25+0j), (9.247136054161433e-34+0j), (-1.0147454708106518e-18+0j), (-5.00832011506716e-25+0j), (2.3811886521100296e-25+0j), (-1.7210041943791264e-24+0j), (-6.043671025789655e-26+0j), (3.828272329017291e-34+0j), (1.0285660131691984e-24+0j), (1.8421536587354407e-34+0j), (1.1725535442709993e-40+0j), (2.4153835445261797e-23+0j), (8.465242049496132e-26+0j), (6.324764315467515e-34+0j), (7.995984372741462e-26+0j), (1.3233025487392426e-34+0j), (-5.201619899573721e-42+0j), (2.184113216235002e-34+0j), (-3.486430579240145e-42+0j), (-1.0912081220906088e-65+0j), (-1.4177213954668852e-18+0j), (-3.1872504925825584e-25+0j), (7.285731870230294e-25+0j), (2.9364305900457273e-24+0j), (1.5201045251450887e-26+0j), (-1.220697833980618e-34+0j), (-7.532402307052009e-25+0j), (8.936852352948798e-35+0j), (-2.719864267315897e-40+0j), (4.304933866648655e-24+0j), (-2.7544179850064594e-26+0j), (1.9229227803035702e-34+0j), (-3.944030827121939e-26+0j), (3.061043502909393e-35+0j), (-4.450523922695619e-42+0j), (1.4005473113991694e-34+0j), (2.3037346753499993e-42+0j), (-6.437598422464367e-66+0j), (-1.2497862704935454e-24+0j), (-6.137853434109405e-35+0j), (5.93505951580133e-41+0j), (2.0448115862964346e-34+0j), (7.868290877183848e-43+0j), (-3.73118634239796e-67+0j), (-5.3224818921217364e-42+0j), (2.701599624468876e-68+0j), (4.822001631087043e-93+0j), (1.619727075923232e-23+0j), (-5.133563029511069e-26+0j), (6.473357599256898e-35+0j), (-1.0112483774290647e-25+0j), (1.2897491872556406e-34+0j), (-3.693822751960218e-42+0j), (9.70551865904737e-34+0j), (-8.381516439742812e-43+0j), (-3.89850078813953e-66+0j), (-1.0118476053357377e-24+0j), (9.964289483234009e-35+0j), (-9.903256507076347e-41+0j), (1.1243765444507268e-35+0j), (2.939573853537385e-42+0j), (-3.410303256819901e-67+0j), (-2.7119504342386225e-41+0j), (-2.9300031151926347e-67+0j), (1.7024031105850479e-93+0j), (6.319259899016082e-34+0j), (1.7228964618873626e-42+0j), (6.031111213424258e-67+0j), (-2.0060025422579858e-42+0j), (-9.558212448189629e-69+0j), (-1.057139572832642e-95+0j), (-5.428416660651565e-68+0j), (5.91413599530104e-95+0j), (1.2067043734111217e-265+0j), (1.1019898643778447e-18+0j), (5.5586294825493376e-24+0j), (-2.1591315459998398e-23+0j), (-4.673287733176175e-24+0j), (-1.7034357571923092e-25+0j), (2.1930981141931745e-34+0j), (-2.6270310012646956e-24+0j), (-3.238522989122292e-35+0j), (7.210431694928062e-39+0j), (1.0958931425977197e-23+0j), (1.9620931227098294e-25+0j), (3.066611527067087e-33+0j), (-7.575642003680321e-25+0j), (-6.375114909136195e-35+0j), (1.0094954136995982e-41+0j), (7.167644570484217e-34+0j), (-1.2203067546726237e-40+0j), (-4.294864680131637e-66+0j), (-7.105424193191934e-24+0j), (4.00041021570338e-34+0j), (-2.3459137591261763e-41+0j), (3.6211858585255208e-34+0j), (-1.949906813107983e-42+0j), (-1.6578164368601495e-66+0j), (-1.2262552666536825e-40+0j), (-6.644108978189739e-67+0j), (3.0428385126482286e-94+0j), (5.509959920140822e-24+0j), (-5.62342915924073e-26+0j), (3.159550909565236e-33+0j), (-4.0244530514371114e-25+0j), (5.384162786089828e-35+0j), (-1.3767196892605598e-40+0j), (-2.231820491508929e-34+0j), (-2.7377868746746114e-42+0j), (-3.93944222114437e-66+0j), (2.711395147395582e-26+0j), (5.848011944286639e-35+0j), (-8.197245691684099e-42+0j), (2.1181563983952595e-35+0j), (-6.854714181706904e-42+0j), (-2.9492569375796784e-68+0j), (-1.0514467864753224e-41+0j), (-7.386941999526056e-68+0j), (-2.4216497790396065e-94+0j), (3.390648261007199e-34+0j), (4.339208275935816e-43+0j), (-5.4257544836207946e-67+0j), (-1.5995865760844797e-41+0j), (-1.3616346101183092e-67+0j), (1.5632484381213687e-94+0j), (9.402345662202222e-68+0j), (1.644826263290115e-95+0j), (-1.3590442928937816e-267+0j), (-1.3695727425680415e-25+0j), (6.038076375980127e-36+0j), (-4.430092991085763e-40+0j), (4.393026033801216e-34+0j), (-5.0074962716877237e-42+0j), (-5.79817536689833e-67+0j), (-1.6929071420806161e-40+0j), (-4.480304803183714e-67+0j), (3.934287777558032e-93+0j), (2.0686863407095217e-34+0j), (-1.2029512352861907e-41+0j), (-2.2426026048693973e-67+0j), (-4.842301793739057e-42+0j), (-7.442935797498509e-68+0j), (4.526143843134082e-95+0j), (-1.7701232003075846e-67+0j), (-2.2679926497163048e-95+0j), (-8.390388169831093e-268+0j), (-2.1396533538689307e-40+0j), (-1.760556908042475e-67+0j), (-3.931175255629206e-95+0j), (-3.55585841719677e-67+0j), (6.18089475579669e-96+0j), (1.379661385824063e-269+0j), (-5.278797457470282e-95+0j), (5.28154565404014e-270+0j), 0j, 0j, (4.135960237489078e-16+0j), (-2.9896874906287785e-17+0j), (8.558149848514329e-16+0j), (-2.6125206598753836e-16+0j), (0.002358711007946839+0j), (-4.286664339464563e-17+0j), (0.002370413061542909+0j), (-8.419742030728631e-19+0j), (3.3499136624370873e-16+0j), (1.4190444609304964e-15+0j), (0.002380448873624651+0j), (3.4206578541917665e-17+0j), (-0.09405186334170663+0j), (4.801118310867901e-19+0j), (0.003643662312907425+0j), (-3.996786357061222e-19+0j), (1.1069682210278076e-18+0j), (9.276704838329097e-17+0j), (-0.0035700319970263096+0j), (1.217418427213835e-17+0j), (0.010130047876202775+0j), (-1.059829758394423e-18+0j), (2.282571868110029e-18+0j), (2.8866186368633202e-18+0j), (2.2968646640067877e-18+0j), (8.387860170685553e-22+0j), (4.910622689729971e-16+0j), (3.126839065448195e-16+0j), (-0.0033344575525739314+0j), (-5.800481622797449e-18+0j), (-0.012772120658719943+0j), (-1.238728065973437e-19+0j), (0.0019822301935981165+0j), (-9.507236249673872e-19+0j), (1.910781359467314e-18+0j), (-1.1372196587200456e-15+0j), (0.1386195857253669+0j), (-1.2126084528371732e-18+0j), (-0.07046287799955397+0j), (4.184486517700694e-17+0j), (4.465312625482275e-19+0j), (9.344410833022489e-19+0j), (1.8507669897506462e-19+0j), (-1.4703748513998394e-20+0j), (-0.007035540287485968+0j), (-4.853845785474623e-19+0j), (1.1519078640031403e-18+0j), (-1.1579815068095171e-18+0j), (1.9608812728937053e-19+0j), (-2.2155617435568158e-21+0j), (4.535861432546778e-19+0j), (-1.2587334020659715e-22+0j), (-4.509988855738867e-22+0j), (6.811500148640182e-17+0j), (0.001345104890463747+0j), (1.99669845535859e-17+0j), (-0.0011578533985751739+0j), (5.318086335039271e-19+0j), (1.785545452812065e-18+0j), (-6.819801718128855e-18+0j), (6.598386659111e-19+0j), (2.5716578083803787e-22+0j), (0.0007547140950756454+0j), (-1.3086608030389637e-18+0j), (1.4416500762268192e-18+0j), (1.0848728978776069e-18+0j), (2.9010878443459787e-19+0j), (5.100537046095081e-22+0j), (1.328147661294743e-18+0j), (-1.1336036183996084e-22+0j), (8.68262674116856e-24+0j), (3.83157581391309e-18+0j), (9.537590986083422e-19+0j), (-1.0268226524968898e-21+0j), (8.364450354136216e-19+0j), (-1.1767593947015108e-22+0j), (8.266344841483432e-24+0j), (1.2345830941393588e-23+0j), (-4.479459875597941e-25+0j), (1.949805704020375e-31+0j), (-4.290187996525141e-16+0j), (2.569084810340183e-16+0j), (-0.006549677756147287+0j), (5.816744655384731e-17+0j), (0.1868382787841538+0j), (-5.381401066608784e-19+0j), (-0.0032509302354601474+0j), (7.311033773980798e-19+0j), (7.3845411159050805e-19+0j), (-6.716361408004579e-16+0j), (0.08657868275362453+0j), (5.245595033615181e-19+0j), (-0.21621641415154771+0j), (1.925024210041912e-17+0j), (2.3073096014730707e-19+0j), (7.379942182358958e-19+0j), (1.32560656245298e-19+0j), (-3.994739139644065e-22+0j), (-0.0008351689201632735+0j), (-5.339672892755508e-20+0j), (5.929230630780102e-19+0j), (-9.17233901737465e-19+0j), (4.8492636230308694e-20+0j), (1.6328060150228664e-23+0j), (3.8709405689521525e-19+0j), (-3.6164171435488137e-23+0j), (-1.7025653595804905e-24+0j), (9.670337989577116e-16+0j), (-0.28251350533233166+0j), (1.3085601403495012e-18+0j), (0.15086642222383312+0j), (-4.022679505748213e-17+0j), (1.257843926672636e-19+0j), (4.942063023538403e-19+0j), (5.405128994666504e-20+0j), (1.4834412989116915e-22+0j), (0.1737628598676272+0j), (-8.744046322569985e-18+0j), (1.6855955650360577e-19+0j), (2.919438682572072e-17+0j), (-7.835054762102278e-20+0j), (1.6709496786915664e-22+0j), (1.3167391862864116e-19+0j), (7.831322469777708e-24+0j), (2.472427102101888e-23+0j), (2.869741544743037e-19+0j), (2.218167530622199e-20+0j), (-4.709321386277588e-23+0j), (9.145308852386274e-20+0j), (-1.64725546392652e-23+0j), (-4.282892008273035e-25+0j), (-1.7512602702122656e-24+0j), (-7.6536764138739855e-25+0j), (2.6582402232148707e-31+0j), (-0.009734148321725908+0j), (1.859764217184503e-18+0j), (2.0328790734103208e-19+0j), (-7.768955629709765e-19+0j), (3.703122166277394e-20+0j), (-4.6805172716680465e-23+0j), (2.490276864927643e-19+0j), (-9.398771359496348e-24+0j), (-6.4833806519874656e-24+0j), (-7.239740554747935e-19+0j), (1.0037340424963459e-19+0j), (-1.2178368360441435e-23+0j), (8.75620309224133e-20+0j), (-2.2253449381475685e-24+0j), (-9.740909924451452e-25+0j), (-8.174155539662686e-24+0j), (1.1867088511843549e-25+0j), (-1.5379696602721535e-32+0j), (-1.9481757786848908e-19+0j), (-1.7590100059186048e-23+0j), (-1.8196127288966383e-24+0j), (-1.2877174722192412e-23+0j), (1.633627196422191e-25+0j), (-6.620061290226632e-33+0j), (5.64079369842978e-25+0j), (4.942928720035016e-34+0j), (1.1053725168974813e-37+0j), (-3.57109090562413e-18+0j), (0.0012102384757079242+0j), (1.8087522508704412e-17+0j), (0.007074651529048737+0j), (-3.0450388026775306e-19+0j), (5.166900978251232e-19+0j), (1.4905033692766066e-17+0j), (8.584678920422334e-19+0j), (-1.3176289994441022e-21+0j), (-0.003621267397452377+0j), (5.101051132433124e-19+0j), (1.0426975580700437e-18+0j), (4.57493371485628e-19+0j), (1.0577323928838075e-19+0j), (-7.356784179281105e-23+0j), (6.51368336438557e-19+0j), (-5.021041899356404e-23+0j), (-2.091420389831927e-24+0j), (1.570411790056682e-18+0j), (7.085430603782222e-19+0j), (9.177568492914852e-23+0j), (4.0234064994579266e-19+0j), (2.7849894586738805e-23+0j), (2.157857515712542e-26+0j), (-2.655690808732919e-22+0j), (1.3879339560784594e-25+0j), (-4.271927203144632e-33+0j), (-0.007873971086058865+0j), (7.172041248239389e-19+0j), (5.149960319306146e-19+0j), (-1.2349687734779993e-18+0j), (2.5172760401213738e-20+0j), (1.3502544409948e-23+0j), (8.809142651444724e-20+0j), (-1.7120021221366334e-23+0j), (2.171129952531466e-24+0j), (2.8327402872100104e-19+0j), (3.705769144237564e-20+0j), (1.0307601568711619e-23+0j), (6.410980619530986e-20+0j), (-5.7502688184223224e-24+0j), (2.746176112130773e-25+0j), (-3.9010126220942505e-24+0j), (1.8553940698064155e-26+0j), (-1.0467954987593032e-32+0j), (2.202285662861181e-20+0j), (-1.2611250558331213e-23+0j), (4.649306058130897e-25+0j), (-1.811722658681252e-23+0j), (1.397707064470093e-27+0j), (3.914660674467152e-34+0j), (4.618183329711706e-25+0j), (1.9548705234689844e-34+0j), (6.415741613112721e-38+0j), (1.4427102850921248e-18+0j), (1.1501119236937297e-19+0j), (-2.740007752341482e-22+0j), (1.8994713842177685e-19+0j), (9.897464000224674e-24+0j), (1.508393355269378e-25+0j), (-1.2542643956172953e-23+0j), (-1.045456056904222e-26+0j), (-1.1522364604072993e-33+0j), (2.39710324072967e-19+0j), (3.400389333264818e-24+0j), (6.322297221130622e-27+0j), (3.826094144643965e-24+0j), (2.595851059217629e-26+0j), (3.8437684927967315e-34+0j), (1.1817316372067834e-25+0j), (1.193484924519638e-35+0j), (-3.525421217020819e-39+0j), (-5.0634549882707115e-23+0j), (9.2166387638907e-27+0j), (2.4671642777042802e-34+0j), (2.2460192826717164e-26+0j), (1.1059140392144835e-35+0j), (-1.2033490314282138e-39+0j), (3.1130980790980166e-34+0j), (6.367554317897986e-41+0j), (6.881001438873793e-61+0j), (-2.940356291780688e-16+0j), (-6.880889087679254e-16+0j), (0.004596789105550721+0j), (-1.1673537028515646e-15+0j), (-0.06290838650856824+0j), (1.1191813558378628e-18+0j), (-0.001342721484763332+0j), (6.4420149787919565e-19+0j), (7.589415207398531e-19+0j), (1.1959021013058235e-15+0j), (-0.11990336759427571+0j), (1.870728011641176e-18+0j), (0.15273682699728766+0j), (-2.906851001915575e-17+0j), (1.3552527156068805e-20+0j), (2.8015351618875397e-18+0j), (-9.825582188149884e-20+0j), (3.0819205937072606e-22+0j), (0.003156456789914524+0j), (7.687453814228278e-19+0j), (-2.100641709190665e-18+0j), (1.9909918820917632e-18+0j), (-3.9985249066323315e-19+0j), (-6.155324163999862e-23+0j), (2.1141942363467336e-18+0j), (-9.973914834332778e-23+0j), (-1.1266041129494798e-22+0j), (4.0680620764371733e-16+0j), (0.08571925612927088+0j), (-3.1885070529675286e-18+0j), (-0.029546706993407525+0j), (7.243734482976001e-18+0j), (-1.4399560103323106e-19+0j), (4.510277198582804e-19+0j), (8.983843196815923e-20+0j), (-5.071332623842e-22+0j), (-0.08941278135016961+0j), (1.669273432341668e-17+0j), (-1.971469184734384e-19+0j), (-1.0004988524745895e-17+0j), (2.2896359355467806e-19+0j), (-1.2733983556991285e-22+0j), (7.962109704190423e-20+0j), (6.534325733366084e-24+0j), (-2.870023292968091e-24+0j), (3.1046015903495224e-19+0j), (8.24798332388875e-20+0j), (-5.975362175951345e-23+0j), (-6.081696561285876e-19+0j), (-3.345407053132474e-24+0j), (-3.4482876722600055e-24+0j), (9.008529951144357e-24+0j), (1.2830442367811537e-25+0j), (-1.0392195111366505e-32+0j), (0.001181528158113709+0j), (3.804358556710691e-19+0j), (-1.8973538018496328e-19+0j), (-7.203541720433642e-19+0j), (-4.421511984667448e-19+0j), (-1.669528408800599e-23+0j), (-1.3552527156068805e-18+0j), (-8.882042130328096e-23+0j), (6.241144019555221e-23+0j), (8.465506395251068e-19+0j), (-3.1170812458958252e-19+0j), (-7.693908753952274e-24+0j), (-8.131516293641283e-20+0j), (9.029191456185697e-24+0j), (4.643346961252771e-25+0j), (-5.296393327311261e-23+0j), (5.217693305703022e-25+0j), (3.3545945543079504e-34+0j), (3.63207727782644e-18+0j), (-1.5037883129144092e-23+0j), (6.05067689000842e-24+0j), (-3.186010668794718e-23+0j), (-2.5036118993528084e-26+0j), (2.405024707583937e-34+0j), (6.450981760883333e-24+0j), (-1.2106985699194919e-34+0j), (3.722404465230464e-40+0j), (-1.2172473315766319e-15+0j), (0.13505470253340798+0j), (1.9377979963269304e-18+0j), (-0.11918125423207183+0j), (2.914075058596689e-17+0j), (2.2552252220645747e-20+0j), (-1.6047784697374223e-18+0j), (5.638063055161437e-20+0j), (-3.614650479713921e-22+0j), (-0.055105236425131314+0j), (-1.796648880731329e-18+0j), (-8.893845946170154e-20+0j), (-1.866082894566473e-17+0j), (-2.9005584487539447e-19+0j), (-6.371279828236751e-24+0j), (-4.438452643612534e-19+0j), (9.845178008740602e-24+0j), (2.5228740568767182e-24+0j), (1.4401064814998486e-18+0j), (4.282810339554556e-20+0j), (-6.205617589501576e-23+0j), (8.999725064576941e-20+0j), (-5.40273288967504e-24+0j), (2.335911156555581e-25+0j), (2.1525198963887704e-24+0j), (1.3390039031708737e-25+0j), (-4.385602282314947e-33+0j), (0.0756026609401255+0j), (-1.805851622750429e-17+0j), (-2.405573570202213e-19+0j), (1.1344550951758769e-17+0j), (7.623296525288703e-21+0j), (1.5044394477676808e-24+0j), (-2.236166980751353e-19+0j), (2.4640084022801225e-25+0j), (1.1529202992621703e-24+0j), (8.767688815340362e-18+0j), (4.147285067993868e-19+0j), (1.4685722722769575e-23+0j), (8.682087709356578e-21+0j), (-4.480976925731097e-23+0j), (7.712089560938459e-26+0j), (5.930864599285896e-24+0j), (7.273959680852609e-26+0j), (1.0742721190132476e-33+0j), (-1.9142944607947188e-19+0j), (-7.813300832175308e-24+0j), (1.2333377009200996e-24+0j), (-6.630497415130703e-25+0j), (-3.1856589098543066e-26+0j), (2.4280428155184568e-34+0j), (3.5101529149217364e-25+0j), (-5.623783083637199e-35+0j), (-4.6536526614358525e-39+0j), (1.0894432463243065e-19+0j), (1.249373597200093e-20+0j), (3.105887137764609e-23+0j), (2.4775713707188285e-20+0j), (-5.740700423601922e-24+0j), (3.4859566648713393e-25+0j), (1.1002148381808825e-23+0j), (2.0662543292117198e-26+0j), (-3.897740443477523e-34+0j), (-3.7947076036992655e-19+0j), (5.13200777689162e-24+0j), (5.536847927472397e-26+0j), (6.026140009419518e-25+0j), (-1.5671030705081982e-26+0j), (-1.0502914028874369e-34+0j), (1.0826667366949322e-25+0j), (5.495368648401703e-35+0j), (1.870277146273814e-40+0j), (9.666057822079618e-24+0j), (6.524433665481796e-26+0j), (2.693038042690927e-35+0j), (4.116853344010872e-26+0j), (7.978246036895801e-36+0j), (1.3475885861087583e-40+0j), (2.0791512277829647e-34+0j), (-6.224359311610392e-42+0j), (-1.6539179947453653e-62+0j), (0.0004452308854351643+0j), (5.313311536434035e-19+0j), (-3.876022766635678e-18+0j), (4.653971853979373e-19+0j), (6.776263578034403e-20+0j), (-1.1123924328148577e-23+0j), (-1.463672932855431e-18+0j), (-2.298073330031396e-23+0j), (3.3129167903651014e-23+0j), (-5.1058033825493255e-20+0j), (2.236166980751353e-19+0j), (1.5702753460104983e-23+0j), (-2.727446090158847e-19+0j), (3.320971770385649e-24+0j), (7.637248462559956e-25+0j), (2.926824250873325e-23+0j), (7.819789394987971e-25+0j), (4.211982685053457e-34+0j), (-5.149960319306146e-19+0j), (-4.374135688835954e-24+0j), (5.3663417180345146e-24+0j), (-2.86021016117962e-25+0j), (5.014093658224794e-26+0j), (-7.493973932808823e-35+0j), (5.3208361490758335e-24+0j), (-1.3503045952914661e-34+0j), (-4.847793643031855e-39+0j), (2.7140590069292945e-19+0j), (-2.498747194400186e-20+0j), (1.6379857165619048e-23+0j), (1.6432439176733427e-19+0j), (1.8345509818256625e-24+0j), (-1.0216082744346078e-24+0j), (1.8110731770936407e-23+0j), (1.185967401137316e-25+0j), (-9.872472875682485e-34+0j), (-9.317362419797304e-20+0j), (-3.4107472087214055e-24+0j), (1.523963530126987e-26+0j), (2.7493254051017868e-24+0j), (1.8495195884196546e-25+0j), (-1.400854640591041e-35+0j), (5.58033221142014e-26+0j), (3.0165443621333533e-35+0j), (2.336973448319752e-40+0j), (3.5736787401705024e-24+0j), (5.579887575735461e-26+0j), (-1.0396788355542343e-35+0j), (1.1724897511692072e-25+0j), (1.4643443933950545e-35+0j), (2.269398160153075e-40+0j), (4.609458148814374e-34+0j), (1.698216089632818e-42+0j), (-1.4153435018376434e-62+0j), (3.2017845406212553e-19+0j), (1.6543612251060553e-24+0j), (7.194897625189282e-24+0j), (7.797631301732789e-24+0j), (-6.352016751057219e-26+0j), (-2.8374686458535615e-34+0j), (4.3651300262488e-25+0j), (4.5287047720115817e-35+0j), (-1.4041932971946233e-40+0j), (9.234696057330286e-24+0j), (1.5316468205876044e-25+0j), (1.9770896625869296e-34+0j), (1.1689693016283165e-26+0j), (-4.811613847194222e-35+0j), (2.356806974836982e-41+0j), (1.7333648062373703e-34+0j), (-1.279105238235693e-41+0j), (4.61959394075303e-64+0j), (-2.4106941126598934e-25+0j), (4.982533337375704e-35+0j), (1.8536525573331027e-40+0j), (3.6052683623010722e-34+0j), (7.20267410662956e-42+0j), (2.084640377016439e-64+0j), (-1.9424799312470614e-41+0j), (-9.078808529013167e-66+0j), (1.691977911113569e-89+0j), (1.6576180667882481e-16+0j), (-0.003397565901914831+0j), (1.130330416727005e-18+0j), (0.0019360494339501345+0j), (-1.9403146183331572e-19+0j), (2.3854565377049233e-19+0j), (2.731779522511997e-18+0j), (1.2213156308222943e-19+0j), (-4.632146293926825e-22+0j), (0.002502980761256917+0j), (-2.4932201653248443e-19+0j), (2.600391148070702e-19+0j), (4.1556735436777604e-19+0j), (2.5278639519620526e-21+0j), (-9.331165228613636e-24+0j), (1.8677076486957322e-19+0j), (-1.0612285037456083e-23+0j), (3.820961932750195e-25+0j), (1.3183548300575515e-18+0j), (1.4743667238145165e-19+0j), (8.910241269515099e-23+0j), (1.2260801911505997e-19+0j), (-3.860308453313897e-24+0j), (5.035231365291676e-27+0j), (3.772511465514064e-24+0j), (1.0252993434397432e-25+0j), (-3.2226434767182254e-34+0j), (-0.0006811492076234078+0j), (1.5314858441267827e-19+0j), (1.6263032587282567e-19+0j), (5.127002821498262e-20+0j), (3.218725199566341e-20+0j), (-7.585995533265632e-24+0j), (1.5754812818929986e-19+0j), (-6.683227008052128e-25+0j), (9.981966623269168e-25+0j), (3.6733434346428805e-20+0j), (8.152692117322641e-21+0j), (-1.3014999743393397e-23+0j), (1.4293680984916318e-20+0j), (9.679348803102643e-25+0j), (1.4068833483571905e-25+0j), (2.013829362397166e-24+0j), (-1.989281110828983e-26+0j), (-4.181308022215505e-35+0j), (-1.0672615135404184e-19+0j), (-3.270756378052957e-24+0j), (5.315857262596624e-25+0j), (2.824046310044321e-24+0j), (1.8230277554221062e-26+0j), (6.370646735579082e-35+0j), (1.5930999035794086e-25+0j), (-8.453468900780111e-36+0j), (1.2791206455091828e-40+0j), (8.335148773081584e-19+0j), (3.3947492339176256e-20+0j), (3.547167474993384e-23+0j), (3.22931311140702e-20+0j), (1.106677186716453e-25+0j), (1.3651765722617445e-25+0j), (3.3239089692707014e-23+0j), (2.267021240566468e-28+0j), (1.5257009568559991e-33+0j), (4.658681209898652e-20+0j), (1.7367561689345796e-26+0j), (5.056716486046401e-26+0j), (-9.17249595267542e-25+0j), (4.822489698025621e-27+0j), (1.0015493115212873e-34+0j), (2.4506856470060788e-26+0j), (-7.864203569823105e-37+0j), (6.863560201698805e-42+0j), (1.7271434254879184e-23+0j), (3.285528827145366e-27+0j), (7.541759270496561e-34+0j), (1.0479921940253104e-26+0j), (4.1416289535510954e-35+0j), (1.894411326363893e-42+0j), (-1.486526353486352e-34+0j), (9.921193127419705e-43+0j), (1.0456584784655566e-64+0j), (-0.0017627013305946033+0j), (4.912900020883588e-19+0j), (-8.131516293641283e-20+0j), (-1.9148172070185447e-19+0j), (-1.4293680984916318e-20+0j), (1.1630892311195062e-25+0j), (1.9989977555201488e-19+0j), (-2.3244935169181365e-24+0j), (1.2427938605615295e-24+0j), (-1.4171242905086183e-19+0j), (-1.9058241313221758e-21+0j), (-1.0395139952347445e-23+0j), (4.129285617864714e-21+0j), (7.660935644473889e-25+0j), (6.25118895054068e-27+0j), (2.567975749322339e-24+0j), (5.938146975710555e-26+0j), (-1.0407831573955341e-35+0j), (-2.0837010502455788e-19+0j), (-6.731914935481378e-25+0j), (3.0511772648040583e-25+0j), (6.246263756412389e-25+0j), (3.760336236259042e-26+0j), (-5.5735938370431545e-36+0j), (1.7917186909636609e-25+0j), (4.440148664024207e-36+0j), (2.032041015274523e-41+0j), (5.995480688486144e-20+0j), (-3.983701830055381e-21+0j), (-1.8327152778246464e-24+0j), (-1.4823076576950256e-20+0j), (2.599075801649772e-25+0j), (9.319993661948558e-26+0j), (2.1398451588407913e-24+0j), (7.286135492715828e-27+0j), (-7.093435712203148e-35+0j), (3.801060350803673e-20+0j), (5.19663699036152e-25+0j), (5.178425420038924e-26+0j), (-2.4506437337233864e-25+0j), (4.0038716735403226e-28+0j), (-2.036943375461419e-35+0j), (3.390761065740673e-26+0j), (-7.278877838158315e-37+0j), (6.333545407523244e-42+0j), (-2.0235228852005218e-25+0j), (3.0447256366731476e-27+0j), (1.432296488554319e-34+0j), (1.3397646665515403e-26+0j), (-5.353586121770784e-37+0j), (9.449107627089488e-42+0j), (3.282447566138873e-35+0j), (-6.5300508437536476e-43+0j), (3.638177281884271e-64+0j), (9.910285482875314e-20+0j), (1.7130272329055316e-25+0j), (3.957848276753622e-25+0j), (2.0396787565394481e-25+0j), (9.976354777680108e-27+0j), (-2.962187649422252e-36+0j), (7.119165235804452e-26+0j), (3.1235942736384315e-36+0j), (1.424840278525474e-41+0j), (-9.735427094641133e-25+0j), (1.020751518023566e-26+0j), (4.0467301337511435e-35+0j), (-8.820851606237916e-27+0j), (1.858225123469436e-36+0j), (-6.488011889823903e-43+0j), (3.8010921897616783e-35+0j), (6.221765181602188e-43+0j), (-9.922463231052328e-66+0j), (7.545588956271325e-27+0j), (9.594369878460283e-36+0j), (6.793494955046713e-42+0j), (9.715745956968115e-36+0j), (-9.7740567886656e-44+0j), (-6.411386166307737e-66+0j), (-5.4140917792269714e-42+0j), (1.9814474884356773e-67+0j), (-1.6479230463428374e-91+0j), (2.5327536259872116e-19+0j), (1.2943722225229777e-20+0j), (-1.5777373458874757e-23+0j), (7.835054762102278e-21+0j), (-7.2923564256078685e-25+0j), (1.3036936958107854e-26+0j), (-5.62224322594636e-25+0j), (9.52388701572942e-27+0j), (3.5652130658503126e-34+0j), (2.8904999325053e-20+0j), (2.7454883856587918e-24+0j), (1.421552559599981e-26+0j), (6.000391589473104e-25+0j), (-1.9601420721367318e-27+0j), (3.216903442523833e-36+0j), (1.0251216649972571e-26+0j), (-1.8015536450614736e-35+0j), (-1.0830697170430473e-41+0j), (5.838731901887973e-24+0j), (4.187933540673603e-27+0j), (6.617239908634229e-34+0j), (5.6039679416919605e-27+0j), (1.9270058833205044e-35+0j), (-7.067884136870472e-42+0j), (3.2618418726560256e-34+0j), (3.3350903450930646e-43+0j), (7.522067073391272e-65+0j), (-1.1329065669526267e-20+0j), (-9.935860873439688e-26+0j), (2.535421477614837e-25+0j), (1.4040966806464998e-24+0j), (9.641260985663277e-27+0j), (6.509825080175589e-35+0j), (-4.672848978887415e-26+0j), (-4.3743427913084966e-36+0j), (4.51778624898321e-42+0j), (6.694589186067595e-25+0j), (8.367811174916307e-27+0j), (3.2576787974236286e-35+0j), (8.507344379456499e-27+0j), (5.8454512546329246e-36+0j), (4.2319213622609476e-43+0j), (3.328170648077282e-35+0j), (6.060615858204834e-44+0j), (-1.0312841228389927e-65+0j), (1.1732843818003701e-26+0j), (8.048905559076424e-36+0j), (-2.6400463067879554e-42+0j), (1.2411079183050643e-35+0j), (3.7432185228276676e-43+0j), (-2.6515816491983034e-66+0j), (-3.0049093944365296e-42+0j), (1.2703299543937767e-67+0j), (-1.3266381882387373e-91+0j), (-3.684043536254543e-25+0j), (-1.9106158392893048e-28+0j), (3.4947613645276357e-34+0j), (1.7199515376926863e-27+0j), (2.6794370106491824e-35+0j), (1.1918043439082569e-42+0j), (1.2162841172360097e-34+0j), (-2.1150848695902708e-44+0j), (-1.08750154686231e-67+0j), (2.5045624306639447e-27+0j), (5.891086814832368e-36+0j), (2.3850099862808387e-42+0j), (8.763174145538258e-36+0j), (-3.0828566215145976e-44+0j), (1.8314425176781207e-67+0j), (-1.1812508148488106e-43+0j), (-4.921028939482732e-68+0j), (2.59592498132023e-93+0j), (-2.6483459745054357e-35+0j), (-9.165367768224507e-44+0j), (-5.994261011885404e-68+0j), (-2.39118445763927e-43+0j), (-2.919678915259167e-69+0j), (1.037942524530525e-93+0j), (-1.4968221818766035e-70+0j), (-2.0740012076033412e-95+0j), (4.933820166180381e-266+0j), (1.2461819770548388e-15+0j), (1.4529664364021366e-16+0j), (0.0044736933074311+0j), (-2.1636270791484946e-16+0j), (-0.12030823555454606+0j), (5.087524647526594e-19+0j), (0.0010978009855655863+0j), (-2.670055005149752e-19+0j), (1.7889335846010823e-18+0j), (1.2476727550420064e-15+0j), (-0.06575344087275493+0j), (3.018018353834264e-20+0j), (0.16604080378327862+0j), (-1.1444952707512024e-17+0j), (-6.268043809681823e-20+0j), (7.881672461622223e-19+0j), (2.498747194400186e-20+0j), (-1.3208130486466867e-23+0j), (0.00012071348421156418+0j), (4.1771007445169945e-19+0j), (-2.0599841277224584e-18+0j), (8.436822350410198e-19+0j), (-2.541098841762901e-20+0j), (-3.3032315082307916e-23+0j), (-5.095750210681871e-18+0j), (-4.093886296901416e-23+0j), (8.88596796544092e-24+0j), (-2.946319403729358e-17+0j), (0.18607511192383036+0j), (-6.733019385418304e-19+0j), (-0.08345891431933039+0j), (2.044708061168368e-17+0j), (1.3129010682441655e-19+0j), (1.7719294284624246e-19+0j), (1.4293680984916318e-20+0j), (1.9699760628259522e-23+0j), (-0.14619788616024298+0j), (8.34233788055059e-18+0j), (-1.1180834903756764e-19+0j), (-2.1777918029367044e-17+0j), (-3.0556713572198885e-19+0j), (-1.5596096986228744e-23+0j), (-2.236166980751353e-19+0j), (-5.449257947146073e-24+0j), (-7.775921751013494e-25+0j), (4.2988642901399935e-19+0j), (1.598774687942492e-20+0j), (-4.3638908463732084e-23+0j), (-2.439454888092385e-19+0j), (1.2881393560876056e-24+0j), (1.3854877230632723e-24+0j), (-2.906383573089531e-24+0j), (-4.665746481767247e-26+0j), (-4.824378186209169e-33+0j), (0.005609945152117701+0j), (-9.829893506905077e-19+0j), (-1.2468324983583301e-18+0j), (7.939641042963869e-19+0j), (-1.6771252355635147e-19+0j), (-6.643999705649458e-24+0j), (1.0842021724855044e-18+0j), (-1.56608299177701e-23+0j), (2.683409381321787e-23+0j), (1.002238393874498e-18+0j), (-1.2536087619363645e-19+0j), (1.4029049354420478e-23+0j), (-3.3881317890172014e-20+0j), (-4.27076244123177e-24+0j), (-1.7255820004344426e-26+0j), (1.7107972452320994e-23+0j), (3.488941072431652e-25+0j), (3.3495725302005506e-34+0j), (9.215718466126788e-19+0j), (-6.093489945510908e-24+0j), (5.4076269088595804e-24+0j), (4.237708958634344e-24+0j), (4.138400969828884e-25+0j), (-1.2162463522423961e-34+0j), (2.440131305688233e-24+0j), (-1.1547487122401973e-34+0j), (2.0326576724700787e-39+0j), (1.5421420650890694e-16+0j), (-0.009468597595093433+0j), (1.055562854205952e-18+0j), (-0.05938999986795371+0j), (1.568814700304453e-18+0j), (3.655807435239361e-20+0j), (-3.177176058453583e-19+0j), (2.345222472710344e-20+0j), (-1.698856319532377e-22+0j), (0.03327433398467713+0j), (-1.5335118145859359e-18+0j), (3.032377951170395e-19+0j), (3.260356918454871e-18+0j), (-7.999167395632799e-20+0j), (6.347572919020329e-24+0j), (-1.1858461261560205e-19+0j), (-5.678800045789451e-24+0j), (1.9475854396455961e-25+0j), (2.256412217927166e-19+0j), (1.3870164511289168e-20+0j), (-2.783262876470769e-23+0j), (4.2351647362715017e-20+0j), (-2.456974053972947e-24+0j), (-1.9123945472566812e-26+0j), (7.144271392432453e-24+0j), (7.49336624234907e-27+0j), (4.74139319824016e-34+0j), (0.058202566776118396+0j), (-4.699160266719428e-18+0j), (1.9820570965750628e-19+0j), (8.541133044639848e-18+0j), (1.5564230405797769e-19+0j), (-5.924048760689937e-24+0j), (-5.082197683525802e-21+0j), (-1.0566572085992012e-24+0j), (4.111029061235795e-25+0j), (-1.7351135805014485e-18+0j), (3.3034284942917713e-20+0j), (1.4198827202789327e-24+0j), (-4.5528020914918643e-20+0j), (1.429082503906706e-23+0j), (-2.8448876099446e-26+0j), (8.105971738834856e-25+0j), (1.3734946246914716e-26+0j), (4.804566491664131e-34+0j), (9.317362419797304e-20+0j), (5.635135838838933e-25+0j), (2.263786857049394e-25+0j), (-6.215752001154579e-25+0j), (1.924731204663838e-26+0j), (-6.537014198503346e-35+0j), (2.8234752859413027e-25+0j), (-3.052597686488316e-35+0j), (-2.090533083052323e-39+0j), (-1.048067623213949e-19+0j), (-1.6623021589865644e-20+0j), (-2.1994641599764875e-23+0j), (3.356368053495165e-20+0j), (2.2257113399172287e-24+0j), (6.049107775564085e-26+0j), (-5.850041011825221e-24+0j), (4.011780781709378e-27+0j), (-6.266326203718412e-35+0j), (-7.538593230563273e-20+0j), (4.530504385665886e-25+0j), (6.046638121772253e-25+0j), (-3.322051044066725e-26+0j), (1.0137184485076364e-26+0j), (-3.516510578541881e-35+0j), (2.8895071087725473e-26+0j), (-4.068500455580209e-36+0j), (8.285976721518984e-41+0j), (6.0398725000576054e-24+0j), (2.3141807673735933e-26+0j), (1.6570246487742837e-34+0j), (1.2690488360906839e-26+0j), (-5.375756143247581e-36+0j), (9.307493014663877e-41+0j), (3.3358598261057947e-35+0j), (-3.9801686161250533e-42+0j), (-1.055913273276998e-62+0j), (-0.0019391562223237046+0j), (1.9148873067273353e-19+0j), (-4.87890977618477e-18+0j), (-1.4423669517310678e-19+0j), (1.6940658945086007e-21+0j), (-9.79404371165046e-24+0j), (2.846030702774449e-19+0j), (-2.11455594859669e-24+0j), (9.542939165458666e-24+0j), (8.356056556617493e-20+0j), (1.1011428314305904e-19+0j), (-2.2975544581493057e-26+0j), (-2.710505431213761e-20+0j), (9.038322141209814e-26+0j), (2.0125939965177178e-25+0j), (-5.845003495832583e-24+0j), (1.714264863413779e-25+0j), (-9.529632831014213e-35+0j), (-2.913793338554793e-19+0j), (3.484013654239461e-24+0j), (1.6565704040755864e-24+0j), (6.673182656543517e-24+0j), (1.0841460769482488e-25+0j), (5.745976237979841e-35+0j), (2.7529668162813202e-24+0j), (5.630355635302856e-35+0j), (-1.0443177592764654e-40+0j), (5.636316899513634e-20+0j), (-9.317362419797304e-21+0j), (7.875284661745591e-24+0j), 0j, (3.1210615972564806e-25+0j), (1.9801218466255452e-26+0j), (-2.9532932807557316e-24+0j), (1.4379457058240288e-26+0j), (1.0136474901964007e-35+0j), (-2.541098841762901e-21+0j), (2.0538151439610087e-25+0j), (2.5965620800831216e-25+0j), (3.482095144517964e-25+0j), (3.4699898481120614e-26+0j), (-1.1088985493466555e-36+0j), (2.6436455387775373e-26+0j), (9.153627028277518e-36+0j), (8.421898666487194e-41+0j), (-1.852674545791376e-24+0j), (9.453517406276328e-27+0j), (1.7060584900322902e-34+0j), (-9.401071801344509e-26+0j), (5.493073478462567e-36+0j), (3.28624590974012e-42+0j), (6.279872082894678e-35+0j), (9.320809552993358e-43+0j), (2.52364221448789e-64+0j), (5.421010862427522e-20+0j), (1.3865776576633516e-24+0j), (-6.049340532653901e-25+0j), (2.312006162796225e-24+0j), (1.0117817523948075e-25+0j), (3.679934426828356e-37+0j), (6.105434511682073e-25+0j), (6.954305975847771e-35+0j), (-5.779727125431862e-41+0j), (3.919616335215262e-24+0j), (-1.9135169163769382e-26+0j), (7.445039962652888e-35+0j), (1.2751663248604845e-26+0j), (3.659138709202267e-37+0j), (-4.9347599663517104e-42+0j), (1.5043763094611966e-34+0j), (6.148897661457297e-42+0j), (1.652738138882169e-64+0j), (-7.568737690170135e-25+0j), (9.525078830728757e-36+0j), (-1.3588110948864886e-40+0j), (6.449126978477661e-35+0j), (3.712740281228603e-42+0j), (-6.808468391171699e-66+0j), (3.214999066700428e-41+0j), (-2.159396088618285e-66+0j), (-1.1055878425825642e-91+0j), (6.533080418877693e-16+0j), (-0.09126404075949228+0j), (1.3074242793958495e-18+0j), (0.07096324625342894+0j), (-1.621426992864673e-17+0j), (4.412512259602871e-20+0j), (-1.6228691229791757e-19+0j), (2.0699367648526965e-20+0j), (-7.591530892233192e-23+0j), (0.05522986393719635+0j), (-3.161189148944321e-18+0j), (1.545835128739098e-20+0j), (1.6197445364528354e-17+0j), (3.486334671339497e-19+0j), (2.2619679241713123e-24+0j), (2.0752307207730358e-20+0j), (-4.291786509365016e-25+0j), (3.3332967801235953e-26+0j), (-2.5036583396785544e-21+0j), (4.3556022334592225e-20+0j), (2.597669318441001e-23+0j), (1.3023131564034868e-20+0j), (-1.5532118642621271e-24+0j), (1.6502575128991973e-26+0j), (-1.9431006597513098e-23+0j), (-1.0896519891192292e-26+0j), (-3.5312865080934063e-34+0j), (-0.05054697892666424+0j), (1.1765162430732036e-17+0j), (-3.4728350837426314e-19+0j), (-6.574060740844462e-18+0j), (9.883815703273617e-20+0j), (2.618033128919365e-24+0j), (-1.0926725019580474e-19+0j), (-1.3973592708188414e-24+0j), (2.5774934975070883e-25+0j), (-7.367073669915513e-18+0j), (-4.199165836013194e-19+0j), (-2.0410548307492904e-24+0j), (1.0264980529538052e-19+0j), (1.9441070963067096e-25+0j), (2.9118901349308485e-26+0j), (1.157568181434071e-24+0j), (8.850165567761281e-28+0j), (-2.9902343534418653e-35+0j), (1.3891340334970526e-19+0j), (7.676338712060338e-25+0j), (1.9156410098452545e-25+0j), (5.602048386772702e-25+0j), (3.4100252633660616e-26+0j), (-2.3249952384289066e-35+0j), (9.163106448121214e-26+0j), (5.878968385712887e-36+0j), (7.276156522409753e-41+0j), (4.989254001181445e-19+0j), (7.583591855886158e-21+0j), (-1.9657717647357453e-23+0j), (2.159934015498466e-20+0j), (5.769665551914063e-25+0j), (2.7065646972591594e-26+0j), (8.320273739547056e-26+0j), (-1.0577307989738587e-28+0j), (1.4059233860071112e-35+0j), (1.725829630030637e-20+0j), (3.4109083364308197e-25+0j), (3.3501992917102046e-26+0j), (3.155443620884047e-26+0j), (1.765325397479586e-27+0j), (-6.885682726428248e-36+0j), (2.3858939056640347e-26+0j), (4.2684679436753525e-36+0j), (-2.973924775086074e-43+0j), (1.9253759668165444e-24+0j), (3.759024785134383e-27+0j), (1.8011245293563791e-34+0j), (7.034281696941826e-27+0j), (5.4377594524505e-36+0j), (-6.905036475958035e-42+0j), (7.453018548050641e-35+0j), (3.3210773604498165e-43+0j), (2.2790032527900556e-64+0j), (-0.015865473020100506+0j), (-1.0343771117606038e-19+0j), (-4.404571325722362e-20+0j), (-5.341957382421164e-18+0j), (-5.403011412298368e-19+0j), (-2.9253351544502025e-24+0j), (2.253107639696439e-19+0j), (2.0090652504029155e-24+0j), (3.4232548088584554e-25+0j), (2.514156311976923e-18+0j), (-2.338869725605937e-19+0j), (-7.964350864522022e-24+0j), (2.687212025164268e-19+0j), (-3.277109911732092e-24+0j), (3.6330083405656786e-26+0j), (3.6168956960021303e-25+0j), (1.3927450305519726e-26+0j), (-4.051562833243495e-35+0j), (-1.4568966692773966e-19+0j), (-5.573775611929581e-25+0j), (7.621018071168552e-25+0j), (7.209557584995871e-25+0j), (1.961186895751946e-26+0j), (-4.0487084712285345e-36+0j), (7.088127732320606e-27+0j), (-2.2857705761042756e-35+0j), (1.6929902271537956e-40+0j), (2.524488577581178e-18+0j), (3.7280037591029894e-19+0j), (-3.741686141682315e-24+0j), (-3.415130964210932e-19+0j), (-2.829953300502616e-24+0j), (1.4281563304563243e-25+0j), (1.5156226799830256e-24+0j), (-3.8855517987718644e-28+0j), (-2.6822278568494296e-36+0j), (-1.291195848970774e-19+0j), (4.489767132185559e-24+0j), (8.282533478012482e-27+0j), (-4.222867088956703e-24+0j), (-1.0806261819566924e-25+0j), (-7.66647026715793e-39+0j), (1.4834159800060435e-26+0j), (-1.9315480095632935e-37+0j), (-2.2276425404768937e-42+0j), (-2.88786200183308e-25+0j), (1.70471834008013e-27+0j), (3.246088583836639e-35+0j), (1.7731851583417214e-26+0j), (1.0810202432871358e-36+0j), (-2.075724975159897e-42+0j), (1.301665336363407e-35+0j), (8.975316664000453e-43+0j), (4.1427336753513636e-64+0j), (2.5199230180815435e-20+0j), (1.0529084274165889e-25+0j), (5.052969878229087e-27+0j), (-5.371322349213661e-25+0j), (1.9578059002404528e-26+0j), (4.5379197465862223e-36+0j), (-5.1540168686470154e-26+0j), (-5.364852113008447e-38+0j), (8.553525826238683e-42+0j), (7.18481890700814e-25+0j), (1.9486672618521683e-26+0j), (5.817759023072847e-35+0j), (-3.0029580211760384e-27+0j), (2.8862558370579023e-36+0j), (-2.1019476964872256e-43+0j), (2.026692200087465e-35+0j), (-4.004210361808165e-43+0j), (-1.7651980362724946e-65+0j), (5.185561675820403e-26+0j), (3.059199474845133e-36+0j), (-1.1073060465094704e-41+0j), (-8.299688535170357e-36+0j), (-8.099505123797443e-43+0j), (-9.5645339103795e-66+0j), (4.4806518396786026e-42+0j), (4.670758314858223e-67+0j), (-2.3123286479887886e-91+0j), (-4.637232380967731e-20+0j), (1.0587911840678754e-21+0j), (1.0387417477362679e-23+0j), (1.8131799027162367e-21+0j), (1.9818710294048524e-25+0j), (1.8492662473216129e-26+0j), (7.615473852386395e-25+0j), (-1.7051023655739647e-28+0j), (1.217042881678604e-34+0j), (7.517417406881916e-21+0j), (-4.613006138242806e-25+0j), (-5.094408691904896e-27+0j), (2.9930013832809365e-25+0j), (2.197891495299751e-27+0j), (7.863683228466944e-36+0j), (1.2665043442328347e-26+0j), (-1.4557013100234415e-36+0j), (4.531039893075071e-42+0j), (3.0112019820861957e-24+0j), (1.1041429509152181e-27+0j), (9.597456860504477e-35+0j), (2.946629831661877e-27+0j), (1.5027599616809272e-36+0j), (1.2163270670339412e-42+0j), (6.019528092220208e-35+0j), (-1.7446165880843973e-43+0j), (-6.002029092260537e-66+0j), (-8.179161896924338e-21+0j), (-1.5408031200776803e-25+0j), (1.5718225874610503e-25+0j), (3.939129598566809e-25+0j), (-6.319078053831676e-27+0j), (-8.203410889249293e-36+0j), (2.2425940749728238e-26+0j), (2.2853547962764064e-37+0j), (3.7498746905332105e-42+0j), (6.902848465045942e-26+0j), (-3.818451250003773e-27+0j), (-3.568585098458665e-36+0j), (-2.9789422636172616e-27+0j), (1.5147276335064223e-37+0j), (-8.477855709165143e-44+0j), (1.58372779465805e-35+0j), (-2.368194404708941e-43+0j), (-1.0014649959094886e-65+0j), (-4.0428743414711875e-26+0j), (2.8832865528743243e-36+0j), (-7.274490652926207e-42+0j), (5.255734834489865e-36+0j), (1.9526218288826123e-43+0j), (3.972208538161278e-67+0j), (-1.7027965870397035e-42+0j), (1.6073279694310985e-67+0j), (5.063194408918828e-93+0j), (1.1523806321213376e-24+0j), (1.1675423387418946e-28+0j), (6.994559769701223e-35+0j), (4.040022417886383e-28+0j), (3.451085118001839e-36+0j), (5.9555184733804726e-43+0j), (1.7692019478501384e-35+0j), (2.848577034510292e-44+0j), (-8.636717400878458e-68+0j), (-5.6095475943166805e-27+0j), (1.2863039494702655e-36+0j), (7.684370453741216e-43+0j), (2.0058987641956656e-36+0j), (5.640226318907389e-44+0j), (2.9671388194391234e-67+0j), (-1.2871145347708496e-43+0j), (-1.9289842829178148e-68+0j), (3.302753073909329e-93+0j), (3.505539404035855e-35+0j), (3.697019464081959e-44+0j), (-6.90965571996224e-68+0j), (-1.4102481635012666e-43+0j), (-7.362298261177475e-69+0j), (-1.0147985581562153e-94+0j), (-7.486893529918105e-69+0j), (-3.782210876746198e-95+0j), (9.844444565266257e-266+0j), (0.001687083631506418+0j), (-3.1160175730113884e-19+0j), (-1.328147661294743e-18+0j), (3.08064220153946e-19+0j), (-3.134021904840911e-20+0j), (1.9761057189075976e-24+0j), (-3.015437292225309e-19+0j), (2.9403685836845906e-25+0j), (4.226478918583542e-24+0j), (1.6485984002141948e-19+0j), (6.437450399132683e-20+0j), (3.830936642145697e-25+0j), (-4.086933970501999e-20+0j), (1.7213576040646654e-25+0j), (1.65302500417358e-25+0j), (-9.367376150701218e-25+0j), (2.4943784961338576e-25+0j), (4.4943427992085163e-35+0j), (-6.06475590234079e-19+0j), (-6.0887440108578575e-25+0j), (1.3609821218505434e-24+0j), (4.736699528181461e-25+0j), (-1.4040494800502146e-26+0j), (-4.6667911351615853e-35+0j), (6.638921988562056e-25+0j), (1.9737097575959087e-35+0j), (5.099089903770294e-40+0j), (-3.4197973468752984e-20+0j), (-9.343832199399e-21+0j), (1.3142801334216563e-24+0j), (3.128727948920572e-20+0j), (4.800565707068154e-25+0j), (1.3331452443368025e-25+0j), (1.0181228169399631e-24+0j), (2.746020059538097e-27+0j), (1.1429817296724e-34+0j), (-1.8714134178399698e-20+0j), (-1.4504943236479788e-25+0j), (5.431944685869244e-26+0j), (2.8460839282925752e-25+0j), (5.82247841849629e-27+0j), (-2.4658931130078705e-36+0j), (-1.0857914944772744e-26+0j), (1.44581868062976e-36+0j), (-1.9562126561974446e-41+0j), (2.203256953846077e-24+0j), (2.2639135860924245e-27+0j), (8.03163991182687e-35+0j), (-7.607844294272785e-26+0j), (8.782041822212476e-36+0j), (-5.061490053141239e-42+0j), (-4.590456376621222e-36+0j), (3.5733110840282835e-43+0j), (-7.860354251235684e-66+0j), (-2.448454613156962e-19+0j), (5.398206728863593e-25+0j), (8.120099723050967e-25+0j), (4.3295210833425835e-25+0j), (-5.858608794704283e-26+0j), (4.857743372949611e-35+0j), (-5.478429412404722e-25+0j), (1.5645223192944634e-35+0j), (-5.983824702359834e-41+0j), (8.654750763360765e-26+0j), (-5.602971159407533e-26+0j), (3.956648919777524e-35+0j), (-4.4632700665417556e-26+0j), (9.007083920820494e-36+0j), (4.1548499467230826e-42+0j), (3.426376489093048e-35+0j), (-3.758983130551322e-43+0j), (7.051776526170965e-68+0j), (2.6613758067990307e-25+0j), (2.1663695795772427e-35+0j), (1.922581493053649e-41+0j), (3.8880102953112505e-35+0j), (-1.8908771152983e-42+0j), (-4.093918134842933e-68+0j), (-2.8314987094763335e-42+0j), (-1.494116091302421e-67+0j), (2.720777251641042e-93+0j), (-9.846362914071327e-21+0j), (-1.1527589016538994e-20+0j), (-1.3449762889656163e-25+0j), (6.392451773809798e-21+0j), (5.264163483848438e-25+0j), (3.526792716799321e-27+0j), (-1.4836138598927778e-24+0j), (-7.60707888140859e-28+0j), (5.010349376457702e-35+0j), (8.523269031746397e-21+0j), (-1.9987842072127909e-25+0j), (1.698980044795339e-26+0j), (2.428366301759945e-25+0j), (6.2727781704890265e-27+0j), (1.1450598790610156e-35+0j), (-1.8311869984531154e-26+0j), (8.22257980025179e-36+0j), (-7.920138920363866e-42+0j), (4.833634756215019e-25+0j), (-3.2036623743959277e-27+0j), (7.310933036028882e-35+0j), (-7.20884008440795e-27+0j), (4.020557057703512e-36+0j), (7.707141553786494e-44+0j), (1.423767374916783e-35+0j), (-4.333515500924497e-43+0j), (-3.5845140361992844e-66+0j), (8.880611056369305e-21+0j), (3.7644442397146683e-26+0j), (-1.6564097157915864e-26+0j), (1.5155595711106079e-25+0j), (2.0487203299435876e-27+0j), (2.3378431319611316e-36+0j), (-2.8409389464757815e-26+0j), (1.40457314749571e-36+0j), (5.580671134173584e-42+0j), (-9.195909344342379e-26+0j), (3.3384800095873624e-27+0j), (1.7001477153551872e-36+0j), (-1.2792249589386048e-27+0j), (-5.679143235910556e-37+0j), (2.1019476964872256e-44+0j), (1.45967736045183e-35+0j), (6.892636821397694e-44+0j), (5.309519017140844e-67+0j), (-2.1540547416911928e-26+0j), (2.070068111582285e-36+0j), (-1.7498714573256153e-42+0j), (-1.8702814050020242e-37+0j), (-1.4459648528751706e-43+0j), (1.56122423207596e-67+0j), (-7.174867090228114e-43+0j), (-3.0062241094603132e-68+0j), (3.152276496086317e-93+0j), (3.986650579497323e-25+0j), (1.9647317984360187e-28+0j), (1.0588526040177282e-34+0j), (-1.4653088284565344e-27+0j), (1.0466186266294759e-36+0j), (-1.0334576174395526e-44+0j), (-7.562672838192305e-36+0j), (-3.5262362137423717e-44+0j), (-1.1135182664831626e-67+0j), (-2.7479149546817735e-26+0j), (1.3095709816537545e-36+0j), (-1.3274675514857033e-42+0j), (2.790258067924898e-36+0j), (-1.0153940044228655e-43+0j), (-3.739564036300795e-68+0j), (-1.0637606967305768e-42+0j), (-2.847323609413714e-69+0j), (-8.886799698722047e-95+0j), (-1.589608832331396e-36+0j), (-3.4298969443200405e-43+0j), (-7.441912970271341e-69+0j), (5.241184685902392e-46+0j), (-1.4760631740164158e-69+0j), (-3.684589741229032e-95+0j), (3.019127896551882e-70+0j), (2.2580951825708097e-96+0j), (-1.4682502180326325e-267+0j), (8.762820537141754e-20+0j), (1.1495281110880584e-26+0j), (-3.6010936357693653e-25+0j), (3.132566654632638e-25+0j), (-5.1110430297287103e-26+0j), (3.092189092162875e-35+0j), (-1.6283939261540858e-25+0j), (1.0994402936640652e-35+0j), (-5.05532433989821e-41+0j), (2.309469186125034e-25+0j), (-4.5002737904458686e-26+0j), (7.614548190577445e-36+0j), (-9.079224155268323e-27+0j), (4.732608520945479e-36+0j), (4.3869400048768804e-43+0j), (3.8571367249946243e-35+0j), (4.041870258036894e-43+0j), (7.31981648817899e-69+0j), (-1.5778430326240255e-25+0j), (1.9180299130909002e-35+0j), (1.430340374997949e-41+0j), (1.3435069580070745e-35+0j), (-6.407437228125226e-43+0j), (-7.023467940419637e-68+0j), (-1.0615164296588065e-41+0j), (-3.728629267498023e-68+0j), (-5.2119807360946526e-95+0j), (3.1583466290152605e-25+0j), (-5.825877406855635e-28+0j), (4.2225189656982616e-35+0j), (-2.5224689996724292e-26+0j), (3.0764589040429455e-36+0j), (1.5236493364911777e-42+0j), (2.941518354946764e-35+0j), (-8.942035825472739e-44+0j), (-1.2653512646249231e-67+0j), (-2.239532942187952e-26+0j), (2.396513917997744e-36+0j), (-3.625859776440464e-44+0j), (1.9831607989911313e-36+0j), (-6.325001466903618e-44+0j), (-1.4104280635724827e-68+0j), (-6.501956451690573e-43+0j), (-9.170779464059505e-69+0j), (-5.9158003032598015e-95+0j), (4.631787399996821e-36+0j), (-4.977620150522548e-44+0j), (-3.871313704753015e-68+0j), (-1.1989599978828218e-42+0j), (-8.885315226709342e-70+0j), (-2.4501850826599e-96+0j), (-1.6048723587615939e-68+0j), (9.847716777561715e-97+0j), (-7.672254729802554e-268+0j), (-3.2142137583230126e-26+0j), (3.371037662182594e-36+0j), (-1.0363259002337174e-42+0j), (1.1339211854761557e-35+0j), (-8.012012919386693e-43+0j), (-2.208768507877714e-68+0j), (-6.333393520450954e-42+0j), (-7.005108188501158e-69+0j), (-4.1835824183647716e-95+0j), (-6.519877899474808e-37+0j), (-8.049843872556869e-43+0j), (-4.039159467305865e-68+0j), (-2.3847766646127064e-43+0j), (-2.182617607755008e-69+0j), (2.17081894939393e-96+0j), (-4.3539507497901226e-70+0j), (7.753508755584521e-97+0j), (2.1139388038460662e-269+0j), (-5.860880878411645e-42+0j), (-1.6275455010653867e-69+0j), (4.931609060124658e-96+0j), (-2.5324037196255484e-69+0j), (-2.969631637235522e-97+0j), (5.864569874483181e-270+0j), (1.1870908212646534e-96+0j), (-2.2420533671161087e-271+0j), 0j, (-7.28084110471379e-17+0j), (0.0021891356947064196+0j), (2.5798784146317937e-18+0j), (0.00012780697328537016+0j), (2.694961946361546e-19+0j), (1.1666555359447902e-19+0j), (2.2923657140653986e-18+0j), (1.0328508000582125e-19+0j), (-1.2099168103227295e-22+0j), (-0.002096360342807301+0j), (2.2629503121180936e-19+0j), (1.0736142606448257e-19+0j), (-4.013314739120883e-19+0j), (1.1653320469647054e-20+0j), (-1.6526514511208498e-23+0j), (9.645587686858345e-20+0j), (-3.34041485747075e-24+0j), (9.624605518818835e-27+0j), (1.064098237057218e-18+0j), (1.2323667638060028e-19+0j), (2.4662213618288204e-23+0j), (7.385068508873431e-20+0j), (2.1040965809447808e-24+0j), (1.4008353765528856e-26+0j), (-1.1532157431819352e-24+0j), (6.051818139182234e-27+0j), (-8.197656993634699e-34+0j), (-0.0004864224193654863+0j), (-1.170832734204666e-19+0j), (-6.776263578034403e-20+0j), (-3.1679451438656147e-20+0j), (-1.1858461261560205e-20+0j), (-1.297460737188508e-23+0j), (1.2197274440461925e-19+0j), (-5.719166743036574e-25+0j), (2.2704618074026334e-25+0j), (2.7680015063518805e-19+0j), (3.65282958503417e-21+0j), (6.094947796551668e-25+0j), (1.011145580784821e-20+0j), (-4.942835978507662e-25+0j), (1.372418421057079e-26+0j), (1.3675945088401131e-24+0j), (9.438850728231387e-27+0j), (2.6321786593870263e-35+0j), (7.369186641112413e-20+0j), (-1.954860432010085e-25+0j), (4.408494827961927e-26+0j), (1.563888345608068e-24+0j), (1.4363248699652958e-26+0j), (-1.947635149109807e-35+0j), (2.1223616354349539e-26+0j), (5.19160409077399e-36+0j), (2.1114568563376524e-40+0j), (3.540705065182067e-19+0j), (1.9362643778641272e-20+0j), (2.048567141147465e-23+0j), (2.789914770018852e-20+0j), (-6.559283763604087e-25+0j), (2.4992644591707442e-26+0j), (1.0234744493209825e-23+0j), (1.2624838280727426e-27+0j), (-4.1790989193554246e-34+0j), (3.2928405824510926e-20+0j), (-1.5550026163716585e-24+0j), (6.519190963836857e-27+0j), (-9.673327964182135e-26+0j), (1.3124022254134624e-27+0j), (6.324826562714754e-35+0j), (-5.5671978349914065e-27+0j), (4.1618011583808085e-35+0j), (1.7849524035389556e-41+0j), (3.6253775284550666e-24+0j), (4.5363831419412144e-27+0j), (5.794986488994772e-34+0j), (2.5608030542189717e-27+0j), (-2.032753183646387e-35+0j), (2.7103942385673108e-42+0j), (7.187753076994105e-34+0j), (1.14906474074635e-43+0j), (8.274827189020223e-65+0j), (0.00045617210294274474+0j), (-3.39526953761102e-21+0j), (-2.947674656444965e-19+0j), (1.3876979220530946e-19+0j), (1.418780186650953e-20+0j), (-1.0863082796683767e-24+0j), (1.1011428314305904e-20+0j), (-9.175442756373634e-26+0j), (1.3893360476428185e-24+0j), (3.728028756533265e-20+0j), (-8.046812998915853e-21+0j), (2.2060842313303892e-24+0j), (-4.235164736271502e-21+0j), (-2.08915611251491e-25+0j), (1.8414720141028924e-26+0j), (-2.6439083446152926e-24+0j), (-2.260046100086881e-27+0j), (-1.8918313098694035e-35+0j), (1.1921988732604277e-19+0j), (-4.75386514147907e-25+0j), (2.2795755053434983e-25+0j), (2.76921732168784e-25+0j), (-2.481760998365842e-27+0j), (-4.2061252736155374e-36+0j), (1.9781769763925432e-25+0j), (-7.002732806000118e-37+0j), (2.856511690760143e-41+0j), (-8.361980619185959e-21+0j), (-6.432156443212343e-21+0j), (-2.742459159782743e-24+0j), (6.776263578034403e-21+0j), (9.077580208559227e-26+0j), (3.7253408170025195e-26+0j), (1.8148092223407674e-24+0j), (-2.348282154113671e-28+0j), (-2.9103433549958856e-35+0j), (9.26442286059391e-22+0j), (8.661061650602533e-26+0j), (1.9341559680761165e-26+0j), (2.393971966292309e-25+0j), (3.6097294238152245e-27+0j), (-8.326950079328983e-36+0j), (-4.7044630071588225e-27+0j), (4.7846675548326735e-36+0j), (-2.1669708668130245e-42+0j), (9.41003774845557e-25+0j), (4.778465715056557e-28+0j), (1.6985131780482276e-35+0j), (1.4699929936046904e-27+0j), (2.2031951651210886e-36+0j), (8.57594660166788e-43+0j), (5.138944886359719e-36+0j), (1.233142648605839e-43+0j), (-8.097890465023387e-66+0j), (-3.5469504666273827e-20+0j), (-7.107321211679228e-26+0j), (2.329819827559972e-25+0j), (-3.7865323450608567e-26+0j), (1.4353615827513093e-27+0j), (3.2081500130638675e-36+0j), (-2.698678007644859e-26+0j), (9.091988101516875e-37+0j), (1.6333534900170068e-41+0j), (4.45397177975025e-25+0j), (1.053720509237579e-27+0j), (4.044234073664162e-35+0j), (-1.292815975203248e-26+0j), (2.4976473321469986e-36+0j), (7.531979245745892e-43+0j), (1.2999615569756895e-35+0j), (-8.84919980221122e-43+0j), (-3.1506337794484697e-66+0j), (-8.227434847902739e-26+0j), (1.1371032738704549e-35+0j), (-1.1218795505384485e-41+0j), (1.1267960224555944e-35+0j), (2.9865173520922664e-44+0j), (1.9664208073259348e-67+0j), (-1.186286731204978e-42+0j), (2.5039295474302823e-68+0j), (2.863645960728337e-93+0j), (1.9990010289862712e-19+0j), (7.345363839470886e-21+0j), (-1.1292954065909499e-23+0j), (7.4313906231764e-21+0j), (4.1348933208064555e-25+0j), (8.078444097853281e-27+0j), (1.1946862958352542e-23+0j), (1.262132737050657e-27+0j), (-2.776402316998787e-34+0j), (6.961552035246281e-21+0j), (6.101365785341394e-25+0j), (1.1931539556900772e-26+0j), (6.31290672568546e-25+0j), (2.123440622954414e-27+0j), (2.0499891816625126e-35+0j), (-1.3102804794722666e-26+0j), (1.0336246163432594e-35+0j), (3.884399343108393e-42+0j), (3.4412005951913065e-25+0j), (2.0558086170944e-27+0j), (9.040050803920366e-34+0j), (8.592415255647686e-29+0j), (1.819450029081246e-35+0j), (5.640226318907389e-44+0j), (1.34854998307328e-34+0j), (5.219836779609944e-44+0j), (-1.0600592873700617e-66+0j), (1.2123159057577174e-20+0j), (5.090361649210145e-26+0j), (-7.212991451836734e-27+0j), (3.5890015743935153e-25+0j), (-3.131256138810043e-29+0j), (1.6738951307537285e-35+0j), (-1.0467219688853723e-26+0j), (5.534294500083879e-36+0j), (2.8670566580085757e-42+0j), (4.762384839255457e-25+0j), (-3.3847901757097414e-28+0j), (3.143159646728688e-35+0j), (-2.177616948184439e-27+0j), (3.4879582187740132e-37+0j), (2.049399004075045e-43+0j), (1.6275504529952427e-35+0j), (5.491338357072877e-44+0j), (1.2020931567675908e-67+0j), (-3.4099633210187413e-26+0j), (4.7016254055227024e-36+0j), (1.7595053842678484e-42+0j), (2.599582922404753e-36+0j), (-9.18069447017806e-44+0j), (-4.94043561583516e-68+0j), (-1.530436875927751e-42+0j), (-1.5947970190910783e-68+0j), (-5.0419710725694456e-95+0j), (8.794095153659004e-25+0j), (-7.718581566103019e-28+0j), (3.80922757933655e-34+0j), (-2.5168830688451223e-27+0j), (2.285350993432634e-35+0j), (-6.0360931350791495e-43+0j), (5.564261416561362e-35+0j), (9.123766720064864e-44+0j), (-5.64054809393245e-67+0j), (-4.993239776661308e-27+0j), (1.207520451571022e-35+0j), (-6.243660470107263e-43+0j), (5.793087925981605e-36+0j), (-6.332117435667767e-44+0j), (-1.0800592404346764e-68+0j), (-3.5592980993850354e-43+0j), (-1.069026693363332e-68+0j), (-4.1939699301922926e-95+0j), (3.894804603299666e-35+0j), (-1.6170765325423338e-43+0j), (-2.3316604999978384e-67+0j), (-1.925882207795789e-43+0j), (-7.508737689292227e-69+0j), (-5.645412850798969e-97+0j), (9.704572650269967e-69+0j), (9.145092346587576e-97+0j), (-7.196368106753468e-268+0j), (0.00037069844265942786+0j), (2.6196543315504553e-20+0j), (-1.8973538018496328e-18+0j), (1.479180457085356e-19+0j), (-1.1858461261560205e-20+0j), (5.366090908816123e-25+0j), (-3.049318610115481e-19+0j), (5.335476509680418e-25+0j), (5.064728674969424e-24+0j), (-5.928934608129585e-20+0j), (1.2451384324638215e-19+0j), (2.895939937502543e-24+0j), (-1.002675251312278e-19+0j), (-1.5423808418881223e-26+0j), (9.628836609698529e-26+0j), (-1.1744308721440753e-24+0j), (2.830030923478755e-26+0j), (4.169238685625904e-35+0j), (-5.675120746603812e-20+0j), (-1.7153496394105022e-24+0j), (1.606462616446633e-24+0j), (3.235213235619996e-25+0j), (-2.855376989868389e-26+0j), (9.995259172535939e-36+0j), (2.3622521020323198e-25+0j), (6.593942161613908e-36+0j), (-7.836887976631404e-41+0j), (-2.657517475217845e-20+0j), (-1.8925892415213273e-20+0j), (1.0339757656912846e-24+0j), (1.8740603958001395e-20+0j), (3.4167143526932463e-25+0j), (1.8925233416743742e-25+0j), (7.47410997817079e-25+0j), (6.69970606222358e-27+0j), (2.0301728752536525e-35+0j), (3.6925342544367155e-20+0j), (8.794852460128016e-26+0j), (8.686714402897006e-26+0j), (3.4111607719204904e-25+0j), (1.2226509364872987e-26+0j), (4.7470037910201825e-36+0j), (3.304684410802642e-26+0j), (1.5482212676137352e-36+0j), (6.378710609606567e-42+0j), (1.1630207880109586e-24+0j), (3.491238335460158e-27+0j), (2.474273299335111e-35+0j), (7.850346831862612e-27+0j), (4.890545855265234e-36+0j), (2.483100878783576e-42+0j), (6.21029964994736e-35+0j), (2.7325320054333933e-44+0j), (-5.225581225258704e-66+0j), (-4.923379005915621e-21+0j), (1.1841748820453652e-25+0j), (-2.3465719325577737e-25+0j), (5.59422288659291e-25+0j), (-2.1438500124509716e-26+0j), (5.242882753877829e-35+0j), (-3.481142548947434e-25+0j), (2.0418487272344713e-35+0j), (5.453853623152188e-42+0j), (5.700750663233955e-25+0j), (7.03991944928368e-27+0j), (3.7921933660572097e-35+0j), (-1.1281195481494149e-27+0j), (3.1204959466818707e-37+0j), (-1.6269075170811126e-42+0j), (4.822258829475605e-35+0j), (-3.6873417465627155e-42+0j), (-6.628430811253142e-68+0j), (-5.398907755697251e-25+0j), (5.953905498133243e-35+0j), (-1.0691346763412624e-40+0j), (3.183806128692875e-35+0j), (5.20144473726568e-43+0j), (-9.876243328300325e-68+0j), (-9.343420254347779e-42+0j), (-1.255648856520882e-67+0j), (2.6394859524267557e-93+0j), (4.653400536674282e-20+0j), (-4.116050728063866e-21+0j), (-8.788794008375919e-25+0j), (6.352747104407253e-22+0j), (4.1140673929086208e-25+0j), (1.5164520554915883e-26+0j), (2.3414906279897655e-24+0j), (3.2051043025259092e-27+0j), (-6.263582618270692e-37+0j), (3.758708703440958e-21+0j), (2.4957034686296106e-25+0j), (-4.096132016822654e-26+0j), (7.201984520305749e-26+0j), (4.957880365887362e-27+0j), (-2.140951774254874e-37+0j), (-5.5208054744935795e-27+0j), (-3.04109752370376e-36+0j), (1.440534821325912e-42+0j), (1.3440170341048675e-24+0j), (8.198569744497675e-28+0j), (4.1779931329890737e-35+0j), (1.2030953253484236e-27+0j), (3.2003002249316066e-36+0j), (7.626566892087817e-43+0j), (5.147197953616508e-35+0j), (5.535128934083027e-44+0j), (1.1419684718616e-67+0j), (2.911014011696615e-20+0j), (4.896932955249953e-26+0j), (-3.6183342521817214e-26+0j), (3.483609757455988e-27+0j), (2.4318180668744093e-27+0j), (6.111997549255925e-36+0j), (-2.5479800241088097e-26+0j), (-7.875413957537704e-37+0j), (-4.9949283760858104e-42+0j), (7.601779227071758e-26+0j), (2.8894044568405913e-29+0j), (1.8644427486548552e-35+0j), (-6.014738913595096e-27+0j), (1.1717540217768933e-36+0j), (2.3191489584575723e-43+0j), (6.757915249954514e-36+0j), (8.372758324340782e-44+0j), (2.2079572896902825e-67+0j), (-1.5209175459527156e-27+0j), (2.3281533072231566e-36+0j), (-4.970405652960126e-42+0j), (-6.7883152514551215e-37+0j), (-1.4692833351330758e-43+0j), (-3.86246843627956e-68+0j), (-9.151902165793885e-43+0j), (-1.3230681281560997e-68+0j), (-1.0230369500820999e-94+0j), (1.2683621178505516e-25+0j), (-5.498166080447751e-28+0j), (8.498551728329131e-35+0j), (-1.0085992180200163e-27+0j), (-1.4792242795623501e-36+0j), (-2.4951870780383774e-43+0j), (5.49579329487554e-35+0j), (-1.8950372201142643e-44+0j), (-1.5098441299965625e-67+0j), (-4.686865520612343e-27+0j), (2.656172387858273e-36+0j), (-3.622006205663571e-42+0j), (1.0079174079956759e-36+0j), (-3.31768359073153e-44+0j), (-1.7042596603516807e-68+0j), (-7.151466500638315e-43+0j), (-1.2025084189009258e-68+0j), (-5.372958834719151e-95+0j), (2.69461872606602e-36+0j), (-1.3864096681413659e-43+0j), (-1.1905316873074986e-68+0j), (-1.7241513356098097e-43+0j), (-2.6445430215813976e-69+0j), (2.295436909922596e-96+0j), (-8.236537010378519e-69+0j), (1.9385692343575615e-96+0j), (-1.4316244967394662e-267+0j), (-5.669826790683473e-20+0j), (1.098189043376275e-25+0j), (-1.5168692290712725e-25+0j), (3.7505602877827785e-25+0j), (-3.622049847976322e-26+0j), (3.1708904397804264e-35+0j), (-4.968513296887656e-25+0j), (9.463069411864533e-36+0j), (-4.3159992701204366e-43+0j), (4.178627769391908e-25+0j), (-2.9342023531063706e-26+0j), (4.737318674084528e-35+0j), (-1.2820742221205286e-26+0j), (4.148423875027979e-36+0j), (1.0456313978483744e-42+0j), (2.1724299597370458e-35+0j), (-8.565436863185444e-44+0j), (-1.1073030727705331e-67+0j), (-1.6996165975167744e-25+0j), (4.1722713436726607e-35+0j), (-2.819399373048429e-41+0j), (5.013070612806409e-36+0j), (-6.688791685415443e-43+0j), (-2.813647777713474e-68+0j), (-2.4797596577577014e-41+0j), (-3.989392279843618e-68+0j), (-3.2695910775437163e-95+0j), (6.098809876008478e-25+0j), (-1.8008050066957144e-28+0j), (7.916695186227795e-35+0j), (-1.3000409666684731e-26+0j), (-7.985530793461277e-37+0j), (1.489667848731301e-42+0j), (2.3555224520461832e-35+0j), (-1.1212029861236417e-43+0j), (-7.017245416240776e-68+0j), (-9.000508498119134e-27+0j), (8.615199500997347e-37+0j), (-1.4254708628344202e-42+0j), (3.692556282930884e-36+0j), (2.361133174166054e-44+0j), (-1.3348543415935993e-68+0j), (-3.3279606917735995e-43+0j), (-5.793398375267094e-69+0j), (2.877869463701727e-96+0j), (1.1249649941101953e-35+0j), (-9.280454878429309e-44+0j), (-2.1511683053716308e-68+0j), (-3.110670480193701e-43+0j), (-3.9011860111156137e-70+0j), (1.6482188415745546e-96+0j), (-5.797463072884908e-69+0j), (1.2764765496647997e-97+0j), (2.6139529283477882e-269+0j), (-1.157022289687658e-25+0j), (4.2761130127868764e-36+0j), (-5.527947279453363e-42+0j), (-1.3657526125015681e-37+0j), (-5.743982617310814e-43+0j), (-5.66345694203849e-68+0j), (-1.6108673708361774e-41+0j), (-1.0393787812809527e-68+0j), (-7.936275606664153e-96+0j), (2.0004473225355998e-36+0j), (-7.420162744261536e-43+0j), (-3.663560554591953e-68+0j), (-5.785865909423843e-43+0j), (-1.4195625385235661e-70+0j), (3.2950686620482857e-96+0j), (-5.842130526320512e-69+0j), (1.2946564350631612e-96+0j), (1.628166905039491e-269+0j), (-8.938797204400344e-42+0j), (-1.0649719913882951e-69+0j), (1.6526298523026482e-96+0j), (-3.79905639802652e-69+0j), (-3.283553518408838e-97+0j), (-4.320458481255519e-271+0j), (-4.872276065305624e-97+0j), (-1.4886063016418433e-271+0j), 0j, (5.738648940527373e-20+0j), (2.058025364031933e-21+0j), (3.224711919249694e-24+0j), (1.6378176128549948e-22+0j), (4.336589277053364e-25+0j), (6.214236170766107e-27+0j), (1.685259329041752e-25+0j), (3.516367468312748e-28+0j), (1.814410435157389e-34+0j), (-2.2267702089927505e-21+0j), (2.9738162860659615e-25+0j), (-1.5223703787804877e-27+0j), (1.2676679202539571e-25+0j), (-1.1328333200523318e-27+0j), (3.164320110013614e-36+0j), (7.285693610794738e-28+0j), (1.275304107410451e-35+0j), (2.543356712749543e-43+0j), (6.81500091464053e-25+0j), (-9.054526604602922e-28+0j), (2.257987113960561e-34+0j), (-1.4663452277767746e-27+0j), (2.8078383958723394e-36+0j), (-3.7414668997472616e-43+0j), (1.0702795203710342e-34+0j), (-2.0975686387862106e-44+0j), (-4.195552960827292e-67+0j), (-1.313562812734208e-20+0j), (2.8914118759065746e-26+0j), (-7.267846534380155e-26+0j), (1.452419144256718e-25+0j), (2.2355583159892733e-27+0j), (1.383987903777924e-35+0j), (-3.1228596731277385e-26+0j), (2.523293324911298e-36+0j), (2.6423234167924832e-42+0j), (1.1202613715043589e-25+0j), (-3.078377859823748e-27+0j), (2.0021351305759007e-35+0j), (-3.0548914268292065e-27+0j), (1.3556503674086623e-36+0j), (-3.835178734548984e-43+0j), (3.959473216793803e-36+0j), (1.0130949991298326e-43+0j), (-2.21998992369125e-68+0j), (-2.5623822859345149e-26+0j), (1.1311039711259074e-36+0j), (-2.3908779236001988e-42+0j), (2.5493315880592604e-36+0j), (-2.3344209221898623e-43+0j), (-2.1272049005885007e-68+0j), (-2.251601993419415e-42+0j), (-8.923450602763723e-69+0j), (-2.316764273304886e-95+0j), (4.618291506307787e-25+0j), (-3.7658398101726636e-28+0j), (3.293000295056896e-35+0j), (-1.7001924659775857e-27+0j), (4.1494548052284756e-36+0j), (3.05012316519951e-43+0j), (3.2794655832659976e-35+0j), (-3.225244419574165e-44+0j), (-1.053622726687762e-67+0j), (-4.1636078577565e-27+0j), (5.289764100396695e-36+0j), (-4.710771321866944e-44+0j), (-7.2905880278719956e-37+0j), (-8.899066321781529e-45+0j), (-2.8931621724738964e-69+0j), (-3.6732083378326894e-44+0j), (-1.5238852141652963e-69+0j), (-1.841461600436511e-96+0j), (2.5555644911989628e-36+0j), (-4.816347666127353e-44+0j), (2.0160562566152054e-68+0j), (-1.6798920625800974e-43+0j), (-3.3925829734796594e-71+0j), (-4.3718775193904046e-97+0j), (3.777223252098861e-69+0j), (1.357799677003808e-97+0j), (1.2408046248716014e-269+0j), (1.3402807465196707e-20+0j), (1.4412488738387886e-26+0j), (-8.144863637343389e-26+0j), (1.6329420738074944e-26+0j), (-1.4185493918788701e-27+0j), (3.2699357118256644e-36+0j), (-6.017031117546339e-26+0j), (4.004267520772525e-37+0j), (-3.648280551869661e-42+0j), (1.2377227602917675e-25+0j), (-4.1209134266204266e-27+0j), (1.4317251940522034e-35+0j), (-1.7971961580236538e-27+0j), (8.74188826071584e-37+0j), (-8.284082405895227e-44+0j), (2.33875177031916e-36+0j), (2.2981294814927e-43+0j), (-2.628672614898358e-68+0j), (-1.629471085824522e-26+0j), (2.303296948770939e-36+0j), (2.172778954801144e-43+0j), (1.9442183084661523e-36+0j), (-1.2023387145902613e-43+0j), (-7.254022295084235e-69+0j), (-1.6774692120614577e-42+0j), (-4.507836501873672e-69+0j), (1.870453690484621e-95+0j), (8.115169904189593e-26+0j), (1.1674210085899677e-29+0j), (7.016420049706893e-36+0j), (-1.6576925847088037e-27+0j), (7.659308027080583e-37+0j), (-2.533668047586047e-43+0j), (-2.6189360300616526e-38+0j), (-2.0457725969163919e-44+0j), (-6.956302078059338e-69+0j), (-1.954600107911362e-27+0j), (6.780912623443183e-37+0j), (-2.478601396995783e-43+0j), (3.4355829455859553e-37+0j), (1.034860284359409e-44+0j), (-1.0321203406979826e-69+0j), (-9.656318295868386e-44+0j), (-6.67875572603635e-70+0j), (3.4996619296127565e-96+0j), (2.8495058093083e-36+0j), (-7.236563907868822e-45+0j), (-3.5351418348845665e-69+0j), (-6.700967518507562e-44+0j), (-4.053572292761226e-70+0j), (-4.773162104252776e-97+0j), (6.850273042920303e-70+0j), (3.0949153186804137e-99+0j), (1.729331964577206e-269+0j), (-8.099925167196197e-27+0j), (2.8585385149685612e-39+0j), (-2.553291699908721e-42+0j), (1.3038549968429153e-36+0j), (-8.243413618066464e-44+0j), (-2.4875457004003215e-69+0j), (-8.959898018940873e-43+0j), (-1.189408234425503e-69+0j), (2.586559529027281e-96+0j), (8.33285901440467e-37+0j), (-8.95396704713859e-44+0j), (-4.373488660038238e-69+0j), (-3.2547613639265393e-44+0j), (-1.6149955259156168e-72+0j), (3.4806569772242486e-97+0j), (-8.257656807606269e-70+0j), (7.821582090755295e-98+0j), (-4.605292713724859e-271+0j), (-2.4675015120653584e-43+0j), (-1.03316015642604e-69+0j), (-2.719235614104682e-97+0j), (-2.5078760342474046e-70+0j), (1.76375426359425e-98+0j), (-1.777822015562036e-271+0j), (-9.698845787627584e-98+0j), (7.452149301398185e-273+0j), 0j, (4.430692494437178e-25+0j), (-1.0070302493211979e-28+0j), (4.2285256499226e-35+0j), (-6.573183492754081e-28+0j), (4.456584396672354e-37+0j), (-9.66430665503391e-44+0j), (1.9889480593802792e-35+0j), (-5.132666162249113e-44+0j), (-9.554715966482271e-68+0j), (-3.2382986678355416e-27+0j), (2.7744750896381775e-36+0j), (-2.897649849872292e-43+0j), (-2.654654403730625e-38+0j), (-1.631130570851529e-44+0j), (-1.8597627651332472e-69+0j), (-1.733070244536799e-43+0j), (8.907046652651203e-70+0j), (8.843909239179505e-98+0j), (8.513215168923517e-36+0j), (-2.4090889877004123e-44+0j), (1.6438293535192985e-69+0j), (-5.710830071489184e-44+0j), (-5.326823695736443e-70+0j), (1.4092980032624498e-97+0j), (-6.582265920870447e-69+0j), (-4.603519237547997e-98+0j), (-1.9108428112462536e-271+0j), (-4.2311540727660494e-27+0j), (2.7141601888071484e-37+0j), (1.5427009744016563e-43+0j), (7.244200592436882e-37+0j), (-5.683114570551412e-44+0j), (-4.018080832478832e-69+0j), (-4.1293489917135046e-43+0j), (-3.570785045925186e-70+0j), (4.0033423591304e-96+0j), (8.983573035414472e-37+0j), (-7.511730380302233e-44+0j), (-1.7333305288867399e-69+0j), (-1.7587374455113447e-44+0j), (-3.051453468828469e-70+0j), (-5.4698171224247676e-98+0j), (-5.3417020100328104e-70+0j), (-4.197357656064822e-98+0j), (-2.7074670261559784e-271+0j), (-2.2680546776900354e-43+0j), (9.246190849134761e-71+0j), (-2.310730749105864e-97+0j), (-3.9621312184493284e-70+0j), (7.975781580198777e-99+0j), (7.538320487690324e-273+0j), (1.1272737989162328e-98+0j), (3.1533034002686707e-273+0j), 0j, (-1.396443367817735e-36+0j), (-5.935943144643757e-45+0j), (-3.913605521992993e-69+0j), (-3.4742363735634085e-44+0j), (-8.1756819864826e-71+0j), (3.729266846018025e-98+0j), (-8.27573218785296e-70+0j), (7.790998842520057e-99+0j), (3.32938854702604e-273+0j), (-2.520215528338682e-44+0j), (5.601191448366384e-71+0j), (7.821716023689834e-99+0j), (-1.3873914967606252e-70+0j), (3.0614174989785105e-99+0j), (3.5696949275673335e-273+0j), (-5.786813838158448e-99+0j), (-1.249002724186407e-274+0j), 0j, (-2.905433714996657e-70+0j), (2.3387685757916495e-99+0j), (-4.9659040780909366e-275+0j), (1.3105983307271435e-99+0j), (-5.113412940464627e-275+0j), 0j, (7.661734760745137e-277+0j), 0j, 0j, (5.776425887095427e-17+0j), (9.263152311173029e-18+0j), (-0.00022258684333429702+0j), (2.893734539572627e-17+0j), (0.0043647994253026645+0j), (2.7884564248276647e-19+0j), (1.2590560129425057e-05+0j), (9.067993300682672e-19+0j), (2.2551405187698492e-17+0j), (-4.2035510513125826e-17+0j), (0.004669751160580358+0j), (1.802912388190805e-18+0j), (-0.007815878234412715+0j), (1.1234093470549791e-18+0j), (4.87890977618477e-19+0j), (1.0425675866881242e-18+0j), (-4.0826988057657276e-19+0j), (-2.412474257466606e-22+0j), (-8.596326671828418e-05+0j), (-1.1265265942105934e-19+0j), (-4.683753385137379e-17+0j), (5.19070814605251e-19+0j), (-1.2739375526704677e-18+0j), (-6.329084861795287e-23+0j), (-1.0408340855860843e-16+0j), (-7.088268853985659e-23+0j), (-1.5666050484592952e-22+0j), (8.191867391133152e-17+0j), (-0.006452282290499302+0j), (2.966374034072448e-19+0j), (0.002633757104164953+0j), (-4.985644054085572e-19+0j), (-2.574980159653073e-19+0j), (1.4016460560477301e-18+0j), (-5.590417451878382e-20+0j), (-1.1491285452106587e-22+0j), (0.005708076652180287+0j), (-5.280927934393153e-19+0j), (-9.215718466126788e-19+0j), (1.0094928930446822e-18+0j), (-5.421010862427522e-19+0j), (4.398417620799216e-25+0j), (-2.656295322589486e-18+0j), (-3.255718192336549e-25+0j), (6.292756688264541e-24+0j), (7.761807579522288e-19+0j), (-2.778268066994105e-19+0j), (6.167887857100638e-23+0j), (-9.486769009248164e-19+0j), (-1.1349253194624291e-23+0j), (6.834133182826758e-24+0j), (4.403627473900817e-23+0j), (4.673129453319998e-25+0j), (8.508461373411958e-34+0j), (-0.00015586988605421884+0j), (4.447091362585319e-19+0j), (-8.760353553682876e-17+0j), (6.986220665474863e-19+0j), (1.0299920638612292e-18+0j), (-1.1463554750665685e-22+0j), (3.279711571768651e-17+0j), (-7.628653409173123e-23+0j), (1.315093023453755e-22+0j), (5.968210142972236e-19+0j), (-1.463672932855431e-18+0j), (-7.617281746464176e-23+0j), (1.748276003132876e-18+0j), (-3.519656391112095e-24+0j), (4.3171252697453305e-24+0j), (-1.8359532189555872e-23+0j), (3.633163195131376e-24+0j), (-7.467172661610074e-34+0j), (1.680513367352532e-17+0j), (-3.432969219131794e-23+0j), (5.712032888537331e-23+0j), (1.700243899708606e-23+0j), (4.61335100250496e-24+0j), (-1.875544399420154e-34+0j), (5.2170064313367767e-23+0j), (1.1501370550473795e-33+0j), (-1.999305430738796e-40+0j), (2.1725124547651922e-17+0j), (-0.0033616892517299396+0j), (1.4386765747736088e-18+0j), (0.004567348052183284+0j), (-7.10979087611711e-19+0j), (-2.202285662861181e-20+0j), (1.3124755554597704e-18+0j), (7.030373462210693e-20+0j), (-2.0965572369222775e-22+0j), (0.0006749595316184265+0j), (2.3076631674250496e-19+0j), (-3.0357660829594124e-18+0j), (5.497903253654628e-19+0j), (-1.6263032587282567e-19+0j), (-4.8959276857158795e-24+0j), (-1.870248747537495e-18+0j), (3.129210810103223e-24+0j), (9.06322256414268e-24+0j), (1.2306023482904143e-18+0j), (2.0328790734103208e-20+0j), (2.4789759749865653e-23+0j), (-3.7269449679189215e-20+0j), (1.453987388594868e-24+0j), (1.220249533611581e-25+0j), (-1.2786655198288651e-23+0j), (3.4339376639630945e-25+0j), (8.562469828702473e-34+0j), (-0.0034189053224323914+0j), (6.174057455840031e-19+0j), (-3.0357660829594124e-18+0j), (-3.9155288704740887e-19+0j), (-1.0842021724855044e-19+0j), (-1.7050891786669362e-24+0j), (1.8905775382715984e-18+0j), (-1.1365071180624472e-24+0j), (8.751392311151928e-24+0j), (-5.067594337290432e-20+0j), (-6.776263578034403e-21+0j), (7.426160187173976e-25+0j), (-2.202285662861181e-20+0j), (8.959634096865802e-25+0j), (4.233920865650086e-25+0j), (-1.0687108890699762e-24+0j), (1.9491689255236205e-25+0j), (5.991766768798567e-35+0j), (-3.4220131069073734e-19+0j), (8.283742743903111e-25+0j), (2.404785973615042e-24+0j), (2.167310140116966e-24+0j), (6.554273567835476e-26+0j), (2.925230319401984e-35+0j), (9.96182702974203e-25+0j), (2.6005209549598883e-35+0j), (1.498728729386232e-42+0j), (5.336615316852896e-19+0j), (-5.082197683525802e-21+0j), (2.767009700411795e-23+0j), (-3.3881317890172014e-21+0j), (-7.916376956073898e-25+0j), (8.16343527413624e-25+0j), (5.9938282667416654e-24+0j), (9.95983460879664e-26+0j), (2.333077336062661e-34+0j), (-7.403067959002585e-19+0j), (1.5388467450327321e-24+0j), (1.679974918079694e-24+0j), (3.554695591347264e-24+0j), (9.314203044368723e-26+0j), (5.043928308163058e-35+0j), (6.254684259828088e-25+0j), (1.0152477237169537e-35+0j), (6.808401528317551e-41+0j), (8.51899095701585e-24+0j), (1.2295222985992922e-25+0j), (1.2940969812031894e-33+0j), (3.645277786610087e-27+0j), (4.0757013988029755e-35+0j), (-1.6498708865109618e-42+0j), (1.1024509879436637e-33+0j), (-1.447261053954671e-41+0j), (1.350578537208063e-64+0j), (3.547555001141489e-05+0j), (1.114948668617818e-19+0j), (-9.540979117872439e-18+0j), (2.6947586392029265e-19+0j), (-6.2341624917916505e-19+0j), (-2.2139642014328207e-23+0j), (1.6154612370034016e-17+0j), (1.1809679892362132e-23+0j), (1.6816134115420846e-22+0j), (6.641561554327309e-19+0j), (-1.5043305143236374e-18+0j), (6.725208351164294e-24+0j), (-7.928228386300251e-19+0j), (3.2793534459993183e-24+0j), (5.239732741788402e-24+0j), (-7.615877749169868e-24+0j), (3.612306724993889e-24+0j), (9.553476769647933e-35+0j), (-3.333921680392926e-18+0j), (-9.96817261611754e-24+0j), (3.7278888197524917e-23+0j), (-1.1470668650637688e-24+0j), (7.2155221354907005e-25+0j), (5.632818892898554e-34+0j), (3.033858697439393e-24+0j), (1.2197470065615685e-34+0j), (3.975777840794036e-39+0j), (3.235807253674487e-19+0j), (-4.828087799349512e-20+0j), (-5.203801479774712e-23+0j), (-5.929230630780102e-20+0j), (5.133528117943839e-25+0j), (3.3202347789567094e-24+0j), (1.6420827628884713e-23+0j), (2.6333524533562354e-25+0j), (1.0819529644054135e-33+0j), (-1.4433441421213278e-18+0j), (3.2190573642810696e-24+0j), (2.381336709285792e-24+0j), (4.759923593231168e-25+0j), (2.1767419916712186e-25+0j), (6.654115361569953e-35+0j), (-6.875930413434151e-25+0j), (4.6988692022230903e-35+0j), (-2.438879004648165e-41+0j), (7.449472274378927e-24+0j), (6.919696754501187e-26+0j), (6.299004975369864e-34+0j), (5.138972243388254e-25+0j), (7.716819077926549e-36+0j), (6.984045097325589e-41+0j), (3.26840278345508e-34+0j), (4.450523922695619e-42+0j), (6.836176254742797e-65+0j), (-2.7159264420761886e-17+0j), (1.0295329010730799e-24+0j), (5.0064469467634936e-23+0j), (9.376867725112837e-24+0j), (-7.794018672356676e-25+0j), (8.685287866199286e-34+0j), (-2.789791602813468e-23+0j), (-2.298481469730318e-35+0j), (-2.4562407898193978e-39+0j), (1.4865421115729582e-23+0j), (-7.167633822167118e-26+0j), (1.1620887388869654e-33+0j), (-5.436372492154811e-25+0j), (5.788738647554515e-35+0j), (6.605160441441458e-41+0j), (1.0154277058639299e-33+0j), (4.645584668929634e-41+0j), (-7.381629010667731e-66+0j), (4.194189356306035e-24+0j), (6.7616994076922364e-34+0j), (-6.675113260780553e-40+0j), (1.0189798442922556e-33+0j), (-3.095748567386386e-41+0j), (-5.405507390864894e-66+0j), (-1.0535382244333272e-40+0j), (-3.0970241474184477e-66+0j), (-7.038824436337696e-92+0j), (-6.929491838192706e-17+0j), (0.0022691571194941966+0j), (2.908036581204483e-18+0j), (-0.0017176508958175831+0j), (4.872399897679188e-19+0j), (7.623296525288703e-21+0j), (1.5184194675814214e-18+0j), (8.279747059410786e-20+0j), (-2.006985818480482e-22+0j), (-0.0014113119602811774+0j), (1.2305531234593463e-19+0j), (7.453889935837843e-20+0j), (-3.0736034801853637e-19+0j), (5.717472393966527e-21+0j), (-1.1916057381028074e-23+0j), (3.218725199566341e-20+0j), (-9.897726216632228e-25+0j), (5.353852874245006e-25+0j), (1.4842563243584904e-18+0j), (1.0079692072326174e-19+0j), (-4.397444787756126e-23+0j), (2.964615315390051e-20+0j), (-1.4231002117878194e-24+0j), (2.2559192115038874e-25+0j), (2.980435144605128e-23+0j), (2.940881384680478e-26+0j), (-2.7268814893238104e-34+0j), (0.001192168254206991+0j), (-2.579196243718652e-19+0j), (-3.3068166260807885e-18+0j), (2.0543722264730763e-19+0j), (-1.5246593050577406e-20+0j), (-5.403244963280242e-24+0j), (5.861467994999758e-19+0j), (-1.0420537013607478e-25+0j), (3.2618127952441683e-24+0j), (2.234269609225419e-19+0j), (8.470329472543003e-20+0j), (-4.592906089686975e-24+0j), (-4.002230675776569e-20+0j), (-1.2032085179665378e-24+0j), (2.80348726170148e-25+0j), (-3.554291694563791e-25+0j), (2.2460440183228645e-25+0j), (-1.9881064782947826e-36+0j), (1.2874900798265365e-19+0j), (1.5727740748444774e-24+0j), (2.5054530636073687e-24+0j), (3.232789854919157e-24+0j), (1.077774862990331e-25+0j), (1.827343850495411e-35+0j), (5.078576533413852e-26+0j), (2.1147934017517687e-35+0j), (9.981663072346292e-41+0j), (4.87045749882466e-19+0j), (1.9958213819679452e-20+0j), (-1.5541948228047122e-23+0j), (5.7809998650106e-20+0j), (-1.4075802904039558e-24+0j), (2.631514430024085e-25+0j), (5.6464770329547495e-24+0j), (1.2687182445874629e-26+0j), (2.4618991765179873e-34+0j), (3.282252670610414e-20+0j), (-5.188054183712715e-25+0j), (1.312814730654654e-25+0j), (1.085371631388244e-24+0j), (6.566589526113073e-27+0j), (1.0173115112536604e-35+0j), (-3.8967408049450365e-26+0j), (3.692016067939513e-35+0j), (-5.392196490721896e-42+0j), (1.3250641775403896e-23+0j), (5.449477099584311e-27+0j), (1.3862855303165822e-33+0j), (1.908421241800586e-26+0j), (2.7795250180287325e-35+0j), (-6.1096613044562024e-43+0j), (4.702557095128365e-34+0j), (8.085492139154194e-43+0j), (-4.571698526753947e-66+0j), (0.00047202638269621246+0j), (-8.160053298325187e-21+0j), (-2.710505431213761e-20+0j), (1.5104750032328402e-19+0j), (2.922263668027336e-20+0j), (2.337768476630432e-25+0j), (-1.7110065534536867e-19+0j), (4.121766675343578e-25+0j), (4.156427097569183e-24+0j), (6.151387700143864e-21+0j), (4.743384504624082e-20+0j), (-3.073433750582925e-24+0j), (9.423241538204091e-21+0j), (7.07324242057368e-26+0j), (1.6665990396416982e-25+0j), (3.812785635986612e-25+0j), (1.4439300442635023e-25+0j), (4.8058318700068183e-35+0j), (-6.378158092824882e-19+0j), (8.653488585912411e-25+0j), (1.7975517784622593e-25+0j), (-4.0318996410208e-25+0j), (5.500576173358063e-26+0j), (3.0599503376459465e-35+0j), (5.985781398376734e-25+0j), (5.695620970027899e-36+0j), (9.536726233009106e-41+0j), (1.1542192631532722e-20+0j), (-1.3817224952085774e-20+0j), (6.325023629189655e-25+0j), (-3.2081372877256625e-20+0j), (9.552158929140188e-26+0j), (1.902465954312619e-25+0j), (1.928405192692593e-24+0j), (1.675641242087865e-26+0j), (-1.826859848732066e-35+0j), (1.4823076576950256e-20+0j), (-6.486582342578918e-25+0j), (1.1579920133348333e-25+0j), (3.672179068240019e-25+0j), (1.1815627575454598e-26+0j), (1.6689859495122329e-35+0j), (-5.4889578276942144e-27+0j), (8.22164050428596e-36+0j), (6.950440383051093e-42+0j), (5.101721246245328e-25+0j), (2.9020773388162152e-27+0j), (1.1079633770098416e-34+0j), (-3.572963354890452e-26+0j), (4.721183084363351e-36+0j), (2.1467892473456198e-42+0j), (2.0521725436787384e-35+0j), (-5.9344989964156e-43+0j), (-7.431940333508278e-66+0j), (-2.8343839997497025e-19+0j), (8.080460024359868e-26+0j), (-2.0407315593024054e-25+0j), (4.7271069795739735e-25+0j), (2.5931610828310105e-26+0j), (4.677999962390214e-35+0j), (-3.0129900078451824e-25+0j), (7.170622128382344e-36+0j), (1.9483653847972257e-41+0j), (6.890731561541747e-25+0j), (-2.2892353996983348e-26+0j), (1.0221554014034442e-34+0j), (-3.7684777255275426e-26+0j), (5.359527346999829e-36+0j), (-2.2651989675810668e-42+0j), (5.873967661566353e-35+0j), (-3.797518838320254e-42+0j), (8.971871156680053e-68+0j), (-4.241985295222028e-25+0j), (2.924866931321099e-35+0j), (4.681738169309214e-42+0j), (2.6306020175668993e-35+0j), (-5.444920345442117e-43+0j), (6.6057538493182175e-68+0j), (-9.71056045200088e-43+0j), (-1.2197652007170366e-67+0j), (2.6264213493722363e-93+0j), (2.9180333988940825e-19+0j), (4.658681209898652e-21+0j), (-5.428372769879244e-24+0j), (-6.988021814847978e-21+0j), (1.9160863407966618e-24+0j), (1.463227945266071e-25+0j), (7.559332199483626e-24+0j), (1.306011614611977e-26+0j), (7.780939998876501e-34+0j), (-2.2499312661442353e-20+0j), (9.865178936331885e-25+0j), (-3.026260751971497e-26+0j), (7.0904080338712895e-25+0j), (7.125563033759425e-27+0j), (4.1207124686602347e-35+0j), (-4.9441536953809963e-26+0j), (2.3022239455649596e-35+0j), (1.0089348943138683e-42+0j), (5.849031269866538e-24+0j), (1.1629160232907159e-26+0j), (6.675847514270928e-34+0j), (-4.9560929859565867e-26+0j), (1.7526100563928799e-35+0j), (-2.1047502934158752e-42+0j), (4.294188885056172e-34+0j), (9.84412171188184e-43+0j), (-2.6230531510793195e-66+0j), (2.482865326639168e-20+0j), (2.4587216693928496e-25+0j), (4.3229435733222655e-25+0j), (1.0851696829965074e-24+0j), (6.2828457763658716e-27+0j), (1.25557825762006e-34+0j), (-3.525652098702394e-25+0j), (2.0839853370936752e-35+0j), (-3.646739123558904e-41+0j), (1.1124074723319785e-24+0j), (5.4917570096842154e-27+0j), (7.280059852470632e-35+0j), (-2.0203787472667282e-26+0j), (6.09817295517822e-36+0j), (8.975316664000453e-43+0j), (4.018797942720996e-35+0j), (-3.528995020094011e-42+0j), (8.50274820054555e-68+0j), (-3.633241266910384e-25+0j), (1.316464981938558e-35+0j), (-1.708182828011952e-42+0j), (1.462170992929352e-35+0j), (-8.944663260093348e-43+0j), (-1.4768340677360381e-67+0j), (-3.532498266254823e-42+0j), (-9.941906906011727e-68+0j), (1.954730978737773e-93+0j), (2.5580550345782794e-24+0j), (2.616445021444456e-29+0j), (3.473185002136315e-34+0j), (-6.226195317843169e-26+0j), (8.609544458749776e-36+0j), (-2.87704090956689e-42+0j), (1.1084245107873657e-34+0j), (-7.296604894316333e-43+0j), (-5.609695888798831e-67+0j), (-5.606675820036592e-26+0j), (7.520842241229273e-36+0j), (1.601158657799144e-42+0j), (5.1038637485474925e-36+0j), (4.598010586065806e-46+0j), (-4.799532009574448e-68+0j), (-1.4361338683363918e-42+0j), (-1.4855505085818497e-68+0j), (-1.1981795735305977e-95+0j), (3.191757469485365e-35+0j), (-3.348008565311059e-43+0j), (-1.803226772584741e-67+0j), (-7.073355058896459e-43+0j), (-1.0262167372965967e-68+0j), (-8.77179244135506e-96+0j), (-2.7923706730192204e-68+0j), (2.4057071207859343e-96+0j), (-6.987971372795129e-268+0j), (-3.9295455394174047e-05+0j), (4.754504034020723e-20+0j), (1.531435568635775e-17+0j), (4.764642996828684e-20+0j), (-1.9566461081574338e-19+0j), (5.688482298435958e-24+0j), (-1.5429552167184335e-17+0j), (6.514855117422039e-25+0j), (3.117404821186902e-23+0j), (9.740939677962572e-20+0j), (3.2610768469290563e-19+0j), (4.3023085375560795e-24+0j), (-5.8106460181645e-19+0j), (-1.0985992510469899e-25+0j), (3.62996402705977e-25+0j), (9.25448699972047e-24+0j), (7.781109414251096e-25+0j), (1.6445413780350466e-34+0j), (-1.0785270517389006e-17+0j), (7.585181593625908e-25+0j), (-7.426465024411576e-24+0j), (4.655920171486829e-24+0j), (2.4246175584721203e-25+0j), (4.187720120050219e-34+0j), (-1.0032680699946635e-23+0j), (3.5176400905727912e-34+0j), (-8.19703549691445e-40+0j), (2.964615668573512e-19+0j), (7.067431153653068e-21+0j), (9.493189998753107e-24+0j), (-1.573363699524863e-19+0j), (7.518538624352837e-25+0j), (4.767122488720011e-25+0j), (7.686761634669407e-24+0j), (-3.9883078795273625e-28+0j), (2.7374135284063194e-34+0j), (-9.682645378300721e-20+0j), (6.434075760727408e-25+0j), (1.3554236010630603e-25+0j), (7.1863335199461645e-25+0j), (-2.8859054072639154e-26+0j), (5.57364691150275e-35+0j), (-2.646822730616225e-25+0j), (6.325087338753771e-35+0j), (4.5830867574207467e-41+0j), (3.4090908009051905e-24+0j), (-8.497611904507997e-26+0j), (6.91863397686666e-34+0j), (-2.2879888645328053e-25+0j), (5.025228139344149e-35+0j), (-2.9441280735464407e-41+0j), (3.1537545538897083e-34+0j), (-3.6983769719692735e-42+0j), (-1.4119142574665e-66+0j), (7.450634152946834e-18+0j), (1.4483233784368106e-24+0j), (-1.7550312068341413e-23+0j), (4.7351344281455025e-24+0j), (-3.34005919890268e-25+0j), (4.13987073607222e-34+0j), (-3.593519765506449e-24+0j), (2.7557142372555865e-34+0j), (1.0064265700627269e-40+0j), (2.537733977659786e-24+0j), (-3.1264453683216824e-25+0j), (4.625725703856611e-34+0j), (-2.351509644930747e-25+0j), (3.263586247436047e-35+0j), (-4.5433949784187463e-41+0j), (2.2032037810607484e-34+0j), (-2.939486272383365e-42+0j), (-1.2016319560124118e-66+0j), (-7.978662407877941e-24+0j), (3.365697953445317e-34+0j), (-2.723605734215644e-40+0j), (1.9867807009054267e-34+0j), (-1.2199091362333715e-41+0j), (-1.0717542911127145e-66+0j), (-2.432462550293463e-40+0j), (-4.0949515484312506e-67+0j), (2.385417827928475e-93+0j), (1.3298419121764937e-19+0j), (7.940933880509066e-21+0j), (1.0129731329506804e-23+0j), (-2.3022090808575866e-20+0j), (3.704743246407542e-25+0j), (4.603076600110538e-27+0j), (7.120094447456567e-24+0j), (4.777686545987648e-28+0j), (2.6158057966530914e-34+0j), (-8.179161896924338e-20+0j), (3.9036624122680725e-25+0j), (-5.707425069538404e-25+0j), (9.302500229855842e-25+0j), (-3.8823405582195325e-26+0j), (3.9043527480748475e-35+0j), (-5.281804228434471e-25+0j), (4.4418971616485184e-35+0j), (-7.858201528240709e-41+0j), (4.442309260127463e-24+0j), (-2.3074823758296738e-26+0j), (5.550395588043202e-34+0j), (-6.950010980581221e-26+0j), (1.9121776165044794e-35+0j), (-4.661069016960423e-42+0j), (2.6733372241268485e-34+0j), (1.5379250645964867e-43+0j), (-1.0780012941658979e-66+0j), (3.2892009877558592e-19+0j), (9.091464160491117e-26+0j), (-3.2167222675168958e-25+0j), (3.7645704574595037e-25+0j), (-2.24667516802431e-26+0j), (4.0466753911858556e-35+0j), (-3.983096796202155e-25+0j), (5.4140789321226504e-36+0j), (-5.917683414843702e-42+0j), (4.85483933734736e-25+0j), (-5.8977234874730896e-27+0j), (1.1591092122653896e-35+0j), (-6.125982161606692e-26+0j), (3.9543142236928647e-36+0j), (2.041516700213218e-42+0j), (3.77005884311389e-35+0j), (-7.767572550060502e-43+0j), (-1.2534928099235333e-67+0j), (-2.3315746974017837e-25+0j), (1.4176201470425759e-36+0j), (-2.876130065565079e-41+0j), (6.203037449803005e-36+0j), (-1.3055941582691331e-42+0j), (-8.599780643556165e-68+0j), (-2.0712368174477944e-41+0j), (-3.7090222383503156e-68+0j), (2.0692819644677496e-95+0j), (2.076420742061022e-24+0j), (-1.6120477477871826e-26+0j), (3.3646047575775124e-34+0j), (-2.6496260084563345e-26+0j), (1.7580947226775096e-35+0j), (-2.105801267264119e-42+0j), (5.047664865263311e-35+0j), (-7.834873193102977e-43+0j), (-4.0961365537414474e-67+0j), (-1.1456942470886843e-25+0j), (1.3996595815212546e-35+0j), (-2.669346581865447e-41+0j), (2.3327861637229725e-36+0j), (-1.1613917881747074e-42+0j), (-6.062937539783411e-68+0j), (-5.6171131049792186e-42+0j), (-4.036464467189702e-68+0j), (2.023818714534639e-95+0j), (-9.573806888833474e-37+0j), (-2.1799551356086825e-42+0j), (-5.6238367683688214e-68+0j), (-2.6657291380905258e-42+0j), (-4.601975454345726e-69+0j), (-4.296964881204298e-96+0j), (-3.5352801086283497e-68+0j), (2.9212948084155304e-98+0j), (9.369741175469308e-270+0j), (6.5731609591505825e-18+0j), (6.207767344237604e-25+0j), (-9.092523999679525e-24+0j), (1.3678469443297839e-24+0j), (-6.43188166266487e-25+0j), (1.666613660625515e-34+0j), (-6.202750413066258e-24+0j), (2.0072940868787412e-34+0j), (1.665443224850045e-41+0j), (1.6691287012517927e-24+0j), (-2.591331356654345e-25+0j), (2.557588941187403e-34+0j), (-1.839449975469433e-25+0j), (2.419419964626565e-35+0j), (2.203541835150775e-42+0j), (1.0718078592088882e-34+0j), (-1.9029282820914935e-41+0j), (-7.484000101586608e-67+0j), (-1.563982205577665e-24+0j), (1.4560479775307905e-34+0j), (-3.3148906277298224e-40+0j), (8.950241310449069e-35+0j), (-2.7856258905293503e-41+0j), (-3.1815629765839493e-67+0j), (-4.135863885201677e-40+0j), (-2.340553783584839e-67+0j), (1.8763894272673194e-93+0j), (2.1344493502490796e-24+0j), (-2.091439804871605e-26+0j), (5.299902307985993e-34+0j), (-6.196186866148959e-26+0j), (6.275836852128895e-36+0j), (-5.6075585484578164e-42+0j), (9.829529366168873e-35+0j), (-1.9776755127545447e-42+0j), (-5.47169781323459e-67+0j), (-6.831219894851874e-26+0j), (2.1195824286214084e-35+0j), (-1.676202569621439e-41+0j), (1.6699954472906236e-35+0j), (-8.212758503589944e-43+0j), (-5.603855078223344e-68+0j), (-1.577520641174618e-41+0j), (-2.069173016942004e-68+0j), (3.983194951637829e-95+0j), (7.479469876798928e-35+0j), (-1.686609126556795e-42+0j), (-2.4728766640124555e-67+0j), (-1.5366581484653618e-41+0j), (-3.9872472547950855e-69+0j), (6.178402242411182e-96+0j), (-6.548590631720684e-68+0j), (4.094338952084482e-96+0j), (7.53872199050953e-270+0j), (-4.114461823361231e-26+0j), (2.2359567097982767e-35+0j), (-5.987394072291931e-40+0j), (5.38142242895351e-35+0j), (-1.3132579882053455e-41+0j), (-2.0665185793218894e-67+0j), (-8.9042129461902e-41+0j), (-7.646373363038756e-68+0j), (-5.18254969465892e-95+0j), (1.7671284206497622e-35+0j), (-1.1123678266751844e-41+0j), (-1.2532835540531952e-67+0j), (-4.5368383145360425e-42+0j), (-1.5829484386659024e-68+0j), (-4.349638269868128e-96+0j), (-1.0735903390114084e-68+0j), (-2.8075835459880108e-96+0j), (-1.868135871884139e-271+0j), (-5.088787490914118e-41+0j), (-1.8418440984329086e-68+0j), (-1.2239037849117026e-95+0j), (-8.462726838152304e-69+0j), (2.9861597854261134e-97+0j), (-7.113259819924929e-272+0j), (-5.680776971768795e-97+0j), (2.4299202637170184e-273+0j), 0j, (8.991170031809673e-17+0j), (0.002321597608482496+0j), (1.3086448246848053e-18+0j), (-0.0018956940789787245+0j), (5.363938695979127e-19+0j), (5.717472393966527e-20+0j), (1.0912152763237783e-18+0j), (5.320425699941074e-20+0j), (-4.737270023390864e-23+0j), (-0.0013628000118285869+0j), (1.7220596688997907e-19+0j), (1.1858461261560205e-20+0j), (-3.735101216754568e-19+0j), (1.1117307432712692e-21+0j), (1.668507641999642e-24+0j), (5.082197683525802e-21+0j), (-2.328762681470955e-24+0j), (1.966764100402697e-25+0j), (7.047825610229119e-19+0j), (5.346895479542771e-20+0j), (1.5489255540684982e-23+0j), (4.658681209898652e-20+0j), (-2.5420272025732576e-24+0j), (2.808325583750364e-26+0j), (-3.334571844354393e-24+0j), (2.7146492672757985e-26+0j), (-1.2473551673884348e-34+0j), (0.0013893616059800094+0j), (-3.1240160087368747e-19+0j), (-7.792703114739563e-20+0j), (2.2299201134108244e-19+0j), (5.135137242729196e-21+0j), (-7.195462116489357e-24+0j), (2.710505431213761e-20+0j), (-1.9710163033490113e-25+0j), (4.186732127014649e-25+0j), (2.108108255400964e-19+0j), (1.5828928201814738e-20+0j), (-1.2323621791649623e-24+0j), (-2.6469779601696886e-23+0j), (-2.5036551865542384e-25+0j), (2.886193364694054e-26+0j), (1.6349741794993438e-24+0j), (8.944857322598776e-27+0j), (-2.915072452569134e-36+0j), (6.988021814847978e-21+0j), (7.022755322639535e-26+0j), (2.0692172585003662e-25+0j), (4.850800369512628e-25+0j), (4.752665687363557e-27+0j), (1.2795410981100366e-35+0j), (-1.4135625864379815e-27+0j), (2.1534760194096235e-36+0j), (-2.1096657225549015e-42+0j), (3.7523657926968563e-19+0j), (1.2705494208814505e-20+0j), (7.108583389127582e-24+0j), (2.5040411503205254e-20+0j), (1.3041827138348273e-24+0j), (3.793388655662875e-26+0j), (6.57382404780912e-24+0j), (6.442566951573366e-27+0j), (7.0855593260807875e-34+0j), (3.1075521252392144e-20+0j), (1.1933130467714454e-24+0j), (1.1617399297293854e-25+0j), (3.849136346499196e-25+0j), (4.416273487827118e-27+0j), (2.5233237499035554e-37+0j), (-1.6171729690144543e-26+0j), (3.201947350382931e-36+0j), (2.090737308772627e-42+0j), (9.649498053957219e-24+0j), (1.145259601801659e-28+0j), (8.327595826359936e-34+0j), (3.65797767351902e-27+0j), (5.289154848860018e-35+0j), (1.6955711418330287e-42+0j), (6.2449892467798e-34+0j), (5.612200349620892e-43+0j), (-3.509074420440241e-66+0j), (0.0003855728828853437+0j), (1.4258631643598985e-20+0j), (1.3417001884508117e-18+0j), (1.490817311732738e-19+0j), (-2.0328790734103208e-20+0j), (5.343037448560386e-25+0j), (-2.456395547037471e-19+0j), (-2.42741966867368e-25+0j), (4.9362442408606234e-24+0j), (-2.3290489705921203e-20+0j), (9.317362419797304e-21+0j), (-1.1292954065909499e-24+0j), (5.823351512373315e-21+0j), (2.428934281611704e-25+0j), (1.834382064056787e-25+0j), (6.4259978250579445e-25+0j), (1.815950163617064e-25+0j), (1.8151776561559557e-35+0j), (-2.1429933565533799e-19+0j), (5.606087354607434e-25+0j), (1.241803458948253e-25+0j), (6.4497267610869925e-25+0j), (3.8669719414997743e-26+0j), (3.9006703688866664e-35+0j), (4.077709817649264e-25+0j), (6.741696506621944e-36+0j), (-7.883148163449644e-41+0j), (-3.4674987874226236e-20+0j), (-1.0164395367051604e-20+0j), (-1.3995023547344926e-24+0j), (-2.4352197233561135e-21+0j), (1.4030364515898828e-25+0j), (1.9490878656656866e-25+0j), (6.963180547077245e-25+0j), (6.737553095287718e-27+0j), (6.368007110782872e-35+0j), (8.787966827763366e-21+0j), (-1.6163444403616443e-25+0j), (2.3254674524579613e-27+0j), (1.315946207653483e-25+0j), (3.095610593436197e-27+0j), (3.8833908566295876e-36+0j), (-8.135092825550884e-27+0j), (-8.334652871252336e-37+0j), (3.67700717038832e-42+0j), (9.785914192575278e-25+0j), (4.704294579745088e-27+0j), (8.95345188792866e-35+0j), (-8.192512010403297e-27+0j), (5.201365804925782e-36+0j), (-4.0637655465419695e-43+0j), (4.139304092900097e-35+0j), (6.20775219695894e-43+0j), (-3.4300083926631936e-66+0j), (-5.897466895258066e-20+0j), (1.9062666002484706e-25+0j), (-6.742491235490764e-25+0j), (3.3874318358914424e-25+0j), (9.958661230096437e-27+0j), (-2.961780129408051e-36+0j), (-3.3828120308795903e-25+0j), (1.0851243527769851e-35+0j), (5.262155993232553e-41+0j), (7.319367023002636e-25+0j), (2.7842112415420104e-26+0j), (6.675489902902833e-35+0j), (-3.796031916351784e-26+0j), (9.818543026285379e-36+0j), (-6.21651031236097e-42+0j), (2.694337528414028e-35+0j), (-3.585046958667004e-42+0j), (-1.2512564144434478e-68+0j), (-7.7157593541473385e-25+0j), (2.878669092826093e-35+0j), (-1.8442489088978917e-41+0j), (2.7628884212268263e-35+0j), (-1.495535786050661e-42+0j), (-3.1922211649747006e-68+0j), (-1.6251821683469127e-41+0j), (-7.862377503908151e-68+0j), (1.7936923919703855e-93+0j), (1.6961842450819438e-19+0j), (5.1880768019325896e-21+0j), (-9.919705002100762e-25+0j), (2.8322664173815668e-21+0j), (2.8712012595148122e-25+0j), (5.2778367602729e-27+0j), (7.232377753262105e-24+0j), (3.0202915259984332e-27+0j), (3.0689678228559587e-34+0j), (1.1249656330721176e-20+0j), (5.059816954959987e-25+0j), (6.208104048681421e-26+0j), (2.6518348189909533e-25+0j), (-1.4066734829995284e-28+0j), (2.5291537658443934e-35+0j), (-2.649226290414633e-26+0j), (9.656703580209927e-36+0j), (4.685942064702188e-42+0j), (3.721202040334073e-24+0j), (-9.69493781594206e-28+0j), (5.382953544884661e-34+0j), (-3.6571171712958925e-27+0j), (1.8646688251807792e-35+0j), (-6.3618950280346695e-43+0j), (3.27857011499407e-34+0j), (-3.215979975625455e-43+0j), (-8.497989060230959e-67+0j), (7.815202427401005e-21+0j), (7.058096291193437e-26+0j), (-1.0602440063331272e-26+0j), (2.705477360545982e-25+0j), (-3.255986262414049e-27+0j), (2.2549314497039025e-35+0j), (-4.6697849752008284e-26+0j), (4.386416928186117e-36+0j), (-4.713267384756522e-42+0j), (2.074704180731261e-25+0j), (-2.3260318104356237e-27+0j), (2.1624306742080272e-35+0j), (-7.354208168164538e-27+0j), (-2.3022375280707146e-37+0j), (-7.824500300173697e-43+0j), (9.880995315720163e-36+0j), (1.1319864157123913e-43+0j), (8.210379096378027e-68+0j), (-2.6399405328697647e-26+0j), (3.534906592446252e-36+0j), (2.3156457122967602e-43+0j), (7.73778846968171e-36+0j), (-1.1476415469935201e-43+0j), (-2.283538478030907e-68+0j), (-1.2707149636805482e-42+0j), (-1.2201669137598392e-68+0j), (-3.2743042373968055e-95+0j), (1.9060267865332834e-24+0j), (-3.846793833114066e-27+0j), (1.967401508308828e-34+0j), (-4.020207725923858e-27+0j), (1.0948642075135125e-35+0j), (-2.957615571265567e-43+0j), (1.2705897424306465e-34+0j), (-1.0373987693704661e-43+0j), (-6.560569469889006e-67+0j), (-1.2084244187792717e-26+0j), (8.385557910630787e-36+0j), (-2.758631189331443e-42+0j), (4.310654341879189e-36+0j), (-1.5506243319294304e-43+0j), (-2.6452818482032203e-68+0j), (-9.320933793053066e-43+0j), (-8.141164691088869e-69+0j), (5.601967386604963e-96+0j), (1.1193566286108796e-35+0j), (-1.40176373920555e-43+0j), (-1.0930821319757607e-67+0j), (-2.458656157623191e-43+0j), (-4.3015187538885315e-69+0j), (3.0827067567023143e-96+0j), (7.381934399898831e-69+0j), (-5.981666709695276e-97+0j), (-4.4314843856290795e-268+0j), (-0.00011075716875448734+0j), (1.8848670368907326e-20+0j), (1.0130514049161432e-18+0j), (-1.6675475009354502e-20+0j), (-1.8740603958001395e-20+0j), (-2.809102129055814e-25+0j), (-6.598386659111e-19+0j), (-4.1778073540504785e-26+0j), (1.6904805946229826e-24+0j), (4.076667804778261e-21+0j), (-1.2811373327221293e-20+0j), (1.6054897143058032e-25+0j), (-3.4728350837426314e-20+0j), (2.702953005649275e-26+0j), (4.613479013492194e-26+0j), (2.8479771944651057e-25+0j), (4.3247687998747313e-26+0j), (2.624151627177726e-35+0j), (-2.0720543472208322e-19+0j), (-4.397426230064008e-26+0j), (-2.402750529435941e-25+0j), (1.2826247230169475e-25+0j), (-2.061600342518463e-26+0j), (2.502164582539449e-35+0j), (-7.677469225502603e-25+0j), (9.310696667377775e-36+0j), (-1.6799887029097367e-40+0j), (2.604629841586352e-20+0j), (2.8024879153296578e-21+0j), (-1.1236408516223257e-24+0j), (-1.63649412387491e-20+0j), (-1.7531644757631766e-26+0j), (5.437695325871271e-26+0j), (9.348695924465584e-25+0j), (-1.7942943584698283e-27+0j), (5.043320794832028e-35+0j), (-3.851352932046897e-21+0j), (1.3233930545987694e-25+0j), (-2.5973663403979783e-26+0j), (5.828104367772835e-26+0j), (-2.2455377465445797e-27+0j), (3.69369564858431e-36+0j), (-2.806385449792704e-26+0j), (2.9868877881439076e-36+0j), (-7.076557244840326e-44+0j), (5.284358323022096e-25+0j), (-2.992706657008973e-27+0j), (1.0004446421579683e-34+0j), (-3.305284435556654e-26+0j), (9.355584954342552e-37+0j), (7.693128569143246e-43+0j), (2.0539653593288017e-35+0j), (-4.288849112374143e-43+0j), (9.311624314768177e-68+0j), (3.824287582404162e-19+0j), (4.5779176051785757e-26+0j), (-2.2491384444588054e-25+0j), (1.7899569483826846e-25+0j), (-1.964019687484466e-26+0j), (1.1457918276956772e-35+0j), (-3.2461882311535193e-25+0j), (8.407097221681776e-36+0j), (7.67981623373216e-42+0j), (2.1517601139532495e-25+0j), (-2.6746053379622825e-26+0j), (1.634209836961021e-35+0j), (-1.8469182029384528e-26+0j), (2.248045800597427e-36+0j), (-3.9069952808456306e-43+0j), (1.3854218942551618e-35+0j), (-1.479858759481027e-42+0j), (-5.3050235991599736e-68+0j), (-4.013122285211773e-25+0j), (6.961895670093107e-36+0j), (-2.4495748130246046e-41+0j), (7.167661630340138e-36+0j), (-8.0505691539886e-43+0j), (-5.933787581911715e-68+0j), (-3.1350571788395475e-41+0j), (-3.6598268114161825e-68+0j), (2.0265832594773717e-95+0j), (2.3637525625837968e-20+0j), (1.852884572118782e-21+0j), (1.2155273698624691e-24+0j), (-5.492479267352104e-22+0j), (-1.4022791451208706e-26+0j), (1.557953561623057e-27+0j), (1.1048344076418568e-24+0j), (-2.925025488666467e-28+0j), (2.617910956798282e-35+0j), (-9.568825326013424e-21+0j), (6.035101469302829e-26+0j), (-3.395150791307902e-26+0j), (1.2858432755102492e-26+0j), (-7.543828640245637e-28+0j), (2.782230860343095e-36+0j), (-2.0339133727051078e-26+0j), (2.585990456220261e-36+0j), (-1.4419361197902368e-42+0j), (8.074906443587112e-25+0j), (-1.7359957080881555e-27+0j), (7.874748697257479e-35+0j), (-3.2006129737726897e-27+0j), (2.8082481439510988e-36+0j), (1.1070257868166055e-43+0j), (3.695576806573719e-35+0j), (-2.701878601526288e-43+0j), (-7.40128535934398e-68+0j), (5.795227371546512e-21+0j), (1.012029655308036e-26+0j), (-2.975056975355448e-26+0j), (-2.792567604482382e-28+0j), (-3.343103515956183e-27+0j), (5.828156922243287e-36+0j), (-2.1669312002891596e-26+0j), (1.4418949972855027e-36+0j), (-4.135406930530576e-42+0j), (5.557840621644117e-26+0j), (-2.9069343053534433e-27+0j), (3.018600915606868e-36+0j), (-3.750543580209665e-27+0j), (3.248446512609058e-37+0j), (-3.7292055381844194e-43+0j), (2.8449470468589005e-36+0j), (-4.483060321414161e-44+0j), (-3.846365780543048e-68+0j), (-1.7709455744851552e-26+0j), (2.630410278598675e-36+0j), (-2.486341381482327e-42+0j), (2.526882454728203e-36+0j), (-1.5048084407325604e-43+0j), (-1.4959747627489557e-68+0j), (-1.1551078403737508e-42+0j), (-5.286615129276238e-69+0j), (-5.292349448033216e-95+0j), (2.259723617441797e-25+0j), (-1.1730854857352502e-27+0j), (3.195780593172546e-35+0j), (-3.143610707305732e-27+0j), (1.3883091840865344e-36+0j), (-1.4245074701401969e-43+0j), (2.02138796653453e-35+0j), (-7.346142984559065e-44+0j), (-5.718439084187321e-68+0j), (-1.105509672576726e-26+0j), (1.770330830740077e-36+0j), (-7.303501910195431e-43+0j), (1.3517570819940858e-36+0j), (-5.309196925821282e-44+0j), (-2.7800288006636113e-69+0j), (-1.3279424055551571e-42+0j), (-3.299229506538968e-69+0j), (8.1596357972605e-96+0j), (1.483160752075995e-36+0j), (-5.728765391799787e-44+0j), (-3.3208365959512933e-69+0j), (-8.38158486251939e-44+0j), (-4.512656599983525e-70+0j), (1.0616177672943232e-96+0j), (-2.0055739194450125e-69+0j), (1.9773633384305614e-97+0j), (2.6575981224214598e-269+0j), (2.6583930526229203e-19+0j), (5.0596749599970476e-26+0j), (-6.326657438985272e-27+0j), (4.897090727430997e-26+0j), (-2.385364437324347e-26+0j), (8.798341686713145e-36+0j), (-2.4582127131620198e-25+0j), (4.7401643923300414e-36+0j), (-4.1143524211040954e-41+0j), (1.7649500576871786e-25+0j), (-1.3381308354798994e-26+0j), (2.555419484956488e-35+0j), (-7.658412575056502e-27+0j), (7.0940382067136624e-37+0j), (-9.889663911972396e-43+0j), (9.12406984999964e-36+0j), (-3.754713549292832e-43+0j), (-3.517853099533911e-68+0j), (-6.099235860897297e-26+0j), (5.4169423998801296e-36+0j), (-1.223552512240616e-41+0j), (5.895358352040771e-36+0j), (-7.286204632276422e-43+0j), (-1.1739392343817543e-68+0j), (-1.2743022330109984e-41+0j), (-1.3629072009448281e-68+0j), (4.665244013404854e-96+0j), (1.7882293430002506e-25+0j), (-7.200287085675407e-28+0j), (-4.7408061393825544e-36+0j), (-7.707762689688201e-27+0j), (1.6232721068425697e-36+0j), (-1.0927172157785388e-42+0j), (5.9274024440429725e-36+0j), (-1.3700155708731908e-43+0j), (-3.290864491279815e-68+0j), (-4.23756356762097e-27+0j), (1.7277675499423862e-36+0j), (-1.1680260605917452e-42+0j), (5.074784229037945e-37+0j), (-1.3357562866404058e-43+0j), (-2.8430012470616387e-69+0j), (-5.652444916234869e-43+0j), (-1.6082652906712833e-69+0j), (2.389165517605754e-96+0j), (4.126256045422828e-36+0j), (-1.0584250886129962e-43+0j), (-4.783350423483498e-69+0j), (-5.729053622746123e-43+0j), (-9.380222637219421e-70+0j), (4.1600910388521043e-97+0j), (-2.5268964281671063e-69+0j), (3.692217619387767e-98+0j), (8.03169658622789e-270+0j), (-1.4088069691115745e-26+0j), (6.842113967454916e-37+0j), (-2.0304803800422347e-41+0j), (9.400186864826242e-37+0j), (-4.62670111156982e-43+0j), (-5.969619311325131e-69+0j), (-4.3654912177532143e-42+0j), (-1.5459420597120572e-69+0j), (7.932680125096164e-96+0j), (9.468552653688768e-37+0j), (-6.065724473761116e-43+0j), (-5.70533748287481e-69+0j), (-1.889702595574097e-43+0j), (-2.9057342860348285e-70+0j), (2.1523141394480655e-97+0j), (-1.627241262886207e-69+0j), (1.647463139352667e-97+0j), (-5.438997428726304e-271+0j), (-3.623465214463516e-42+0j), (-1.9201539722920634e-69+0j), (7.777387534982152e-97+0j), (-1.4363851321003755e-69+0j), (3.203615100865536e-98+0j), (-1.0195773804928808e-271+0j), (8.821605154431707e-98+0j), (7.525756451294043e-273+0j), 0j, (5.278074052578359e-20+0j), (1.3540946627493063e-21+0j), (5.886997567513015e-24+0j), (2.735899876019139e-21+0j), (1.5884503187530294e-25+0j), (-1.0250568373391283e-27+0j), (2.5953397564026453e-24+0j), (3.6179274378774364e-28+0j), (3.0550445356637235e-34+0j), (7.877240973342483e-21+0j), (1.6862059621280172e-25+0j), (-2.5366703991306503e-26+0j), (1.0079749102552e-25+0j), (-7.744283849420942e-28+0j), (1.6408630468057653e-35+0j), (-2.1074599757016123e-26+0j), (1.4538675478717312e-35+0j), (-1.0010525904520412e-42+0j), (2.3207278089643484e-24+0j), (-3.377897731907756e-27+0j), (2.004545741164086e-34+0j), (-4.1033246988557445e-27+0j), (1.0518466792920471e-35+0j), (-2.8687206999349614e-43+0j), (1.0274712607870495e-34+0j), (-2.0606750776551587e-43+0j), (-3.413198570737356e-67+0j), (4.0663785322800564e-20+0j), (1.0775445534866411e-26+0j), (-7.294790580455783e-26+0j), (1.7778873765328031e-25+0j), (-1.779688378159898e-27+0j), (1.6021867809535897e-35+0j), (-2.750757976505668e-26+0j), (9.071239306369864e-37+0j), (-3.449164798204507e-42+0j), (1.1307847759800072e-25+0j), (-1.0686359187552235e-27+0j), (1.0583622025242965e-35+0j), (-1.6842180326468602e-27+0j), (4.854260177057681e-37+0j), (-9.290170912703436e-44+0j), (5.689248952019758e-36+0j), (-7.198076096043494e-44+0j), (-1.1772729566772395e-68+0j), (-2.6036354176819495e-26+0j), (8.122074410589512e-37+0j), (-8.396624188811314e-43+0j), (5.4233457445285086e-36+0j), (-7.07683093594664e-44+0j), (-1.4788016653359749e-68+0j), (-1.9615941075753327e-42+0j), (-2.478165743110351e-69+0j), (1.3802900133137487e-95+0j), (7.763590375950692e-25+0j), (-8.794813017082755e-28+0j), (1.1893576682696511e-34+0j), (-3.245768194531853e-27+0j), (3.851174115723303e-36+0j), (-3.363116314379561e-44+0j), (2.1243550976304823e-35+0j), (-5.807759487359504e-44+0j), (-6.081264845328194e-68+0j), (-4.2319429336712705e-27+0j), (5.976881466796628e-36+0j), (-9.052032281100111e-43+0j), (2.564022889134511e-36+0j), (-3.526971758590589e-44+0j), (-6.450089104127644e-69+0j), (-6.545081617198499e-43+0j), (-7.458778272163556e-69+0j), (-2.2958670231099867e-97+0j), (2.106677469234538e-35+0j), (-3.8365677169135244e-44+0j), (-4.489702787690276e-68+0j), (-2.5431185159585796e-43+0j), (-1.6116728442371695e-69+0j), (3.3498475501296134e-97+0j), (7.53683209775352e-69+0j), (2.4192332933258763e-97+0j), (-1.9925250085928447e-271+0j), (2.221476253072411e-19+0j), (3.6359782412994266e-26+0j), (-5.024671095658322e-25+0j), (5.489604153342499e-26+0j), (-8.994151307622164e-27+0j), (8.29548330994514e-36+0j), (-1.5677979402543433e-25+0j), (2.3421384319509863e-36+0j), (-2.1500209919289689e-41+0j), (1.9755680307721365e-25+0j), (-1.6712461305654552e-26+0j), (1.3506833995612247e-35+0j), (2.792567604482382e-28+0j), (1.5436656830027187e-36+0j), (-1.4199860930638988e-42+0j), (4.7598977502075394e-36+0j), (-6.978356875895064e-43+0j), (-1.9550585488678803e-68+0j), (-1.0064918517533845e-25+0j), (4.8028313925691e-36+0j), (-1.4452860989315133e-41+0j), (2.068002989374477e-36+0j), (-1.0927777699358106e-42+0j), (-9.707594545148875e-69+0j), (-1.4086903137241305e-41+0j), (-8.869915590515843e-69+0j), (-1.094166289399175e-95+0j), (1.209473651275803e-25+0j), (-9.613749244315318e-28+0j), (1.248893962876579e-35+0j), (-5.515814056918467e-27+0j), (5.3654047365152366e-37+0j), (-1.5255350682136773e-42+0j), (4.179337943943093e-36+0j), (-1.7276751086035953e-44+0j), (-1.1793207227506894e-68+0j), (-3.4413070914135114e-27+0j), (8.0819345554559e-37+0j), (-1.0045202567690325e-42+0j), (1.7445481283047813e-37+0j), (-9.9444038438045e-44+0j), (-2.155142412580452e-69+0j), (-5.1495456335886405e-43+0j), (-7.975483991036235e-70+0j), (-1.1845317808605822e-97+0j), (1.9533992222614353e-36+0j), (-9.967274156875816e-44+0j), (-6.3723919189472e-69+0j), (-4.1005211930139806e-43+0j), (-3.600267112912145e-70+0j), (7.740934036856142e-97+0j), (-8.43606146821696e-71+0j), (8.65758436109994e-98+0j), (-4.1734048387121755e-271+0j), (-1.6295820193893187e-26+0j), (7.849759503459586e-37+0j), (-1.6772362325072084e-41+0j), (2.2300295412806826e-36+0j), (-3.1395622114250904e-43+0j), (-6.280314268631737e-69+0j), (-2.413198021328296e-42+0j), (-6.570090911566056e-70+0j), (3.3677820540849356e-97+0j), (1.6372234078292982e-36+0j), (-5.208951998783387e-43+0j), (-4.3103371984478473e-69+0j), (-1.875872588402902e-43+0j), (-3.589171988150238e-70+0j), (1.3007633901327432e-97+0j), (-1.3839093935992112e-69+0j), (6.027161945891421e-98+0j), (1.1400159748226075e-272+0j), (-2.4667119773699972e-42+0j), (-1.3046823712438153e-69+0j), (9.430148552504567e-97+0j), (-9.9870698215826e-70+0j), (-3.1127276264098276e-98+0j), (5.427052394354514e-273+0j), (7.672064034759367e-98+0j), (-2.3254809617060014e-274+0j), 0j, (4.190139222151346e-25+0j), (-3.8235101999930916e-28+0j), (5.404303060197052e-35+0j), (-1.2353808294793926e-27+0j), (1.542632547224125e-36+0j), (-2.1515610517841946e-43+0j), (1.2672415701768154e-35+0j), (-1.0723422813690347e-43+0j), (-7.192710639198663e-68+0j), (-2.0017838508048938e-27+0j), (7.682467345425114e-37+0j), (-1.3718038001390662e-42+0j), (6.4308419528865814e-37+0j), (-4.605789400478058e-44+0j), (-1.0465297844795879e-69+0j), (-9.143643109018524e-43+0j), (-8.82462610043669e-70+0j), (1.2400955179003746e-96+0j), (4.399231416840542e-36+0j), (-5.051200293885503e-44+0j), (-9.064179333222213e-69+0j), (-5.48301269500094e-44+0j), (-4.679903456579738e-70+0j), (2.531348704343466e-98+0j), (6.361797461742911e-69+0j), (8.720425700525725e-98+0j), (-3.510670577044078e-272+0j), (-1.590996860362696e-27+0j), (2.026777659783992e-37+0j), (-2.243300428994104e-42+0j), (4.195012366148898e-37+0j), (-7.298629780860698e-44+0j), (-1.7636251085050692e-69+0j), (-2.417577688419665e-43+0j), (-3.365276643187757e-70+0j), (4.742520644733228e-97+0j), (3.245269958795216e-37+0j), (-3.1884441913867897e-44+0j), (-1.3847392963987278e-69+0j), (-5.224754265761807e-44+0j), (-1.3817130244571329e-70+0j), (-1.1543466798026012e-98+0j), (-2.6859285642785636e-70+0j), (-2.707785861173977e-101+0j), (9.654971630401272e-273+0j), (-1.79562074322591e-43+0j), (-8.562890613961047e-71+0j), (-2.8629868922617327e-98+0j), (-1.9763091028696256e-70+0j), (2.172635184657147e-99+0j), (8.575393010328046e-274+0j), (-6.153603378539098e-99+0j), (-1.580656708474218e-274+0j), 0j, (6.30067199563721e-37+0j), (-2.605536525701011e-44+0j), (-2.636913063437843e-69+0j), (-1.703553707781043e-44+0j), (-1.6529709521668896e-70+0j), (-2.88527903019613e-98+0j), (-1.2531525508845998e-69+0j), (2.6616223456408883e-99+0j), (-1.1141613662591068e-274+0j), (-7.516258277588042e-44+0j), (3.950805301236166e-72+0j), (-2.8431662452181433e-98+0j), (-1.443243326040132e-70+0j), (2.167197994761736e-99+0j), (-1.4984098648084e-274+0j), (-1.9993284277143897e-98+0j), (5.369604622165126e-276+0j), 0j, (-2.8138046575205892e-70+0j), (7.060246268950361e-100+0j), (-3.1440573293722244e-276+0j), (1.040968750907755e-99+0j), (2.6976371180650453e-276+0j), 0j, (-3.8925516318945613e-278+0j), 0j, 0j, (-8.901099949614787e-06+0j), (1.2706445971211075e-20+0j), (6.315477654728063e-18+0j), (1.4426177419382986e-20+0j), (-9.359714067160019e-20+0j), (4.156097901938796e-25+0j), (-1.0604005466676586e-17+0j), (8.053701862454771e-25+0j), (1.8927152905020276e-23+0j), (4.09223637457584e-20+0j), (-3.7269449679189215e-20+0j), (2.6342148218119368e-24+0j), (-2.998496633280223e-19+0j), (1.742814620686677e-25+0j), (3.346563746532845e-25+0j), (2.4263089525191287e-24+0j), (1.8606927988668576e-25+0j), (3.1015885444293316e-34+0j), (-2.1192764340302594e-18+0j), (1.544400325805488e-24+0j), (-1.9341591577294497e-23+0j), (1.7259014428787385e-24+0j), (-4.728069694316194e-25+0j), (2.1908881880422202e-34+0j), (-1.1897476753015654e-23+0j), (2.0974773525990506e-34+0j), (-9.57367110826715e-40+0j), (4.8280885830303276e-20+0j), (1.0587911840678754e-21+0j), (4.750230070427812e-24+0j), (-8.732380290599803e-20+0j), (1.4918937439539775e-25+0j), (6.922230297271566e-26+0j), (2.0845112995049687e-24+0j), (4.879456151876589e-28+0j), (2.263641414248621e-34+0j), (2.0090562717687936e-19+0j), (1.7224935637681837e-25+0j), (-3.9990792973435305e-25+0j), (3.4589972972130926e-25+0j), (-1.8981223460188314e-26+0j), (2.7300812756701744e-35+0j), (-4.719939591556791e-25+0j), (2.32316548791444e-35+0j), (-7.166240346557115e-42+0j), (2.146761891257769e-24+0j), (-7.512626982726675e-26+0j), (2.738900702924693e-34+0j), (-4.366352333458873e-25+0j), (1.5191538555506302e-35+0j), (-1.1860590202045252e-41+0j), (1.0190981140508004e-34+0j), (-3.6622935365129094e-42+0j), (-3.686648838722467e-67+0j), (8.88521679802e-18+0j), (5.333330808018217e-25+0j), (-1.4665916904431668e-23+0j), (1.2749001970330234e-24+0j), (-4.624549009457118e-25+0j), (1.6966847485658492e-34+0j), (-5.4926999894197884e-24+0j), (1.0713785094308982e-34+0j), (-1.2756720569980972e-40+0j), (1.5872260066281264e-24+0j), (-6.114043344990194e-25+0j), (3.1253407805220923e-34+0j), (-3.3509423351911853e-25+0j), (1.654694106474584e-35+0j), (-6.180076552288524e-42+0j), (1.0773884218545347e-34+0j), (-1.4738200389113274e-41+0j), (-4.3113850409663004e-67+0j), (-4.505654314907952e-24+0j), (8.66000680664361e-35+0j), (-4.460248934038033e-40+0j), (1.1948112761470916e-34+0j), (-2.0446958961041548e-41+0j), (-3.1913965846374004e-67+0j), (-3.774963625819636e-40+0j), (-2.6532028682382783e-67+0j), (-1.3825672915695916e-93+0j), (7.522711511944396e-20+0j), (5.135137242729196e-21+0j), (3.6273970123724324e-24+0j), (7.279189390466644e-22+0j), (3.668392535894958e-25+0j), (-1.1040402325629824e-26+0j), (3.927896219276462e-24+0j), (2.2118399023868506e-28+0j), (4.745222257728523e-34+0j), (2.882558998624791e-20+0j), (1.741804878727994e-25+0j), (-2.718411223976174e-25+0j), (3.1390037596192413e-25+0j), (-3.237691516797735e-26+0j), (3.059545179660437e-35+0j), (-2.531026460122931e-25+0j), (2.757996369738018e-35+0j), (-1.4307257320756382e-41+0j), (2.7847042389791387e-24+0j), (-3.789786898248042e-26+0j), (2.9990407781039856e-34+0j), (-3.5875990540670793e-26+0j), (1.1327030550212003e-35+0j), (-2.3890387193657725e-42+0j), (1.5019260361849655e-34+0j), (2.5888989128400995e-43+0j), (-5.0440812935886995e-67+0j), (2.7264038425870303e-19+0j), (3.946828881001766e-26+0j), (-5.343784265800587e-25+0j), (2.5995806726291135e-25+0j), (-2.0324587075765373e-26+0j), (1.785882086918967e-35+0j), (-1.9923500228397387e-25+0j), (5.349931239701849e-36+0j), (-1.5483121894632944e-41+0j), (2.298125366307956e-25+0j), (-1.3994732425247365e-26+0j), (3.134503031017805e-35+0j), (-2.8092191679137645e-26+0j), (3.8022013976504636e-36+0j), (-2.6872525488048976e-42+0j), (1.0979082949008691e-35+0j), (-1.4922077021978896e-42+0j), (-6.506129680573343e-68+0j), (-1.263344962493346e-25+0j), (3.6234048206413366e-36+0j), (-2.1033183415476433e-41+0j), (7.674133838839215e-36+0j), (-1.2718754139713172e-42+0j), (-2.780996028278103e-68+0j), (-1.315574578151959e-41+0j), (-8.079490578136396e-69+0j), (7.383374169201289e-95+0j), (1.5600923469321444e-24+0j), (-1.2178714600259434e-26+0j), (2.3075011319976244e-34+0j), (-1.9404006116173838e-26+0j), (7.675061436590303e-36+0j), (-2.930312146499738e-42+0j), (5.544525660501672e-35+0j), (-1.3086731832151593e-42+0j), (-2.3286227829220384e-67+0j), (-1.9178628555552195e-25+0j), (9.960366707300572e-36+0j), (-2.3535158032951384e-41+0j), (5.403455121026307e-36+0j), (-1.2741826299975395e-42+0j), (-2.846340511598053e-68+0j), (-1.6231595910712564e-41+0j), (-1.7575901494516182e-68+0j), (-3.1775785443210176e-95+0j), (2.3563618651565513e-35+0j), (-2.2094864059799028e-42+0j), (-7.562878360030044e-69+0j), (-2.8496121000629813e-42+0j), (-3.719481702569212e-69+0j), (2.504908324751686e-96+0j), (-5.182178867558369e-69+0j), (-4.1185155352371447e-97+0j), (9.383150460360959e-270+0j), (2.9346581423258307e-18+0j), (3.7474048441618945e-25+0j), (-5.8145726483270526e-24+0j), (5.1599865127049515e-25+0j), (-9.659689016107576e-26+0j), (7.584639673213463e-35+0j), (-3.1297484562700896e-24+0j), (3.894902567234528e-35+0j), (-2.9196403828823645e-40+0j), (1.3000964143457825e-24+0j), (-1.8705154240238544e-25+0j), (1.2688988477440875e-34+0j), (-7.266513342352828e-26+0j), (9.299568310624305e-36+0j), (-1.4443161376507384e-41+0j), (6.584538094493258e-35+0j), (-1.778878335537139e-41+0j), (-1.454654692123489e-67+0j), (-3.9133811786203954e-26+0j), (6.174682810726116e-35+0j), (-3.076336423550671e-40+0j), (3.4627489893975167e-35+0j), (-1.1481473281579873e-41+0j), (-9.88199192435847e-68+0j), (-2.4957501701205067e-40+0j), (-4.995807420465759e-68+0j), (6.748714832478017e-96+0j), (8.660856546767175e-25+0j), (-8.620671972255217e-27+0j), (1.418551593354799e-34+0j), (-8.129842717026703e-26+0j), (5.1003282444084505e-36+0j), (-1.5057718334267837e-41+0j), (1.6964928457534294e-35+0j), (-1.6352961494896196e-42+0j), (-1.496609973453035e-67+0j), (-3.609334464225587e-26+0j), (7.349287705836555e-36+0j), (-1.6010168858060737e-41+0j), (4.902585132172329e-36+0j), (-1.2955100094570144e-42+0j), (-1.5946848108525642e-68+0j), (-8.17634971783095e-42+0j), (-7.383912670631358e-69+0j), (3.96583992708168e-96+0j), (1.955840029915209e-35+0j), (-1.5150503040446922e-42+0j), (-2.8370336306864253e-68+0j), (-6.324530033972993e-42+0j), (-5.637515601911159e-69+0j), (4.0264520220839656e-97+0j), (-1.729766522140354e-68+0j), (2.2569112790876904e-97+0j), (-2.031466036616041e-271+0j), (-8.112606106247624e-26+0j), (1.1160050315676182e-35+0j), (-3.899993933916805e-40+0j), (1.5304503394380868e-35+0j), (-1.0779959527635397e-41+0j), (-5.834710530809881e-68+0j), (-8.76779282875255e-41+0j), (-2.093972625657862e-68+0j), (-7.005956155183291e-95+0j), (1.2957800285373355e-35+0j), (-1.0219683384876207e-41+0j), (-4.3313814944137894e-68+0j), (-3.660002186018006e-42+0j), (-4.434803513877629e-69+0j), (-1.9757572502584045e-96+0j), (-8.152046973554295e-69+0j), (-6.677720528463707e-97+0j), (-1.368322787972925e-271+0j), (-5.510434529658061e-41+0j), (-1.6600522179147908e-68+0j), (5.4099987896466265e-96+0j), (-1.1981842250511698e-68+0j), (5.751319240738143e-98+0j), (4.493950078752478e-273+0j), (3.2152678128856337e-96+0j), (1.4981942266416975e-273+0j), 0j, (4.9604366973579964e-20+0j), (1.3416869535610109e-21+0j), (4.240714278076423e-24+0j), (7.792868550862074e-21+0j), (1.367758591908399e-25+0j), (-6.680333637982871e-28+0j), (3.0272063921313195e-24+0j), (-2.1760362156240186e-27+0j), (3.1080279633241523e-34+0j), (1.1330720030751373e-20+0j), (1.334752651633952e-25+0j), (-3.8063369899507395e-26+0j), (1.4035570997873286e-25+0j), (-1.9503160100840033e-27+0j), (1.4188904902417738e-35+0j), (-2.144039401328194e-26+0j), (1.3819746865804575e-35+0j), (-1.3281682007178657e-42+0j), (2.5189149119048337e-24+0j), (-1.4970101247586761e-27+0j), (2.2366373316657805e-34+0j), (-2.0282131566445773e-26+0j), (9.41019679459869e-36+0j), (-1.5299551795806393e-42+0j), (1.1652790071405208e-34+0j), (-2.5723679700187677e-43+0j), (-3.1617093132301566e-67+0j), (2.64910381434395e-19+0j), (3.6530176368522004e-26+0j), (-4.015897305797011e-25+0j), (1.453807539449907e-25+0j), (-1.429340001440334e-26+0j), (2.289595979660997e-35+0j), (-1.711733501020969e-25+0j), (3.759606655468929e-36+0j), (-1.3632882434800064e-41+0j), (1.2607653873332733e-25+0j), (-1.5355244451379296e-26+0j), (1.2832118977384227e-35+0j), (-9.98737349055062e-27+0j), (1.2926102262616019e-36+0j), (-1.2795716078808511e-42+0j), (5.866252073335748e-36+0j), (-1.1941307183119212e-42+0j), (-2.43948690035677e-68+0j), (-1.6455322938548218e-25+0j), (5.000462884945323e-36+0j), (-1.8625599386746862e-41+0j), (7.065362884126589e-36+0j), (-8.060695724922197e-43+0j), (-1.8140843397297065e-68+0j), (-1.9564052663135127e-41+0j), (-1.17182027141341e-68+0j), (1.2341278831677538e-95+0j), (9.460359185599619e-25+0j), (-2.0610963301161986e-27+0j), (1.0673965999213967e-34+0j), (-7.224092347174568e-27+0j), (3.626462136583972e-36+0j), (-1.2500485482428203e-42+0j), (2.505321624924175e-35+0j), (-1.3542543842883655e-43+0j), (-9.658408288808637e-68+0j), (-1.1494590857588516e-26+0j), (4.228358740106443e-36+0j), (-1.7318543717970015e-42+0j), (1.4631752646198583e-36+0j), (-1.3850925296922325e-43+0j), (-5.817241726621736e-69+0j), (-8.430288194887867e-43+0j), (-3.335213890894923e-69+0j), (1.7971404749336212e-96+0j), (8.433991250795409e-36+0j), (-2.2191097273917663e-43+0j), (-8.768029342417485e-69+0j), (-9.217359670292281e-43+0j), (-1.8026521597809047e-69+0j), (4.0203112326795996e-96+0j), (-7.847061179917469e-69+0j), (1.4574257584877784e-97+0j), (-2.2911786767549745e-271+0j), (1.1030122596149603e-19+0j), (1.7602839454327963e-26+0j), (-1.8695441812444102e-25+0j), (3.0392246880223617e-26+0j), (-7.394882832941537e-27+0j), (4.020897571128395e-36+0j), (-7.93862283360113e-26+0j), (1.9904073147345838e-36+0j), (-1.4015217962675688e-41+0j), (1.1450355482328247e-25+0j), (-6.360191048344408e-27+0j), (1.109017490088944e-35+0j), (-4.2733581311953736e-27+0j), (6.343831907661974e-37+0j), (-9.21797119885795e-43+0j), (6.881796691069182e-36+0j), (-8.434475668814463e-43+0j), (-1.263363980475294e-68+0j), (-2.6504937554520776e-26+0j), (2.7862856955677908e-36+0j), (-1.6020812705185267e-41+0j), (1.6506242996413446e-36+0j), (-4.584361952707838e-43+0j), (-3.5829107251425123e-69+0j), (-9.69550128310375e-42+0j), (-4.27466202607351e-69+0j), (-6.735023030886742e-97+0j), (1.1483960956890663e-25+0j), (-8.202674300101233e-28+0j), (1.3065010662337527e-35+0j), (-2.6425361210706606e-27+0j), (6.084738492117435e-37+0j), (-9.002193130640433e-43+0j), (2.931617273401603e-36+0j), (-1.1113997128093772e-43+0j), (-1.806158740327983e-68+0j), (-5.154417154714091e-27+0j), (7.533192916586e-37+0j), (-1.716508511466007e-42+0j), (7.514911254521772e-37+0j), (-1.104061370021348e-43+0j), (-1.1817373981758202e-69+0j), (-7.162032260269075e-43+0j), (-6.274478939638875e-70+0j), (1.5561859628414315e-96+0j), (1.6649575002104912e-36+0j), (-7.5773264409232e-44+0j), (-3.863349488564252e-69+0j), (-3.641023012104348e-43+0j), (-3.1949799057470676e-70+0j), (3.0251099702560426e-97+0j), (-1.6768839585573467e-69+0j), (1.2793829695568922e-97+0j), (-2.455870725742297e-271+0j), (-1.4067214180933964e-26+0j), (4.5433019917969535e-37+0j), (-1.152579926681675e-41+0j), (8.884294109565996e-37+0j), (-3.844838320032434e-43+0j), (-1.5694288326881503e-69+0j), (-3.0383003829434763e-42+0j), (-9.057151403299754e-70+0j), (-2.3618437263201076e-96+0j), (7.729500175039685e-37+0j), (-3.99289280691975e-43+0j), (-3.1047815853438685e-69+0j), (-2.024651020339594e-43+0j), (-9.535853845896391e-71+0j), (-1.0502239572794349e-97+0j), (-1.9713788869648933e-70+0j), (-1.9057830159135014e-98+0j), (1.1378748343218857e-272+0j), (-1.4464421204511576e-42+0j), (-4.193775965665889e-70+0j), (-1.8000094386226006e-99+0j), (-2.0895714171294626e-70+0j), (1.3438952299822721e-98+0j), (3.995276447260897e-273+0j), (5.6307875480721045e-98+0j), (-2.36153650922152e-274+0j), 0j, (4.299471399310452e-25+0j), (-6.526837914572346e-28+0j), (5.372989329587499e-35+0j), (-1.947648271184102e-27+0j), (1.743098350787002e-36+0j), (-6.30463200631624e-43+0j), (8.644365058716355e-36+0j), (-1.1065921574700401e-43+0j), (-5.038343044106251e-68+0j), (-2.6131264004478898e-27+0j), (1.7060117828027282e-36+0j), (-1.3405960006847347e-42+0j), (8.427143219841247e-37+0j), (-8.799596710330738e-44+0j), (-7.572383052773305e-70+0j), (-6.960962309340607e-43+0j), (-2.4555974509727725e-69+0j), (8.951209914350727e-98+0j), (3.0679047391534474e-36+0j), (-1.1249320274471624e-43+0j), (-7.276357106913752e-69+0j), (-2.4753814104389464e-43+0j), (-4.5776036708209676e-70+0j), (5.997625926868903e-98+0j), (-1.2285446149105464e-69+0j), (-1.7340988336647058e-98+0j), (3.0677852043187825e-273+0j), (-9.979768378386224e-27+0j), (2.960117029489505e-37+0j), (-1.1236119126072478e-41+0j), (9.350712483961331e-37+0j), (-3.2313026149766485e-43+0j), (-3.4131230925323155e-69+0j), (-2.372453952408709e-42+0j), (-6.175758294868414e-70+0j), (-4.160787751720758e-98+0j), (4.161472001475053e-37+0j), (-3.865984165312799e-43+0j), (-1.0618693308263554e-69+0j), (-1.3684376507712937e-43+0j), (-2.4230428239312893e-71+0j), (8.630069874254366e-99+0j), (-4.128823680002554e-70+0j), (1.1510306637945789e-98+0j), (4.764924202868176e-273+0j), (-1.0804037099125851e-42+0j), (-3.248548000797075e-70+0j), (1.2974702660409222e-97+0j), (-2.4913287418863938e-70+0j), (4.72381396929875e-99+0j), (-1.6520683400397958e-274+0j), (5.731291760144435e-99+0j), (-6.543227874900368e-275+0j), 0j, (-1.315767106529323e-37+0j), (-4.134564010032895e-44+0j), (2.4155501286175123e-71+0j), (-1.1565495410176245e-43+0j), (-2.3912210395696767e-70+0j), (-2.2134085939631436e-99+0j), (-9.167835963511497e-70+0j), (3.055410106945901e-99+0j), (-9.554198470105873e-275+0j), (-1.2377224977556998e-43+0j), (-2.3905356732331444e-70+0j), (6.705601262464562e-99+0j), (-1.8466201051245145e-72+0j), (1.12312472822385e-99+0j), (-9.623847635094872e-275+0j), (1.9255792931892528e-99+0j), (4.490040131308806e-276+0j), 0j, (-3.6898962857844965e-70+0j), (-1.0003072721755015e-99+0j), (1.4975694272902556e-276+0j), (3.401841505287711e-99+0j), (1.5913192119801335e-276+0j), 0j, (-3.128610986853152e-278+0j), 0j, 0j, (4.368200194188405e-19+0j), (6.261977865644392e-26+0j), (-1.9029857641846745e-24+0j), (1.3323347929594496e-25+0j), (-4.995895555809929e-26+0j), (1.6907095776241948e-35+0j), (-3.0489079556339496e-25+0j), (8.49264714672212e-36+0j), (-2.913814265564049e-40+0j), (2.14002778014035e-25+0j), (-6.416436830886666e-26+0j), (2.5891131096393393e-35+0j), (-1.231150562875145e-26+0j), (1.9854855058277277e-36+0j), (-1.6453638768353595e-41+0j), (7.443393493094636e-36+0j), (-1.170564135066143e-41+0j), (-2.9216525800872795e-68+0j), (-2.166977240814963e-25+0j), (1.0782952114208196e-35+0j), (-1.8181620958378474e-40+0j), (6.117749148078182e-36+0j), (-8.095559182272169e-42+0j), (-1.1782758355007705e-68+0j), (-8.125815220904295e-41+0j), (-7.424932236842877e-69+0j), (-2.3737155797017352e-95+0j), (3.9834339991562843e-25+0j), (-2.6593980229197597e-27+0j), (4.8036751423041e-35+0j), (-6.894619659728355e-27+0j), (2.0010829795176177e-36+0j), (-1.1624428988685041e-41+0j), (1.2582301885320762e-35+0j), (-6.527792892126562e-43+0j), (-1.654602161783924e-68+0j), (-1.2193287426533095e-26+0j), (1.8770021096944516e-36+0j), (-7.202659224675654e-42+0j), (5.637687677557129e-37+0j), (-6.65883533854473e-43+0j), (-2.9428212355518213e-69+0j), (-2.8708487872703134e-42+0j), (-1.9202424757224256e-69+0j), (-2.5775781218198003e-97+0j), (3.673081873143031e-36+0j), (-5.441760282270081e-43+0j), (-1.5912834245164423e-68+0j), (-3.187904774012999e-42+0j), (-6.553331881158275e-70+0j), (6.848604775904515e-98+0j), (-5.649541037810212e-70+0j), (-8.991410728886732e-99+0j), (4.442531098669548e-273+0j), (-1.5791315295315568e-25+0j), (1.2706874322645906e-36+0j), (-6.95249433216981e-41+0j), (2.5056346795924787e-36+0j), (-2.0570765207047848e-42+0j), (-9.692571561644419e-69+0j), (-9.632267775429563e-42+0j), (-1.3257726413880717e-69+0j), (5.095175320549152e-96+0j), (1.1582541340135737e-36+0j), (-2.079587469869771e-42+0j), (-4.922183191419894e-69+0j), (-5.534373330135661e-43+0j), (-3.967342527678945e-70+0j), (-1.9393295680679646e-97+0j), (-1.1859402218622827e-69+0j), (1.6481218903803672e-97+0j), (-1.211221235839034e-274+0j), (-7.35970584488834e-42+0j), (-3.443793394061075e-69+0j), (-6.076399501015256e-97+0j), (-2.9723851626392617e-69+0j), (-1.3308428233430045e-98+0j), (-6.962374620607539e-275+0j), (-5.219542706876581e-97+0j), (3.152281640696646e-276+0j), 0j, (3.2943225832482063e-25+0j), (-1.6153652667630388e-27+0j), (3.3928905397152714e-35+0j), (-1.012564601609389e-27+0j), (9.794202520102323e-37+0j), (-7.377046131600682e-43+0j), (8.872793283208595e-36+0j), (-3.5368860050956167e-43+0j), (-2.3005422577099464e-68+0j), (-1.1874513665113941e-26+0j), (1.359953403398061e-36+0j), (-6.905253810341802e-42+0j), (5.133949954786237e-37+0j), (-2.9037978501690346e-43+0j), (-1.8267605354363223e-69+0j), (-3.0605598089126546e-42+0j), (-1.4235449274372128e-69+0j), (-5.953314693458314e-97+0j), (1.8037585163051248e-36+0j), (-3.4415729704113725e-43+0j), (-6.119958825846895e-69+0j), (-2.184474131251624e-43+0j), (-3.580040122961577e-70+0j), (-4.672957810898193e-98+0j), (-1.239619882026812e-69+0j), (2.771341582707332e-98+0j), (1.3609179704564438e-273+0j), (-6.071227600976458e-27+0j), (1.502170008618285e-37+0j), (-3.628497987648384e-42+0j), (3.5555778890035543e-37+0j), (-1.1509736159384743e-43+0j), (-1.3882162426664797e-69+0j), (-5.829030551666537e-43+0j), (-2.709452105635672e-70+0j), (4.5972881527436004e-98+0j), (2.643830735656763e-37+0j), (-1.465533994161582e-43+0j), (-8.24436152759661e-70+0j), (-1.2169098627107856e-43+0j), (-6.5156230012720225e-71+0j), (1.3410021036451823e-99+0j), (-2.263278620553148e-70+0j), (-1.1961310629855099e-99+0j), (-2.305706676805569e-274+0j), (-3.1245979564119615e-43+0j), (-4.037139351047734e-71+0j), (9.37342086107704e-98+0j), (-2.150358989461577e-70+0j), (1.142419854627936e-99+0j), (-4.3301529641592524e-275+0j), (-7.76084351157909e-99+0j), (5.6040456719019915e-276+0j), 0j, (-2.939585231497932e-37+0j), (-5.86964879083272e-44+0j), (-1.7769294029949306e-69+0j), (-6.158950864331411e-44+0j), (-5.228018632251349e-71+0j), (9.685467868815105e-99+0j), (-7.193571905355718e-70+0j), (4.5155750940119416e-99+0j), (2.565472173349532e-276+0j), (-1.8202830489828618e-43+0j), (-1.8478360776570718e-70+0j), (2.9086308458294774e-99+0j), (-5.3598963805550635e-71+0j), (2.0623064058272284e-99+0j), (4.329645529445028e-276+0j), (-1.5503851174195377e-98+0j), (-1.6936801193324349e-277+0j), 0j, (-5.139531758342367e-70+0j), (2.270144053664762e-99+0j), (1.981147916479132e-277+0j), (-1.058435531320795e-99+0j), (-1.0897582814198451e-277+0j), 0j, (1.6219300063096991e-279+0j), 0j, 0j, (-7.639555803670529e-26+0j), (2.5297357776670783e-37+0j), (-3.2122715210048376e-41+0j), (1.4581999698232137e-37+0j), (-9.19602353752838e-43+0j), (-1.6211755094770934e-69+0j), (-5.750002576141355e-42+0j), (-5.2520285626214807e-70+0j), (1.7760030136279938e-96+0j), (2.808547153712827e-37+0j), (-1.0161543752460926e-42+0j), (-1.0674735451631004e-69+0j), (-2.2805903634703317e-43+0j), (-1.886780361586157e-70+0j), (5.770507436911608e-98+0j), (-6.492806934842947e-70+0j), (8.019562944566698e-98+0j), (-3.10418149093223e-275+0j), (-9.557342213762269e-43+0j), (-1.2280217149253656e-70+0j), (-3.94413123807016e-97+0j), (-4.685838588575551e-70+0j), (-1.3245832121450517e-98+0j), (1.1227417257205612e-276+0j), (-4.243903011150242e-98+0j), (4.775741526214194e-277+0j), 0j, (-2.051286271874312e-37+0j), (-4.2661171716728237e-44+0j), (-2.5846093111475893e-69+0j), (-1.7696693442564035e-43+0j), (-4.877210556608569e-71+0j), (1.0703497139728369e-97+0j), (-5.6722189255424054e-70+0j), (2.4854607916213494e-99+0j), (1.845465303289119e-276+0j), (-3.159390900716304e-44+0j), (-5.721924566384623e-71+0j), (-3.196190707748122e-99+0j), (-8.639705929734611e-71+0j), (2.0668481318122016e-99+0j), (1.259684692770248e-276+0j), (-5.0437190754529835e-99+0j), (-1.0204316886707888e-277+0j), 0j, (-3.198025723856288e-70+0j), (-4.16142877703489e-100+0j), (-1.9821531925398664e-278+0j), (-6.987786714135838e-99+0j), (-2.3424005883131765e-278+0j), 0j, (7.522339544343603e-280+0j), 0j, 0j, (3.1030320907512195e-43+0j), (-1.3628468148335624e-70+0j), (-2.0624663300387526e-97+0j), (-2.4071488979592084e-70+0j), (-1.9745351366341113e-99+0j), (-3.8595608545400394e-278+0j), (2.937043137100999e-98+0j), (1.4457522012649799e-279+0j), 0j, (-2.8273544605367902e-70+0j), (-4.813471226214856e-99+0j), (-7.067208224120211e-279+0j), (2.9497277475839823e-100+0j), (2.1102951239890924e-279+0j), 0j, (-2.839551395432627e-281+0j), 0j, 0j, (2.1890739117731095e-98+0j), (1.4604138956786085e-280+0j), 0j, (-1.4744881881887886e-281+0j), 0j, 0j, 0j, 0j, 0j]
eigenstate_data = np.array(eigenstate_data)

basis_8_3 = list(itertools.product(range(3), repeat=8))

eigenstate_qobj = qt.Qobj(eigenstate_data, dims=[[3]*8, [1]*8])
eigenstate_density_matrix = eigenstate_qobj * eigenstate_qobj.dag()

In [134]:
two_qubit_basis = list(itertools.product(range(nmax+1), repeat=2))

ramp_states = [T2_to_states[5][ramp_time_index], T2_to_states[5][0], eigenstate_density_matrix]
ramp_state_qobjs = []

ramp_state_labels = ['ramp 5mus', 'ramp 5mus initial', 'eigenstate']

for ramp_state in ramp_states:

    rho_reduced, local_basis = partial_trace_from_restricted_basis(
        rho_restricted=ramp_state,
        basis_restricted=reduced_basis,
        keep=(0,1),
        total_sites=8
    )

    rho = convert_density_matrix_to_basis(
        rho=rho_reduced,
        local_basis=local_basis,
        basis=two_qubit_basis
    )

    ramp_state_qobjs.append(rho)


In [135]:
# Representative initial states (|1,0>, |1,1>, |2,1>)
init_states = [
    fock_state(1, 0, nmax),
    fock_state(1, 1, nmax),
    fock_state(2, 1, nmax),
    current_state,
    current_2_state,
]
init_states.extend(ramp_state_qobjs)

state_labels = ["|1,0>", "|1,1>", "|2,1>", "|1,0> + i|0,1>", "|2,1> + i|1,2>"]
state_labels.extend(ramp_state_labels)

# Analyze mapping at t_star
print("===== Strongly interacting case (U/J = 20, nmax=3) =====")
results = analyze_time(H=H, j0=j0, D=D, times=[t_star], init_states=init_states, state_labels=state_labels)
summarize_results(results)

# -----------------------------------------------------------------------------
# Output interpretation (printed automatically)
# - off-diagonal norm: measures how non-diagonal j(t*) is in Fock basis; smaller is better.
# - operator Frobenius norm: ||j(0) - j(t*)||, global measure of mapping accuracy.
# - bias per state: difference between expected <j(0)> and measured <D(t*)>.
#   Small biases indicate accurate mapping for that initial state.

print("\nInterpretation:")
print("- The smaller the off-diagonal and difference norms, the closer the density measurement at t* reproduces the current operator.")
print("- If biases for all representative states are small, the mapping remains robust even with higher Fock states.")


===== Strongly interacting case (U/J = 20, nmax=3) =====


TypeError: incompatible dimensions [[4, 4, 4, 4], [4, 4, 4, 4]] and [[4, 4], [4, 4]]

# 2. Correlators

In [11]:
import qutip as qt

def build_correlation_operators(n_sites: int, nmax: int = 3, J: float = 1.0, U: float = 0.0, pairs=None):
    """
    Build multi-site operators and Hamiltonians for correlation analysis.

    Arguments:
        n_sites: number of qubits (sites)
        nmax: maximum number of bosons per site
        J: tunneling amplitude
        U: on-site interaction strength
        pairs: list of tuples (i, j) specifying which site pairs are tunnel-coupled.
               If None, assumes nearest-neighbor pairs [(0,1), (1,2), ...].

    Returns:
        dict with keys:
            'a_list', 'n_list', 'H_swap', 'H_int', 'H',
            'j_ops', 'D_ops', 'dims'
    """
    d = nmax + 1
    a_local = qt.destroy(d)
    I_local = qt.qeye(d)

    # --- build local annihilation and number operators
    a_list = []
    n_list = []
    for site in range(n_sites):
        ops = [I_local] * n_sites
        ops[site] = a_local
        a_site = qt.tensor(ops)
        a_list.append(a_site)
        n_list.append(a_site.dag() * a_site)

    # --- define coupling pairs
    if pairs is None:
        pairs = [(i, i+1) for i in range(n_sites-1)]  # nearest-neighbor default

    # --- swap (tunneling) Hamiltonian: H_swap = J * Σ(a_i^† a_j + h.c.)
    H_swap = 0
    for (i, j) in pairs:
        H_swap += J * (a_list[i].dag() * a_list[j] + a_list[j].dag() * a_list[i])

    # --- on-site interaction: H_int = (U/2) Σ n_i (n_i - 1)
    H_int = 0
    for n_i in n_list:
        H_int += (U / 2.0) * n_i * (n_i - qt.qeye(n_i.dims[0]))

    # --- total Hamiltonian
    H_total = H_swap + H_int

    # --- current operators and density-difference operators for each coupled pair
    j_ops = {}
    D_ops = {}
    for (i, j) in pairs:
        j_ops[(i, j)] = 1j * J * (a_list[i].dag() * a_list[j] - a_list[j].dag() * a_list[i])
        D_ops[(i, j)] = J * (n_list[j] - n_list[i])

    return {
        'a_list': a_list,
        'n_list': n_list,
        'H_swap': H_swap,
        'H_int': H_int,
        'H': H_total,
        'j_ops': j_ops,
        'D_ops': D_ops,
        'dims': [d] * n_sites,
        'pairs': pairs,
    }


In [12]:
def analyze_two_point_correlators(H, a_list, n_list, pairs, J, t, init_states, state_labels):
    """
    Compute connected correlators:
        C_jj = <j_ij(0) j_kl(0)>_c
        C_DD = <D_ij(t) D_kl(t)>_c

    Arguments:
        H: Hamiltonian
        a_list, n_list: annihilation & number operators (lists of qutip.Qobj)
        pairs: list of tuples [(i,j), (k,l)] specifying operator pairs to analyze
        J: tunneling amplitude
        t: evolution time
        init_states: list of qutip.Qobj states (ket or density matrix)
        state_labels: list of state names

    Returns: dict mapping state_label -> dict of connected correlators
    """
    results = {}
    for state, label in zip(init_states, state_labels):
        results[label] = {}



        # for (i, j), (k, l) in itertools.combinations_with_replacement(pairs, 2):
        for (i, j), (k, l) in [pairs]:
            # define j_ij(0), j_kl(0)
            j_ij = 1j * J * (a_list[i].dag() * a_list[j] - a_list[j].dag() * a_list[i])
            j_kl = 1j * J * (a_list[k].dag() * a_list[l] - a_list[l].dag() * a_list[k])
            # define D_ij(t), D_kl(t)
            D_ij = J * (n_list[j] - n_list[i])
            D_kl = J * (n_list[l] - n_list[k])
            U = (-1j * H * t).expm()
            D_ij_t = U.dag() * D_ij * U
            D_kl_t = U.dag() * D_kl * U

            # compute connected correlators
            C_jj = qt.expect(j_ij * j_kl, state) - qt.expect(j_ij, state) * qt.expect(j_kl, state)
            C_DD = qt.expect(D_ij_t * D_kl_t, state) - qt.expect(D_ij_t, state) * qt.expect(D_kl_t, state)

            results[label][((i, j), (k, l))] = {"C_jj": C_jj, "C_DD": C_DD}
    return results


def summarize_two_point_results(results):
    print("\n===== Connected two-point correlator results =====")
    for label, data in results.items():
        print(f"\nState: {label}")
        for key, vals in data.items():
            (i, j), (k, l) = key
            print(f"  Pairs ({i},{j}) & ({k},{l}):")
            print(f"    <j_ij j_kl>_c = {vals['C_jj']:.4e}")
            print(f"    <D_ij D_kl>_c = {vals['C_DD']:.4e}")
            print(f"    Difference ΔC = {vals['C_DD'] - vals['C_jj']:.4e}")


In [20]:
nmax = 3

# define some useful initial states

# this state is defined on only the basis states with 4 total particle on 8 qubits with up to 3 particles per site
# let's convert this to 8 qubits with up to nmax particles per site, then trace out the 6 rightmost qubits
ramp_state_5mus = T2_to_states[5][ramp_time_index]

reduced_basis = []
basis = []
reduced_state_to_index = {}

for i, state in enumerate(itertools.product(range(nmax+1), repeat=8)):
    basis.append(state)
    if sum(state) == 4 and not any(n > 2 for n in state):
        reduced_basis.append(state)
        reduced_state_to_index[state] = i

print(f'len(basis) = {len(basis)}')
print(f'len(reduced_basis) = {len(reduced_basis)}')


len(basis) = 65536
len(reduced_basis) = 266


In [138]:
### import eigenstate from exact diagonalization of triangle lattice Hamiltonian

eigenstate_data = [0j, (-4.7707597104803106e-26+0j), (5.990912875824446e-17+0j), (4.50251895018005e-21+0j), (-2.7541178502900948e-15+0j), (-1.129547904746919e-16+0j), (6.097224294605576e-17+0j), (7.560019496445803e-17+0j), (-8.516242594213299e-06+0j), (2.5921134049814503e-20+0j), (1.6190513310890595e-16+0j), (-8.77280601592725e-18+0j), (1.7338547543538917e-15+0j), (1.8444169160939788e-15+0j), (-0.00011414095426704862+0j), (-2.1203235785113026e-16+0j), (0.0003595705075394941+0j), (-1.7299059760892983e-17+0j), (-6.467091258330663e-17+0j), (2.0765309163470712e-16+0j), (-4.1993815931164236e-05+0j), (1.5382118322138094e-18+0j), (0.0017700848882049119+0j), (-1.1166435343653441e-17+0j), (-0.00012075215555091485+0j), (2.3035908033527952e-17+0j), (1.389811659854856e-17+0j), (7.453889935837843e-20+0j), (2.1720398103806243e-15+0j), (-9.676334982843676e-17+0j), (-1.1541230482154524e-15+0j), (-1.0891861143471487e-15+0j), (0.00037976628403070653+0j), (2.6291902682773483e-17+0j), (0.000451703090542162+0j), (5.958521338608044e-17+0j), (2.1575352181918417e-15+0j), (-1.4238013979622766e-15+0j), (0.0004831295243123984+0j), (5.034120093439648e-16+0j), (-0.015800458579631818+0j), (-5.974066326977633e-17+0j), (9.546343673103816e-05+0j), (-2.032819031785159e-17+0j), (-6.944357806819536e-18+0j), (-6.560778396252909e-17+0j), (-0.0018870490158483823+0j), (2.777243262555117e-17+0j), (0.003115152581536211+0j), (-3.1036562256561977e-16+0j), (-1.6920489417216303e-16+0j), (1.3157564065537485e-16+0j), (6.075302754562469e-17+0j), (9.194034422677078e-17+0j), (5.8695995112934e-17+0j), (5.1892626480587456e-17+0j), (3.2078220269032696e-05+0j), (2.120563924110086e-16+0j), (-0.0018012998829168576+0j), (1.1999569743760738e-16+0j), (-1.0851816391042627e-05+0j), (2.5232245069030977e-16+0j), (-1.1181098400777665e-16+0j), (1.8973538018496328e-17+0j), (0.0004848532867848321+0j), (-2.768702586460544e-16+0j), (0.0023271105766127374+0j), (1.3818191795777397e-16+0j), (1.567985896428369e-16+0j), (2.3467377804609117e-16+0j), (-1.076919316999306e-17+0j), (-2.5800587687389212e-17+0j), (-0.0002395676307727389+0j), (1.012299820535689e-16+0j), (-1.21495538760233e-17+0j), (4.581266535096497e-17+0j), (4.095078087838301e-18+0j), (-5.415623963167765e-18+0j), (-9.541483778205892e-18+0j), (-5.366923498300802e-18+0j), (-9.540979117872439e-18+0j), (5.421010862427522e-20+0j), (-7.635493799729165e-16+0j), (8.470329472543003e-17+0j), (4.769405356763734e-16+0j), (2.0486000049113606e-16+0j), (0.0013768091280972985+0j), (-8.120674271916428e-17+0j), (-0.0003496833951489645+0j), (-1.3552099890547926e-16+0j), (-1.4554872064531654e-15+0j), (6.074242671350039e-17+0j), (0.0012191621225350828+0j), (-2.1580773192780844e-15+0j), (-0.05077826222986999+0j), (6.085742268390284e-18+0j), (0.0036328384526220538+0j), (-1.972814276856924e-18+0j), (1.5689874983424405e-18+0j), (1.8111597291370352e-16+0j), (-0.0006911216763212008+0j), (4.930115936257911e-17+0j), (0.0036564280393827876+0j), (1.2267642358949383e-17+0j), (1.0916282726230137e-17+0j), (1.9006779533876246e-16+0j), (6.111809660785611e-18+0j), (-3.298831369816817e-19+0j), (-9.690056916589196e-18+0j), (1.717782817031721e-16+0j), (-0.0033533488109832865+0j), (6.632064689093831e-16+0j), (0.054613627836702226+0j), (-1.568981975143456e-18+0j), (0.0008748480313964163+0j), (4.518912622818934e-18+0j), (4.819085492742687e-18+0j), (4.721700461174372e-16+0j), (0.0761515523598111+0j), (3.94655857505858e-17+0j), (-0.09761422902904494+0j), (1.810743021569383e-16+0j), (1.4099112972468937e-17+0j), (-1.5622083527472978e-17+0j), (4.0336680307805738e-19+0j), (-1.5199998359406696e-18+0j), (-0.0002486038701790287+0j), (3.111043804869405e-18+0j), (7.417589360589077e-20+0j), (5.020939400521336e-18+0j), (3.310883615874474e-19+0j), (1.2317881083706107e-19+0j), (3.143739686379961e-18+0j), (5.447960235913283e-20+0j), (5.292090724993298e-20+0j), (3.359671481989457e-17+0j), (-0.0076022617169812035+0j), (1.418907101864592e-16+0j), (0.0035566499030110963+0j), (5.780579462083735e-18+0j), (3.479702280168585e-18+0j), (1.0470164335757563e-16+0j), (5.298327633666453e-18+0j), (3.3268117903787714e-19+0j), (0.006163860789212469+0j), (1.3218952214750103e-18+0j), (1.1662133155902168e-18+0j), (5.732518563444422e-18+0j), (5.34036760228253e-19+0j), (1.7066397617533527e-19+0j), (8.673617379884035e-19+0j), (2.3580297291397395e-20+0j), (6.89463679915147e-20+0j), (4.699182161732462e-17+0j), (3.848560003302231e-18+0j), (-1.1035176501499268e-20+0j), (3.645629804982509e-18+0j), (-4.353343722759796e-21+0j), (-1.9773608149980035e-21+0j), (-2.8917546623720815e-21+0j), (-8.942804648188538e-22+0j), (1.6155871338926322e-26+0j), (-3.8977068100853884e-17+0j), (8.330738442835495e-17+0j), (-0.00016128177881297702+0j), (-2.211772431870429e-17+0j), (0.005759307895171371+0j), (1.867822959921987e-17+0j), (-0.00022351528917908024+0j), (-1.7375902591749894e-17+0j), (-1.942429497436105e-17+0j), (-4.1226787608761306e-17+0j), (0.0011773692583648022+0j), (-3.6403177033998347e-17+0j), (-0.0046752070933520126+0j), (-5.2642575655816884e-21+0j), (5.914711018188333e-18+0j), (1.6056667623426525e-16+0j), (5.065383311717458e-18+0j), (6.1544063376733175e-21+0j), (-1.7020036731041165e-05+0j), (2.909528032927062e-17+0j), (3.816356731324327e-17+0j), (1.8467573950216374e-17+0j), (3.1435461613419944e-18+0j), (1.4949456914301223e-20+0j), (-3.6429322831111644e-17+0j), (-3.3686380317109287e-20+0j), (5.7856861178305255e-21+0j), (-2.927345865710862e-17+0j), (-0.009910731766192064+0j), (4.098781241041518e-17+0j), (0.0074541952321461715+0j), (-5.26929443010718e-18+0j), (6.0435166904993706e-18+0j), (1.0185390262274807e-17+0j), (3.970320385572163e-18+0j), (2.5187213132902345e-20+0j), (0.00451340620541854+0j), (7.498092492964578e-18+0j), (7.371912623788085e-18+0j), (-1.1750633367695908e-17+0j), (5.957739948487473e-19+0j), (-1.139447032922514e-20+0j), 0j, (-5.088059880785569e-22+0j), (-1.549763847898071e-21+0j), (4.771202665354741e-17+0j), (4.269046054161674e-18+0j), (-6.678391575112097e-21+0j), (1.5178830414797062e-18+0j), (-1.7569231890763263e-21+0j), (-3.2978462671396476e-23+0j), (-8.696692249345341e-21+0j), (1.7003013287825062e-24+0j), (-4.091520612713623e-29+0j), (-0.00048126663407117593+0j), (1.2310579905319685e-17+0j), (-3.816419430690415e-17+0j), (8.803874170612075e-17+0j), (3.0357660829594124e-18+0j), (4.552204453428297e-21+0j), (-1.9949319973733282e-17+0j), (-2.6309071098432404e-21+0j), (-1.2371759345340428e-21+0j), (4.8742464618021474e-17+0j), (-2.168404344971009e-19+0j), (-8.66603521952118e-21+0j), (-8.673617379884035e-19+0j), (-1.1199812583121385e-21+0j), (1.10718167904256e-22+0j), (-1.8011691500487557e-21+0j), (-2.543729320539466e-24+0j), (-5.480933061786687e-29+0j), (-4.0766001685454967e-17+0j), (-5.791461355270332e-21+0j), (9.617699477131126e-22+0j), (-2.783068089403824e-21+0j), (-5.2376088480568886e-24+0j), (9.364259537845382e-31+0j), (4.665850250009834e-22+0j), (5.2945130724878035e-31+0j), (2.311115933264683e-33+0j), (5.421010862427522e-20+0j), (-4.673927802949229e-16+0j), (-1.2841019480375193e-17+0j), (-1.2251755599629321e-15+0j), (1.37506206338608e-15+0j), (-0.0014293813534402996+0j), (-9.546400128734867e-17+0j), (-0.0021744516669558052+0j), (-3.214413664095908e-17+0j), (-6.179410282081133e-16+0j), (7.155327762589647e-16+0j), (-0.001520409835160273+0j), (-1.040183564282593e-15+0j), (0.05854781936234524+0j), (3.608063445721387e-18+0j), (-0.0007718788464247998+0j), (4.267835411926908e-18+0j), (5.403691092074746e-18+0j), (-4.3286771736483765e-17+0j), (0.002683450988615863+0j), (3.908976973860626e-17+0j), (-0.006926343226710627+0j), (1.633739395967887e-17+0j), (1.101313924433376e-17+0j), (-3.130507078248741e-17+0j), (4.422001197966677e-18+0j), (-5.94960390139217e-19+0j), (-8.94046663958703e-16+0j), (-5.440390976260701e-16+0j), (0.0007807757959740328+0j), (6.593439986699035e-16+0j), (0.033069046504850416+0j), (-2.0353612216662626e-18+0j), (-0.0005463218150649548+0j), (6.116749361049717e-19+0j), (2.0398282567818975e-18+0j), (-4.868287983026201e-16+0j), (-0.06005122997216667+0j), (1.0394185283138886e-17+0j), (-0.01122424693040089+0j), (-9.8332031956714e-17+0j), (3.882316118921653e-18+0j), (9.025503800329472e-18+0j), (8.935035965080263e-19+0j), (1.5531889565243898e-18+0j), (0.005659695728227896+0j), (2.1801275924742267e-18+0j), (1.069345372232435e-18+0j), (1.8510996889994037e-17+0j), (5.750088046984103e-19+0j), (2.596137837403686e-19+0j), (4.9060148304969076e-18+0j), (4.4048937996555784e-20+0j), (6.372337713992835e-20+0j), (-9.269928574751063e-17+0j), (-0.003543533017215219+0j), (5.041338630810685e-17+0j), (0.0017279323649542462+0j), (3.743370356447583e-18+0j), (2.710064837414934e-18+0j), (7.693003130185753e-18+0j), (9.791700870259712e-19+0j), (-2.312156615945512e-20+0j), (0.003103849814581976+0j), (4.480000493732166e-19+0j), (4.248344033474839e-18+0j), (5.755762606596894e-18+0j), (4.514685608865421e-19+0j), (-3.610809433175756e-20+0j), (2.137911158869854e-18+0j), (-6.644314014629561e-22+0j), (-1.0112509927965977e-21+0j), (-6.3211516927527755e-18+0j), (2.2463313761184045e-18+0j), (-3.637173613273161e-21+0j), (3.6253010142484055e-19+0j), (-3.4219926444293882e-22+0j), (-8.842425947640382e-22+0j), (-2.5290418912930726e-21+0j), (2.067608061344436e-23+0j), (-2.0278300972331116e-29+0j), (1.9607796289400348e-16+0j), (2.5221253037444047e-16+0j), (0.005749608991475586+0j), (-2.505862271157122e-17+0j), (-0.14589414690001493+0j), (2.59053388922304e-18+0j), (0.0004430856743303535+0j), (1.7087137197790883e-18+0j), (3.2273671834674486e-18+0j), (-3.9889153178457315e-16+0j), (-0.08989306564400971+0j), (1.3014342743539057e-17+0j), (0.19635143639091385+0j), (-5.266940795235675e-17+0j), (3.9539997528050423e-20+0j), (9.190616734868007e-18+0j), (3.1432833293447747e-19+0j), (1.0888907489188923e-20+0j), (0.0005942194104370989+0j), (-1.1942914000282431e-18+0j), (1.300804281046905e-18+0j), (3.1220996624901393e-18+0j), (2.7418434283532217e-19+0j), (-4.966265674265095e-21+0j), (1.0842021724855044e-18+0j), (-4.903747820332606e-22+0j), (-1.2228734571820534e-21+0j), (-5.698024517497569e-16+0j), (0.1765535749904091+0j), (1.6003305155461815e-17+0j), (-0.0656951158644541+0j), (4.544945452576602e-17+0j), (6.498506694146931e-19+0j), (-2.6831247484605e-18+0j), (2.0921713797181218e-19+0j), (-2.045108120542637e-20+0j), (-0.13698002571951642+0j), (2.040370956254598e-17+0j), (6.5703449057311e-19+0j), (-3.9165788434112955e-17+0j), (3.477070248478903e-19+0j), (-1.0426102933224823e-20+0j), (-9.67311625764411e-19+0j), (-1.3742631050593537e-21+0j), (-1.7624416314698695e-21+0j), (1.2744536290768509e-17+0j), (1.7279472123987727e-19+0j), (6.685616646386351e-22+0j), (5.21772295508649e-19+0j), (5.058420397167249e-22+0j), (7.32782881838858e-23+0j), (7.106216729266869e-22+0j), (4.623048100907998e-23+0j), (-4.2696697981931184e-29+0j), (0.004573751141417035+0j), (-6.061010338899102e-19+0j), (-8.673617379884035e-19+0j), (3.878177459294517e-19+0j), (4.404571325722362e-20+0j), (-4.515562435121646e-23+0j), (-1.2197274440461925e-18+0j), (-4.2740682551232725e-22+0j), (4.1447353861182644e-22+0j), (5.568093458460377e-18+0j), (2.913793338554793e-19+0j), (-2.3327708916662195e-22+0j), (8.639736061993863e-20+0j), (-4.39899281399047e-23+0j), (4.6315707174697345e-23+0j), (-2.585802893822532e-22+0j), (-7.349917633709583e-25+0j), (1.0595274970088822e-30+0j), (-9.75781955236954e-18+0j), (2.4615504772981327e-23+0j), (7.932527118032022e-23+0j), (1.2857004987771968e-22+0j), (-1.6967419390743424e-25+0j), (5.373839626326756e-31+0j), (-2.784762133974011e-24+0j), (-2.4495392092816385e-32+0j), (-1.1199632499436611e-35+0j), (-4.4655576979246714e-17+0j), (0.0008555207857561557+0j), (-8.299290611985063e-18+0j), (-0.008165580285269208+0j), (6.072306502571693e-18+0j), (3.1151058286359655e-19+0j), (1.6789511822089478e-17+0j), (1.8041801776516597e-18+0j), (-9.384077640643469e-22+0j), (0.0032077066823793365+0j), (-1.1976171382833676e-20+0j), (2.236166980751353e-18+0j), (5.1320157797969525e-18+0j), (2.363221922839498e-19+0j), (-1.0222714515398112e-22+0j), (7.352245982167327e-19+0j), (-2.9955144869873532e-22+0j), (8.976533595259838e-23+0j), (2.3871047700630916e-17+0j), (7.775762455794477e-19+0j), (-3.1639159463104508e-21+0j), (1.170599533105443e-18+0j), (-2.7881079580347913e-22+0j), (-8.676823683134528e-24+0j), (-2.3828868260994114e-22+0j), (1.1833205456850109e-24+0j), (4.466828574614783e-31+0j), (0.006459908851319755+0j), (8.306402776369358e-19+0j), (-2.2768245622195593e-18+0j), (2.608323938291896e-18+0j), (2.0837010502455788e-19+0j), (-6.750754242994307e-22+0j), (-2.7647155398380363e-18+0j), (4.065983924878549e-23+0j), (4.1114644832429716e-22+0j), (2.3270294286877614e-18+0j), (2.862971361719535e-19+0j), (-1.0153261572582019e-22+0j), (1.6517142471458857e-19+0j), (4.059433912094882e-23+0j), (2.3769528839078445e-24+0j), (-2.9376233035246996e-22+0j), (-1.1482084454012368e-24+0j), (1.0768250389099222e-30+0j), (-4.336808689942018e-19+0j), (3.0164976969652096e-22+0j), (5.497685044775166e-23+0j), (-5.39807403040855e-22+0j), (-4.57072664498942e-25+0j), (-3.264631694119216e-32+0j), (-1.0116936603432774e-23+0j), (-1.9488099133684347e-32+0j), (-1.382174179907699e-35+0j), (1.0610675962351968e-18+0j), (5.233604822847508e-19+0j), (1.0894261171877983e-21+0j), (8.639736061993863e-19+0j), (-9.806507230582002e-23+0j), (-5.220271657512394e-24+0j), (-1.566863378392406e-22+0j), (-7.303281717736872e-25+0j), (-1.4720989057784073e-32+0j), (1.4230153513872246e-19+0j), (2.1685366371492057e-23+0j), (2.539157772986097e-24+0j), (-1.7038238685906112e-23+0j), (1.6685215495257087e-25+0j), (-2.2532231487273723e-32+0j), (1.780461893815656e-24+0j), (2.806393580252361e-34+0j), (2.294794610163783e-37+0j), (2.7021926563540336e-22+0j), (1.6451628812321889e-25+0j), (1.5433322471335055e-33+0j), (1.1291854567911688e-26+0j), (-2.7813748360340392e-34+0j), (1.3331935722487784e-37+0j), (2.9242439143377512e-33+0j), (-3.188047753336653e-39+0j), (-1.5930919111324523e-58+0j), (7.318364664277155e-17+0j), (3.3152869555533315e-17+0j), (-9.048448720377939e-05+0j), (-1.3234042767901188e-17+0j), (0.00023307555696059363+0j), (1.4302660866888143e-17+0j), (2.452039536160076e-05+0j), (1.5990386354782157e-17+0j), (2.081649906843929e-17+0j), (2.879234394306818e-17+0j), (0.0032594452950725963+0j), (4.518187267276205e-17+0j), (-0.0015590203089898835+0j), (4.300007782126574e-18+0j), (-3.255623032965937e-19+0j), (2.917486624221039e-17+0j), (-2.4936649967166602e-18+0j), (-2.2643419685465463e-20+0j), (-0.00021912423707534832+0j), (1.7339772240566177e-17+0j), (-5.898066045851281e-17+0j), (-6.715741850685041e-18+0j), (-3.415236843329339e-18+0j), (-4.5897302517369444e-21+0j), (-9.71445146547012e-17+0j), (-9.828705474230676e-21+0j), (3.08707591137135e-21+0j), (6.033585089881832e-17+0j), (-0.0009802997805908608+0j), (6.085779468915284e-17+0j), (0.00023823487297677646+0j), (7.487719764849986e-19+0j), (3.0899761915836876e-18+0j), (6.417916147647053e-17+0j), (1.1079190950086248e-18+0j), (-9.75145715629818e-21+0j), (0.0006614991696787994+0j), (4.365531119227469e-18+0j), (-5.421010862427522e-19+0j), (3.538207982219344e-18+0j), (2.303929616531697e-19+0j), (1.0809182569241104e-21+0j), (-1.8973538018496328e-19+0j), (-6.320139698264497e-22+0j), (1.9687545760248702e-22+0j), (2.2445578092440568e-17+0j), (1.0062751413381088e-18+0j), (-9.500984477631077e-21+0j), (-5.421010862427522e-20+0j), (-3.1892297789215333e-22+0j), (1.439314350240915e-22+0j), (-2.7525269570310983e-21+0j), (-9.020109719461613e-25+0j), (4.326031740404647e-31+0j), (-2.103863599666516e-05+0j), (-3.58793374793479e-20+0j), (-1.33939625883428e-16+0j), (4.741229683768552e-17+0j), (-4.851804721872632e-18+0j), (-1.839449240963297e-21+0j), (8.673617379884035e-17+0j), (1.4438517184606798e-21+0j), (1.420231275223521e-21+0j), (1.5588403528087987e-17+0j), (3.0899761915836876e-18+0j), (-7.418204642706994e-21+0j), (2.0057740190981832e-18+0j), (-7.042329521540151e-22+0j), (4.710566262442435e-23+0j), (-4.1425345283610724e-22+0j), (-2.5767401985901416e-23+0j), (-3.521077541694606e-32+0j), (-5.551115123125783e-17+0j), (-2.664151836290316e-22+0j), (4.856058853197959e-22+0j), (-4.0285501914206727e-22+0j), (2.2000572649891715e-23+0j), (-1.6174990346410808e-32+0j), (-7.342102795786111e-23+0j), (-3.923652030756592e-32+0j), (1.908810693673632e-37+0j), (-4.0427188506553247e-17+0j), (-0.00681392054785557+0j), (7.498612021415495e-18+0j), (0.0019422209211010148+0j), (-9.049346832033824e-19+0j), (2.608861477543245e-18+0j), (5.5198147650014914e-17+0j), (9.84252284709497e-19+0j), (-5.55434300373844e-21+0j), (0.005404823403939068+0j), (7.063560674694571e-18+0j), (-3.3068166260807885e-18+0j), (6.94349068029108e-18+0j), (1.0028870095490916e-18+0j), (2.3552169967590795e-22+0j), (-5.095750210681871e-18+0j), (1.8031830505781162e-22+0j), (-7.673124596200852e-23+0j), (1.6715862767697184e-17+0j), (2.0853951161400874e-18+0j), (-2.871750152558806e-21+0j), (1.4907779871675686e-19+0j), (1.3205974258095295e-22+0j), (1.6245231234294655e-23+0j), (-8.803189047212564e-22+0j), (4.574051890920153e-24+0j), (2.770056493168231e-31+0j), (-0.0002279113790832428+0j), (5.191016256635871e-19+0j), (3.017978391067072e-18+0j), (-3.0929463641587664e-19+0j), (-7.962109704190423e-20+0j), (-4.9157658834788665e-22+0j), (-4.933119884809045e-18+0j), (-1.0874457707168337e-22+0j), (5.544632960166273e-23+0j), (-4.1194903228290766e-19+0j), (-6.352747104407253e-19+0j), (-9.897299929376775e-22+0j), (6.369687763352339e-19+0j), (7.146676732071732e-23+0j), (5.482443268475355e-25+0j), (-1.4543686844397247e-22+0j), (-2.351680837340571e-24+0j), (-3.104624872453225e-32+0j), (5.800481622797449e-18+0j), (1.4810686299569591e-22+0j), (-1.94611425966338e-23+0j), (-6.982292136523267e-23+0j), (1.3952174780014859e-24+0j), (-1.067730722539662e-32+0j), (8.40419438812934e-24+0j), (5.691824461918135e-34+0j), (2.274326992853066e-37+0j), (6.304007792754061e-18+0j), (4.1843427594362437e-19+0j), (-4.961197388725384e-22+0j), (-2.1006417091906648e-19+0j), (6.249351306023603e-23+0j), (-9.69987349109542e-24+0j), (4.1009924875446393e-22+0j), (1.668614761453529e-24+0j), (-2.0271335427741053e-32+0j), (7.589415207398531e-18+0j), (-1.1190253350697893e-22+0j), (4.168606351622917e-23+0j), (1.5439882879207446e-23+0j), (1.787409996164754e-24+0j), (1.6180740350496252e-33+0j), (9.506448710335985e-24+0j), (-5.077695703525685e-34+0j), (-6.697986499099298e-39+0j), (-5.283824254030196e-22+0j), (5.613650521943571e-25+0j), (-4.68815358972348e-33+0j), (1.830551026397222e-25+0j), (7.071859421558309e-34+0j), (-3.556005360575041e-39+0j), (-1.7055890384553675e-33+0j), (-1.2920539216043558e-40+0j), (1.1433085963291198e-60+0j), (-0.0002403182613346544+0j), (4.925137494274489e-19+0j), (-3.8163916471489756e-17+0j), (7.362089591505478e-18+0j), (-1.1275702593849246e-17+0j), (-5.804845857928296e-22+0j), (-3.122502256758253e-17+0j), (7.166343542643656e-23+0j), (5.289518774962735e-23+0j), (1.3279768257038288e-17+0j), (-8.429671891074797e-18+0j), (-7.299185280770823e-22+0j), (-3.2526065174565133e-19+0j), (-4.153882250780275e-23+0j), (-4.625597344366265e-24+0j), (-5.179591241814206e-22+0j), (-1.1981735528418017e-23+0j), (-1.5521559485411592e-32+0j), (-1.5092094240998222e-16+0j), (-1.241739620109529e-21+0j), (1.8933182883950202e-22+0j), (-7.4116726270153605e-22+0j), (1.0834171688687678e-23+0j), (-1.4352697861463819e-33+0j), (1.5155932713987015e-22+0j), (-7.75684330599848e-33+0j), (-3.6197669173452035e-38+0j), (6.2991828049897595e-18+0j), (2.439454888092385e-19+0j), (-1.4767573923714763e-21+0j), (-8.348356728138384e-18+0j), (-3.3469933640313465e-23+0j), (5.1208256146400097e-23+0j), (-1.4774769788506626e-22+0j), (-1.1395666744812548e-24+0j), (-9.17064562656297e-33+0j), (-1.111307226797642e-18+0j), (6.037074714823324e-23+0j), (1.5997814339806364e-23+0j), (-6.437834992303035e-23+0j), (2.26938212914988e-25+0j), (9.704032800404306e-34+0j), (8.101942782094848e-24+0j), (-2.425313923750122e-34+0j), (-9.377208660855334e-39+0j), (5.875114754006139e-24+0j), (9.16433833983971e-25+0j), (3.795703584899814e-33+0j), (3.1147255361163248e-24+0j), (-9.027603759773236e-35+0j), (-4.915856126154405e-39+0j), (9.171821074996044e-33+0j), (3.900778624662959e-40+0j), (1.4084062305472516e-60+0j), (5.963111948670274e-19+0j), (-5.898145909722424e-23+0j), (-1.85066443618394e-23+0j), (-1.2743751312145083e-23+0j), (1.0272127258831352e-23+0j), (5.790219951237684e-33+0j), (2.5611956287791523e-23+0j), (-1.951384448034827e-33+0j), (1.1649300279525977e-38+0j), (-5.2991257991678335e-24+0j), (1.9244891300031925e-24+0j), (7.062295856659408e-33+0j), (2.7944747584168752e-24+0j), (1.358534111855428e-33+0j), (-5.169492129754501e-40+0j), (-1.5572095089579396e-34+0j), (4.693686787985901e-40+0j), (-1.8829833409023993e-62+0j), (-3.0516580806835964e-23+0j), (-3.793404069993766e-33+0j), (9.662842398462928e-39+0j), (-2.4755220769769377e-33+0j), (2.1560579952326474e-40+0j), (-1.1066230799354697e-62+0j), (4.3600336730633796e-39+0j), (2.1064596678699095e-64+0j), (-3.770183781348654e-87+0j), (2.202285662861181e-20+0j), (-2.4328277023130673e-15+0j), (-7.868597266813548e-17+0j), (3.1933819737844926e-16+0j), (-6.750717064345213e-16+0j), (-0.0018629334018180677+0j), (-6.561794835789614e-17+0j), (-6.007811382591565e-05+0j), (4.659315174401848e-18+0j), (2.7206156164721884e-15+0j), (-1.995125120885302e-15+0j), (-0.0017455336911048511+0j), (2.3089440515794424e-15+0j), (0.07079445397362016+0j), (1.1998269182192548e-19+0j), (-0.00508497954370973+0j), (1.5323944023332021e-18+0j), (4.215053728941119e-18+0j), (-2.4157379655692646e-17+0j), (0.0019914895617923543+0j), (4.6042453780375e-17+0j), (-0.007103431925281722+0j), (-2.8873956230132197e-18+0j), (2.069900035997384e-18+0j), (3.006989030384988e-17+0j), (3.722328238620556e-18+0j), (-4.278693208541436e-21+0j), (-7.058291668152195e-16+0j), (1.0924150039420821e-15+0j), (0.004426967829194563+0j), (-6.526795434409066e-16+0j), (-0.0538199699002521+0j), (2.5444199684943557e-18+0j), (-0.001925081327300399+0j), (1.216810895451938e-18+0j), (1.8109572153252267e-18+0j), (2.730224358225841e-16+0j), (-0.11921759725101264+0j), (-4.795992328092781e-19+0j), (0.13958038725418812+0j), (-4.096460693570336e-17+0j), (-4.752297241467088e-20+0j), (3.982359036263185e-18+0j), (2.600075917462916e-19+0j), (1.5211332645330919e-21+0j), (0.0019759103803974956+0j), (1.6726492923417225e-18+0j), (1.9783495853238197e-18+0j), (2.5341738271318427e-18+0j), (9.19856669101132e-20+0j), (-4.180883213734762e-21+0j), (2.625802136488331e-18+0j), (-9.167272515368398e-22+0j), (-8.58063595602893e-22+0j), (-3.458604930228759e-17+0j), (0.006603136737591641+0j), (4.094456712633315e-18+0j), (-0.002494097301093418+0j), (2.2625938700722874e-18+0j), (4.63126321826129e-18+0j), (2.088796606595999e-17+0j), (1.2976544751935881e-18+0j), (-2.898285502399893e-21+0j), (-0.007783703036708023+0j), (5.576482977778459e-18+0j), (2.8051861022530186e-18+0j), (-1.655472038024654e-19+0j), (4.2859867131067597e-19+0j), (-1.70821857042874e-21+0j), (1.849919956803392e-18+0j), (-3.6676785174285627e-22+0j), (-4.306689807914805e-22+0j), (1.4874752395852965e-17+0j), (1.650020181251377e-18+0j), (2.6237429691745775e-21+0j), (2.6495190590114515e-18+0j), (-5.597077346190234e-23+0j), (9.211419927460834e-24+0j), (-3.6369425829725667e-22+0j), (8.03722334700561e-24+0j), (-2.4260677197881624e-29+0j), (2.9889039851525682e-16+0j), (9.455869247332327e-16+0j), (0.002672924009922194+0j), (-3.7604197099944114e-16+0j), (-0.08369188500853364+0j), (5.943526864626415e-18+0j), (0.003231993172189091+0j), (1.2843903647357292e-18+0j), (-1.2486272205862441e-19+0j), (9.802271841441446e-16+0j), (-0.0297488210523301+0j), (1.7433828699388048e-18+0j), (0.08317196669159756+0j), (6.480541010797514e-18+0j), (4.365374695417407e-19+0j), (1.367825528559158e-18+0j), (3.11413972563912e-19+0j), (-5.1885511374867965e-22+0j), (0.00015811527666930247+0j), (2.5924445512654194e-18+0j), (1.5178830414797062e-18+0j), (2.6911525055833534e-18+0j), (-2.371692252312041e-20+0j), (-9.323320953367114e-22+0j), (5.421010862427522e-19+0j), (-7.759206145884458e-23+0j), (6.569033768848256e-23+0j), (-6.042800808347959e-16+0j), (0.1500721888363539+0j), (9.516140729172574e-19+0j), (-0.12025511636442017+0j), (2.0101979497941914e-17+0j), (-1.019248150210149e-19+0j), (1.6561573851562559e-18+0j), (1.2281977735187355e-19+0j), (-1.1710001676637027e-21+0j), (-0.06353023806753615+0j), (-4.6279750427118455e-18+0j), (4.336808689942018e-19+0j), (2.151276416515289e-18+0j), (2.879912020664621e-19+0j), (4.819995581042746e-22+0j), (2.1429933565533799e-19+0j), (3.153315905096411e-23+0j), (5.677260436514827e-23+0j), (2.1950952527091047e-18+0j), (1.5500702934753696e-19+0j), (-1.722894371360026e-23+0j), (-6.378158092824882e-19+0j), (-2.5575717186839127e-23+0j), (2.785524950143408e-24+0j), (-1.2812824143412516e-22+0j), (-9.746070800404828e-25+0j), (8.015132593202188e-31+0j), (0.007313226359099404+0j), (-8.38536104418344e-21+0j), (-1.4094628242311558e-18+0j), (2.068325152617741e-18+0j), (-5.421010862427522e-20+0j), (-3.731858322882528e-23+0j), (-3.308510691975297e-18+0j), (-2.0513193225792441e-22+0j), (-1.158825278618195e-22+0j), (9.945967697878052e-19+0j), (-5.421010862427522e-20+0j), (-4.1165633296921076e-22+0j), (-2.2404021454876244e-19+0j), (-2.5857401644382396e-23+0j), (5.543704727260622e-24+0j), (1.3601994717041134e-22+0j), (8.092959130972642e-24+0j), (7.321771482149325e-31+0j), (4.607859233063394e-18+0j), (-2.0227725734586968e-23+0j), (1.4733318037132964e-23+0j), (-7.049960291247082e-23+0j), (-8.665758947514575e-25+0j), (-3.2005521175096345e-32+0j), (9.853819074288742e-24+0j), (-1.3913599293553846e-32+0j), (-1.4024408028567458e-35+0j), (-1.6738726290460582e-16+0j), (-0.0010901001894210057+0j), (-1.2340569361539712e-17+0j), (-0.0026840646327200104+0j), (1.976491030944563e-18+0j), (6.352747104407253e-19+0j), (-1.6027926124975608e-18+0j), (5.836057006582129e-19+0j), (-5.3024089630300805e-21+0j), (0.0017441313648045448+0j), (1.5516827212934088e-18+0j), (1.6296913905172739e-18+0j), (2.6182000314442156e-19+0j), (2.7316812548951186e-19+0j), (-3.184235318599533e-23+0j), (5.539595475043124e-19+0j), (-6.601308295703273e-23+0j), (-9.822703706966391e-24+0j), (-2.658792720077459e-18+0j), (4.688327363052552e-19+0j), (-1.744933646522693e-21+0j), (4.764560328305439e-19+0j), (-1.6483420080810066e-23+0j), (4.402089753868545e-25+0j), (-3.674408016953434e-22+0j), (5.11407856743634e-25+0j), (-1.2123217072127867e-32+0j), (0.0036336639190927035+0j), (-2.730464222336998e-19+0j), (9.147955830346444e-20+0j), (8.338678823773211e-19+0j), (2.50801161726078e-20+0j), (-1.8083127371052297e-22+0j), (3.9979955110402976e-19+0j), (-2.761946150637991e-23+0j), (2.0153953400408816e-23+0j), (4.036754314258586e-19+0j), (6.961552035246281e-20+0j), (-6.285208953588011e-23+0j), (1.98840984367947e-19+0j), (-7.50887816114809e-24+0j), (-1.078488425559738e-24+0j), (-4.917233010897266e-23+0j), (-2.2475198159785117e-25+0j), (-2.4818798323047436e-32+0j), (-1.6263032587282567e-19+0j), (4.1427711830732794e-24+0j), (1.671762901948064e-24+0j), (-2.79786729905994e-23+0j), (2.527469294909147e-25+0j), (9.398758754205681e-34+0j), (2.954953059892942e-25+0j), (4.489246445746414e-34+0j), (1.722148779140068e-37+0j), (1.0630344960938958e-17+0j), (1.6977716636528382e-19+0j), (-4.619429258616823e-22+0j), (1.7406527066075872e-19+0j), (-1.7605120504207754e-24+0j), (1.5720809619316276e-25+0j), (-3.3923703579489045e-22+0j), (2.127038322519271e-26+0j), (-1.365967341199303e-32+0j), (-3.9302328752599536e-19+0j), (-5.483939107066406e-23+0j), (2.0527341826537263e-24+0j), (3.130015766089399e-23+0j), (-1.498506944419116e-25+0j), (7.114279155588646e-34+0j), (8.477109184858977e-25+0j), (2.766332640940841e-34+0j), (1.4522343147348636e-37+0j), (-1.7061587260313478e-24+0j), (4.4325781108159696e-26+0j), (2.908956472552063e-33+0j), (3.346821587046868e-26+0j), (1.2368083198321703e-35+0j), (-3.202096921003907e-39+0j), (1.2539303363657326e-33+0j), (-1.2497223255370932e-39+0j), (5.9493881195928396e-61+0j), (-1.07073773675595e-15+0j), (-9.719059324703183e-16+0j), (-0.007782021217223236+0j), (3.43312617917535e-16+0j), (0.1619927573380449+0j), (1.441252768803222e-18+0j), (0.000716143405581117+0j), (9.072321340475158e-19+0j), (1.3008815851488102e-18+0j), (-1.3805959413887292e-15+0j), (0.15404912289628286+0j), (9.421091208891243e-19+0j), (-0.28309083296897886+0j), (3.959053108645922e-17+0j), (2.5397691312705534e-19+0j), (-1.3001047590875458e-18+0j), (1.8634724839594607e-19+0j), (-3.0551201675930905e-22+0j), (-0.00138708450541603+0j), (-6.983602293792528e-19+0j), (-1.463672932855431e-18+0j), (1.5059286007534352e-19+0j), (-1.6940658945086007e-20+0j), (-1.880693336450632e-22+0j), (-6.776263578034403e-19+0j), (3.162514105773534e-22+0j), (7.199668582460202e-23+0j), (3.4193026014761596e-17+0j), (-0.2150912451367855+0j), (3.33928881561685e-18+0j), (0.08288772541958181+0j), (-2.2223956676538398e-17+0j), (-2.574980159653073e-19+0j), (-8.028006561852e-19+0j), (1.411368648362478e-19+0j), (-3.605778830234889e-22+0j), (0.19644755753314094+0j), (-2.383063238406085e-17+0j), (2.270048298641525e-19+0j), (2.802685712865268e-17+0j), (-1.4584848560534984e-19+0j), (8.14716810516435e-23+0j), (3.1509625637859973e-19+0j), (2.6394021949020802e-23+0j), (2.752270577848384e-23+0j), (1.3339931094852604e-18+0j), (2.3656042030036507e-19+0j), (2.3173017473186705e-22+0j), (-4.0657581468206416e-19+0j), (-6.334222157432795e-23+0j), (-6.683302262843306e-24+0j), (-1.5386369848852148e-22+0j), (-3.7228374969020162e-25+0j), (4.968932287910327e-31+0j), (-0.0046220038286997845+0j), (1.5382233345557371e-18+0j), (1.6940658945086007e-19+0j), (-7.082048076025873e-19+0j), (9.147955830346444e-20+0j), (-1.8288481610197414e-23+0j), (-3.74049749507499e-18+0j), (-5.120675452877067e-23+0j), (-5.415985089869332e-24+0j), (-5.308521301200585e-19+0j), (-3.9471735342050396e-19+0j), (5.405984816095569e-23+0j), (-5.793705359219414e-19+0j), (4.679250246764764e-23+0j), (-2.274125308646545e-24+0j), (2.351398684085607e-23+0j), (3.790214162283725e-24+0j), (-1.6787487555378515e-32+0j), (2.927345865710862e-18+0j), (-1.372707513310762e-22+0j), (2.471430671554683e-23+0j), (-6.039915483328934e-23+0j), (-1.1764017486711327e-24+0j), (-1.0387170056687732e-32+0j), (5.630932197942605e-24+0j), (4.261833747432899e-34+0j), (1.186594374717233e-37+0j), (1.0635345685724995e-15+0j), (-0.07092731906694419+0j), (3.902986834521796e-18+0j), (0.14369337284750888+0j), (-1.9190414961637185e-17+0j), (1.6347735882007997e-19+0j), (7.617841697399901e-19+0j), (2.4034559878340772e-20+0j), (-3.033993141656643e-22+0j), (-0.01433306387532545+0j), (4.451965916759397e-18+0j), (4.0657581468206416e-20+0j), (8.96951528392631e-19+0j), (1.402898318889935e-21+0j), (2.334871378717932e-23+0j), (-2.0328790734103208e-19+0j), (-4.082003465722651e-23+0j), (-1.319324504480092e-25+0j), (1.6564495026967416e-18+0j), (1.2003217868756985e-19+0j), (-3.935089115861065e-22+0j), (1.7110065534536867e-19+0j), (3.0690147392201754e-24+0j), (-5.797343907739148e-25+0j), (-4.202330972830228e-25+0j), (-2.7722060568127377e-25+0j), (-1.744257512872842e-32+0j), (-0.10028976995894616+0j), (1.4999669912619312e-17+0j), (3.6845933205562065e-19+0j), (-1.2140112855510351e-17+0j), (-1.521482931505537e-19+0j), (-1.683063468768228e-23+0j), (-1.8973538018496328e-19+0j), (4.496300213916863e-24+0j), (-8.412079399544414e-24+0j), (-2.015212244940385e-18+0j), (-6.541211935171334e-19+0j), (-2.1804131791435517e-23+0j), (1.2959604092990795e-19+0j), (-2.5605239884113365e-23+0j), (1.3628834315141333e-25+0j), (-5.2418807091085714e-24+0j), (-9.914758536076235e-26+0j), (-3.0915841677139427e-32+0j), (-2.236166980751353e-19+0j), (-1.3980836886763146e-23+0j), (-1.3382426539380794e-24+0j), (1.3812264789522834e-23+0j), (-1.5048864987671214e-25+0j), (1.8868364001922444e-33+0j), (1.4002499737758845e-24+0j), (8.779398586774591e-34+0j), (2.5378472824720955e-37+0j), (6.979111890410099e-19+0j), (4.0234064994579266e-20+0j), (-8.644125459929911e-23+0j), (-2.0328790734103208e-20+0j), (1.7945642372657206e-24+0j), (8.0149066383972545e-25+0j), (1.957710466268547e-23+0j), (-6.054466848968038e-26+0j), (-1.3621004055954793e-34+0j), (-3.5575383784680614e-19+0j), (-7.56860274843014e-24+0j), (-9.118664446577967e-25+0j), (-6.191855977984077e-24+0j), (3.431855339840538e-26+0j), (8.929840971104468e-34+0j), (3.695985834468497e-25+0j), (-9.022393243309972e-35+0j), (-6.45519960541199e-39+0j), (4.401688320978624e-23+0j), (1.39254320452901e-25+0j), (-2.1907828265406613e-34+0j), (6.918611849556683e-26+0j), (-1.729928773973971e-35+0j), (-4.4411480957291056e-39+0j), (7.828221762360776e-34+0j), (1.7830554723591249e-40+0j), (1.2188127578653691e-60+0j), (0.0024973295180318635+0j), (-2.0040846308362157e-20+0j), (-3.0899761915836876e-18+0j), (4.483156247198654e-19+0j), (1.6263032587282567e-19+0j), (-2.5761731349899238e-23+0j), (-1.4094628242311558e-18+0j), (-2.188645920118239e-23+0j), (9.55412898599146e-25+0j), (4.807982576718945e-19+0j), (1.6940658945086007e-20+0j), (-8.304717203833414e-24+0j), (-3.2695471764015993e-19+0j), (1.0198636225138283e-24+0j), (1.4791516866713226e-24+0j), (-1.741811391261756e-23+0j), (1.4909838853444424e-24+0j), (-2.9399076979866947e-34+0j), (2.8731357570865868e-18+0j), (2.4147305332965017e-23+0j), (1.747466805337925e-24+0j), (1.256311439355905e-23+0j), (3.5077818400948487e-25+0j), (-2.5849469749467503e-34+0j), (3.088220554770972e-24+0j), (2.9278065346579562e-34+0j), (-3.0694608862279495e-39+0j), (-5.241201798856168e-22+0j), (3.441071348220595e-20+0j), (-3.855163563341608e-23+0j), (6.606856988583543e-20+0j), (-6.453201970632419e-24+0j), (7.198634904427872e-26+0j), (3.950932421843386e-23+0j), (1.6147101819147199e-25+0j), (8.551542323014634e-35+0j), (-9.994988777600744e-20+0j), (-2.979017021759196e-24+0j), (7.402238488025424e-25+0j), (-1.5157439896010981e-24+0j), (1.8751267580157934e-26+0j), (-1.3832275058697358e-34+0j), (1.756416632845113e-25+0j), (-5.933886299401292e-35+0j), (-4.4795105387233125e-39+0j), (1.4453042499803487e-23+0j), (9.102719029668895e-27+0j), (-8.72219505977837e-34+0j), (3.6039962516822903e-25+0j), (7.068463344910386e-36+0j), (2.3804365679507077e-40+0j), (1.5003342651618905e-34+0j), (5.798166758560767e-41+0j), (-2.258004403942516e-62+0j), (-2.710505431213761e-20+0j), (-3.7139520463263345e-24+0j), (8.180396842522924e-24+0j), (-1.5696236799333868e-23+0j), (2.9194863895929673e-25+0j), (-5.638113094457692e-34+0j), (-1.46992251954845e-24+0j), (2.5650658527666173e-35+0j), (7.715780909798792e-40+0j), (8.384897224902761e-25+0j), (1.2944205421689067e-25+0j), (3.2878978772795073e-35+0j), (1.0391496596070074e-25+0j), (8.773289603474384e-36+0j), (1.5961241515002999e-40+0j), (2.5680233575232003e-34+0j), (1.1521413869320452e-41+0j), (-1.2439867997011314e-62+0j), (6.18526931759628e-25+0j), (3.0985863774880936e-34+0j), (-1.9939357060116e-40+0j), (2.7121479260695363e-34+0j), (-1.476494090412994e-41+0j), (3.903082624448163e-64+0j), (6.788450280575144e-41+0j), (1.1160286707223006e-64+0j), (1.4598600732052665e-89+0j), (-7.533849846058649e-17+0j), (0.009729035625242006+0j), (1.3558651148967665e-17+0j), (-0.007055767402917388+0j), (1.861478241383535e-18+0j), (7.254837193233082e-19+0j), (1.9452195479219176e-17+0j), (6.496742705440484e-19+0j), (1.1250531575015526e-21+0j), (-0.006505761122047564+0j), (1.6564361791169986e-18+0j), (3.9302328752599536e-19+0j), (-1.3055081691227902e-18+0j), (4.5263323118901674e-20+0j), (-1.1807252817243516e-22+0j), (2.778268066994105e-19+0j), (-1.7643061892213368e-22+0j), (1.0687693633845757e-24+0j), (4.581066737978144e-18+0j), (4.516273795641523e-19+0j), (-6.104378574424184e-22+0j), (4.290221877843031e-19+0j), (-5.398052530526024e-23+0j), (8.605276488873214e-25+0j), (1.1555063539796607e-22+0j), (8.153785693206051e-25+0j), (-1.6308720461232105e-33+0j), (0.00368085834011289+0j), (-5.799769196072926e-19+0j), (1.8295911660692887e-19+0j), (2.4196104427476247e-19+0j), (5.1880768019325896e-20+0j), (-1.662915008814896e-22+0j), (-3.3881317890172014e-20+0j), (2.722407491180728e-24+0j), (4.222435742278918e-24+0j), (4.245665553348701e-19+0j), (4.298692207315574e-20+0j), (5.912253792374406e-23+0j), (1.088437337221776e-19+0j), (1.3918878317662014e-24+0j), (4.099964809409444e-26+0j), (-8.307513008785826e-24+0j), (-1.5685993351116424e-26+0j), (6.57417715393923e-34+0j), (3.7269449679189215e-19+0j), (-1.5262772712667707e-24+0j), (1.8504926693870294e-24+0j), (1.9420163033258023e-23+0j), (6.563748593068121e-26+0j), (3.662330710158571e-35+0j), (1.1372852105987482e-25+0j), (-1.2518906929549376e-34+0j), (-4.0438791082135026e-39+0j), (2.0199435492576442e-18+0j), (7.39036246479377e-20+0j), (-1.4346736076712723e-22+0j), (1.359487880343152e-19+0j), (5.402450808467704e-24+0j), (-2.0146080867479037e-25+0j), (2.0028110581440183e-22+0j), (2.182019922238621e-26+0j), (-2.4982606950583376e-33+0j), (1.8634724839594607e-20+0j), (3.3761456112303347e-23+0j), (-9.804847341560858e-26+0j), (-1.3150879269886026e-24+0j), (-1.8748769697168312e-26+0j), (-1.522631148767052e-34+0j), (-3.346300196132595e-26+0j), (3.885012273298766e-35+0j), (1.4978560806249806e-40+0j), (-4.1998803132672866e-23+0j), (7.312139743385848e-26+0j), (2.3133496021199858e-33+0j), (7.200525589402887e-26+0j), (-7.470314526125195e-35+0j), (1.2149219068919265e-40+0j), (3.117900363821984e-33+0j), (-1.7462527550188102e-41+0j), (-1.0083001500180521e-62+0j), (0.003042980832073251+0j), (-2.0466465395094005e-19+0j), (-5.421010862427522e-20+0j), (6.002166305578206e-19+0j), (8.300922883092143e-20+0j), (8.556682853363861e-24+0j), (-4.391018798566293e-18+0j), (-1.5130584959498813e-23+0j), (-8.756547913716523e-24+0j), (8.048732086748225e-19+0j), (8.809142651444724e-20+0j), (-5.231355394802549e-24+0j), (-2.1514636860259229e-19+0j), (-2.1857364818353685e-24+0j), (1.684820445429723e-24+0j), (-2.326962195426573e-23+0j), (3.4437283243172763e-25+0j), (-5.501305296653651e-34+0j), (1.6263032587282567e-18+0j), (8.828825728963583e-24+0j), (9.160846411701934e-25+0j), (1.0838223643935116e-24+0j), (3.231305465262712e-25+0j), (1.611266637061418e-34+0j), (9.195776065295773e-24+0j), (-3.1756356412998345e-34+0j), (1.730131053401786e-40+0j), (5.964457706714078e-20+0j), (-1.5246593050577406e-20+0j), (6.99751411537081e-23+0j), (1.7279472123987727e-19+0j), (-2.0283869457831513e-24+0j), (-3.0536341299395775e-25+0j), (-4.452558141008094e-24+0j), (1.1868679676645002e-25+0j), (-4.677387530837886e-34+0j), (-1.1011428314305904e-20+0j), (-4.3385432159925345e-24+0j), (4.157708541743848e-25+0j), (4.75386514147907e-24+0j), (1.747312171580096e-25+0j), (-1.6074706168277274e-34+0j), (4.975712690353384e-26+0j), (3.116628981733736e-35+0j), (1.2514959723417032e-40+0j), (3.3666412489621616e-23+0j), (4.2747017079223634e-26+0j), (1.975347772813553e-34+0j), (4.6023532484263844e-26+0j), (4.404435848469843e-35+0j), (7.374709207226744e-41+0j), (4.05820661481586e-34+0j), (-1.3037285033150022e-42+0j), (-2.172991307166599e-62+0j), (-4.0826988057657276e-19+0j), (-4.2332421875821695e-24+0j), (8.275350337815106e-24+0j), (2.143641788605439e-23+0j), (-1.7145909052225389e-25+0j), (-8.739045151407808e-35+0j), (2.571991615668953e-25+0j), (3.480274013673491e-35+0j), (-9.369786912555586e-42+0j), (-6.501122626783952e-24+0j), (5.58029409700323e-26+0j), (3.3630829666150186e-34+0j), (9.138658254435254e-26+0j), (1.1181485362042064e-35+0j), (-1.3674495726105282e-41+0j), (3.231137348932526e-34+0j), (-8.508684275380289e-42+0j), (4.7855991203874025e-64+0j), (7.682367744483157e-25+0j), (-7.3283004018524024e-37+0j), (1.5127389203901933e-42+0j), (2.3817154667773002e-34+0j), (3.570508487099634e-42+0j), (2.8147234988077685e-64+0j), (-4.2857312232910205e-41+0j), (-1.102715917229092e-65+0j), (1.699848203716493e-89+0j), (1.4739327795691914e-18+0j), (3.366955965335844e-20+0j), (-1.4864779795688713e-22+0j), (1.291725244562808e-20+0j), (3.672705265907198e-24+0j), (1.3970966431408907e-25+0j), (1.1285845482520371e-22+0j), (1.4835969397074737e-25+0j), (1.8892161334041864e-33+0j), (-7.623296525288703e-20+0j), (4.2029499288216826e-24+0j), (-3.1771142362271964e-26+0j), (-1.0996089930056728e-24+0j), (3.0486588554035527e-26+0j), (-1.574534504896885e-35+0j), (1.3380957517372386e-25+0j), (8.02899605398826e-35+0j), (1.1065611179927878e-40+0j), (-1.7311016139659554e-24+0j), (2.903078336898726e-26+0j), (1.790661989655447e-33+0j), (2.4544210856193154e-26+0j), (2.3920671854281734e-35+0j), (5.4890256474022847e-42+0j), (1.7939510578613306e-33+0j), (-8.942611238187481e-42+0j), (2.272762394302746e-64+0j), (-1.6940658945086007e-21+0j), (1.4029354773940145e-24+0j), (3.776001304784554e-25+0j), (-1.2171429569963618e-24+0j), (8.656804822791245e-27+0j), (1.4879301881887934e-35+0j), (9.822632289905849e-26+0j), (9.232274444961852e-36+0j), (9.405764281330733e-42+0j), (9.89446145313369e-25+0j), (2.5726709496839703e-26+0j), (1.1924001464198643e-34+0j), (-1.76774778744548e-26+0j), (7.560954331297857e-36+0j), (-4.503053117231817e-43+0j), (2.6043317513814177e-35+0j), (-6.0255833965967134e-43+0j), (3.442877736057334e-64+0j), (-8.182216220616564e-26+0j), (-3.264053436420417e-36+0j), (1.9663020051405833e-41+0j), (4.6967796621837256e-35+0j), (5.261875733539688e-43+0j), (-1.1050674248268343e-65+0j), (1.9127724038033753e-43+0j), (-4.3125287367133267e-66+0j), (-2.3046501095570932e-91+0j), (1.2378022774710138e-23+0j), (1.4610027656152666e-26+0j), (1.4328542327861494e-33+0j), (4.059017689570474e-27+0j), (4.829664526506343e-36+0j), (5.582773081870071e-42+0j), (-6.791420549030763e-35+0j), (5.482580241670847e-43+0j), (-6.901199188298897e-66+0j), (3.0673669883360375e-26+0j), (3.5156812968369264e-35+0j), (3.609744844100729e-42+0j), (-1.2999621488841608e-35+0j), (-5.083210179338274e-43+0j), (-6.581028017054045e-66+0j), (5.5950344434329134e-42+0j), (2.026224052656518e-67+0j), (-1.4144271144575407e-91+0j), (5.417661501865892e-35+0j), (1.1911036946760945e-44+0j), (-6.319036367505127e-67+0j), (-4.241117383433079e-43+0j), (6.3098762049501e-68+0j), (2.8497065722739e-93+0j), (-1.158091586055297e-67+0j), (1.0302199730833535e-93+0j), (-1.984832206659219e-263+0j), (4.9873299934333204e-18+0j), (2.636813564802637e-17+0j), (6.625273749604092e-06+0j), (8.822758706071836e-17+0j), (0.0012294559147695632+0j), (4.805592686458596e-18+0j), (-0.00013013413458707728+0j), (6.413901346960018e-18+0j), (-2.4286199643627506e-17+0j), (-3.7499842640842385e-17+0j), (-0.0012676677328911838+0j), (2.315256039554532e-17+0j), (0.0005608103490019227+0j), (-2.848458698501605e-18+0j), (-1.1384122811097797e-18+0j), (6.332958420612159e-18+0j), (-2.2632720350634905e-18+0j), (-1.3953826620854024e-21+0j), (2.47666633410617e-05+0j), (9.466350975066597e-18+0j), (-2.3418766925686896e-17+0j), (-1.5452111965428939e-18+0j), (-8.300922883092143e-19+0j), (4.0626496410208106e-22+0j), (-4.423544863740858e-17+0j), (-2.1030734823727635e-21+0j), (6.551538293363129e-23+0j), (4.2824291747282917e-17+0j), (-0.0032783620663029435+0j), (1.940655981048135e-17+0j), (0.0032601050429366843+0j), (-1.2076786866547774e-18+0j), (-1.4501204056993622e-18+0j), (9.055741454187317e-18+0j), (-3.3711911300721153e-19+0j), (-2.196089904597953e-21+0j), (0.00041648155321934735+0j), (1.0357124882048203e-18+0j), (-5.305814381600937e-18+0j), (1.2965982841669334e-18+0j), (4.353749348887104e-19+0j), (1.7815430672101956e-22+0j), (-4.2825985813177425e-18+0j), (4.47103374396641e-23+0j), (1.9197756657566768e-23+0j), (3.9031873310624e-18+0j), (-1.6771252355635147e-19+0j), (-6.170651291891093e-22+0j), (2.2768245622195593e-18+0j), (-2.9076921071131265e-23+0j), (-3.4259254808900837e-23+0j), (1.1517954062330575e-22+0j), (6.0040497963355986e-24+0j), (-1.7417589261617724e-32+0j), (-0.00021872373807622998+0j), (5.0826605777280904e-18+0j), (1.6479873021779667e-17+0j), (-1.0173383239608602e-18+0j), (-5.637851296924623e-18+0j), (-2.006643345065613e-21+0j), (-4.2500725161431774e-17+0j), (-5.178911238522772e-22+0j), (4.229879487521733e-22+0j), (2.3973650469527124e-18+0j), (-1.0299920638612292e-18+0j), (2.7883270266003974e-22+0j), (4.933119884809045e-18+0j), (-1.4535834279766123e-22+0j), (1.1537592750215858e-23+0j), (-5.5954628875707755e-22+0j), (1.544575155254715e-23+0j), (-6.766045566313267e-33+0j), (-7.719519468096792e-17+0j), (-3.570943296792835e-22+0j), (3.3211168462170725e-22+0j), (-7.824968208924353e-23+0j), (-4.454284361261712e-24+0j), (-1.9053823031348623e-33+0j), (1.2600312300029323e-22+0j), (-2.33521356745654e-33+0j), (1.0656176771108427e-37+0j), (4.66613509983449e-17+0j), (0.0018889753438960275+0j), (-1.3767132854210081e-18+0j), (-0.0018568863471047773+0j), (9.194102088765964e-19+0j), (6.306160292308266e-19+0j), (7.736278272548322e-18+0j), (4.0816400145816598e-19+0j), (-7.6401689986720375e-22+0j), (-0.00047379842965344937+0j), (5.311682687484635e-19+0j), (-4.851804721872632e-18+0j), (3.1333131738299366e-19+0j), (-9.486769009248164e-20+0j), (-2.41293285026761e-22+0j), (4.174178364069192e-18+0j), (3.8340609055464445e-23+0j), (2.5850537016844505e-23+0j), (9.852119583853436e-18+0j), (3.1170812458958252e-19+0j), (-1.0490037276068778e-21+0j), (-4.946672411965114e-19+0j), (2.353851667170781e-23+0j), (2.1429062295832845e-24+0j), (4.264149722247327e-22+0j), (9.918456198309942e-25+0j), (-9.719945320494593e-33+0j), (0.0008370691754092189+0j), (-2.3421178957404366e-19+0j), (-7.697835424647081e-18+0j), (1.0278860903076954e-18+0j), (6.776263578034403e-20+0j), (3.5844404951938405e-23+0j), (5.421010862427522e-20+0j), (-3.4433655300746564e-24+0j), (9.550649936836113e-24+0j), (3.7716070422630606e-19+0j), (4.0657581468206416e-20+0j), (-2.4529419043524185e-23+0j), (-7.962109704190423e-19+0j), (-1.02844106341859e-23+0j), (2.510483270726992e-24+0j), (7.243986302148023e-23+0j), (6.116614890343904e-25+0j), (5.552356648967879e-34+0j), (-1.0842021724855044e-18+0j), (-8.232746455323668e-24+0j), (5.9828689219082775e-24+0j), (-3.2168077723792422e-24+0j), (2.7638080455495095e-25+0j), (1.1441573244682962e-34+0j), (3.2071493737766617e-24+0j), (-5.070303634851188e-34+0j), (-2.982188708218197e-39+0j), (4.041773899779524e-18+0j), (4.743384504624082e-20+0j), (1.4741867067784165e-22+0j), (-1.4907779871675686e-19+0j), (-3.554180495955976e-23+0j), (1.3468694904269006e-24+0j), (2.5921213521798686e-23+0j), (1.4081056918928009e-24+0j), (-1.7875992385504076e-33+0j), (7.318364664277155e-19+0j), (-1.5663341621255385e-23+0j), (-2.6038165003079273e-24+0j), (6.013099802892687e-24+0j), (5.951526647682855e-25+0j), (3.353692270558198e-34+0j), (2.3556904444094116e-24+0j), (-3.573727823465341e-35+0j), (-2.9971177956788345e-39+0j), (6.212255647243949e-23+0j), (1.151932789781357e-25+0j), (-1.5017388488975974e-33+0j), (5.761991539034145e-25+0j), (3.2935319849688e-35+0j), (-1.0705941875917397e-40+0j), (1.7845114307042134e-34+0j), (2.7656430716038368e-40+0j), (-1.0572495298902982e-62+0j), (-1.1297141411216477e-05+0j), (2.213052291067895e-19+0j), (-3.469446951953614e-17+0j), (3.138052915466696e-19+0j), (-1.4094628242311558e-18+0j), (-8.14908188191963e-23+0j), (-3.252606517456513e-17+0j), (2.812407436448844e-22+0j), (1.762325032567978e-22+0j), (4.202090953756539e-18+0j), (5.149960319306146e-19+0j), (5.595125951132715e-22+0j), (2.3445871979999033e-18+0j), (-1.9514312305426133e-23+0j), (2.3797143588178222e-24+0j), (8.242985740091668e-23+0j), (6.591410603044286e-25+0j), (-2.7724328137731356e-34+0j), (9.920449878242366e-18+0j), (3.445724239166206e-23+0j), (6.54251047073251e-23+0j), (-3.395317920588756e-23+0j), (2.1087158252281885e-24+0j), (-4.7422001567449e-33+0j), (2.0702616302781215e-23+0j), (-3.190626867692665e-33+0j), (1.512476363861133e-39+0j), (3.625602664713874e-18+0j), (6.437450399132683e-20+0j), (1.4222121033951525e-22+0j), (-1.1926223897340549e-18+0j), (2.1514759934369065e-24+0j), (6.665706528866766e-24+0j), (-2.3235240883375587e-23+0j), (2.2655820907396016e-25+0j), (6.430726435520601e-33+0j), (-2.574980159653073e-19+0j), (-1.99104958380928e-23+0j), (2.760634144842198e-24+0j), (1.4159005641435028e-23+0j), (-4.0258619446546016e-27+0j), (9.611330836340064e-35+0j), (3.835911341649957e-25+0j), (-1.5128378114567637e-34+0j), (2.2251007809238714e-40+0j), (1.8482316811731712e-23+0j), (2.289968418807691e-25+0j), (-2.6032960285033535e-33+0j), (6.462057015411851e-25+0j), (-2.8816879538283e-35+0j), (2.9338540402062955e-40+0j), (1.9976388076455528e-33+0j), (-4.0422859575123e-41+0j), (1.6145086044036304e-64+0j), (-3.713392440762853e-18+0j), (9.128067306493372e-24+0j), (4.5735654235736556e-23+0j), (4.1352568279115813e-23+0j), (8.076016969504924e-26+0j), (9.725721589531939e-34+0j), (-1.237214467073012e-23+0j), (9.61925107391184e-34+0j), (4.362690404565716e-39+0j), (-1.1684249269738294e-22+0j), (2.5997504768197532e-24+0j), (1.710163341560123e-33+0j), (-2.696512842320244e-25+0j), (6.888927819780768e-35+0j), (-1.0421833495904497e-40+0j), (1.7764693876960804e-33+0j), (4.271287715385971e-41+0j), (1.3568127175742236e-64+0j), (-2.084137892914945e-23+0j), (2.6814912357251686e-33+0j), (5.6338924498486405e-40+0j), (3.4539379681839124e-33+0j), (1.3647526003752258e-40+0j), (-1.8674390529600556e-65+0j), (-3.5828399135856923e-40+0j), (-8.327490599256324e-66+0j), (-6.774846949231237e-92+0j), (7.235694248625135e-17+0j), (0.0035964622282034925+0j), (6.512966206548106e-18+0j), (-0.005213825486052969+0j), (1.5822380807035587e-18+0j), (2.8672065264558066e-19+0j), (9.042419836154582e-18+0j), (4.183912625517716e-19+0j), (-1.0480778159780746e-21+0j), (-0.0006080811329916221+0j), (2.885399981187269e-19+0j), (3.5067164016328034e-19+0j), (1.4192682912806718e-19+0j), (-3.8963515573697816e-20+0j), (-5.994175027486294e-23+0j), (1.4568966692773966e-19+0j), (-3.9998884509894964e-23+0j), (2.5332333474692014e-24+0j), (5.047747865848652e-18+0j), (4.0149361699853836e-19+0j), (-1.1671387146420718e-21+0j), (-1.7787691892340307e-19+0j), (2.6219944336200615e-23+0j), (1.3454596218852037e-24+0j), (-1.3164433919941986e-22+0j), (2.0139991154032083e-25+0j), (-3.1539461174882324e-33+0j), (0.0037051924586153635+0j), (-5.244562449220343e-19+0j), (-3.7947076036992655e-18+0j), (1.1377394890138961e-18+0j), (2.879912020664621e-20+0j), (-8.334995230855734e-23+0j), (-9.75781955236954e-19+0j), (1.6966965762485584e-23+0j), (1.1555576902228759e-23+0j), (7.63639180577823e-19+0j), (3.3881317890172014e-19+0j), (-6.35565393975256e-23+0j), (-4.0826988057657276e-19+0j), (1.3493943870239047e-23+0j), (1.1319299493323794e-24+0j), (2.411371583151362e-23+0j), (8.398655338498014e-25+0j), (-3.996579773750004e-35+0j), (-6.478107980600889e-18+0j), (-9.257307386783215e-25+0j), (1.3417413820289317e-23+0j), (-1.1663003425268836e-23+0j), (1.7376525316381777e-25+0j), (-1.8329047193765179e-34+0j), (3.3190773958163614e-24+0j), (-1.8468439701608586e-35+0j), (1.0247330391500853e-39+0j), (3.7296004760238666e-19+0j), (4.086933970501999e-20+0j), (-2.275540709211757e-22+0j), (2.371692252312041e-20+0j), (4.9485651068010855e-24+0j), (1.3370139306632317e-24+0j), (7.560301551763962e-23+0j), (1.2839279740510014e-25+0j), (-1.965077947031449e-33+0j), (9.656175598699024e-20+0j), (4.220839488058522e-24+0j), (1.2107027115522798e-24+0j), (2.070374912083408e-24+0j), (7.331907350510677e-26+0j), (-1.477045156417927e-34+0j), (1.522810249993811e-25+0j), (-6.454504596980199e-35+0j), (-1.3192826271099542e-40+0j), (7.085965169253085e-24+0j), (1.0427912067722412e-26+0j), (8.42014455428926e-34+0j), (1.5995496096999433e-25+0j), (3.277488840364665e-34+0j), (3.9043993510068056e-41+0j), (7.233473192215509e-34+0j), (2.1148460226833537e-41+0j), (-6.794636845630608e-63+0j), (8.516367566742e-05+0j), (1.1328208136981004e-19+0j), (-4.716279450311944e-18+0j), (1.491540439748569e-19+0j), (3.8624702394796095e-19+0j), (-1.3256140691029491e-24+0j), (-4.8382521947165635e-18+0j), (-1.2382983615454473e-23+0j), (4.8837034843873444e-24+0j), (1.8944718267120838e-20+0j), (1.8973538018496328e-19+0j), (4.822334480627972e-23+0j), (2.473336205982557e-19+0j), (6.06247722484224e-25+0j), (3.7906212772961355e-25+0j), (9.800967931370618e-24+0j), (5.541191501109357e-25+0j), (2.39939978344426e-35+0j), (1.0706496453294356e-18+0j), (-2.226279070504047e-24+0j), (1.3101017375425352e-24+0j), (1.4112153614552142e-24+0j), (3.367870622764385e-25+0j), (-1.7020245584663653e-35+0j), (1.3571414526901821e-24+0j), (1.5667970773596487e-34+0j), (-4.9157759305817e-40+0j), (2.4913340211336653e-19+0j), (-3.5045988192646677e-20+0j), (5.205866213991064e-24+0j), (9.571472303973594e-20+0j), (5.174510654818528e-24+0j), (1.4250543996415394e-25+0j), (-9.288010432748742e-24+0j), (7.083539875670477e-26+0j), (7.129756650933633e-35+0j), (8.978549240895584e-20+0j), (-1.8510589586574833e-24+0j), (1.4933798428672295e-25+0j), (3.999991795126421e-24+0j), (-3.173977813722389e-27+0j), (7.005495762997808e-36+0j), (2.790932888991969e-25+0j), (-3.414403138645656e-35+0j), (8.80349523300926e-41+0j), (-3.9392053292137104e-24+0j), (2.606701567874335e-26+0j), (2.5637983507280818e-34+0j), (-4.62925238583519e-26+0j), (4.6858850262494964e-36+0j), (5.570647916001268e-41+0j), (4.792433071117367e-34+0j), (3.5774767585622486e-42+0j), (5.156156617758926e-64+0j), (-5.065257024580716e-19+0j), (-1.1208135741380136e-25+0j), (2.680154194731886e-24+0j), (4.422467830639344e-24+0j), (6.354721781579749e-28+0j), (5.298281925026666e-35+0j), (-1.180181596523851e-24+0j), (1.130605188998854e-34+0j), (-2.2565163272562005e-41+0j), (-1.1888701821532407e-24+0j), (-7.818238313224505e-26+0j), (5.5083074088525876e-37+0j), (7.418375585767887e-26+0j), (1.2010327978530856e-35+0j), (-1.2021303751881318e-41+0j), (1.6433380837163541e-34+0j), (1.0442476156148537e-41+0j), (1.6399582567538906e-64+0j), (-3.512501694005919e-25+0j), (5.413464282258802e-35+0j), (1.1849379814330653e-41+0j), (5.757100260867487e-35+0j), (8.404988189020253e-42+0j), (-1.3243440045451786e-65+0j), (-6.038054952929204e-41+0j), (-2.896975095565474e-66+0j), (-2.3091042159791937e-91+0j), (5.35339141275863e-19+0j), (3.705769144237564e-20+0j), (6.470103353813213e-23+0j), (3.9387032047324966e-20+0j), (-2.8014280901698242e-24+0j), (2.024512023575706e-25+0j), (2.4976977089980093e-24+0j), (7.34586184007817e-27+0j), (2.414966083714032e-33+0j), (1.249373597200093e-19+0j), (2.762250102172928e-24+0j), (2.557407811389159e-25+0j), (1.2761118873834428e-24+0j), (-9.553619669476802e-27+0j), (-3.721277332269056e-35+0j), (7.111469096655965e-26+0j), (1.0343810753696202e-34+0j), (-5.713154988753405e-41+0j), (1.024443801601318e-23+0j), (-1.0897050437319714e-26+0j), (-2.523647757013438e-34+0j), (-8.792679127736474e-27+0j), (1.6802507300280226e-34+0j), (-5.773349673018246e-42+0j), (1.3611681260767945e-33+0j), (4.74759919713248e-42+0j), (-7.040189092514335e-66+0j), (-1.1468826105823227e-18+0j), (2.1354022942225866e-24+0j), (1.46470272622236e-24+0j), (-2.259398606748846e-24+0j), (3.9775302856220865e-26+0j), (3.177342949269189e-34+0j), (-9.209873301843662e-25+0j), (3.7259417503893463e-35+0j), (-8.938571240844437e-41+0j), (2.6190686924316933e-24+0j), (-1.6331856155500475e-26+0j), (5.806082053640581e-34+0j), (4.0808062480618774e-26+0j), (4.0317094608086953e-35+0j), (-2.2420775429197073e-43+0j), (-6.086227271407189e-35+0j), (1.700615816304598e-41+0j), (-1.1133192864789865e-65+0j), (-4.213024050555766e-25+0j), (9.091627037349363e-35+0j), (1.0922841269719084e-40+0j), (1.2232972125881015e-34+0j), (-5.809783433090692e-42+0j), (1.7468057202548156e-67+0j), (-9.604499674482296e-42+0j), (-1.8419424556964097e-67+0j), (5.596767276172434e-93+0j), (5.7810756360471794e-24+0j), (-4.778902193976532e-28+0j), (-3.5218285712342426e-35+0j), (-5.39727971411412e-26+0j), (1.4532037253237306e-35+0j), (2.4405014054681014e-41+0j), (2.3280463553038926e-34+0j), (-1.7810503481568425e-42+0j), (-1.8616918608160733e-66+0j), (-1.64653706715706e-25+0j), (3.156895398483754e-35+0j), (-3.0203587100057107e-41+0j), (7.168990262510777e-35+0j), (-2.3436716815832566e-43+0j), (1.4743552607476632e-67+0j), (4.560175527529036e-42+0j), (-1.533496051151444e-67+0j), (2.89562922679032e-93+0j), (-7.987414107768763e-35+0j), (5.574540453392163e-43+0j), (-1.3946034364578915e-67+0j), (1.3649522854063921e-43+0j), (-1.720688478050109e-68+0j), (-6.754573373968438e-95+0j), (-5.662036340491552e-68+0j), (-3.399702857476568e-95+0j), (1.0253314378498982e-265+0j), (-0.00011720859516435001+0j), (5.3163967375236877e-20+0j), (1.3010426069826053e-17+0j), (5.371981320354699e-19+0j), (-4.607859233063394e-19+0j), (-1.1439033693236367e-22+0j), (-2.927345865710862e-18+0j), (-2.0043721389232394e-24+0j), (1.4245792066764176e-22+0j), (7.505834616025947e-19+0j), (-3.3881317890172014e-18+0j), (-5.75423477870472e-23+0j), (3.178067618098135e-18+0j), (-4.8281496430924995e-27+0j), (8.320525706891305e-24+0j), (6.721488711846907e-23+0j), (6.2442293312388484e-24+0j), (-1.5239139988128829e-33+0j), (-2.3798237686056822e-17+0j), (3.9652970614260764e-23+0j), (2.74456754298554e-23+0j), (5.389275561239042e-23+0j), (3.840826399824113e-24+0j), (1.2946332976308928e-33+0j), (2.9078263679152356e-24+0j), (-1.9997330816971247e-34+0j), (-1.5710957250980826e-39+0j), (1.3484950102696173e-18+0j), (1.1096131609031334e-19+0j), (1.8783203098864531e-22+0j), (1.0842021724855044e-19+0j), (2.4015702745313977e-24+0j), (8.601400200946433e-24+0j), (1.4420730757125635e-23+0j), (7.039095605020644e-26+0j), (-2.564078735977289e-33+0j), (-9.554531645028508e-19+0j), (8.004426454871046e-24+0j), (3.5996378692013533e-25+0j), (-5.7999578106745495e-25+0j), (2.2399493618525753e-25+0j), (2.495282096836231e-34+0j), (1.860537777537844e-26+0j), (2.346476565431e-35+0j), (8.203305475492966e-41+0j), (-3.462203227931911e-24+0j), (1.898912151674938e-25+0j), (1.794262615521765e-33+0j), (1.1552287492619627e-24+0j), (1.5154437154067281e-34+0j), (-4.236339506925959e-40+0j), (3.698570269007629e-33+0j), (5.806980836162042e-42+0j), (1.6877679708438918e-64+0j), (-8.080694316806025e-18+0j), (1.9181058247140276e-24+0j), (-5.585100383457667e-24+0j), (-2.4923662714561637e-23+0j), (7.075108531022088e-25+0j), (1.3914282664439632e-33+0j), (4.706266851414828e-24+0j), (1.1847360710502546e-33+0j), (3.6483085778389477e-40+0j), (3.9056011168287437e-23+0j), (1.5737692026665743e-24+0j), (2.2568754957711548e-33+0j), (-4.581211647866425e-25+0j), (1.5348722284964408e-34+0j), (1.0916675556476055e-40+0j), (3.6064331036479987e-34+0j), (7.532259505438757e-41+0j), (-1.499184442966732e-65+0j), (8.443758312161234e-24+0j), (4.484351954596979e-34+0j), (2.3381281448583876e-39+0j), (1.85499697125483e-34+0j), (-6.841139102833757e-41+0j), (-7.140663028070985e-66+0j), (5.10739659083252e-40+0j), (-2.9760429669226954e-66+0j), (-6.899238198445243e-92+0j), (7.741938160422696e-19+0j), (3.134021904840911e-20+0j), (1.290660249524146e-22+0j), (5.69629657028517e-20+0j), (-8.27867237084932e-24+0j), (1.6224027999926004e-25+0j), (9.96817261611754e-24+0j), (1.5360989957778398e-25+0j), (9.247136054161433e-34+0j), (-1.0147454708106518e-18+0j), (-5.00832011506716e-25+0j), (2.3811886521100296e-25+0j), (-1.7210041943791264e-24+0j), (-6.043671025789655e-26+0j), (3.828272329017291e-34+0j), (1.0285660131691984e-24+0j), (1.8421536587354407e-34+0j), (1.1725535442709993e-40+0j), (2.4153835445261797e-23+0j), (8.465242049496132e-26+0j), (6.324764315467515e-34+0j), (7.995984372741462e-26+0j), (1.3233025487392426e-34+0j), (-5.201619899573721e-42+0j), (2.184113216235002e-34+0j), (-3.486430579240145e-42+0j), (-1.0912081220906088e-65+0j), (-1.4177213954668852e-18+0j), (-3.1872504925825584e-25+0j), (7.285731870230294e-25+0j), (2.9364305900457273e-24+0j), (1.5201045251450887e-26+0j), (-1.220697833980618e-34+0j), (-7.532402307052009e-25+0j), (8.936852352948798e-35+0j), (-2.719864267315897e-40+0j), (4.304933866648655e-24+0j), (-2.7544179850064594e-26+0j), (1.9229227803035702e-34+0j), (-3.944030827121939e-26+0j), (3.061043502909393e-35+0j), (-4.450523922695619e-42+0j), (1.4005473113991694e-34+0j), (2.3037346753499993e-42+0j), (-6.437598422464367e-66+0j), (-1.2497862704935454e-24+0j), (-6.137853434109405e-35+0j), (5.93505951580133e-41+0j), (2.0448115862964346e-34+0j), (7.868290877183848e-43+0j), (-3.73118634239796e-67+0j), (-5.3224818921217364e-42+0j), (2.701599624468876e-68+0j), (4.822001631087043e-93+0j), (1.619727075923232e-23+0j), (-5.133563029511069e-26+0j), (6.473357599256898e-35+0j), (-1.0112483774290647e-25+0j), (1.2897491872556406e-34+0j), (-3.693822751960218e-42+0j), (9.70551865904737e-34+0j), (-8.381516439742812e-43+0j), (-3.89850078813953e-66+0j), (-1.0118476053357377e-24+0j), (9.964289483234009e-35+0j), (-9.903256507076347e-41+0j), (1.1243765444507268e-35+0j), (2.939573853537385e-42+0j), (-3.410303256819901e-67+0j), (-2.7119504342386225e-41+0j), (-2.9300031151926347e-67+0j), (1.7024031105850479e-93+0j), (6.319259899016082e-34+0j), (1.7228964618873626e-42+0j), (6.031111213424258e-67+0j), (-2.0060025422579858e-42+0j), (-9.558212448189629e-69+0j), (-1.057139572832642e-95+0j), (-5.428416660651565e-68+0j), (5.91413599530104e-95+0j), (1.2067043734111217e-265+0j), (1.1019898643778447e-18+0j), (5.5586294825493376e-24+0j), (-2.1591315459998398e-23+0j), (-4.673287733176175e-24+0j), (-1.7034357571923092e-25+0j), (2.1930981141931745e-34+0j), (-2.6270310012646956e-24+0j), (-3.238522989122292e-35+0j), (7.210431694928062e-39+0j), (1.0958931425977197e-23+0j), (1.9620931227098294e-25+0j), (3.066611527067087e-33+0j), (-7.575642003680321e-25+0j), (-6.375114909136195e-35+0j), (1.0094954136995982e-41+0j), (7.167644570484217e-34+0j), (-1.2203067546726237e-40+0j), (-4.294864680131637e-66+0j), (-7.105424193191934e-24+0j), (4.00041021570338e-34+0j), (-2.3459137591261763e-41+0j), (3.6211858585255208e-34+0j), (-1.949906813107983e-42+0j), (-1.6578164368601495e-66+0j), (-1.2262552666536825e-40+0j), (-6.644108978189739e-67+0j), (3.0428385126482286e-94+0j), (5.509959920140822e-24+0j), (-5.62342915924073e-26+0j), (3.159550909565236e-33+0j), (-4.0244530514371114e-25+0j), (5.384162786089828e-35+0j), (-1.3767196892605598e-40+0j), (-2.231820491508929e-34+0j), (-2.7377868746746114e-42+0j), (-3.93944222114437e-66+0j), (2.711395147395582e-26+0j), (5.848011944286639e-35+0j), (-8.197245691684099e-42+0j), (2.1181563983952595e-35+0j), (-6.854714181706904e-42+0j), (-2.9492569375796784e-68+0j), (-1.0514467864753224e-41+0j), (-7.386941999526056e-68+0j), (-2.4216497790396065e-94+0j), (3.390648261007199e-34+0j), (4.339208275935816e-43+0j), (-5.4257544836207946e-67+0j), (-1.5995865760844797e-41+0j), (-1.3616346101183092e-67+0j), (1.5632484381213687e-94+0j), (9.402345662202222e-68+0j), (1.644826263290115e-95+0j), (-1.3590442928937816e-267+0j), (-1.3695727425680415e-25+0j), (6.038076375980127e-36+0j), (-4.430092991085763e-40+0j), (4.393026033801216e-34+0j), (-5.0074962716877237e-42+0j), (-5.79817536689833e-67+0j), (-1.6929071420806161e-40+0j), (-4.480304803183714e-67+0j), (3.934287777558032e-93+0j), (2.0686863407095217e-34+0j), (-1.2029512352861907e-41+0j), (-2.2426026048693973e-67+0j), (-4.842301793739057e-42+0j), (-7.442935797498509e-68+0j), (4.526143843134082e-95+0j), (-1.7701232003075846e-67+0j), (-2.2679926497163048e-95+0j), (-8.390388169831093e-268+0j), (-2.1396533538689307e-40+0j), (-1.760556908042475e-67+0j), (-3.931175255629206e-95+0j), (-3.55585841719677e-67+0j), (6.18089475579669e-96+0j), (1.379661385824063e-269+0j), (-5.278797457470282e-95+0j), (5.28154565404014e-270+0j), 0j, 0j, (4.135960237489078e-16+0j), (-2.9896874906287785e-17+0j), (8.558149848514329e-16+0j), (-2.6125206598753836e-16+0j), (0.002358711007946839+0j), (-4.286664339464563e-17+0j), (0.002370413061542909+0j), (-8.419742030728631e-19+0j), (3.3499136624370873e-16+0j), (1.4190444609304964e-15+0j), (0.002380448873624651+0j), (3.4206578541917665e-17+0j), (-0.09405186334170663+0j), (4.801118310867901e-19+0j), (0.003643662312907425+0j), (-3.996786357061222e-19+0j), (1.1069682210278076e-18+0j), (9.276704838329097e-17+0j), (-0.0035700319970263096+0j), (1.217418427213835e-17+0j), (0.010130047876202775+0j), (-1.059829758394423e-18+0j), (2.282571868110029e-18+0j), (2.8866186368633202e-18+0j), (2.2968646640067877e-18+0j), (8.387860170685553e-22+0j), (4.910622689729971e-16+0j), (3.126839065448195e-16+0j), (-0.0033344575525739314+0j), (-5.800481622797449e-18+0j), (-0.012772120658719943+0j), (-1.238728065973437e-19+0j), (0.0019822301935981165+0j), (-9.507236249673872e-19+0j), (1.910781359467314e-18+0j), (-1.1372196587200456e-15+0j), (0.1386195857253669+0j), (-1.2126084528371732e-18+0j), (-0.07046287799955397+0j), (4.184486517700694e-17+0j), (4.465312625482275e-19+0j), (9.344410833022489e-19+0j), (1.8507669897506462e-19+0j), (-1.4703748513998394e-20+0j), (-0.007035540287485968+0j), (-4.853845785474623e-19+0j), (1.1519078640031403e-18+0j), (-1.1579815068095171e-18+0j), (1.9608812728937053e-19+0j), (-2.2155617435568158e-21+0j), (4.535861432546778e-19+0j), (-1.2587334020659715e-22+0j), (-4.509988855738867e-22+0j), (6.811500148640182e-17+0j), (0.001345104890463747+0j), (1.99669845535859e-17+0j), (-0.0011578533985751739+0j), (5.318086335039271e-19+0j), (1.785545452812065e-18+0j), (-6.819801718128855e-18+0j), (6.598386659111e-19+0j), (2.5716578083803787e-22+0j), (0.0007547140950756454+0j), (-1.3086608030389637e-18+0j), (1.4416500762268192e-18+0j), (1.0848728978776069e-18+0j), (2.9010878443459787e-19+0j), (5.100537046095081e-22+0j), (1.328147661294743e-18+0j), (-1.1336036183996084e-22+0j), (8.68262674116856e-24+0j), (3.83157581391309e-18+0j), (9.537590986083422e-19+0j), (-1.0268226524968898e-21+0j), (8.364450354136216e-19+0j), (-1.1767593947015108e-22+0j), (8.266344841483432e-24+0j), (1.2345830941393588e-23+0j), (-4.479459875597941e-25+0j), (1.949805704020375e-31+0j), (-4.290187996525141e-16+0j), (2.569084810340183e-16+0j), (-0.006549677756147287+0j), (5.816744655384731e-17+0j), (0.1868382787841538+0j), (-5.381401066608784e-19+0j), (-0.0032509302354601474+0j), (7.311033773980798e-19+0j), (7.3845411159050805e-19+0j), (-6.716361408004579e-16+0j), (0.08657868275362453+0j), (5.245595033615181e-19+0j), (-0.21621641415154771+0j), (1.925024210041912e-17+0j), (2.3073096014730707e-19+0j), (7.379942182358958e-19+0j), (1.32560656245298e-19+0j), (-3.994739139644065e-22+0j), (-0.0008351689201632735+0j), (-5.339672892755508e-20+0j), (5.929230630780102e-19+0j), (-9.17233901737465e-19+0j), (4.8492636230308694e-20+0j), (1.6328060150228664e-23+0j), (3.8709405689521525e-19+0j), (-3.6164171435488137e-23+0j), (-1.7025653595804905e-24+0j), (9.670337989577116e-16+0j), (-0.28251350533233166+0j), (1.3085601403495012e-18+0j), (0.15086642222383312+0j), (-4.022679505748213e-17+0j), (1.257843926672636e-19+0j), (4.942063023538403e-19+0j), (5.405128994666504e-20+0j), (1.4834412989116915e-22+0j), (0.1737628598676272+0j), (-8.744046322569985e-18+0j), (1.6855955650360577e-19+0j), (2.919438682572072e-17+0j), (-7.835054762102278e-20+0j), (1.6709496786915664e-22+0j), (1.3167391862864116e-19+0j), (7.831322469777708e-24+0j), (2.472427102101888e-23+0j), (2.869741544743037e-19+0j), (2.218167530622199e-20+0j), (-4.709321386277588e-23+0j), (9.145308852386274e-20+0j), (-1.64725546392652e-23+0j), (-4.282892008273035e-25+0j), (-1.7512602702122656e-24+0j), (-7.6536764138739855e-25+0j), (2.6582402232148707e-31+0j), (-0.009734148321725908+0j), (1.859764217184503e-18+0j), (2.0328790734103208e-19+0j), (-7.768955629709765e-19+0j), (3.703122166277394e-20+0j), (-4.6805172716680465e-23+0j), (2.490276864927643e-19+0j), (-9.398771359496348e-24+0j), (-6.4833806519874656e-24+0j), (-7.239740554747935e-19+0j), (1.0037340424963459e-19+0j), (-1.2178368360441435e-23+0j), (8.75620309224133e-20+0j), (-2.2253449381475685e-24+0j), (-9.740909924451452e-25+0j), (-8.174155539662686e-24+0j), (1.1867088511843549e-25+0j), (-1.5379696602721535e-32+0j), (-1.9481757786848908e-19+0j), (-1.7590100059186048e-23+0j), (-1.8196127288966383e-24+0j), (-1.2877174722192412e-23+0j), (1.633627196422191e-25+0j), (-6.620061290226632e-33+0j), (5.64079369842978e-25+0j), (4.942928720035016e-34+0j), (1.1053725168974813e-37+0j), (-3.57109090562413e-18+0j), (0.0012102384757079242+0j), (1.8087522508704412e-17+0j), (0.007074651529048737+0j), (-3.0450388026775306e-19+0j), (5.166900978251232e-19+0j), (1.4905033692766066e-17+0j), (8.584678920422334e-19+0j), (-1.3176289994441022e-21+0j), (-0.003621267397452377+0j), (5.101051132433124e-19+0j), (1.0426975580700437e-18+0j), (4.57493371485628e-19+0j), (1.0577323928838075e-19+0j), (-7.356784179281105e-23+0j), (6.51368336438557e-19+0j), (-5.021041899356404e-23+0j), (-2.091420389831927e-24+0j), (1.570411790056682e-18+0j), (7.085430603782222e-19+0j), (9.177568492914852e-23+0j), (4.0234064994579266e-19+0j), (2.7849894586738805e-23+0j), (2.157857515712542e-26+0j), (-2.655690808732919e-22+0j), (1.3879339560784594e-25+0j), (-4.271927203144632e-33+0j), (-0.007873971086058865+0j), (7.172041248239389e-19+0j), (5.149960319306146e-19+0j), (-1.2349687734779993e-18+0j), (2.5172760401213738e-20+0j), (1.3502544409948e-23+0j), (8.809142651444724e-20+0j), (-1.7120021221366334e-23+0j), (2.171129952531466e-24+0j), (2.8327402872100104e-19+0j), (3.705769144237564e-20+0j), (1.0307601568711619e-23+0j), (6.410980619530986e-20+0j), (-5.7502688184223224e-24+0j), (2.746176112130773e-25+0j), (-3.9010126220942505e-24+0j), (1.8553940698064155e-26+0j), (-1.0467954987593032e-32+0j), (2.202285662861181e-20+0j), (-1.2611250558331213e-23+0j), (4.649306058130897e-25+0j), (-1.811722658681252e-23+0j), (1.397707064470093e-27+0j), (3.914660674467152e-34+0j), (4.618183329711706e-25+0j), (1.9548705234689844e-34+0j), (6.415741613112721e-38+0j), (1.4427102850921248e-18+0j), (1.1501119236937297e-19+0j), (-2.740007752341482e-22+0j), (1.8994713842177685e-19+0j), (9.897464000224674e-24+0j), (1.508393355269378e-25+0j), (-1.2542643956172953e-23+0j), (-1.045456056904222e-26+0j), (-1.1522364604072993e-33+0j), (2.39710324072967e-19+0j), (3.400389333264818e-24+0j), (6.322297221130622e-27+0j), (3.826094144643965e-24+0j), (2.595851059217629e-26+0j), (3.8437684927967315e-34+0j), (1.1817316372067834e-25+0j), (1.193484924519638e-35+0j), (-3.525421217020819e-39+0j), (-5.0634549882707115e-23+0j), (9.2166387638907e-27+0j), (2.4671642777042802e-34+0j), (2.2460192826717164e-26+0j), (1.1059140392144835e-35+0j), (-1.2033490314282138e-39+0j), (3.1130980790980166e-34+0j), (6.367554317897986e-41+0j), (6.881001438873793e-61+0j), (-2.940356291780688e-16+0j), (-6.880889087679254e-16+0j), (0.004596789105550721+0j), (-1.1673537028515646e-15+0j), (-0.06290838650856824+0j), (1.1191813558378628e-18+0j), (-0.001342721484763332+0j), (6.4420149787919565e-19+0j), (7.589415207398531e-19+0j), (1.1959021013058235e-15+0j), (-0.11990336759427571+0j), (1.870728011641176e-18+0j), (0.15273682699728766+0j), (-2.906851001915575e-17+0j), (1.3552527156068805e-20+0j), (2.8015351618875397e-18+0j), (-9.825582188149884e-20+0j), (3.0819205937072606e-22+0j), (0.003156456789914524+0j), (7.687453814228278e-19+0j), (-2.100641709190665e-18+0j), (1.9909918820917632e-18+0j), (-3.9985249066323315e-19+0j), (-6.155324163999862e-23+0j), (2.1141942363467336e-18+0j), (-9.973914834332778e-23+0j), (-1.1266041129494798e-22+0j), (4.0680620764371733e-16+0j), (0.08571925612927088+0j), (-3.1885070529675286e-18+0j), (-0.029546706993407525+0j), (7.243734482976001e-18+0j), (-1.4399560103323106e-19+0j), (4.510277198582804e-19+0j), (8.983843196815923e-20+0j), (-5.071332623842e-22+0j), (-0.08941278135016961+0j), (1.669273432341668e-17+0j), (-1.971469184734384e-19+0j), (-1.0004988524745895e-17+0j), (2.2896359355467806e-19+0j), (-1.2733983556991285e-22+0j), (7.962109704190423e-20+0j), (6.534325733366084e-24+0j), (-2.870023292968091e-24+0j), (3.1046015903495224e-19+0j), (8.24798332388875e-20+0j), (-5.975362175951345e-23+0j), (-6.081696561285876e-19+0j), (-3.345407053132474e-24+0j), (-3.4482876722600055e-24+0j), (9.008529951144357e-24+0j), (1.2830442367811537e-25+0j), (-1.0392195111366505e-32+0j), (0.001181528158113709+0j), (3.804358556710691e-19+0j), (-1.8973538018496328e-19+0j), (-7.203541720433642e-19+0j), (-4.421511984667448e-19+0j), (-1.669528408800599e-23+0j), (-1.3552527156068805e-18+0j), (-8.882042130328096e-23+0j), (6.241144019555221e-23+0j), (8.465506395251068e-19+0j), (-3.1170812458958252e-19+0j), (-7.693908753952274e-24+0j), (-8.131516293641283e-20+0j), (9.029191456185697e-24+0j), (4.643346961252771e-25+0j), (-5.296393327311261e-23+0j), (5.217693305703022e-25+0j), (3.3545945543079504e-34+0j), (3.63207727782644e-18+0j), (-1.5037883129144092e-23+0j), (6.05067689000842e-24+0j), (-3.186010668794718e-23+0j), (-2.5036118993528084e-26+0j), (2.405024707583937e-34+0j), (6.450981760883333e-24+0j), (-1.2106985699194919e-34+0j), (3.722404465230464e-40+0j), (-1.2172473315766319e-15+0j), (0.13505470253340798+0j), (1.9377979963269304e-18+0j), (-0.11918125423207183+0j), (2.914075058596689e-17+0j), (2.2552252220645747e-20+0j), (-1.6047784697374223e-18+0j), (5.638063055161437e-20+0j), (-3.614650479713921e-22+0j), (-0.055105236425131314+0j), (-1.796648880731329e-18+0j), (-8.893845946170154e-20+0j), (-1.866082894566473e-17+0j), (-2.9005584487539447e-19+0j), (-6.371279828236751e-24+0j), (-4.438452643612534e-19+0j), (9.845178008740602e-24+0j), (2.5228740568767182e-24+0j), (1.4401064814998486e-18+0j), (4.282810339554556e-20+0j), (-6.205617589501576e-23+0j), (8.999725064576941e-20+0j), (-5.40273288967504e-24+0j), (2.335911156555581e-25+0j), (2.1525198963887704e-24+0j), (1.3390039031708737e-25+0j), (-4.385602282314947e-33+0j), (0.0756026609401255+0j), (-1.805851622750429e-17+0j), (-2.405573570202213e-19+0j), (1.1344550951758769e-17+0j), (7.623296525288703e-21+0j), (1.5044394477676808e-24+0j), (-2.236166980751353e-19+0j), (2.4640084022801225e-25+0j), (1.1529202992621703e-24+0j), (8.767688815340362e-18+0j), (4.147285067993868e-19+0j), (1.4685722722769575e-23+0j), (8.682087709356578e-21+0j), (-4.480976925731097e-23+0j), (7.712089560938459e-26+0j), (5.930864599285896e-24+0j), (7.273959680852609e-26+0j), (1.0742721190132476e-33+0j), (-1.9142944607947188e-19+0j), (-7.813300832175308e-24+0j), (1.2333377009200996e-24+0j), (-6.630497415130703e-25+0j), (-3.1856589098543066e-26+0j), (2.4280428155184568e-34+0j), (3.5101529149217364e-25+0j), (-5.623783083637199e-35+0j), (-4.6536526614358525e-39+0j), (1.0894432463243065e-19+0j), (1.249373597200093e-20+0j), (3.105887137764609e-23+0j), (2.4775713707188285e-20+0j), (-5.740700423601922e-24+0j), (3.4859566648713393e-25+0j), (1.1002148381808825e-23+0j), (2.0662543292117198e-26+0j), (-3.897740443477523e-34+0j), (-3.7947076036992655e-19+0j), (5.13200777689162e-24+0j), (5.536847927472397e-26+0j), (6.026140009419518e-25+0j), (-1.5671030705081982e-26+0j), (-1.0502914028874369e-34+0j), (1.0826667366949322e-25+0j), (5.495368648401703e-35+0j), (1.870277146273814e-40+0j), (9.666057822079618e-24+0j), (6.524433665481796e-26+0j), (2.693038042690927e-35+0j), (4.116853344010872e-26+0j), (7.978246036895801e-36+0j), (1.3475885861087583e-40+0j), (2.0791512277829647e-34+0j), (-6.224359311610392e-42+0j), (-1.6539179947453653e-62+0j), (0.0004452308854351643+0j), (5.313311536434035e-19+0j), (-3.876022766635678e-18+0j), (4.653971853979373e-19+0j), (6.776263578034403e-20+0j), (-1.1123924328148577e-23+0j), (-1.463672932855431e-18+0j), (-2.298073330031396e-23+0j), (3.3129167903651014e-23+0j), (-5.1058033825493255e-20+0j), (2.236166980751353e-19+0j), (1.5702753460104983e-23+0j), (-2.727446090158847e-19+0j), (3.320971770385649e-24+0j), (7.637248462559956e-25+0j), (2.926824250873325e-23+0j), (7.819789394987971e-25+0j), (4.211982685053457e-34+0j), (-5.149960319306146e-19+0j), (-4.374135688835954e-24+0j), (5.3663417180345146e-24+0j), (-2.86021016117962e-25+0j), (5.014093658224794e-26+0j), (-7.493973932808823e-35+0j), (5.3208361490758335e-24+0j), (-1.3503045952914661e-34+0j), (-4.847793643031855e-39+0j), (2.7140590069292945e-19+0j), (-2.498747194400186e-20+0j), (1.6379857165619048e-23+0j), (1.6432439176733427e-19+0j), (1.8345509818256625e-24+0j), (-1.0216082744346078e-24+0j), (1.8110731770936407e-23+0j), (1.185967401137316e-25+0j), (-9.872472875682485e-34+0j), (-9.317362419797304e-20+0j), (-3.4107472087214055e-24+0j), (1.523963530126987e-26+0j), (2.7493254051017868e-24+0j), (1.8495195884196546e-25+0j), (-1.400854640591041e-35+0j), (5.58033221142014e-26+0j), (3.0165443621333533e-35+0j), (2.336973448319752e-40+0j), (3.5736787401705024e-24+0j), (5.579887575735461e-26+0j), (-1.0396788355542343e-35+0j), (1.1724897511692072e-25+0j), (1.4643443933950545e-35+0j), (2.269398160153075e-40+0j), (4.609458148814374e-34+0j), (1.698216089632818e-42+0j), (-1.4153435018376434e-62+0j), (3.2017845406212553e-19+0j), (1.6543612251060553e-24+0j), (7.194897625189282e-24+0j), (7.797631301732789e-24+0j), (-6.352016751057219e-26+0j), (-2.8374686458535615e-34+0j), (4.3651300262488e-25+0j), (4.5287047720115817e-35+0j), (-1.4041932971946233e-40+0j), (9.234696057330286e-24+0j), (1.5316468205876044e-25+0j), (1.9770896625869296e-34+0j), (1.1689693016283165e-26+0j), (-4.811613847194222e-35+0j), (2.356806974836982e-41+0j), (1.7333648062373703e-34+0j), (-1.279105238235693e-41+0j), (4.61959394075303e-64+0j), (-2.4106941126598934e-25+0j), (4.982533337375704e-35+0j), (1.8536525573331027e-40+0j), (3.6052683623010722e-34+0j), (7.20267410662956e-42+0j), (2.084640377016439e-64+0j), (-1.9424799312470614e-41+0j), (-9.078808529013167e-66+0j), (1.691977911113569e-89+0j), (1.6576180667882481e-16+0j), (-0.003397565901914831+0j), (1.130330416727005e-18+0j), (0.0019360494339501345+0j), (-1.9403146183331572e-19+0j), (2.3854565377049233e-19+0j), (2.731779522511997e-18+0j), (1.2213156308222943e-19+0j), (-4.632146293926825e-22+0j), (0.002502980761256917+0j), (-2.4932201653248443e-19+0j), (2.600391148070702e-19+0j), (4.1556735436777604e-19+0j), (2.5278639519620526e-21+0j), (-9.331165228613636e-24+0j), (1.8677076486957322e-19+0j), (-1.0612285037456083e-23+0j), (3.820961932750195e-25+0j), (1.3183548300575515e-18+0j), (1.4743667238145165e-19+0j), (8.910241269515099e-23+0j), (1.2260801911505997e-19+0j), (-3.860308453313897e-24+0j), (5.035231365291676e-27+0j), (3.772511465514064e-24+0j), (1.0252993434397432e-25+0j), (-3.2226434767182254e-34+0j), (-0.0006811492076234078+0j), (1.5314858441267827e-19+0j), (1.6263032587282567e-19+0j), (5.127002821498262e-20+0j), (3.218725199566341e-20+0j), (-7.585995533265632e-24+0j), (1.5754812818929986e-19+0j), (-6.683227008052128e-25+0j), (9.981966623269168e-25+0j), (3.6733434346428805e-20+0j), (8.152692117322641e-21+0j), (-1.3014999743393397e-23+0j), (1.4293680984916318e-20+0j), (9.679348803102643e-25+0j), (1.4068833483571905e-25+0j), (2.013829362397166e-24+0j), (-1.989281110828983e-26+0j), (-4.181308022215505e-35+0j), (-1.0672615135404184e-19+0j), (-3.270756378052957e-24+0j), (5.315857262596624e-25+0j), (2.824046310044321e-24+0j), (1.8230277554221062e-26+0j), (6.370646735579082e-35+0j), (1.5930999035794086e-25+0j), (-8.453468900780111e-36+0j), (1.2791206455091828e-40+0j), (8.335148773081584e-19+0j), (3.3947492339176256e-20+0j), (3.547167474993384e-23+0j), (3.22931311140702e-20+0j), (1.106677186716453e-25+0j), (1.3651765722617445e-25+0j), (3.3239089692707014e-23+0j), (2.267021240566468e-28+0j), (1.5257009568559991e-33+0j), (4.658681209898652e-20+0j), (1.7367561689345796e-26+0j), (5.056716486046401e-26+0j), (-9.17249595267542e-25+0j), (4.822489698025621e-27+0j), (1.0015493115212873e-34+0j), (2.4506856470060788e-26+0j), (-7.864203569823105e-37+0j), (6.863560201698805e-42+0j), (1.7271434254879184e-23+0j), (3.285528827145366e-27+0j), (7.541759270496561e-34+0j), (1.0479921940253104e-26+0j), (4.1416289535510954e-35+0j), (1.894411326363893e-42+0j), (-1.486526353486352e-34+0j), (9.921193127419705e-43+0j), (1.0456584784655566e-64+0j), (-0.0017627013305946033+0j), (4.912900020883588e-19+0j), (-8.131516293641283e-20+0j), (-1.9148172070185447e-19+0j), (-1.4293680984916318e-20+0j), (1.1630892311195062e-25+0j), (1.9989977555201488e-19+0j), (-2.3244935169181365e-24+0j), (1.2427938605615295e-24+0j), (-1.4171242905086183e-19+0j), (-1.9058241313221758e-21+0j), (-1.0395139952347445e-23+0j), (4.129285617864714e-21+0j), (7.660935644473889e-25+0j), (6.25118895054068e-27+0j), (2.567975749322339e-24+0j), (5.938146975710555e-26+0j), (-1.0407831573955341e-35+0j), (-2.0837010502455788e-19+0j), (-6.731914935481378e-25+0j), (3.0511772648040583e-25+0j), (6.246263756412389e-25+0j), (3.760336236259042e-26+0j), (-5.5735938370431545e-36+0j), (1.7917186909636609e-25+0j), (4.440148664024207e-36+0j), (2.032041015274523e-41+0j), (5.995480688486144e-20+0j), (-3.983701830055381e-21+0j), (-1.8327152778246464e-24+0j), (-1.4823076576950256e-20+0j), (2.599075801649772e-25+0j), (9.319993661948558e-26+0j), (2.1398451588407913e-24+0j), (7.286135492715828e-27+0j), (-7.093435712203148e-35+0j), (3.801060350803673e-20+0j), (5.19663699036152e-25+0j), (5.178425420038924e-26+0j), (-2.4506437337233864e-25+0j), (4.0038716735403226e-28+0j), (-2.036943375461419e-35+0j), (3.390761065740673e-26+0j), (-7.278877838158315e-37+0j), (6.333545407523244e-42+0j), (-2.0235228852005218e-25+0j), (3.0447256366731476e-27+0j), (1.432296488554319e-34+0j), (1.3397646665515403e-26+0j), (-5.353586121770784e-37+0j), (9.449107627089488e-42+0j), (3.282447566138873e-35+0j), (-6.5300508437536476e-43+0j), (3.638177281884271e-64+0j), (9.910285482875314e-20+0j), (1.7130272329055316e-25+0j), (3.957848276753622e-25+0j), (2.0396787565394481e-25+0j), (9.976354777680108e-27+0j), (-2.962187649422252e-36+0j), (7.119165235804452e-26+0j), (3.1235942736384315e-36+0j), (1.424840278525474e-41+0j), (-9.735427094641133e-25+0j), (1.020751518023566e-26+0j), (4.0467301337511435e-35+0j), (-8.820851606237916e-27+0j), (1.858225123469436e-36+0j), (-6.488011889823903e-43+0j), (3.8010921897616783e-35+0j), (6.221765181602188e-43+0j), (-9.922463231052328e-66+0j), (7.545588956271325e-27+0j), (9.594369878460283e-36+0j), (6.793494955046713e-42+0j), (9.715745956968115e-36+0j), (-9.7740567886656e-44+0j), (-6.411386166307737e-66+0j), (-5.4140917792269714e-42+0j), (1.9814474884356773e-67+0j), (-1.6479230463428374e-91+0j), (2.5327536259872116e-19+0j), (1.2943722225229777e-20+0j), (-1.5777373458874757e-23+0j), (7.835054762102278e-21+0j), (-7.2923564256078685e-25+0j), (1.3036936958107854e-26+0j), (-5.62224322594636e-25+0j), (9.52388701572942e-27+0j), (3.5652130658503126e-34+0j), (2.8904999325053e-20+0j), (2.7454883856587918e-24+0j), (1.421552559599981e-26+0j), (6.000391589473104e-25+0j), (-1.9601420721367318e-27+0j), (3.216903442523833e-36+0j), (1.0251216649972571e-26+0j), (-1.8015536450614736e-35+0j), (-1.0830697170430473e-41+0j), (5.838731901887973e-24+0j), (4.187933540673603e-27+0j), (6.617239908634229e-34+0j), (5.6039679416919605e-27+0j), (1.9270058833205044e-35+0j), (-7.067884136870472e-42+0j), (3.2618418726560256e-34+0j), (3.3350903450930646e-43+0j), (7.522067073391272e-65+0j), (-1.1329065669526267e-20+0j), (-9.935860873439688e-26+0j), (2.535421477614837e-25+0j), (1.4040966806464998e-24+0j), (9.641260985663277e-27+0j), (6.509825080175589e-35+0j), (-4.672848978887415e-26+0j), (-4.3743427913084966e-36+0j), (4.51778624898321e-42+0j), (6.694589186067595e-25+0j), (8.367811174916307e-27+0j), (3.2576787974236286e-35+0j), (8.507344379456499e-27+0j), (5.8454512546329246e-36+0j), (4.2319213622609476e-43+0j), (3.328170648077282e-35+0j), (6.060615858204834e-44+0j), (-1.0312841228389927e-65+0j), (1.1732843818003701e-26+0j), (8.048905559076424e-36+0j), (-2.6400463067879554e-42+0j), (1.2411079183050643e-35+0j), (3.7432185228276676e-43+0j), (-2.6515816491983034e-66+0j), (-3.0049093944365296e-42+0j), (1.2703299543937767e-67+0j), (-1.3266381882387373e-91+0j), (-3.684043536254543e-25+0j), (-1.9106158392893048e-28+0j), (3.4947613645276357e-34+0j), (1.7199515376926863e-27+0j), (2.6794370106491824e-35+0j), (1.1918043439082569e-42+0j), (1.2162841172360097e-34+0j), (-2.1150848695902708e-44+0j), (-1.08750154686231e-67+0j), (2.5045624306639447e-27+0j), (5.891086814832368e-36+0j), (2.3850099862808387e-42+0j), (8.763174145538258e-36+0j), (-3.0828566215145976e-44+0j), (1.8314425176781207e-67+0j), (-1.1812508148488106e-43+0j), (-4.921028939482732e-68+0j), (2.59592498132023e-93+0j), (-2.6483459745054357e-35+0j), (-9.165367768224507e-44+0j), (-5.994261011885404e-68+0j), (-2.39118445763927e-43+0j), (-2.919678915259167e-69+0j), (1.037942524530525e-93+0j), (-1.4968221818766035e-70+0j), (-2.0740012076033412e-95+0j), (4.933820166180381e-266+0j), (1.2461819770548388e-15+0j), (1.4529664364021366e-16+0j), (0.0044736933074311+0j), (-2.1636270791484946e-16+0j), (-0.12030823555454606+0j), (5.087524647526594e-19+0j), (0.0010978009855655863+0j), (-2.670055005149752e-19+0j), (1.7889335846010823e-18+0j), (1.2476727550420064e-15+0j), (-0.06575344087275493+0j), (3.018018353834264e-20+0j), (0.16604080378327862+0j), (-1.1444952707512024e-17+0j), (-6.268043809681823e-20+0j), (7.881672461622223e-19+0j), (2.498747194400186e-20+0j), (-1.3208130486466867e-23+0j), (0.00012071348421156418+0j), (4.1771007445169945e-19+0j), (-2.0599841277224584e-18+0j), (8.436822350410198e-19+0j), (-2.541098841762901e-20+0j), (-3.3032315082307916e-23+0j), (-5.095750210681871e-18+0j), (-4.093886296901416e-23+0j), (8.88596796544092e-24+0j), (-2.946319403729358e-17+0j), (0.18607511192383036+0j), (-6.733019385418304e-19+0j), (-0.08345891431933039+0j), (2.044708061168368e-17+0j), (1.3129010682441655e-19+0j), (1.7719294284624246e-19+0j), (1.4293680984916318e-20+0j), (1.9699760628259522e-23+0j), (-0.14619788616024298+0j), (8.34233788055059e-18+0j), (-1.1180834903756764e-19+0j), (-2.1777918029367044e-17+0j), (-3.0556713572198885e-19+0j), (-1.5596096986228744e-23+0j), (-2.236166980751353e-19+0j), (-5.449257947146073e-24+0j), (-7.775921751013494e-25+0j), (4.2988642901399935e-19+0j), (1.598774687942492e-20+0j), (-4.3638908463732084e-23+0j), (-2.439454888092385e-19+0j), (1.2881393560876056e-24+0j), (1.3854877230632723e-24+0j), (-2.906383573089531e-24+0j), (-4.665746481767247e-26+0j), (-4.824378186209169e-33+0j), (0.005609945152117701+0j), (-9.829893506905077e-19+0j), (-1.2468324983583301e-18+0j), (7.939641042963869e-19+0j), (-1.6771252355635147e-19+0j), (-6.643999705649458e-24+0j), (1.0842021724855044e-18+0j), (-1.56608299177701e-23+0j), (2.683409381321787e-23+0j), (1.002238393874498e-18+0j), (-1.2536087619363645e-19+0j), (1.4029049354420478e-23+0j), (-3.3881317890172014e-20+0j), (-4.27076244123177e-24+0j), (-1.7255820004344426e-26+0j), (1.7107972452320994e-23+0j), (3.488941072431652e-25+0j), (3.3495725302005506e-34+0j), (9.215718466126788e-19+0j), (-6.093489945510908e-24+0j), (5.4076269088595804e-24+0j), (4.237708958634344e-24+0j), (4.138400969828884e-25+0j), (-1.2162463522423961e-34+0j), (2.440131305688233e-24+0j), (-1.1547487122401973e-34+0j), (2.0326576724700787e-39+0j), (1.5421420650890694e-16+0j), (-0.009468597595093433+0j), (1.055562854205952e-18+0j), (-0.05938999986795371+0j), (1.568814700304453e-18+0j), (3.655807435239361e-20+0j), (-3.177176058453583e-19+0j), (2.345222472710344e-20+0j), (-1.698856319532377e-22+0j), (0.03327433398467713+0j), (-1.5335118145859359e-18+0j), (3.032377951170395e-19+0j), (3.260356918454871e-18+0j), (-7.999167395632799e-20+0j), (6.347572919020329e-24+0j), (-1.1858461261560205e-19+0j), (-5.678800045789451e-24+0j), (1.9475854396455961e-25+0j), (2.256412217927166e-19+0j), (1.3870164511289168e-20+0j), (-2.783262876470769e-23+0j), (4.2351647362715017e-20+0j), (-2.456974053972947e-24+0j), (-1.9123945472566812e-26+0j), (7.144271392432453e-24+0j), (7.49336624234907e-27+0j), (4.74139319824016e-34+0j), (0.058202566776118396+0j), (-4.699160266719428e-18+0j), (1.9820570965750628e-19+0j), (8.541133044639848e-18+0j), (1.5564230405797769e-19+0j), (-5.924048760689937e-24+0j), (-5.082197683525802e-21+0j), (-1.0566572085992012e-24+0j), (4.111029061235795e-25+0j), (-1.7351135805014485e-18+0j), (3.3034284942917713e-20+0j), (1.4198827202789327e-24+0j), (-4.5528020914918643e-20+0j), (1.429082503906706e-23+0j), (-2.8448876099446e-26+0j), (8.105971738834856e-25+0j), (1.3734946246914716e-26+0j), (4.804566491664131e-34+0j), (9.317362419797304e-20+0j), (5.635135838838933e-25+0j), (2.263786857049394e-25+0j), (-6.215752001154579e-25+0j), (1.924731204663838e-26+0j), (-6.537014198503346e-35+0j), (2.8234752859413027e-25+0j), (-3.052597686488316e-35+0j), (-2.090533083052323e-39+0j), (-1.048067623213949e-19+0j), (-1.6623021589865644e-20+0j), (-2.1994641599764875e-23+0j), (3.356368053495165e-20+0j), (2.2257113399172287e-24+0j), (6.049107775564085e-26+0j), (-5.850041011825221e-24+0j), (4.011780781709378e-27+0j), (-6.266326203718412e-35+0j), (-7.538593230563273e-20+0j), (4.530504385665886e-25+0j), (6.046638121772253e-25+0j), (-3.322051044066725e-26+0j), (1.0137184485076364e-26+0j), (-3.516510578541881e-35+0j), (2.8895071087725473e-26+0j), (-4.068500455580209e-36+0j), (8.285976721518984e-41+0j), (6.0398725000576054e-24+0j), (2.3141807673735933e-26+0j), (1.6570246487742837e-34+0j), (1.2690488360906839e-26+0j), (-5.375756143247581e-36+0j), (9.307493014663877e-41+0j), (3.3358598261057947e-35+0j), (-3.9801686161250533e-42+0j), (-1.055913273276998e-62+0j), (-0.0019391562223237046+0j), (1.9148873067273353e-19+0j), (-4.87890977618477e-18+0j), (-1.4423669517310678e-19+0j), (1.6940658945086007e-21+0j), (-9.79404371165046e-24+0j), (2.846030702774449e-19+0j), (-2.11455594859669e-24+0j), (9.542939165458666e-24+0j), (8.356056556617493e-20+0j), (1.1011428314305904e-19+0j), (-2.2975544581493057e-26+0j), (-2.710505431213761e-20+0j), (9.038322141209814e-26+0j), (2.0125939965177178e-25+0j), (-5.845003495832583e-24+0j), (1.714264863413779e-25+0j), (-9.529632831014213e-35+0j), (-2.913793338554793e-19+0j), (3.484013654239461e-24+0j), (1.6565704040755864e-24+0j), (6.673182656543517e-24+0j), (1.0841460769482488e-25+0j), (5.745976237979841e-35+0j), (2.7529668162813202e-24+0j), (5.630355635302856e-35+0j), (-1.0443177592764654e-40+0j), (5.636316899513634e-20+0j), (-9.317362419797304e-21+0j), (7.875284661745591e-24+0j), 0j, (3.1210615972564806e-25+0j), (1.9801218466255452e-26+0j), (-2.9532932807557316e-24+0j), (1.4379457058240288e-26+0j), (1.0136474901964007e-35+0j), (-2.541098841762901e-21+0j), (2.0538151439610087e-25+0j), (2.5965620800831216e-25+0j), (3.482095144517964e-25+0j), (3.4699898481120614e-26+0j), (-1.1088985493466555e-36+0j), (2.6436455387775373e-26+0j), (9.153627028277518e-36+0j), (8.421898666487194e-41+0j), (-1.852674545791376e-24+0j), (9.453517406276328e-27+0j), (1.7060584900322902e-34+0j), (-9.401071801344509e-26+0j), (5.493073478462567e-36+0j), (3.28624590974012e-42+0j), (6.279872082894678e-35+0j), (9.320809552993358e-43+0j), (2.52364221448789e-64+0j), (5.421010862427522e-20+0j), (1.3865776576633516e-24+0j), (-6.049340532653901e-25+0j), (2.312006162796225e-24+0j), (1.0117817523948075e-25+0j), (3.679934426828356e-37+0j), (6.105434511682073e-25+0j), (6.954305975847771e-35+0j), (-5.779727125431862e-41+0j), (3.919616335215262e-24+0j), (-1.9135169163769382e-26+0j), (7.445039962652888e-35+0j), (1.2751663248604845e-26+0j), (3.659138709202267e-37+0j), (-4.9347599663517104e-42+0j), (1.5043763094611966e-34+0j), (6.148897661457297e-42+0j), (1.652738138882169e-64+0j), (-7.568737690170135e-25+0j), (9.525078830728757e-36+0j), (-1.3588110948864886e-40+0j), (6.449126978477661e-35+0j), (3.712740281228603e-42+0j), (-6.808468391171699e-66+0j), (3.214999066700428e-41+0j), (-2.159396088618285e-66+0j), (-1.1055878425825642e-91+0j), (6.533080418877693e-16+0j), (-0.09126404075949228+0j), (1.3074242793958495e-18+0j), (0.07096324625342894+0j), (-1.621426992864673e-17+0j), (4.412512259602871e-20+0j), (-1.6228691229791757e-19+0j), (2.0699367648526965e-20+0j), (-7.591530892233192e-23+0j), (0.05522986393719635+0j), (-3.161189148944321e-18+0j), (1.545835128739098e-20+0j), (1.6197445364528354e-17+0j), (3.486334671339497e-19+0j), (2.2619679241713123e-24+0j), (2.0752307207730358e-20+0j), (-4.291786509365016e-25+0j), (3.3332967801235953e-26+0j), (-2.5036583396785544e-21+0j), (4.3556022334592225e-20+0j), (2.597669318441001e-23+0j), (1.3023131564034868e-20+0j), (-1.5532118642621271e-24+0j), (1.6502575128991973e-26+0j), (-1.9431006597513098e-23+0j), (-1.0896519891192292e-26+0j), (-3.5312865080934063e-34+0j), (-0.05054697892666424+0j), (1.1765162430732036e-17+0j), (-3.4728350837426314e-19+0j), (-6.574060740844462e-18+0j), (9.883815703273617e-20+0j), (2.618033128919365e-24+0j), (-1.0926725019580474e-19+0j), (-1.3973592708188414e-24+0j), (2.5774934975070883e-25+0j), (-7.367073669915513e-18+0j), (-4.199165836013194e-19+0j), (-2.0410548307492904e-24+0j), (1.0264980529538052e-19+0j), (1.9441070963067096e-25+0j), (2.9118901349308485e-26+0j), (1.157568181434071e-24+0j), (8.850165567761281e-28+0j), (-2.9902343534418653e-35+0j), (1.3891340334970526e-19+0j), (7.676338712060338e-25+0j), (1.9156410098452545e-25+0j), (5.602048386772702e-25+0j), (3.4100252633660616e-26+0j), (-2.3249952384289066e-35+0j), (9.163106448121214e-26+0j), (5.878968385712887e-36+0j), (7.276156522409753e-41+0j), (4.989254001181445e-19+0j), (7.583591855886158e-21+0j), (-1.9657717647357453e-23+0j), (2.159934015498466e-20+0j), (5.769665551914063e-25+0j), (2.7065646972591594e-26+0j), (8.320273739547056e-26+0j), (-1.0577307989738587e-28+0j), (1.4059233860071112e-35+0j), (1.725829630030637e-20+0j), (3.4109083364308197e-25+0j), (3.3501992917102046e-26+0j), (3.155443620884047e-26+0j), (1.765325397479586e-27+0j), (-6.885682726428248e-36+0j), (2.3858939056640347e-26+0j), (4.2684679436753525e-36+0j), (-2.973924775086074e-43+0j), (1.9253759668165444e-24+0j), (3.759024785134383e-27+0j), (1.8011245293563791e-34+0j), (7.034281696941826e-27+0j), (5.4377594524505e-36+0j), (-6.905036475958035e-42+0j), (7.453018548050641e-35+0j), (3.3210773604498165e-43+0j), (2.2790032527900556e-64+0j), (-0.015865473020100506+0j), (-1.0343771117606038e-19+0j), (-4.404571325722362e-20+0j), (-5.341957382421164e-18+0j), (-5.403011412298368e-19+0j), (-2.9253351544502025e-24+0j), (2.253107639696439e-19+0j), (2.0090652504029155e-24+0j), (3.4232548088584554e-25+0j), (2.514156311976923e-18+0j), (-2.338869725605937e-19+0j), (-7.964350864522022e-24+0j), (2.687212025164268e-19+0j), (-3.277109911732092e-24+0j), (3.6330083405656786e-26+0j), (3.6168956960021303e-25+0j), (1.3927450305519726e-26+0j), (-4.051562833243495e-35+0j), (-1.4568966692773966e-19+0j), (-5.573775611929581e-25+0j), (7.621018071168552e-25+0j), (7.209557584995871e-25+0j), (1.961186895751946e-26+0j), (-4.0487084712285345e-36+0j), (7.088127732320606e-27+0j), (-2.2857705761042756e-35+0j), (1.6929902271537956e-40+0j), (2.524488577581178e-18+0j), (3.7280037591029894e-19+0j), (-3.741686141682315e-24+0j), (-3.415130964210932e-19+0j), (-2.829953300502616e-24+0j), (1.4281563304563243e-25+0j), (1.5156226799830256e-24+0j), (-3.8855517987718644e-28+0j), (-2.6822278568494296e-36+0j), (-1.291195848970774e-19+0j), (4.489767132185559e-24+0j), (8.282533478012482e-27+0j), (-4.222867088956703e-24+0j), (-1.0806261819566924e-25+0j), (-7.66647026715793e-39+0j), (1.4834159800060435e-26+0j), (-1.9315480095632935e-37+0j), (-2.2276425404768937e-42+0j), (-2.88786200183308e-25+0j), (1.70471834008013e-27+0j), (3.246088583836639e-35+0j), (1.7731851583417214e-26+0j), (1.0810202432871358e-36+0j), (-2.075724975159897e-42+0j), (1.301665336363407e-35+0j), (8.975316664000453e-43+0j), (4.1427336753513636e-64+0j), (2.5199230180815435e-20+0j), (1.0529084274165889e-25+0j), (5.052969878229087e-27+0j), (-5.371322349213661e-25+0j), (1.9578059002404528e-26+0j), (4.5379197465862223e-36+0j), (-5.1540168686470154e-26+0j), (-5.364852113008447e-38+0j), (8.553525826238683e-42+0j), (7.18481890700814e-25+0j), (1.9486672618521683e-26+0j), (5.817759023072847e-35+0j), (-3.0029580211760384e-27+0j), (2.8862558370579023e-36+0j), (-2.1019476964872256e-43+0j), (2.026692200087465e-35+0j), (-4.004210361808165e-43+0j), (-1.7651980362724946e-65+0j), (5.185561675820403e-26+0j), (3.059199474845133e-36+0j), (-1.1073060465094704e-41+0j), (-8.299688535170357e-36+0j), (-8.099505123797443e-43+0j), (-9.5645339103795e-66+0j), (4.4806518396786026e-42+0j), (4.670758314858223e-67+0j), (-2.3123286479887886e-91+0j), (-4.637232380967731e-20+0j), (1.0587911840678754e-21+0j), (1.0387417477362679e-23+0j), (1.8131799027162367e-21+0j), (1.9818710294048524e-25+0j), (1.8492662473216129e-26+0j), (7.615473852386395e-25+0j), (-1.7051023655739647e-28+0j), (1.217042881678604e-34+0j), (7.517417406881916e-21+0j), (-4.613006138242806e-25+0j), (-5.094408691904896e-27+0j), (2.9930013832809365e-25+0j), (2.197891495299751e-27+0j), (7.863683228466944e-36+0j), (1.2665043442328347e-26+0j), (-1.4557013100234415e-36+0j), (4.531039893075071e-42+0j), (3.0112019820861957e-24+0j), (1.1041429509152181e-27+0j), (9.597456860504477e-35+0j), (2.946629831661877e-27+0j), (1.5027599616809272e-36+0j), (1.2163270670339412e-42+0j), (6.019528092220208e-35+0j), (-1.7446165880843973e-43+0j), (-6.002029092260537e-66+0j), (-8.179161896924338e-21+0j), (-1.5408031200776803e-25+0j), (1.5718225874610503e-25+0j), (3.939129598566809e-25+0j), (-6.319078053831676e-27+0j), (-8.203410889249293e-36+0j), (2.2425940749728238e-26+0j), (2.2853547962764064e-37+0j), (3.7498746905332105e-42+0j), (6.902848465045942e-26+0j), (-3.818451250003773e-27+0j), (-3.568585098458665e-36+0j), (-2.9789422636172616e-27+0j), (1.5147276335064223e-37+0j), (-8.477855709165143e-44+0j), (1.58372779465805e-35+0j), (-2.368194404708941e-43+0j), (-1.0014649959094886e-65+0j), (-4.0428743414711875e-26+0j), (2.8832865528743243e-36+0j), (-7.274490652926207e-42+0j), (5.255734834489865e-36+0j), (1.9526218288826123e-43+0j), (3.972208538161278e-67+0j), (-1.7027965870397035e-42+0j), (1.6073279694310985e-67+0j), (5.063194408918828e-93+0j), (1.1523806321213376e-24+0j), (1.1675423387418946e-28+0j), (6.994559769701223e-35+0j), (4.040022417886383e-28+0j), (3.451085118001839e-36+0j), (5.9555184733804726e-43+0j), (1.7692019478501384e-35+0j), (2.848577034510292e-44+0j), (-8.636717400878458e-68+0j), (-5.6095475943166805e-27+0j), (1.2863039494702655e-36+0j), (7.684370453741216e-43+0j), (2.0058987641956656e-36+0j), (5.640226318907389e-44+0j), (2.9671388194391234e-67+0j), (-1.2871145347708496e-43+0j), (-1.9289842829178148e-68+0j), (3.302753073909329e-93+0j), (3.505539404035855e-35+0j), (3.697019464081959e-44+0j), (-6.90965571996224e-68+0j), (-1.4102481635012666e-43+0j), (-7.362298261177475e-69+0j), (-1.0147985581562153e-94+0j), (-7.486893529918105e-69+0j), (-3.782210876746198e-95+0j), (9.844444565266257e-266+0j), (0.001687083631506418+0j), (-3.1160175730113884e-19+0j), (-1.328147661294743e-18+0j), (3.08064220153946e-19+0j), (-3.134021904840911e-20+0j), (1.9761057189075976e-24+0j), (-3.015437292225309e-19+0j), (2.9403685836845906e-25+0j), (4.226478918583542e-24+0j), (1.6485984002141948e-19+0j), (6.437450399132683e-20+0j), (3.830936642145697e-25+0j), (-4.086933970501999e-20+0j), (1.7213576040646654e-25+0j), (1.65302500417358e-25+0j), (-9.367376150701218e-25+0j), (2.4943784961338576e-25+0j), (4.4943427992085163e-35+0j), (-6.06475590234079e-19+0j), (-6.0887440108578575e-25+0j), (1.3609821218505434e-24+0j), (4.736699528181461e-25+0j), (-1.4040494800502146e-26+0j), (-4.6667911351615853e-35+0j), (6.638921988562056e-25+0j), (1.9737097575959087e-35+0j), (5.099089903770294e-40+0j), (-3.4197973468752984e-20+0j), (-9.343832199399e-21+0j), (1.3142801334216563e-24+0j), (3.128727948920572e-20+0j), (4.800565707068154e-25+0j), (1.3331452443368025e-25+0j), (1.0181228169399631e-24+0j), (2.746020059538097e-27+0j), (1.1429817296724e-34+0j), (-1.8714134178399698e-20+0j), (-1.4504943236479788e-25+0j), (5.431944685869244e-26+0j), (2.8460839282925752e-25+0j), (5.82247841849629e-27+0j), (-2.4658931130078705e-36+0j), (-1.0857914944772744e-26+0j), (1.44581868062976e-36+0j), (-1.9562126561974446e-41+0j), (2.203256953846077e-24+0j), (2.2639135860924245e-27+0j), (8.03163991182687e-35+0j), (-7.607844294272785e-26+0j), (8.782041822212476e-36+0j), (-5.061490053141239e-42+0j), (-4.590456376621222e-36+0j), (3.5733110840282835e-43+0j), (-7.860354251235684e-66+0j), (-2.448454613156962e-19+0j), (5.398206728863593e-25+0j), (8.120099723050967e-25+0j), (4.3295210833425835e-25+0j), (-5.858608794704283e-26+0j), (4.857743372949611e-35+0j), (-5.478429412404722e-25+0j), (1.5645223192944634e-35+0j), (-5.983824702359834e-41+0j), (8.654750763360765e-26+0j), (-5.602971159407533e-26+0j), (3.956648919777524e-35+0j), (-4.4632700665417556e-26+0j), (9.007083920820494e-36+0j), (4.1548499467230826e-42+0j), (3.426376489093048e-35+0j), (-3.758983130551322e-43+0j), (7.051776526170965e-68+0j), (2.6613758067990307e-25+0j), (2.1663695795772427e-35+0j), (1.922581493053649e-41+0j), (3.8880102953112505e-35+0j), (-1.8908771152983e-42+0j), (-4.093918134842933e-68+0j), (-2.8314987094763335e-42+0j), (-1.494116091302421e-67+0j), (2.720777251641042e-93+0j), (-9.846362914071327e-21+0j), (-1.1527589016538994e-20+0j), (-1.3449762889656163e-25+0j), (6.392451773809798e-21+0j), (5.264163483848438e-25+0j), (3.526792716799321e-27+0j), (-1.4836138598927778e-24+0j), (-7.60707888140859e-28+0j), (5.010349376457702e-35+0j), (8.523269031746397e-21+0j), (-1.9987842072127909e-25+0j), (1.698980044795339e-26+0j), (2.428366301759945e-25+0j), (6.2727781704890265e-27+0j), (1.1450598790610156e-35+0j), (-1.8311869984531154e-26+0j), (8.22257980025179e-36+0j), (-7.920138920363866e-42+0j), (4.833634756215019e-25+0j), (-3.2036623743959277e-27+0j), (7.310933036028882e-35+0j), (-7.20884008440795e-27+0j), (4.020557057703512e-36+0j), (7.707141553786494e-44+0j), (1.423767374916783e-35+0j), (-4.333515500924497e-43+0j), (-3.5845140361992844e-66+0j), (8.880611056369305e-21+0j), (3.7644442397146683e-26+0j), (-1.6564097157915864e-26+0j), (1.5155595711106079e-25+0j), (2.0487203299435876e-27+0j), (2.3378431319611316e-36+0j), (-2.8409389464757815e-26+0j), (1.40457314749571e-36+0j), (5.580671134173584e-42+0j), (-9.195909344342379e-26+0j), (3.3384800095873624e-27+0j), (1.7001477153551872e-36+0j), (-1.2792249589386048e-27+0j), (-5.679143235910556e-37+0j), (2.1019476964872256e-44+0j), (1.45967736045183e-35+0j), (6.892636821397694e-44+0j), (5.309519017140844e-67+0j), (-2.1540547416911928e-26+0j), (2.070068111582285e-36+0j), (-1.7498714573256153e-42+0j), (-1.8702814050020242e-37+0j), (-1.4459648528751706e-43+0j), (1.56122423207596e-67+0j), (-7.174867090228114e-43+0j), (-3.0062241094603132e-68+0j), (3.152276496086317e-93+0j), (3.986650579497323e-25+0j), (1.9647317984360187e-28+0j), (1.0588526040177282e-34+0j), (-1.4653088284565344e-27+0j), (1.0466186266294759e-36+0j), (-1.0334576174395526e-44+0j), (-7.562672838192305e-36+0j), (-3.5262362137423717e-44+0j), (-1.1135182664831626e-67+0j), (-2.7479149546817735e-26+0j), (1.3095709816537545e-36+0j), (-1.3274675514857033e-42+0j), (2.790258067924898e-36+0j), (-1.0153940044228655e-43+0j), (-3.739564036300795e-68+0j), (-1.0637606967305768e-42+0j), (-2.847323609413714e-69+0j), (-8.886799698722047e-95+0j), (-1.589608832331396e-36+0j), (-3.4298969443200405e-43+0j), (-7.441912970271341e-69+0j), (5.241184685902392e-46+0j), (-1.4760631740164158e-69+0j), (-3.684589741229032e-95+0j), (3.019127896551882e-70+0j), (2.2580951825708097e-96+0j), (-1.4682502180326325e-267+0j), (8.762820537141754e-20+0j), (1.1495281110880584e-26+0j), (-3.6010936357693653e-25+0j), (3.132566654632638e-25+0j), (-5.1110430297287103e-26+0j), (3.092189092162875e-35+0j), (-1.6283939261540858e-25+0j), (1.0994402936640652e-35+0j), (-5.05532433989821e-41+0j), (2.309469186125034e-25+0j), (-4.5002737904458686e-26+0j), (7.614548190577445e-36+0j), (-9.079224155268323e-27+0j), (4.732608520945479e-36+0j), (4.3869400048768804e-43+0j), (3.8571367249946243e-35+0j), (4.041870258036894e-43+0j), (7.31981648817899e-69+0j), (-1.5778430326240255e-25+0j), (1.9180299130909002e-35+0j), (1.430340374997949e-41+0j), (1.3435069580070745e-35+0j), (-6.407437228125226e-43+0j), (-7.023467940419637e-68+0j), (-1.0615164296588065e-41+0j), (-3.728629267498023e-68+0j), (-5.2119807360946526e-95+0j), (3.1583466290152605e-25+0j), (-5.825877406855635e-28+0j), (4.2225189656982616e-35+0j), (-2.5224689996724292e-26+0j), (3.0764589040429455e-36+0j), (1.5236493364911777e-42+0j), (2.941518354946764e-35+0j), (-8.942035825472739e-44+0j), (-1.2653512646249231e-67+0j), (-2.239532942187952e-26+0j), (2.396513917997744e-36+0j), (-3.625859776440464e-44+0j), (1.9831607989911313e-36+0j), (-6.325001466903618e-44+0j), (-1.4104280635724827e-68+0j), (-6.501956451690573e-43+0j), (-9.170779464059505e-69+0j), (-5.9158003032598015e-95+0j), (4.631787399996821e-36+0j), (-4.977620150522548e-44+0j), (-3.871313704753015e-68+0j), (-1.1989599978828218e-42+0j), (-8.885315226709342e-70+0j), (-2.4501850826599e-96+0j), (-1.6048723587615939e-68+0j), (9.847716777561715e-97+0j), (-7.672254729802554e-268+0j), (-3.2142137583230126e-26+0j), (3.371037662182594e-36+0j), (-1.0363259002337174e-42+0j), (1.1339211854761557e-35+0j), (-8.012012919386693e-43+0j), (-2.208768507877714e-68+0j), (-6.333393520450954e-42+0j), (-7.005108188501158e-69+0j), (-4.1835824183647716e-95+0j), (-6.519877899474808e-37+0j), (-8.049843872556869e-43+0j), (-4.039159467305865e-68+0j), (-2.3847766646127064e-43+0j), (-2.182617607755008e-69+0j), (2.17081894939393e-96+0j), (-4.3539507497901226e-70+0j), (7.753508755584521e-97+0j), (2.1139388038460662e-269+0j), (-5.860880878411645e-42+0j), (-1.6275455010653867e-69+0j), (4.931609060124658e-96+0j), (-2.5324037196255484e-69+0j), (-2.969631637235522e-97+0j), (5.864569874483181e-270+0j), (1.1870908212646534e-96+0j), (-2.2420533671161087e-271+0j), 0j, (-7.28084110471379e-17+0j), (0.0021891356947064196+0j), (2.5798784146317937e-18+0j), (0.00012780697328537016+0j), (2.694961946361546e-19+0j), (1.1666555359447902e-19+0j), (2.2923657140653986e-18+0j), (1.0328508000582125e-19+0j), (-1.2099168103227295e-22+0j), (-0.002096360342807301+0j), (2.2629503121180936e-19+0j), (1.0736142606448257e-19+0j), (-4.013314739120883e-19+0j), (1.1653320469647054e-20+0j), (-1.6526514511208498e-23+0j), (9.645587686858345e-20+0j), (-3.34041485747075e-24+0j), (9.624605518818835e-27+0j), (1.064098237057218e-18+0j), (1.2323667638060028e-19+0j), (2.4662213618288204e-23+0j), (7.385068508873431e-20+0j), (2.1040965809447808e-24+0j), (1.4008353765528856e-26+0j), (-1.1532157431819352e-24+0j), (6.051818139182234e-27+0j), (-8.197656993634699e-34+0j), (-0.0004864224193654863+0j), (-1.170832734204666e-19+0j), (-6.776263578034403e-20+0j), (-3.1679451438656147e-20+0j), (-1.1858461261560205e-20+0j), (-1.297460737188508e-23+0j), (1.2197274440461925e-19+0j), (-5.719166743036574e-25+0j), (2.2704618074026334e-25+0j), (2.7680015063518805e-19+0j), (3.65282958503417e-21+0j), (6.094947796551668e-25+0j), (1.011145580784821e-20+0j), (-4.942835978507662e-25+0j), (1.372418421057079e-26+0j), (1.3675945088401131e-24+0j), (9.438850728231387e-27+0j), (2.6321786593870263e-35+0j), (7.369186641112413e-20+0j), (-1.954860432010085e-25+0j), (4.408494827961927e-26+0j), (1.563888345608068e-24+0j), (1.4363248699652958e-26+0j), (-1.947635149109807e-35+0j), (2.1223616354349539e-26+0j), (5.19160409077399e-36+0j), (2.1114568563376524e-40+0j), (3.540705065182067e-19+0j), (1.9362643778641272e-20+0j), (2.048567141147465e-23+0j), (2.789914770018852e-20+0j), (-6.559283763604087e-25+0j), (2.4992644591707442e-26+0j), (1.0234744493209825e-23+0j), (1.2624838280727426e-27+0j), (-4.1790989193554246e-34+0j), (3.2928405824510926e-20+0j), (-1.5550026163716585e-24+0j), (6.519190963836857e-27+0j), (-9.673327964182135e-26+0j), (1.3124022254134624e-27+0j), (6.324826562714754e-35+0j), (-5.5671978349914065e-27+0j), (4.1618011583808085e-35+0j), (1.7849524035389556e-41+0j), (3.6253775284550666e-24+0j), (4.5363831419412144e-27+0j), (5.794986488994772e-34+0j), (2.5608030542189717e-27+0j), (-2.032753183646387e-35+0j), (2.7103942385673108e-42+0j), (7.187753076994105e-34+0j), (1.14906474074635e-43+0j), (8.274827189020223e-65+0j), (0.00045617210294274474+0j), (-3.39526953761102e-21+0j), (-2.947674656444965e-19+0j), (1.3876979220530946e-19+0j), (1.418780186650953e-20+0j), (-1.0863082796683767e-24+0j), (1.1011428314305904e-20+0j), (-9.175442756373634e-26+0j), (1.3893360476428185e-24+0j), (3.728028756533265e-20+0j), (-8.046812998915853e-21+0j), (2.2060842313303892e-24+0j), (-4.235164736271502e-21+0j), (-2.08915611251491e-25+0j), (1.8414720141028924e-26+0j), (-2.6439083446152926e-24+0j), (-2.260046100086881e-27+0j), (-1.8918313098694035e-35+0j), (1.1921988732604277e-19+0j), (-4.75386514147907e-25+0j), (2.2795755053434983e-25+0j), (2.76921732168784e-25+0j), (-2.481760998365842e-27+0j), (-4.2061252736155374e-36+0j), (1.9781769763925432e-25+0j), (-7.002732806000118e-37+0j), (2.856511690760143e-41+0j), (-8.361980619185959e-21+0j), (-6.432156443212343e-21+0j), (-2.742459159782743e-24+0j), (6.776263578034403e-21+0j), (9.077580208559227e-26+0j), (3.7253408170025195e-26+0j), (1.8148092223407674e-24+0j), (-2.348282154113671e-28+0j), (-2.9103433549958856e-35+0j), (9.26442286059391e-22+0j), (8.661061650602533e-26+0j), (1.9341559680761165e-26+0j), (2.393971966292309e-25+0j), (3.6097294238152245e-27+0j), (-8.326950079328983e-36+0j), (-4.7044630071588225e-27+0j), (4.7846675548326735e-36+0j), (-2.1669708668130245e-42+0j), (9.41003774845557e-25+0j), (4.778465715056557e-28+0j), (1.6985131780482276e-35+0j), (1.4699929936046904e-27+0j), (2.2031951651210886e-36+0j), (8.57594660166788e-43+0j), (5.138944886359719e-36+0j), (1.233142648605839e-43+0j), (-8.097890465023387e-66+0j), (-3.5469504666273827e-20+0j), (-7.107321211679228e-26+0j), (2.329819827559972e-25+0j), (-3.7865323450608567e-26+0j), (1.4353615827513093e-27+0j), (3.2081500130638675e-36+0j), (-2.698678007644859e-26+0j), (9.091988101516875e-37+0j), (1.6333534900170068e-41+0j), (4.45397177975025e-25+0j), (1.053720509237579e-27+0j), (4.044234073664162e-35+0j), (-1.292815975203248e-26+0j), (2.4976473321469986e-36+0j), (7.531979245745892e-43+0j), (1.2999615569756895e-35+0j), (-8.84919980221122e-43+0j), (-3.1506337794484697e-66+0j), (-8.227434847902739e-26+0j), (1.1371032738704549e-35+0j), (-1.1218795505384485e-41+0j), (1.1267960224555944e-35+0j), (2.9865173520922664e-44+0j), (1.9664208073259348e-67+0j), (-1.186286731204978e-42+0j), (2.5039295474302823e-68+0j), (2.863645960728337e-93+0j), (1.9990010289862712e-19+0j), (7.345363839470886e-21+0j), (-1.1292954065909499e-23+0j), (7.4313906231764e-21+0j), (4.1348933208064555e-25+0j), (8.078444097853281e-27+0j), (1.1946862958352542e-23+0j), (1.262132737050657e-27+0j), (-2.776402316998787e-34+0j), (6.961552035246281e-21+0j), (6.101365785341394e-25+0j), (1.1931539556900772e-26+0j), (6.31290672568546e-25+0j), (2.123440622954414e-27+0j), (2.0499891816625126e-35+0j), (-1.3102804794722666e-26+0j), (1.0336246163432594e-35+0j), (3.884399343108393e-42+0j), (3.4412005951913065e-25+0j), (2.0558086170944e-27+0j), (9.040050803920366e-34+0j), (8.592415255647686e-29+0j), (1.819450029081246e-35+0j), (5.640226318907389e-44+0j), (1.34854998307328e-34+0j), (5.219836779609944e-44+0j), (-1.0600592873700617e-66+0j), (1.2123159057577174e-20+0j), (5.090361649210145e-26+0j), (-7.212991451836734e-27+0j), (3.5890015743935153e-25+0j), (-3.131256138810043e-29+0j), (1.6738951307537285e-35+0j), (-1.0467219688853723e-26+0j), (5.534294500083879e-36+0j), (2.8670566580085757e-42+0j), (4.762384839255457e-25+0j), (-3.3847901757097414e-28+0j), (3.143159646728688e-35+0j), (-2.177616948184439e-27+0j), (3.4879582187740132e-37+0j), (2.049399004075045e-43+0j), (1.6275504529952427e-35+0j), (5.491338357072877e-44+0j), (1.2020931567675908e-67+0j), (-3.4099633210187413e-26+0j), (4.7016254055227024e-36+0j), (1.7595053842678484e-42+0j), (2.599582922404753e-36+0j), (-9.18069447017806e-44+0j), (-4.94043561583516e-68+0j), (-1.530436875927751e-42+0j), (-1.5947970190910783e-68+0j), (-5.0419710725694456e-95+0j), (8.794095153659004e-25+0j), (-7.718581566103019e-28+0j), (3.80922757933655e-34+0j), (-2.5168830688451223e-27+0j), (2.285350993432634e-35+0j), (-6.0360931350791495e-43+0j), (5.564261416561362e-35+0j), (9.123766720064864e-44+0j), (-5.64054809393245e-67+0j), (-4.993239776661308e-27+0j), (1.207520451571022e-35+0j), (-6.243660470107263e-43+0j), (5.793087925981605e-36+0j), (-6.332117435667767e-44+0j), (-1.0800592404346764e-68+0j), (-3.5592980993850354e-43+0j), (-1.069026693363332e-68+0j), (-4.1939699301922926e-95+0j), (3.894804603299666e-35+0j), (-1.6170765325423338e-43+0j), (-2.3316604999978384e-67+0j), (-1.925882207795789e-43+0j), (-7.508737689292227e-69+0j), (-5.645412850798969e-97+0j), (9.704572650269967e-69+0j), (9.145092346587576e-97+0j), (-7.196368106753468e-268+0j), (0.00037069844265942786+0j), (2.6196543315504553e-20+0j), (-1.8973538018496328e-18+0j), (1.479180457085356e-19+0j), (-1.1858461261560205e-20+0j), (5.366090908816123e-25+0j), (-3.049318610115481e-19+0j), (5.335476509680418e-25+0j), (5.064728674969424e-24+0j), (-5.928934608129585e-20+0j), (1.2451384324638215e-19+0j), (2.895939937502543e-24+0j), (-1.002675251312278e-19+0j), (-1.5423808418881223e-26+0j), (9.628836609698529e-26+0j), (-1.1744308721440753e-24+0j), (2.830030923478755e-26+0j), (4.169238685625904e-35+0j), (-5.675120746603812e-20+0j), (-1.7153496394105022e-24+0j), (1.606462616446633e-24+0j), (3.235213235619996e-25+0j), (-2.855376989868389e-26+0j), (9.995259172535939e-36+0j), (2.3622521020323198e-25+0j), (6.593942161613908e-36+0j), (-7.836887976631404e-41+0j), (-2.657517475217845e-20+0j), (-1.8925892415213273e-20+0j), (1.0339757656912846e-24+0j), (1.8740603958001395e-20+0j), (3.4167143526932463e-25+0j), (1.8925233416743742e-25+0j), (7.47410997817079e-25+0j), (6.69970606222358e-27+0j), (2.0301728752536525e-35+0j), (3.6925342544367155e-20+0j), (8.794852460128016e-26+0j), (8.686714402897006e-26+0j), (3.4111607719204904e-25+0j), (1.2226509364872987e-26+0j), (4.7470037910201825e-36+0j), (3.304684410802642e-26+0j), (1.5482212676137352e-36+0j), (6.378710609606567e-42+0j), (1.1630207880109586e-24+0j), (3.491238335460158e-27+0j), (2.474273299335111e-35+0j), (7.850346831862612e-27+0j), (4.890545855265234e-36+0j), (2.483100878783576e-42+0j), (6.21029964994736e-35+0j), (2.7325320054333933e-44+0j), (-5.225581225258704e-66+0j), (-4.923379005915621e-21+0j), (1.1841748820453652e-25+0j), (-2.3465719325577737e-25+0j), (5.59422288659291e-25+0j), (-2.1438500124509716e-26+0j), (5.242882753877829e-35+0j), (-3.481142548947434e-25+0j), (2.0418487272344713e-35+0j), (5.453853623152188e-42+0j), (5.700750663233955e-25+0j), (7.03991944928368e-27+0j), (3.7921933660572097e-35+0j), (-1.1281195481494149e-27+0j), (3.1204959466818707e-37+0j), (-1.6269075170811126e-42+0j), (4.822258829475605e-35+0j), (-3.6873417465627155e-42+0j), (-6.628430811253142e-68+0j), (-5.398907755697251e-25+0j), (5.953905498133243e-35+0j), (-1.0691346763412624e-40+0j), (3.183806128692875e-35+0j), (5.20144473726568e-43+0j), (-9.876243328300325e-68+0j), (-9.343420254347779e-42+0j), (-1.255648856520882e-67+0j), (2.6394859524267557e-93+0j), (4.653400536674282e-20+0j), (-4.116050728063866e-21+0j), (-8.788794008375919e-25+0j), (6.352747104407253e-22+0j), (4.1140673929086208e-25+0j), (1.5164520554915883e-26+0j), (2.3414906279897655e-24+0j), (3.2051043025259092e-27+0j), (-6.263582618270692e-37+0j), (3.758708703440958e-21+0j), (2.4957034686296106e-25+0j), (-4.096132016822654e-26+0j), (7.201984520305749e-26+0j), (4.957880365887362e-27+0j), (-2.140951774254874e-37+0j), (-5.5208054744935795e-27+0j), (-3.04109752370376e-36+0j), (1.440534821325912e-42+0j), (1.3440170341048675e-24+0j), (8.198569744497675e-28+0j), (4.1779931329890737e-35+0j), (1.2030953253484236e-27+0j), (3.2003002249316066e-36+0j), (7.626566892087817e-43+0j), (5.147197953616508e-35+0j), (5.535128934083027e-44+0j), (1.1419684718616e-67+0j), (2.911014011696615e-20+0j), (4.896932955249953e-26+0j), (-3.6183342521817214e-26+0j), (3.483609757455988e-27+0j), (2.4318180668744093e-27+0j), (6.111997549255925e-36+0j), (-2.5479800241088097e-26+0j), (-7.875413957537704e-37+0j), (-4.9949283760858104e-42+0j), (7.601779227071758e-26+0j), (2.8894044568405913e-29+0j), (1.8644427486548552e-35+0j), (-6.014738913595096e-27+0j), (1.1717540217768933e-36+0j), (2.3191489584575723e-43+0j), (6.757915249954514e-36+0j), (8.372758324340782e-44+0j), (2.2079572896902825e-67+0j), (-1.5209175459527156e-27+0j), (2.3281533072231566e-36+0j), (-4.970405652960126e-42+0j), (-6.7883152514551215e-37+0j), (-1.4692833351330758e-43+0j), (-3.86246843627956e-68+0j), (-9.151902165793885e-43+0j), (-1.3230681281560997e-68+0j), (-1.0230369500820999e-94+0j), (1.2683621178505516e-25+0j), (-5.498166080447751e-28+0j), (8.498551728329131e-35+0j), (-1.0085992180200163e-27+0j), (-1.4792242795623501e-36+0j), (-2.4951870780383774e-43+0j), (5.49579329487554e-35+0j), (-1.8950372201142643e-44+0j), (-1.5098441299965625e-67+0j), (-4.686865520612343e-27+0j), (2.656172387858273e-36+0j), (-3.622006205663571e-42+0j), (1.0079174079956759e-36+0j), (-3.31768359073153e-44+0j), (-1.7042596603516807e-68+0j), (-7.151466500638315e-43+0j), (-1.2025084189009258e-68+0j), (-5.372958834719151e-95+0j), (2.69461872606602e-36+0j), (-1.3864096681413659e-43+0j), (-1.1905316873074986e-68+0j), (-1.7241513356098097e-43+0j), (-2.6445430215813976e-69+0j), (2.295436909922596e-96+0j), (-8.236537010378519e-69+0j), (1.9385692343575615e-96+0j), (-1.4316244967394662e-267+0j), (-5.669826790683473e-20+0j), (1.098189043376275e-25+0j), (-1.5168692290712725e-25+0j), (3.7505602877827785e-25+0j), (-3.622049847976322e-26+0j), (3.1708904397804264e-35+0j), (-4.968513296887656e-25+0j), (9.463069411864533e-36+0j), (-4.3159992701204366e-43+0j), (4.178627769391908e-25+0j), (-2.9342023531063706e-26+0j), (4.737318674084528e-35+0j), (-1.2820742221205286e-26+0j), (4.148423875027979e-36+0j), (1.0456313978483744e-42+0j), (2.1724299597370458e-35+0j), (-8.565436863185444e-44+0j), (-1.1073030727705331e-67+0j), (-1.6996165975167744e-25+0j), (4.1722713436726607e-35+0j), (-2.819399373048429e-41+0j), (5.013070612806409e-36+0j), (-6.688791685415443e-43+0j), (-2.813647777713474e-68+0j), (-2.4797596577577014e-41+0j), (-3.989392279843618e-68+0j), (-3.2695910775437163e-95+0j), (6.098809876008478e-25+0j), (-1.8008050066957144e-28+0j), (7.916695186227795e-35+0j), (-1.3000409666684731e-26+0j), (-7.985530793461277e-37+0j), (1.489667848731301e-42+0j), (2.3555224520461832e-35+0j), (-1.1212029861236417e-43+0j), (-7.017245416240776e-68+0j), (-9.000508498119134e-27+0j), (8.615199500997347e-37+0j), (-1.4254708628344202e-42+0j), (3.692556282930884e-36+0j), (2.361133174166054e-44+0j), (-1.3348543415935993e-68+0j), (-3.3279606917735995e-43+0j), (-5.793398375267094e-69+0j), (2.877869463701727e-96+0j), (1.1249649941101953e-35+0j), (-9.280454878429309e-44+0j), (-2.1511683053716308e-68+0j), (-3.110670480193701e-43+0j), (-3.9011860111156137e-70+0j), (1.6482188415745546e-96+0j), (-5.797463072884908e-69+0j), (1.2764765496647997e-97+0j), (2.6139529283477882e-269+0j), (-1.157022289687658e-25+0j), (4.2761130127868764e-36+0j), (-5.527947279453363e-42+0j), (-1.3657526125015681e-37+0j), (-5.743982617310814e-43+0j), (-5.66345694203849e-68+0j), (-1.6108673708361774e-41+0j), (-1.0393787812809527e-68+0j), (-7.936275606664153e-96+0j), (2.0004473225355998e-36+0j), (-7.420162744261536e-43+0j), (-3.663560554591953e-68+0j), (-5.785865909423843e-43+0j), (-1.4195625385235661e-70+0j), (3.2950686620482857e-96+0j), (-5.842130526320512e-69+0j), (1.2946564350631612e-96+0j), (1.628166905039491e-269+0j), (-8.938797204400344e-42+0j), (-1.0649719913882951e-69+0j), (1.6526298523026482e-96+0j), (-3.79905639802652e-69+0j), (-3.283553518408838e-97+0j), (-4.320458481255519e-271+0j), (-4.872276065305624e-97+0j), (-1.4886063016418433e-271+0j), 0j, (5.738648940527373e-20+0j), (2.058025364031933e-21+0j), (3.224711919249694e-24+0j), (1.6378176128549948e-22+0j), (4.336589277053364e-25+0j), (6.214236170766107e-27+0j), (1.685259329041752e-25+0j), (3.516367468312748e-28+0j), (1.814410435157389e-34+0j), (-2.2267702089927505e-21+0j), (2.9738162860659615e-25+0j), (-1.5223703787804877e-27+0j), (1.2676679202539571e-25+0j), (-1.1328333200523318e-27+0j), (3.164320110013614e-36+0j), (7.285693610794738e-28+0j), (1.275304107410451e-35+0j), (2.543356712749543e-43+0j), (6.81500091464053e-25+0j), (-9.054526604602922e-28+0j), (2.257987113960561e-34+0j), (-1.4663452277767746e-27+0j), (2.8078383958723394e-36+0j), (-3.7414668997472616e-43+0j), (1.0702795203710342e-34+0j), (-2.0975686387862106e-44+0j), (-4.195552960827292e-67+0j), (-1.313562812734208e-20+0j), (2.8914118759065746e-26+0j), (-7.267846534380155e-26+0j), (1.452419144256718e-25+0j), (2.2355583159892733e-27+0j), (1.383987903777924e-35+0j), (-3.1228596731277385e-26+0j), (2.523293324911298e-36+0j), (2.6423234167924832e-42+0j), (1.1202613715043589e-25+0j), (-3.078377859823748e-27+0j), (2.0021351305759007e-35+0j), (-3.0548914268292065e-27+0j), (1.3556503674086623e-36+0j), (-3.835178734548984e-43+0j), (3.959473216793803e-36+0j), (1.0130949991298326e-43+0j), (-2.21998992369125e-68+0j), (-2.5623822859345149e-26+0j), (1.1311039711259074e-36+0j), (-2.3908779236001988e-42+0j), (2.5493315880592604e-36+0j), (-2.3344209221898623e-43+0j), (-2.1272049005885007e-68+0j), (-2.251601993419415e-42+0j), (-8.923450602763723e-69+0j), (-2.316764273304886e-95+0j), (4.618291506307787e-25+0j), (-3.7658398101726636e-28+0j), (3.293000295056896e-35+0j), (-1.7001924659775857e-27+0j), (4.1494548052284756e-36+0j), (3.05012316519951e-43+0j), (3.2794655832659976e-35+0j), (-3.225244419574165e-44+0j), (-1.053622726687762e-67+0j), (-4.1636078577565e-27+0j), (5.289764100396695e-36+0j), (-4.710771321866944e-44+0j), (-7.2905880278719956e-37+0j), (-8.899066321781529e-45+0j), (-2.8931621724738964e-69+0j), (-3.6732083378326894e-44+0j), (-1.5238852141652963e-69+0j), (-1.841461600436511e-96+0j), (2.5555644911989628e-36+0j), (-4.816347666127353e-44+0j), (2.0160562566152054e-68+0j), (-1.6798920625800974e-43+0j), (-3.3925829734796594e-71+0j), (-4.3718775193904046e-97+0j), (3.777223252098861e-69+0j), (1.357799677003808e-97+0j), (1.2408046248716014e-269+0j), (1.3402807465196707e-20+0j), (1.4412488738387886e-26+0j), (-8.144863637343389e-26+0j), (1.6329420738074944e-26+0j), (-1.4185493918788701e-27+0j), (3.2699357118256644e-36+0j), (-6.017031117546339e-26+0j), (4.004267520772525e-37+0j), (-3.648280551869661e-42+0j), (1.2377227602917675e-25+0j), (-4.1209134266204266e-27+0j), (1.4317251940522034e-35+0j), (-1.7971961580236538e-27+0j), (8.74188826071584e-37+0j), (-8.284082405895227e-44+0j), (2.33875177031916e-36+0j), (2.2981294814927e-43+0j), (-2.628672614898358e-68+0j), (-1.629471085824522e-26+0j), (2.303296948770939e-36+0j), (2.172778954801144e-43+0j), (1.9442183084661523e-36+0j), (-1.2023387145902613e-43+0j), (-7.254022295084235e-69+0j), (-1.6774692120614577e-42+0j), (-4.507836501873672e-69+0j), (1.870453690484621e-95+0j), (8.115169904189593e-26+0j), (1.1674210085899677e-29+0j), (7.016420049706893e-36+0j), (-1.6576925847088037e-27+0j), (7.659308027080583e-37+0j), (-2.533668047586047e-43+0j), (-2.6189360300616526e-38+0j), (-2.0457725969163919e-44+0j), (-6.956302078059338e-69+0j), (-1.954600107911362e-27+0j), (6.780912623443183e-37+0j), (-2.478601396995783e-43+0j), (3.4355829455859553e-37+0j), (1.034860284359409e-44+0j), (-1.0321203406979826e-69+0j), (-9.656318295868386e-44+0j), (-6.67875572603635e-70+0j), (3.4996619296127565e-96+0j), (2.8495058093083e-36+0j), (-7.236563907868822e-45+0j), (-3.5351418348845665e-69+0j), (-6.700967518507562e-44+0j), (-4.053572292761226e-70+0j), (-4.773162104252776e-97+0j), (6.850273042920303e-70+0j), (3.0949153186804137e-99+0j), (1.729331964577206e-269+0j), (-8.099925167196197e-27+0j), (2.8585385149685612e-39+0j), (-2.553291699908721e-42+0j), (1.3038549968429153e-36+0j), (-8.243413618066464e-44+0j), (-2.4875457004003215e-69+0j), (-8.959898018940873e-43+0j), (-1.189408234425503e-69+0j), (2.586559529027281e-96+0j), (8.33285901440467e-37+0j), (-8.95396704713859e-44+0j), (-4.373488660038238e-69+0j), (-3.2547613639265393e-44+0j), (-1.6149955259156168e-72+0j), (3.4806569772242486e-97+0j), (-8.257656807606269e-70+0j), (7.821582090755295e-98+0j), (-4.605292713724859e-271+0j), (-2.4675015120653584e-43+0j), (-1.03316015642604e-69+0j), (-2.719235614104682e-97+0j), (-2.5078760342474046e-70+0j), (1.76375426359425e-98+0j), (-1.777822015562036e-271+0j), (-9.698845787627584e-98+0j), (7.452149301398185e-273+0j), 0j, (4.430692494437178e-25+0j), (-1.0070302493211979e-28+0j), (4.2285256499226e-35+0j), (-6.573183492754081e-28+0j), (4.456584396672354e-37+0j), (-9.66430665503391e-44+0j), (1.9889480593802792e-35+0j), (-5.132666162249113e-44+0j), (-9.554715966482271e-68+0j), (-3.2382986678355416e-27+0j), (2.7744750896381775e-36+0j), (-2.897649849872292e-43+0j), (-2.654654403730625e-38+0j), (-1.631130570851529e-44+0j), (-1.8597627651332472e-69+0j), (-1.733070244536799e-43+0j), (8.907046652651203e-70+0j), (8.843909239179505e-98+0j), (8.513215168923517e-36+0j), (-2.4090889877004123e-44+0j), (1.6438293535192985e-69+0j), (-5.710830071489184e-44+0j), (-5.326823695736443e-70+0j), (1.4092980032624498e-97+0j), (-6.582265920870447e-69+0j), (-4.603519237547997e-98+0j), (-1.9108428112462536e-271+0j), (-4.2311540727660494e-27+0j), (2.7141601888071484e-37+0j), (1.5427009744016563e-43+0j), (7.244200592436882e-37+0j), (-5.683114570551412e-44+0j), (-4.018080832478832e-69+0j), (-4.1293489917135046e-43+0j), (-3.570785045925186e-70+0j), (4.0033423591304e-96+0j), (8.983573035414472e-37+0j), (-7.511730380302233e-44+0j), (-1.7333305288867399e-69+0j), (-1.7587374455113447e-44+0j), (-3.051453468828469e-70+0j), (-5.4698171224247676e-98+0j), (-5.3417020100328104e-70+0j), (-4.197357656064822e-98+0j), (-2.7074670261559784e-271+0j), (-2.2680546776900354e-43+0j), (9.246190849134761e-71+0j), (-2.310730749105864e-97+0j), (-3.9621312184493284e-70+0j), (7.975781580198777e-99+0j), (7.538320487690324e-273+0j), (1.1272737989162328e-98+0j), (3.1533034002686707e-273+0j), 0j, (-1.396443367817735e-36+0j), (-5.935943144643757e-45+0j), (-3.913605521992993e-69+0j), (-3.4742363735634085e-44+0j), (-8.1756819864826e-71+0j), (3.729266846018025e-98+0j), (-8.27573218785296e-70+0j), (7.790998842520057e-99+0j), (3.32938854702604e-273+0j), (-2.520215528338682e-44+0j), (5.601191448366384e-71+0j), (7.821716023689834e-99+0j), (-1.3873914967606252e-70+0j), (3.0614174989785105e-99+0j), (3.5696949275673335e-273+0j), (-5.786813838158448e-99+0j), (-1.249002724186407e-274+0j), 0j, (-2.905433714996657e-70+0j), (2.3387685757916495e-99+0j), (-4.9659040780909366e-275+0j), (1.3105983307271435e-99+0j), (-5.113412940464627e-275+0j), 0j, (7.661734760745137e-277+0j), 0j, 0j, (5.776425887095427e-17+0j), (9.263152311173029e-18+0j), (-0.00022258684333429702+0j), (2.893734539572627e-17+0j), (0.0043647994253026645+0j), (2.7884564248276647e-19+0j), (1.2590560129425057e-05+0j), (9.067993300682672e-19+0j), (2.2551405187698492e-17+0j), (-4.2035510513125826e-17+0j), (0.004669751160580358+0j), (1.802912388190805e-18+0j), (-0.007815878234412715+0j), (1.1234093470549791e-18+0j), (4.87890977618477e-19+0j), (1.0425675866881242e-18+0j), (-4.0826988057657276e-19+0j), (-2.412474257466606e-22+0j), (-8.596326671828418e-05+0j), (-1.1265265942105934e-19+0j), (-4.683753385137379e-17+0j), (5.19070814605251e-19+0j), (-1.2739375526704677e-18+0j), (-6.329084861795287e-23+0j), (-1.0408340855860843e-16+0j), (-7.088268853985659e-23+0j), (-1.5666050484592952e-22+0j), (8.191867391133152e-17+0j), (-0.006452282290499302+0j), (2.966374034072448e-19+0j), (0.002633757104164953+0j), (-4.985644054085572e-19+0j), (-2.574980159653073e-19+0j), (1.4016460560477301e-18+0j), (-5.590417451878382e-20+0j), (-1.1491285452106587e-22+0j), (0.005708076652180287+0j), (-5.280927934393153e-19+0j), (-9.215718466126788e-19+0j), (1.0094928930446822e-18+0j), (-5.421010862427522e-19+0j), (4.398417620799216e-25+0j), (-2.656295322589486e-18+0j), (-3.255718192336549e-25+0j), (6.292756688264541e-24+0j), (7.761807579522288e-19+0j), (-2.778268066994105e-19+0j), (6.167887857100638e-23+0j), (-9.486769009248164e-19+0j), (-1.1349253194624291e-23+0j), (6.834133182826758e-24+0j), (4.403627473900817e-23+0j), (4.673129453319998e-25+0j), (8.508461373411958e-34+0j), (-0.00015586988605421884+0j), (4.447091362585319e-19+0j), (-8.760353553682876e-17+0j), (6.986220665474863e-19+0j), (1.0299920638612292e-18+0j), (-1.1463554750665685e-22+0j), (3.279711571768651e-17+0j), (-7.628653409173123e-23+0j), (1.315093023453755e-22+0j), (5.968210142972236e-19+0j), (-1.463672932855431e-18+0j), (-7.617281746464176e-23+0j), (1.748276003132876e-18+0j), (-3.519656391112095e-24+0j), (4.3171252697453305e-24+0j), (-1.8359532189555872e-23+0j), (3.633163195131376e-24+0j), (-7.467172661610074e-34+0j), (1.680513367352532e-17+0j), (-3.432969219131794e-23+0j), (5.712032888537331e-23+0j), (1.700243899708606e-23+0j), (4.61335100250496e-24+0j), (-1.875544399420154e-34+0j), (5.2170064313367767e-23+0j), (1.1501370550473795e-33+0j), (-1.999305430738796e-40+0j), (2.1725124547651922e-17+0j), (-0.0033616892517299396+0j), (1.4386765747736088e-18+0j), (0.004567348052183284+0j), (-7.10979087611711e-19+0j), (-2.202285662861181e-20+0j), (1.3124755554597704e-18+0j), (7.030373462210693e-20+0j), (-2.0965572369222775e-22+0j), (0.0006749595316184265+0j), (2.3076631674250496e-19+0j), (-3.0357660829594124e-18+0j), (5.497903253654628e-19+0j), (-1.6263032587282567e-19+0j), (-4.8959276857158795e-24+0j), (-1.870248747537495e-18+0j), (3.129210810103223e-24+0j), (9.06322256414268e-24+0j), (1.2306023482904143e-18+0j), (2.0328790734103208e-20+0j), (2.4789759749865653e-23+0j), (-3.7269449679189215e-20+0j), (1.453987388594868e-24+0j), (1.220249533611581e-25+0j), (-1.2786655198288651e-23+0j), (3.4339376639630945e-25+0j), (8.562469828702473e-34+0j), (-0.0034189053224323914+0j), (6.174057455840031e-19+0j), (-3.0357660829594124e-18+0j), (-3.9155288704740887e-19+0j), (-1.0842021724855044e-19+0j), (-1.7050891786669362e-24+0j), (1.8905775382715984e-18+0j), (-1.1365071180624472e-24+0j), (8.751392311151928e-24+0j), (-5.067594337290432e-20+0j), (-6.776263578034403e-21+0j), (7.426160187173976e-25+0j), (-2.202285662861181e-20+0j), (8.959634096865802e-25+0j), (4.233920865650086e-25+0j), (-1.0687108890699762e-24+0j), (1.9491689255236205e-25+0j), (5.991766768798567e-35+0j), (-3.4220131069073734e-19+0j), (8.283742743903111e-25+0j), (2.404785973615042e-24+0j), (2.167310140116966e-24+0j), (6.554273567835476e-26+0j), (2.925230319401984e-35+0j), (9.96182702974203e-25+0j), (2.6005209549598883e-35+0j), (1.498728729386232e-42+0j), (5.336615316852896e-19+0j), (-5.082197683525802e-21+0j), (2.767009700411795e-23+0j), (-3.3881317890172014e-21+0j), (-7.916376956073898e-25+0j), (8.16343527413624e-25+0j), (5.9938282667416654e-24+0j), (9.95983460879664e-26+0j), (2.333077336062661e-34+0j), (-7.403067959002585e-19+0j), (1.5388467450327321e-24+0j), (1.679974918079694e-24+0j), (3.554695591347264e-24+0j), (9.314203044368723e-26+0j), (5.043928308163058e-35+0j), (6.254684259828088e-25+0j), (1.0152477237169537e-35+0j), (6.808401528317551e-41+0j), (8.51899095701585e-24+0j), (1.2295222985992922e-25+0j), (1.2940969812031894e-33+0j), (3.645277786610087e-27+0j), (4.0757013988029755e-35+0j), (-1.6498708865109618e-42+0j), (1.1024509879436637e-33+0j), (-1.447261053954671e-41+0j), (1.350578537208063e-64+0j), (3.547555001141489e-05+0j), (1.114948668617818e-19+0j), (-9.540979117872439e-18+0j), (2.6947586392029265e-19+0j), (-6.2341624917916505e-19+0j), (-2.2139642014328207e-23+0j), (1.6154612370034016e-17+0j), (1.1809679892362132e-23+0j), (1.6816134115420846e-22+0j), (6.641561554327309e-19+0j), (-1.5043305143236374e-18+0j), (6.725208351164294e-24+0j), (-7.928228386300251e-19+0j), (3.2793534459993183e-24+0j), (5.239732741788402e-24+0j), (-7.615877749169868e-24+0j), (3.612306724993889e-24+0j), (9.553476769647933e-35+0j), (-3.333921680392926e-18+0j), (-9.96817261611754e-24+0j), (3.7278888197524917e-23+0j), (-1.1470668650637688e-24+0j), (7.2155221354907005e-25+0j), (5.632818892898554e-34+0j), (3.033858697439393e-24+0j), (1.2197470065615685e-34+0j), (3.975777840794036e-39+0j), (3.235807253674487e-19+0j), (-4.828087799349512e-20+0j), (-5.203801479774712e-23+0j), (-5.929230630780102e-20+0j), (5.133528117943839e-25+0j), (3.3202347789567094e-24+0j), (1.6420827628884713e-23+0j), (2.6333524533562354e-25+0j), (1.0819529644054135e-33+0j), (-1.4433441421213278e-18+0j), (3.2190573642810696e-24+0j), (2.381336709285792e-24+0j), (4.759923593231168e-25+0j), (2.1767419916712186e-25+0j), (6.654115361569953e-35+0j), (-6.875930413434151e-25+0j), (4.6988692022230903e-35+0j), (-2.438879004648165e-41+0j), (7.449472274378927e-24+0j), (6.919696754501187e-26+0j), (6.299004975369864e-34+0j), (5.138972243388254e-25+0j), (7.716819077926549e-36+0j), (6.984045097325589e-41+0j), (3.26840278345508e-34+0j), (4.450523922695619e-42+0j), (6.836176254742797e-65+0j), (-2.7159264420761886e-17+0j), (1.0295329010730799e-24+0j), (5.0064469467634936e-23+0j), (9.376867725112837e-24+0j), (-7.794018672356676e-25+0j), (8.685287866199286e-34+0j), (-2.789791602813468e-23+0j), (-2.298481469730318e-35+0j), (-2.4562407898193978e-39+0j), (1.4865421115729582e-23+0j), (-7.167633822167118e-26+0j), (1.1620887388869654e-33+0j), (-5.436372492154811e-25+0j), (5.788738647554515e-35+0j), (6.605160441441458e-41+0j), (1.0154277058639299e-33+0j), (4.645584668929634e-41+0j), (-7.381629010667731e-66+0j), (4.194189356306035e-24+0j), (6.7616994076922364e-34+0j), (-6.675113260780553e-40+0j), (1.0189798442922556e-33+0j), (-3.095748567386386e-41+0j), (-5.405507390864894e-66+0j), (-1.0535382244333272e-40+0j), (-3.0970241474184477e-66+0j), (-7.038824436337696e-92+0j), (-6.929491838192706e-17+0j), (0.0022691571194941966+0j), (2.908036581204483e-18+0j), (-0.0017176508958175831+0j), (4.872399897679188e-19+0j), (7.623296525288703e-21+0j), (1.5184194675814214e-18+0j), (8.279747059410786e-20+0j), (-2.006985818480482e-22+0j), (-0.0014113119602811774+0j), (1.2305531234593463e-19+0j), (7.453889935837843e-20+0j), (-3.0736034801853637e-19+0j), (5.717472393966527e-21+0j), (-1.1916057381028074e-23+0j), (3.218725199566341e-20+0j), (-9.897726216632228e-25+0j), (5.353852874245006e-25+0j), (1.4842563243584904e-18+0j), (1.0079692072326174e-19+0j), (-4.397444787756126e-23+0j), (2.964615315390051e-20+0j), (-1.4231002117878194e-24+0j), (2.2559192115038874e-25+0j), (2.980435144605128e-23+0j), (2.940881384680478e-26+0j), (-2.7268814893238104e-34+0j), (0.001192168254206991+0j), (-2.579196243718652e-19+0j), (-3.3068166260807885e-18+0j), (2.0543722264730763e-19+0j), (-1.5246593050577406e-20+0j), (-5.403244963280242e-24+0j), (5.861467994999758e-19+0j), (-1.0420537013607478e-25+0j), (3.2618127952441683e-24+0j), (2.234269609225419e-19+0j), (8.470329472543003e-20+0j), (-4.592906089686975e-24+0j), (-4.002230675776569e-20+0j), (-1.2032085179665378e-24+0j), (2.80348726170148e-25+0j), (-3.554291694563791e-25+0j), (2.2460440183228645e-25+0j), (-1.9881064782947826e-36+0j), (1.2874900798265365e-19+0j), (1.5727740748444774e-24+0j), (2.5054530636073687e-24+0j), (3.232789854919157e-24+0j), (1.077774862990331e-25+0j), (1.827343850495411e-35+0j), (5.078576533413852e-26+0j), (2.1147934017517687e-35+0j), (9.981663072346292e-41+0j), (4.87045749882466e-19+0j), (1.9958213819679452e-20+0j), (-1.5541948228047122e-23+0j), (5.7809998650106e-20+0j), (-1.4075802904039558e-24+0j), (2.631514430024085e-25+0j), (5.6464770329547495e-24+0j), (1.2687182445874629e-26+0j), (2.4618991765179873e-34+0j), (3.282252670610414e-20+0j), (-5.188054183712715e-25+0j), (1.312814730654654e-25+0j), (1.085371631388244e-24+0j), (6.566589526113073e-27+0j), (1.0173115112536604e-35+0j), (-3.8967408049450365e-26+0j), (3.692016067939513e-35+0j), (-5.392196490721896e-42+0j), (1.3250641775403896e-23+0j), (5.449477099584311e-27+0j), (1.3862855303165822e-33+0j), (1.908421241800586e-26+0j), (2.7795250180287325e-35+0j), (-6.1096613044562024e-43+0j), (4.702557095128365e-34+0j), (8.085492139154194e-43+0j), (-4.571698526753947e-66+0j), (0.00047202638269621246+0j), (-8.160053298325187e-21+0j), (-2.710505431213761e-20+0j), (1.5104750032328402e-19+0j), (2.922263668027336e-20+0j), (2.337768476630432e-25+0j), (-1.7110065534536867e-19+0j), (4.121766675343578e-25+0j), (4.156427097569183e-24+0j), (6.151387700143864e-21+0j), (4.743384504624082e-20+0j), (-3.073433750582925e-24+0j), (9.423241538204091e-21+0j), (7.07324242057368e-26+0j), (1.6665990396416982e-25+0j), (3.812785635986612e-25+0j), (1.4439300442635023e-25+0j), (4.8058318700068183e-35+0j), (-6.378158092824882e-19+0j), (8.653488585912411e-25+0j), (1.7975517784622593e-25+0j), (-4.0318996410208e-25+0j), (5.500576173358063e-26+0j), (3.0599503376459465e-35+0j), (5.985781398376734e-25+0j), (5.695620970027899e-36+0j), (9.536726233009106e-41+0j), (1.1542192631532722e-20+0j), (-1.3817224952085774e-20+0j), (6.325023629189655e-25+0j), (-3.2081372877256625e-20+0j), (9.552158929140188e-26+0j), (1.902465954312619e-25+0j), (1.928405192692593e-24+0j), (1.675641242087865e-26+0j), (-1.826859848732066e-35+0j), (1.4823076576950256e-20+0j), (-6.486582342578918e-25+0j), (1.1579920133348333e-25+0j), (3.672179068240019e-25+0j), (1.1815627575454598e-26+0j), (1.6689859495122329e-35+0j), (-5.4889578276942144e-27+0j), (8.22164050428596e-36+0j), (6.950440383051093e-42+0j), (5.101721246245328e-25+0j), (2.9020773388162152e-27+0j), (1.1079633770098416e-34+0j), (-3.572963354890452e-26+0j), (4.721183084363351e-36+0j), (2.1467892473456198e-42+0j), (2.0521725436787384e-35+0j), (-5.9344989964156e-43+0j), (-7.431940333508278e-66+0j), (-2.8343839997497025e-19+0j), (8.080460024359868e-26+0j), (-2.0407315593024054e-25+0j), (4.7271069795739735e-25+0j), (2.5931610828310105e-26+0j), (4.677999962390214e-35+0j), (-3.0129900078451824e-25+0j), (7.170622128382344e-36+0j), (1.9483653847972257e-41+0j), (6.890731561541747e-25+0j), (-2.2892353996983348e-26+0j), (1.0221554014034442e-34+0j), (-3.7684777255275426e-26+0j), (5.359527346999829e-36+0j), (-2.2651989675810668e-42+0j), (5.873967661566353e-35+0j), (-3.797518838320254e-42+0j), (8.971871156680053e-68+0j), (-4.241985295222028e-25+0j), (2.924866931321099e-35+0j), (4.681738169309214e-42+0j), (2.6306020175668993e-35+0j), (-5.444920345442117e-43+0j), (6.6057538493182175e-68+0j), (-9.71056045200088e-43+0j), (-1.2197652007170366e-67+0j), (2.6264213493722363e-93+0j), (2.9180333988940825e-19+0j), (4.658681209898652e-21+0j), (-5.428372769879244e-24+0j), (-6.988021814847978e-21+0j), (1.9160863407966618e-24+0j), (1.463227945266071e-25+0j), (7.559332199483626e-24+0j), (1.306011614611977e-26+0j), (7.780939998876501e-34+0j), (-2.2499312661442353e-20+0j), (9.865178936331885e-25+0j), (-3.026260751971497e-26+0j), (7.0904080338712895e-25+0j), (7.125563033759425e-27+0j), (4.1207124686602347e-35+0j), (-4.9441536953809963e-26+0j), (2.3022239455649596e-35+0j), (1.0089348943138683e-42+0j), (5.849031269866538e-24+0j), (1.1629160232907159e-26+0j), (6.675847514270928e-34+0j), (-4.9560929859565867e-26+0j), (1.7526100563928799e-35+0j), (-2.1047502934158752e-42+0j), (4.294188885056172e-34+0j), (9.84412171188184e-43+0j), (-2.6230531510793195e-66+0j), (2.482865326639168e-20+0j), (2.4587216693928496e-25+0j), (4.3229435733222655e-25+0j), (1.0851696829965074e-24+0j), (6.2828457763658716e-27+0j), (1.25557825762006e-34+0j), (-3.525652098702394e-25+0j), (2.0839853370936752e-35+0j), (-3.646739123558904e-41+0j), (1.1124074723319785e-24+0j), (5.4917570096842154e-27+0j), (7.280059852470632e-35+0j), (-2.0203787472667282e-26+0j), (6.09817295517822e-36+0j), (8.975316664000453e-43+0j), (4.018797942720996e-35+0j), (-3.528995020094011e-42+0j), (8.50274820054555e-68+0j), (-3.633241266910384e-25+0j), (1.316464981938558e-35+0j), (-1.708182828011952e-42+0j), (1.462170992929352e-35+0j), (-8.944663260093348e-43+0j), (-1.4768340677360381e-67+0j), (-3.532498266254823e-42+0j), (-9.941906906011727e-68+0j), (1.954730978737773e-93+0j), (2.5580550345782794e-24+0j), (2.616445021444456e-29+0j), (3.473185002136315e-34+0j), (-6.226195317843169e-26+0j), (8.609544458749776e-36+0j), (-2.87704090956689e-42+0j), (1.1084245107873657e-34+0j), (-7.296604894316333e-43+0j), (-5.609695888798831e-67+0j), (-5.606675820036592e-26+0j), (7.520842241229273e-36+0j), (1.601158657799144e-42+0j), (5.1038637485474925e-36+0j), (4.598010586065806e-46+0j), (-4.799532009574448e-68+0j), (-1.4361338683363918e-42+0j), (-1.4855505085818497e-68+0j), (-1.1981795735305977e-95+0j), (3.191757469485365e-35+0j), (-3.348008565311059e-43+0j), (-1.803226772584741e-67+0j), (-7.073355058896459e-43+0j), (-1.0262167372965967e-68+0j), (-8.77179244135506e-96+0j), (-2.7923706730192204e-68+0j), (2.4057071207859343e-96+0j), (-6.987971372795129e-268+0j), (-3.9295455394174047e-05+0j), (4.754504034020723e-20+0j), (1.531435568635775e-17+0j), (4.764642996828684e-20+0j), (-1.9566461081574338e-19+0j), (5.688482298435958e-24+0j), (-1.5429552167184335e-17+0j), (6.514855117422039e-25+0j), (3.117404821186902e-23+0j), (9.740939677962572e-20+0j), (3.2610768469290563e-19+0j), (4.3023085375560795e-24+0j), (-5.8106460181645e-19+0j), (-1.0985992510469899e-25+0j), (3.62996402705977e-25+0j), (9.25448699972047e-24+0j), (7.781109414251096e-25+0j), (1.6445413780350466e-34+0j), (-1.0785270517389006e-17+0j), (7.585181593625908e-25+0j), (-7.426465024411576e-24+0j), (4.655920171486829e-24+0j), (2.4246175584721203e-25+0j), (4.187720120050219e-34+0j), (-1.0032680699946635e-23+0j), (3.5176400905727912e-34+0j), (-8.19703549691445e-40+0j), (2.964615668573512e-19+0j), (7.067431153653068e-21+0j), (9.493189998753107e-24+0j), (-1.573363699524863e-19+0j), (7.518538624352837e-25+0j), (4.767122488720011e-25+0j), (7.686761634669407e-24+0j), (-3.9883078795273625e-28+0j), (2.7374135284063194e-34+0j), (-9.682645378300721e-20+0j), (6.434075760727408e-25+0j), (1.3554236010630603e-25+0j), (7.1863335199461645e-25+0j), (-2.8859054072639154e-26+0j), (5.57364691150275e-35+0j), (-2.646822730616225e-25+0j), (6.325087338753771e-35+0j), (4.5830867574207467e-41+0j), (3.4090908009051905e-24+0j), (-8.497611904507997e-26+0j), (6.91863397686666e-34+0j), (-2.2879888645328053e-25+0j), (5.025228139344149e-35+0j), (-2.9441280735464407e-41+0j), (3.1537545538897083e-34+0j), (-3.6983769719692735e-42+0j), (-1.4119142574665e-66+0j), (7.450634152946834e-18+0j), (1.4483233784368106e-24+0j), (-1.7550312068341413e-23+0j), (4.7351344281455025e-24+0j), (-3.34005919890268e-25+0j), (4.13987073607222e-34+0j), (-3.593519765506449e-24+0j), (2.7557142372555865e-34+0j), (1.0064265700627269e-40+0j), (2.537733977659786e-24+0j), (-3.1264453683216824e-25+0j), (4.625725703856611e-34+0j), (-2.351509644930747e-25+0j), (3.263586247436047e-35+0j), (-4.5433949784187463e-41+0j), (2.2032037810607484e-34+0j), (-2.939486272383365e-42+0j), (-1.2016319560124118e-66+0j), (-7.978662407877941e-24+0j), (3.365697953445317e-34+0j), (-2.723605734215644e-40+0j), (1.9867807009054267e-34+0j), (-1.2199091362333715e-41+0j), (-1.0717542911127145e-66+0j), (-2.432462550293463e-40+0j), (-4.0949515484312506e-67+0j), (2.385417827928475e-93+0j), (1.3298419121764937e-19+0j), (7.940933880509066e-21+0j), (1.0129731329506804e-23+0j), (-2.3022090808575866e-20+0j), (3.704743246407542e-25+0j), (4.603076600110538e-27+0j), (7.120094447456567e-24+0j), (4.777686545987648e-28+0j), (2.6158057966530914e-34+0j), (-8.179161896924338e-20+0j), (3.9036624122680725e-25+0j), (-5.707425069538404e-25+0j), (9.302500229855842e-25+0j), (-3.8823405582195325e-26+0j), (3.9043527480748475e-35+0j), (-5.281804228434471e-25+0j), (4.4418971616485184e-35+0j), (-7.858201528240709e-41+0j), (4.442309260127463e-24+0j), (-2.3074823758296738e-26+0j), (5.550395588043202e-34+0j), (-6.950010980581221e-26+0j), (1.9121776165044794e-35+0j), (-4.661069016960423e-42+0j), (2.6733372241268485e-34+0j), (1.5379250645964867e-43+0j), (-1.0780012941658979e-66+0j), (3.2892009877558592e-19+0j), (9.091464160491117e-26+0j), (-3.2167222675168958e-25+0j), (3.7645704574595037e-25+0j), (-2.24667516802431e-26+0j), (4.0466753911858556e-35+0j), (-3.983096796202155e-25+0j), (5.4140789321226504e-36+0j), (-5.917683414843702e-42+0j), (4.85483933734736e-25+0j), (-5.8977234874730896e-27+0j), (1.1591092122653896e-35+0j), (-6.125982161606692e-26+0j), (3.9543142236928647e-36+0j), (2.041516700213218e-42+0j), (3.77005884311389e-35+0j), (-7.767572550060502e-43+0j), (-1.2534928099235333e-67+0j), (-2.3315746974017837e-25+0j), (1.4176201470425759e-36+0j), (-2.876130065565079e-41+0j), (6.203037449803005e-36+0j), (-1.3055941582691331e-42+0j), (-8.599780643556165e-68+0j), (-2.0712368174477944e-41+0j), (-3.7090222383503156e-68+0j), (2.0692819644677496e-95+0j), (2.076420742061022e-24+0j), (-1.6120477477871826e-26+0j), (3.3646047575775124e-34+0j), (-2.6496260084563345e-26+0j), (1.7580947226775096e-35+0j), (-2.105801267264119e-42+0j), (5.047664865263311e-35+0j), (-7.834873193102977e-43+0j), (-4.0961365537414474e-67+0j), (-1.1456942470886843e-25+0j), (1.3996595815212546e-35+0j), (-2.669346581865447e-41+0j), (2.3327861637229725e-36+0j), (-1.1613917881747074e-42+0j), (-6.062937539783411e-68+0j), (-5.6171131049792186e-42+0j), (-4.036464467189702e-68+0j), (2.023818714534639e-95+0j), (-9.573806888833474e-37+0j), (-2.1799551356086825e-42+0j), (-5.6238367683688214e-68+0j), (-2.6657291380905258e-42+0j), (-4.601975454345726e-69+0j), (-4.296964881204298e-96+0j), (-3.5352801086283497e-68+0j), (2.9212948084155304e-98+0j), (9.369741175469308e-270+0j), (6.5731609591505825e-18+0j), (6.207767344237604e-25+0j), (-9.092523999679525e-24+0j), (1.3678469443297839e-24+0j), (-6.43188166266487e-25+0j), (1.666613660625515e-34+0j), (-6.202750413066258e-24+0j), (2.0072940868787412e-34+0j), (1.665443224850045e-41+0j), (1.6691287012517927e-24+0j), (-2.591331356654345e-25+0j), (2.557588941187403e-34+0j), (-1.839449975469433e-25+0j), (2.419419964626565e-35+0j), (2.203541835150775e-42+0j), (1.0718078592088882e-34+0j), (-1.9029282820914935e-41+0j), (-7.484000101586608e-67+0j), (-1.563982205577665e-24+0j), (1.4560479775307905e-34+0j), (-3.3148906277298224e-40+0j), (8.950241310449069e-35+0j), (-2.7856258905293503e-41+0j), (-3.1815629765839493e-67+0j), (-4.135863885201677e-40+0j), (-2.340553783584839e-67+0j), (1.8763894272673194e-93+0j), (2.1344493502490796e-24+0j), (-2.091439804871605e-26+0j), (5.299902307985993e-34+0j), (-6.196186866148959e-26+0j), (6.275836852128895e-36+0j), (-5.6075585484578164e-42+0j), (9.829529366168873e-35+0j), (-1.9776755127545447e-42+0j), (-5.47169781323459e-67+0j), (-6.831219894851874e-26+0j), (2.1195824286214084e-35+0j), (-1.676202569621439e-41+0j), (1.6699954472906236e-35+0j), (-8.212758503589944e-43+0j), (-5.603855078223344e-68+0j), (-1.577520641174618e-41+0j), (-2.069173016942004e-68+0j), (3.983194951637829e-95+0j), (7.479469876798928e-35+0j), (-1.686609126556795e-42+0j), (-2.4728766640124555e-67+0j), (-1.5366581484653618e-41+0j), (-3.9872472547950855e-69+0j), (6.178402242411182e-96+0j), (-6.548590631720684e-68+0j), (4.094338952084482e-96+0j), (7.53872199050953e-270+0j), (-4.114461823361231e-26+0j), (2.2359567097982767e-35+0j), (-5.987394072291931e-40+0j), (5.38142242895351e-35+0j), (-1.3132579882053455e-41+0j), (-2.0665185793218894e-67+0j), (-8.9042129461902e-41+0j), (-7.646373363038756e-68+0j), (-5.18254969465892e-95+0j), (1.7671284206497622e-35+0j), (-1.1123678266751844e-41+0j), (-1.2532835540531952e-67+0j), (-4.5368383145360425e-42+0j), (-1.5829484386659024e-68+0j), (-4.349638269868128e-96+0j), (-1.0735903390114084e-68+0j), (-2.8075835459880108e-96+0j), (-1.868135871884139e-271+0j), (-5.088787490914118e-41+0j), (-1.8418440984329086e-68+0j), (-1.2239037849117026e-95+0j), (-8.462726838152304e-69+0j), (2.9861597854261134e-97+0j), (-7.113259819924929e-272+0j), (-5.680776971768795e-97+0j), (2.4299202637170184e-273+0j), 0j, (8.991170031809673e-17+0j), (0.002321597608482496+0j), (1.3086448246848053e-18+0j), (-0.0018956940789787245+0j), (5.363938695979127e-19+0j), (5.717472393966527e-20+0j), (1.0912152763237783e-18+0j), (5.320425699941074e-20+0j), (-4.737270023390864e-23+0j), (-0.0013628000118285869+0j), (1.7220596688997907e-19+0j), (1.1858461261560205e-20+0j), (-3.735101216754568e-19+0j), (1.1117307432712692e-21+0j), (1.668507641999642e-24+0j), (5.082197683525802e-21+0j), (-2.328762681470955e-24+0j), (1.966764100402697e-25+0j), (7.047825610229119e-19+0j), (5.346895479542771e-20+0j), (1.5489255540684982e-23+0j), (4.658681209898652e-20+0j), (-2.5420272025732576e-24+0j), (2.808325583750364e-26+0j), (-3.334571844354393e-24+0j), (2.7146492672757985e-26+0j), (-1.2473551673884348e-34+0j), (0.0013893616059800094+0j), (-3.1240160087368747e-19+0j), (-7.792703114739563e-20+0j), (2.2299201134108244e-19+0j), (5.135137242729196e-21+0j), (-7.195462116489357e-24+0j), (2.710505431213761e-20+0j), (-1.9710163033490113e-25+0j), (4.186732127014649e-25+0j), (2.108108255400964e-19+0j), (1.5828928201814738e-20+0j), (-1.2323621791649623e-24+0j), (-2.6469779601696886e-23+0j), (-2.5036551865542384e-25+0j), (2.886193364694054e-26+0j), (1.6349741794993438e-24+0j), (8.944857322598776e-27+0j), (-2.915072452569134e-36+0j), (6.988021814847978e-21+0j), (7.022755322639535e-26+0j), (2.0692172585003662e-25+0j), (4.850800369512628e-25+0j), (4.752665687363557e-27+0j), (1.2795410981100366e-35+0j), (-1.4135625864379815e-27+0j), (2.1534760194096235e-36+0j), (-2.1096657225549015e-42+0j), (3.7523657926968563e-19+0j), (1.2705494208814505e-20+0j), (7.108583389127582e-24+0j), (2.5040411503205254e-20+0j), (1.3041827138348273e-24+0j), (3.793388655662875e-26+0j), (6.57382404780912e-24+0j), (6.442566951573366e-27+0j), (7.0855593260807875e-34+0j), (3.1075521252392144e-20+0j), (1.1933130467714454e-24+0j), (1.1617399297293854e-25+0j), (3.849136346499196e-25+0j), (4.416273487827118e-27+0j), (2.5233237499035554e-37+0j), (-1.6171729690144543e-26+0j), (3.201947350382931e-36+0j), (2.090737308772627e-42+0j), (9.649498053957219e-24+0j), (1.145259601801659e-28+0j), (8.327595826359936e-34+0j), (3.65797767351902e-27+0j), (5.289154848860018e-35+0j), (1.6955711418330287e-42+0j), (6.2449892467798e-34+0j), (5.612200349620892e-43+0j), (-3.509074420440241e-66+0j), (0.0003855728828853437+0j), (1.4258631643598985e-20+0j), (1.3417001884508117e-18+0j), (1.490817311732738e-19+0j), (-2.0328790734103208e-20+0j), (5.343037448560386e-25+0j), (-2.456395547037471e-19+0j), (-2.42741966867368e-25+0j), (4.9362442408606234e-24+0j), (-2.3290489705921203e-20+0j), (9.317362419797304e-21+0j), (-1.1292954065909499e-24+0j), (5.823351512373315e-21+0j), (2.428934281611704e-25+0j), (1.834382064056787e-25+0j), (6.4259978250579445e-25+0j), (1.815950163617064e-25+0j), (1.8151776561559557e-35+0j), (-2.1429933565533799e-19+0j), (5.606087354607434e-25+0j), (1.241803458948253e-25+0j), (6.4497267610869925e-25+0j), (3.8669719414997743e-26+0j), (3.9006703688866664e-35+0j), (4.077709817649264e-25+0j), (6.741696506621944e-36+0j), (-7.883148163449644e-41+0j), (-3.4674987874226236e-20+0j), (-1.0164395367051604e-20+0j), (-1.3995023547344926e-24+0j), (-2.4352197233561135e-21+0j), (1.4030364515898828e-25+0j), (1.9490878656656866e-25+0j), (6.963180547077245e-25+0j), (6.737553095287718e-27+0j), (6.368007110782872e-35+0j), (8.787966827763366e-21+0j), (-1.6163444403616443e-25+0j), (2.3254674524579613e-27+0j), (1.315946207653483e-25+0j), (3.095610593436197e-27+0j), (3.8833908566295876e-36+0j), (-8.135092825550884e-27+0j), (-8.334652871252336e-37+0j), (3.67700717038832e-42+0j), (9.785914192575278e-25+0j), (4.704294579745088e-27+0j), (8.95345188792866e-35+0j), (-8.192512010403297e-27+0j), (5.201365804925782e-36+0j), (-4.0637655465419695e-43+0j), (4.139304092900097e-35+0j), (6.20775219695894e-43+0j), (-3.4300083926631936e-66+0j), (-5.897466895258066e-20+0j), (1.9062666002484706e-25+0j), (-6.742491235490764e-25+0j), (3.3874318358914424e-25+0j), (9.958661230096437e-27+0j), (-2.961780129408051e-36+0j), (-3.3828120308795903e-25+0j), (1.0851243527769851e-35+0j), (5.262155993232553e-41+0j), (7.319367023002636e-25+0j), (2.7842112415420104e-26+0j), (6.675489902902833e-35+0j), (-3.796031916351784e-26+0j), (9.818543026285379e-36+0j), (-6.21651031236097e-42+0j), (2.694337528414028e-35+0j), (-3.585046958667004e-42+0j), (-1.2512564144434478e-68+0j), (-7.7157593541473385e-25+0j), (2.878669092826093e-35+0j), (-1.8442489088978917e-41+0j), (2.7628884212268263e-35+0j), (-1.495535786050661e-42+0j), (-3.1922211649747006e-68+0j), (-1.6251821683469127e-41+0j), (-7.862377503908151e-68+0j), (1.7936923919703855e-93+0j), (1.6961842450819438e-19+0j), (5.1880768019325896e-21+0j), (-9.919705002100762e-25+0j), (2.8322664173815668e-21+0j), (2.8712012595148122e-25+0j), (5.2778367602729e-27+0j), (7.232377753262105e-24+0j), (3.0202915259984332e-27+0j), (3.0689678228559587e-34+0j), (1.1249656330721176e-20+0j), (5.059816954959987e-25+0j), (6.208104048681421e-26+0j), (2.6518348189909533e-25+0j), (-1.4066734829995284e-28+0j), (2.5291537658443934e-35+0j), (-2.649226290414633e-26+0j), (9.656703580209927e-36+0j), (4.685942064702188e-42+0j), (3.721202040334073e-24+0j), (-9.69493781594206e-28+0j), (5.382953544884661e-34+0j), (-3.6571171712958925e-27+0j), (1.8646688251807792e-35+0j), (-6.3618950280346695e-43+0j), (3.27857011499407e-34+0j), (-3.215979975625455e-43+0j), (-8.497989060230959e-67+0j), (7.815202427401005e-21+0j), (7.058096291193437e-26+0j), (-1.0602440063331272e-26+0j), (2.705477360545982e-25+0j), (-3.255986262414049e-27+0j), (2.2549314497039025e-35+0j), (-4.6697849752008284e-26+0j), (4.386416928186117e-36+0j), (-4.713267384756522e-42+0j), (2.074704180731261e-25+0j), (-2.3260318104356237e-27+0j), (2.1624306742080272e-35+0j), (-7.354208168164538e-27+0j), (-2.3022375280707146e-37+0j), (-7.824500300173697e-43+0j), (9.880995315720163e-36+0j), (1.1319864157123913e-43+0j), (8.210379096378027e-68+0j), (-2.6399405328697647e-26+0j), (3.534906592446252e-36+0j), (2.3156457122967602e-43+0j), (7.73778846968171e-36+0j), (-1.1476415469935201e-43+0j), (-2.283538478030907e-68+0j), (-1.2707149636805482e-42+0j), (-1.2201669137598392e-68+0j), (-3.2743042373968055e-95+0j), (1.9060267865332834e-24+0j), (-3.846793833114066e-27+0j), (1.967401508308828e-34+0j), (-4.020207725923858e-27+0j), (1.0948642075135125e-35+0j), (-2.957615571265567e-43+0j), (1.2705897424306465e-34+0j), (-1.0373987693704661e-43+0j), (-6.560569469889006e-67+0j), (-1.2084244187792717e-26+0j), (8.385557910630787e-36+0j), (-2.758631189331443e-42+0j), (4.310654341879189e-36+0j), (-1.5506243319294304e-43+0j), (-2.6452818482032203e-68+0j), (-9.320933793053066e-43+0j), (-8.141164691088869e-69+0j), (5.601967386604963e-96+0j), (1.1193566286108796e-35+0j), (-1.40176373920555e-43+0j), (-1.0930821319757607e-67+0j), (-2.458656157623191e-43+0j), (-4.3015187538885315e-69+0j), (3.0827067567023143e-96+0j), (7.381934399898831e-69+0j), (-5.981666709695276e-97+0j), (-4.4314843856290795e-268+0j), (-0.00011075716875448734+0j), (1.8848670368907326e-20+0j), (1.0130514049161432e-18+0j), (-1.6675475009354502e-20+0j), (-1.8740603958001395e-20+0j), (-2.809102129055814e-25+0j), (-6.598386659111e-19+0j), (-4.1778073540504785e-26+0j), (1.6904805946229826e-24+0j), (4.076667804778261e-21+0j), (-1.2811373327221293e-20+0j), (1.6054897143058032e-25+0j), (-3.4728350837426314e-20+0j), (2.702953005649275e-26+0j), (4.613479013492194e-26+0j), (2.8479771944651057e-25+0j), (4.3247687998747313e-26+0j), (2.624151627177726e-35+0j), (-2.0720543472208322e-19+0j), (-4.397426230064008e-26+0j), (-2.402750529435941e-25+0j), (1.2826247230169475e-25+0j), (-2.061600342518463e-26+0j), (2.502164582539449e-35+0j), (-7.677469225502603e-25+0j), (9.310696667377775e-36+0j), (-1.6799887029097367e-40+0j), (2.604629841586352e-20+0j), (2.8024879153296578e-21+0j), (-1.1236408516223257e-24+0j), (-1.63649412387491e-20+0j), (-1.7531644757631766e-26+0j), (5.437695325871271e-26+0j), (9.348695924465584e-25+0j), (-1.7942943584698283e-27+0j), (5.043320794832028e-35+0j), (-3.851352932046897e-21+0j), (1.3233930545987694e-25+0j), (-2.5973663403979783e-26+0j), (5.828104367772835e-26+0j), (-2.2455377465445797e-27+0j), (3.69369564858431e-36+0j), (-2.806385449792704e-26+0j), (2.9868877881439076e-36+0j), (-7.076557244840326e-44+0j), (5.284358323022096e-25+0j), (-2.992706657008973e-27+0j), (1.0004446421579683e-34+0j), (-3.305284435556654e-26+0j), (9.355584954342552e-37+0j), (7.693128569143246e-43+0j), (2.0539653593288017e-35+0j), (-4.288849112374143e-43+0j), (9.311624314768177e-68+0j), (3.824287582404162e-19+0j), (4.5779176051785757e-26+0j), (-2.2491384444588054e-25+0j), (1.7899569483826846e-25+0j), (-1.964019687484466e-26+0j), (1.1457918276956772e-35+0j), (-3.2461882311535193e-25+0j), (8.407097221681776e-36+0j), (7.67981623373216e-42+0j), (2.1517601139532495e-25+0j), (-2.6746053379622825e-26+0j), (1.634209836961021e-35+0j), (-1.8469182029384528e-26+0j), (2.248045800597427e-36+0j), (-3.9069952808456306e-43+0j), (1.3854218942551618e-35+0j), (-1.479858759481027e-42+0j), (-5.3050235991599736e-68+0j), (-4.013122285211773e-25+0j), (6.961895670093107e-36+0j), (-2.4495748130246046e-41+0j), (7.167661630340138e-36+0j), (-8.0505691539886e-43+0j), (-5.933787581911715e-68+0j), (-3.1350571788395475e-41+0j), (-3.6598268114161825e-68+0j), (2.0265832594773717e-95+0j), (2.3637525625837968e-20+0j), (1.852884572118782e-21+0j), (1.2155273698624691e-24+0j), (-5.492479267352104e-22+0j), (-1.4022791451208706e-26+0j), (1.557953561623057e-27+0j), (1.1048344076418568e-24+0j), (-2.925025488666467e-28+0j), (2.617910956798282e-35+0j), (-9.568825326013424e-21+0j), (6.035101469302829e-26+0j), (-3.395150791307902e-26+0j), (1.2858432755102492e-26+0j), (-7.543828640245637e-28+0j), (2.782230860343095e-36+0j), (-2.0339133727051078e-26+0j), (2.585990456220261e-36+0j), (-1.4419361197902368e-42+0j), (8.074906443587112e-25+0j), (-1.7359957080881555e-27+0j), (7.874748697257479e-35+0j), (-3.2006129737726897e-27+0j), (2.8082481439510988e-36+0j), (1.1070257868166055e-43+0j), (3.695576806573719e-35+0j), (-2.701878601526288e-43+0j), (-7.40128535934398e-68+0j), (5.795227371546512e-21+0j), (1.012029655308036e-26+0j), (-2.975056975355448e-26+0j), (-2.792567604482382e-28+0j), (-3.343103515956183e-27+0j), (5.828156922243287e-36+0j), (-2.1669312002891596e-26+0j), (1.4418949972855027e-36+0j), (-4.135406930530576e-42+0j), (5.557840621644117e-26+0j), (-2.9069343053534433e-27+0j), (3.018600915606868e-36+0j), (-3.750543580209665e-27+0j), (3.248446512609058e-37+0j), (-3.7292055381844194e-43+0j), (2.8449470468589005e-36+0j), (-4.483060321414161e-44+0j), (-3.846365780543048e-68+0j), (-1.7709455744851552e-26+0j), (2.630410278598675e-36+0j), (-2.486341381482327e-42+0j), (2.526882454728203e-36+0j), (-1.5048084407325604e-43+0j), (-1.4959747627489557e-68+0j), (-1.1551078403737508e-42+0j), (-5.286615129276238e-69+0j), (-5.292349448033216e-95+0j), (2.259723617441797e-25+0j), (-1.1730854857352502e-27+0j), (3.195780593172546e-35+0j), (-3.143610707305732e-27+0j), (1.3883091840865344e-36+0j), (-1.4245074701401969e-43+0j), (2.02138796653453e-35+0j), (-7.346142984559065e-44+0j), (-5.718439084187321e-68+0j), (-1.105509672576726e-26+0j), (1.770330830740077e-36+0j), (-7.303501910195431e-43+0j), (1.3517570819940858e-36+0j), (-5.309196925821282e-44+0j), (-2.7800288006636113e-69+0j), (-1.3279424055551571e-42+0j), (-3.299229506538968e-69+0j), (8.1596357972605e-96+0j), (1.483160752075995e-36+0j), (-5.728765391799787e-44+0j), (-3.3208365959512933e-69+0j), (-8.38158486251939e-44+0j), (-4.512656599983525e-70+0j), (1.0616177672943232e-96+0j), (-2.0055739194450125e-69+0j), (1.9773633384305614e-97+0j), (2.6575981224214598e-269+0j), (2.6583930526229203e-19+0j), (5.0596749599970476e-26+0j), (-6.326657438985272e-27+0j), (4.897090727430997e-26+0j), (-2.385364437324347e-26+0j), (8.798341686713145e-36+0j), (-2.4582127131620198e-25+0j), (4.7401643923300414e-36+0j), (-4.1143524211040954e-41+0j), (1.7649500576871786e-25+0j), (-1.3381308354798994e-26+0j), (2.555419484956488e-35+0j), (-7.658412575056502e-27+0j), (7.0940382067136624e-37+0j), (-9.889663911972396e-43+0j), (9.12406984999964e-36+0j), (-3.754713549292832e-43+0j), (-3.517853099533911e-68+0j), (-6.099235860897297e-26+0j), (5.4169423998801296e-36+0j), (-1.223552512240616e-41+0j), (5.895358352040771e-36+0j), (-7.286204632276422e-43+0j), (-1.1739392343817543e-68+0j), (-1.2743022330109984e-41+0j), (-1.3629072009448281e-68+0j), (4.665244013404854e-96+0j), (1.7882293430002506e-25+0j), (-7.200287085675407e-28+0j), (-4.7408061393825544e-36+0j), (-7.707762689688201e-27+0j), (1.6232721068425697e-36+0j), (-1.0927172157785388e-42+0j), (5.9274024440429725e-36+0j), (-1.3700155708731908e-43+0j), (-3.290864491279815e-68+0j), (-4.23756356762097e-27+0j), (1.7277675499423862e-36+0j), (-1.1680260605917452e-42+0j), (5.074784229037945e-37+0j), (-1.3357562866404058e-43+0j), (-2.8430012470616387e-69+0j), (-5.652444916234869e-43+0j), (-1.6082652906712833e-69+0j), (2.389165517605754e-96+0j), (4.126256045422828e-36+0j), (-1.0584250886129962e-43+0j), (-4.783350423483498e-69+0j), (-5.729053622746123e-43+0j), (-9.380222637219421e-70+0j), (4.1600910388521043e-97+0j), (-2.5268964281671063e-69+0j), (3.692217619387767e-98+0j), (8.03169658622789e-270+0j), (-1.4088069691115745e-26+0j), (6.842113967454916e-37+0j), (-2.0304803800422347e-41+0j), (9.400186864826242e-37+0j), (-4.62670111156982e-43+0j), (-5.969619311325131e-69+0j), (-4.3654912177532143e-42+0j), (-1.5459420597120572e-69+0j), (7.932680125096164e-96+0j), (9.468552653688768e-37+0j), (-6.065724473761116e-43+0j), (-5.70533748287481e-69+0j), (-1.889702595574097e-43+0j), (-2.9057342860348285e-70+0j), (2.1523141394480655e-97+0j), (-1.627241262886207e-69+0j), (1.647463139352667e-97+0j), (-5.438997428726304e-271+0j), (-3.623465214463516e-42+0j), (-1.9201539722920634e-69+0j), (7.777387534982152e-97+0j), (-1.4363851321003755e-69+0j), (3.203615100865536e-98+0j), (-1.0195773804928808e-271+0j), (8.821605154431707e-98+0j), (7.525756451294043e-273+0j), 0j, (5.278074052578359e-20+0j), (1.3540946627493063e-21+0j), (5.886997567513015e-24+0j), (2.735899876019139e-21+0j), (1.5884503187530294e-25+0j), (-1.0250568373391283e-27+0j), (2.5953397564026453e-24+0j), (3.6179274378774364e-28+0j), (3.0550445356637235e-34+0j), (7.877240973342483e-21+0j), (1.6862059621280172e-25+0j), (-2.5366703991306503e-26+0j), (1.0079749102552e-25+0j), (-7.744283849420942e-28+0j), (1.6408630468057653e-35+0j), (-2.1074599757016123e-26+0j), (1.4538675478717312e-35+0j), (-1.0010525904520412e-42+0j), (2.3207278089643484e-24+0j), (-3.377897731907756e-27+0j), (2.004545741164086e-34+0j), (-4.1033246988557445e-27+0j), (1.0518466792920471e-35+0j), (-2.8687206999349614e-43+0j), (1.0274712607870495e-34+0j), (-2.0606750776551587e-43+0j), (-3.413198570737356e-67+0j), (4.0663785322800564e-20+0j), (1.0775445534866411e-26+0j), (-7.294790580455783e-26+0j), (1.7778873765328031e-25+0j), (-1.779688378159898e-27+0j), (1.6021867809535897e-35+0j), (-2.750757976505668e-26+0j), (9.071239306369864e-37+0j), (-3.449164798204507e-42+0j), (1.1307847759800072e-25+0j), (-1.0686359187552235e-27+0j), (1.0583622025242965e-35+0j), (-1.6842180326468602e-27+0j), (4.854260177057681e-37+0j), (-9.290170912703436e-44+0j), (5.689248952019758e-36+0j), (-7.198076096043494e-44+0j), (-1.1772729566772395e-68+0j), (-2.6036354176819495e-26+0j), (8.122074410589512e-37+0j), (-8.396624188811314e-43+0j), (5.4233457445285086e-36+0j), (-7.07683093594664e-44+0j), (-1.4788016653359749e-68+0j), (-1.9615941075753327e-42+0j), (-2.478165743110351e-69+0j), (1.3802900133137487e-95+0j), (7.763590375950692e-25+0j), (-8.794813017082755e-28+0j), (1.1893576682696511e-34+0j), (-3.245768194531853e-27+0j), (3.851174115723303e-36+0j), (-3.363116314379561e-44+0j), (2.1243550976304823e-35+0j), (-5.807759487359504e-44+0j), (-6.081264845328194e-68+0j), (-4.2319429336712705e-27+0j), (5.976881466796628e-36+0j), (-9.052032281100111e-43+0j), (2.564022889134511e-36+0j), (-3.526971758590589e-44+0j), (-6.450089104127644e-69+0j), (-6.545081617198499e-43+0j), (-7.458778272163556e-69+0j), (-2.2958670231099867e-97+0j), (2.106677469234538e-35+0j), (-3.8365677169135244e-44+0j), (-4.489702787690276e-68+0j), (-2.5431185159585796e-43+0j), (-1.6116728442371695e-69+0j), (3.3498475501296134e-97+0j), (7.53683209775352e-69+0j), (2.4192332933258763e-97+0j), (-1.9925250085928447e-271+0j), (2.221476253072411e-19+0j), (3.6359782412994266e-26+0j), (-5.024671095658322e-25+0j), (5.489604153342499e-26+0j), (-8.994151307622164e-27+0j), (8.29548330994514e-36+0j), (-1.5677979402543433e-25+0j), (2.3421384319509863e-36+0j), (-2.1500209919289689e-41+0j), (1.9755680307721365e-25+0j), (-1.6712461305654552e-26+0j), (1.3506833995612247e-35+0j), (2.792567604482382e-28+0j), (1.5436656830027187e-36+0j), (-1.4199860930638988e-42+0j), (4.7598977502075394e-36+0j), (-6.978356875895064e-43+0j), (-1.9550585488678803e-68+0j), (-1.0064918517533845e-25+0j), (4.8028313925691e-36+0j), (-1.4452860989315133e-41+0j), (2.068002989374477e-36+0j), (-1.0927777699358106e-42+0j), (-9.707594545148875e-69+0j), (-1.4086903137241305e-41+0j), (-8.869915590515843e-69+0j), (-1.094166289399175e-95+0j), (1.209473651275803e-25+0j), (-9.613749244315318e-28+0j), (1.248893962876579e-35+0j), (-5.515814056918467e-27+0j), (5.3654047365152366e-37+0j), (-1.5255350682136773e-42+0j), (4.179337943943093e-36+0j), (-1.7276751086035953e-44+0j), (-1.1793207227506894e-68+0j), (-3.4413070914135114e-27+0j), (8.0819345554559e-37+0j), (-1.0045202567690325e-42+0j), (1.7445481283047813e-37+0j), (-9.9444038438045e-44+0j), (-2.155142412580452e-69+0j), (-5.1495456335886405e-43+0j), (-7.975483991036235e-70+0j), (-1.1845317808605822e-97+0j), (1.9533992222614353e-36+0j), (-9.967274156875816e-44+0j), (-6.3723919189472e-69+0j), (-4.1005211930139806e-43+0j), (-3.600267112912145e-70+0j), (7.740934036856142e-97+0j), (-8.43606146821696e-71+0j), (8.65758436109994e-98+0j), (-4.1734048387121755e-271+0j), (-1.6295820193893187e-26+0j), (7.849759503459586e-37+0j), (-1.6772362325072084e-41+0j), (2.2300295412806826e-36+0j), (-3.1395622114250904e-43+0j), (-6.280314268631737e-69+0j), (-2.413198021328296e-42+0j), (-6.570090911566056e-70+0j), (3.3677820540849356e-97+0j), (1.6372234078292982e-36+0j), (-5.208951998783387e-43+0j), (-4.3103371984478473e-69+0j), (-1.875872588402902e-43+0j), (-3.589171988150238e-70+0j), (1.3007633901327432e-97+0j), (-1.3839093935992112e-69+0j), (6.027161945891421e-98+0j), (1.1400159748226075e-272+0j), (-2.4667119773699972e-42+0j), (-1.3046823712438153e-69+0j), (9.430148552504567e-97+0j), (-9.9870698215826e-70+0j), (-3.1127276264098276e-98+0j), (5.427052394354514e-273+0j), (7.672064034759367e-98+0j), (-2.3254809617060014e-274+0j), 0j, (4.190139222151346e-25+0j), (-3.8235101999930916e-28+0j), (5.404303060197052e-35+0j), (-1.2353808294793926e-27+0j), (1.542632547224125e-36+0j), (-2.1515610517841946e-43+0j), (1.2672415701768154e-35+0j), (-1.0723422813690347e-43+0j), (-7.192710639198663e-68+0j), (-2.0017838508048938e-27+0j), (7.682467345425114e-37+0j), (-1.3718038001390662e-42+0j), (6.4308419528865814e-37+0j), (-4.605789400478058e-44+0j), (-1.0465297844795879e-69+0j), (-9.143643109018524e-43+0j), (-8.82462610043669e-70+0j), (1.2400955179003746e-96+0j), (4.399231416840542e-36+0j), (-5.051200293885503e-44+0j), (-9.064179333222213e-69+0j), (-5.48301269500094e-44+0j), (-4.679903456579738e-70+0j), (2.531348704343466e-98+0j), (6.361797461742911e-69+0j), (8.720425700525725e-98+0j), (-3.510670577044078e-272+0j), (-1.590996860362696e-27+0j), (2.026777659783992e-37+0j), (-2.243300428994104e-42+0j), (4.195012366148898e-37+0j), (-7.298629780860698e-44+0j), (-1.7636251085050692e-69+0j), (-2.417577688419665e-43+0j), (-3.365276643187757e-70+0j), (4.742520644733228e-97+0j), (3.245269958795216e-37+0j), (-3.1884441913867897e-44+0j), (-1.3847392963987278e-69+0j), (-5.224754265761807e-44+0j), (-1.3817130244571329e-70+0j), (-1.1543466798026012e-98+0j), (-2.6859285642785636e-70+0j), (-2.707785861173977e-101+0j), (9.654971630401272e-273+0j), (-1.79562074322591e-43+0j), (-8.562890613961047e-71+0j), (-2.8629868922617327e-98+0j), (-1.9763091028696256e-70+0j), (2.172635184657147e-99+0j), (8.575393010328046e-274+0j), (-6.153603378539098e-99+0j), (-1.580656708474218e-274+0j), 0j, (6.30067199563721e-37+0j), (-2.605536525701011e-44+0j), (-2.636913063437843e-69+0j), (-1.703553707781043e-44+0j), (-1.6529709521668896e-70+0j), (-2.88527903019613e-98+0j), (-1.2531525508845998e-69+0j), (2.6616223456408883e-99+0j), (-1.1141613662591068e-274+0j), (-7.516258277588042e-44+0j), (3.950805301236166e-72+0j), (-2.8431662452181433e-98+0j), (-1.443243326040132e-70+0j), (2.167197994761736e-99+0j), (-1.4984098648084e-274+0j), (-1.9993284277143897e-98+0j), (5.369604622165126e-276+0j), 0j, (-2.8138046575205892e-70+0j), (7.060246268950361e-100+0j), (-3.1440573293722244e-276+0j), (1.040968750907755e-99+0j), (2.6976371180650453e-276+0j), 0j, (-3.8925516318945613e-278+0j), 0j, 0j, (-8.901099949614787e-06+0j), (1.2706445971211075e-20+0j), (6.315477654728063e-18+0j), (1.4426177419382986e-20+0j), (-9.359714067160019e-20+0j), (4.156097901938796e-25+0j), (-1.0604005466676586e-17+0j), (8.053701862454771e-25+0j), (1.8927152905020276e-23+0j), (4.09223637457584e-20+0j), (-3.7269449679189215e-20+0j), (2.6342148218119368e-24+0j), (-2.998496633280223e-19+0j), (1.742814620686677e-25+0j), (3.346563746532845e-25+0j), (2.4263089525191287e-24+0j), (1.8606927988668576e-25+0j), (3.1015885444293316e-34+0j), (-2.1192764340302594e-18+0j), (1.544400325805488e-24+0j), (-1.9341591577294497e-23+0j), (1.7259014428787385e-24+0j), (-4.728069694316194e-25+0j), (2.1908881880422202e-34+0j), (-1.1897476753015654e-23+0j), (2.0974773525990506e-34+0j), (-9.57367110826715e-40+0j), (4.8280885830303276e-20+0j), (1.0587911840678754e-21+0j), (4.750230070427812e-24+0j), (-8.732380290599803e-20+0j), (1.4918937439539775e-25+0j), (6.922230297271566e-26+0j), (2.0845112995049687e-24+0j), (4.879456151876589e-28+0j), (2.263641414248621e-34+0j), (2.0090562717687936e-19+0j), (1.7224935637681837e-25+0j), (-3.9990792973435305e-25+0j), (3.4589972972130926e-25+0j), (-1.8981223460188314e-26+0j), (2.7300812756701744e-35+0j), (-4.719939591556791e-25+0j), (2.32316548791444e-35+0j), (-7.166240346557115e-42+0j), (2.146761891257769e-24+0j), (-7.512626982726675e-26+0j), (2.738900702924693e-34+0j), (-4.366352333458873e-25+0j), (1.5191538555506302e-35+0j), (-1.1860590202045252e-41+0j), (1.0190981140508004e-34+0j), (-3.6622935365129094e-42+0j), (-3.686648838722467e-67+0j), (8.88521679802e-18+0j), (5.333330808018217e-25+0j), (-1.4665916904431668e-23+0j), (1.2749001970330234e-24+0j), (-4.624549009457118e-25+0j), (1.6966847485658492e-34+0j), (-5.4926999894197884e-24+0j), (1.0713785094308982e-34+0j), (-1.2756720569980972e-40+0j), (1.5872260066281264e-24+0j), (-6.114043344990194e-25+0j), (3.1253407805220923e-34+0j), (-3.3509423351911853e-25+0j), (1.654694106474584e-35+0j), (-6.180076552288524e-42+0j), (1.0773884218545347e-34+0j), (-1.4738200389113274e-41+0j), (-4.3113850409663004e-67+0j), (-4.505654314907952e-24+0j), (8.66000680664361e-35+0j), (-4.460248934038033e-40+0j), (1.1948112761470916e-34+0j), (-2.0446958961041548e-41+0j), (-3.1913965846374004e-67+0j), (-3.774963625819636e-40+0j), (-2.6532028682382783e-67+0j), (-1.3825672915695916e-93+0j), (7.522711511944396e-20+0j), (5.135137242729196e-21+0j), (3.6273970123724324e-24+0j), (7.279189390466644e-22+0j), (3.668392535894958e-25+0j), (-1.1040402325629824e-26+0j), (3.927896219276462e-24+0j), (2.2118399023868506e-28+0j), (4.745222257728523e-34+0j), (2.882558998624791e-20+0j), (1.741804878727994e-25+0j), (-2.718411223976174e-25+0j), (3.1390037596192413e-25+0j), (-3.237691516797735e-26+0j), (3.059545179660437e-35+0j), (-2.531026460122931e-25+0j), (2.757996369738018e-35+0j), (-1.4307257320756382e-41+0j), (2.7847042389791387e-24+0j), (-3.789786898248042e-26+0j), (2.9990407781039856e-34+0j), (-3.5875990540670793e-26+0j), (1.1327030550212003e-35+0j), (-2.3890387193657725e-42+0j), (1.5019260361849655e-34+0j), (2.5888989128400995e-43+0j), (-5.0440812935886995e-67+0j), (2.7264038425870303e-19+0j), (3.946828881001766e-26+0j), (-5.343784265800587e-25+0j), (2.5995806726291135e-25+0j), (-2.0324587075765373e-26+0j), (1.785882086918967e-35+0j), (-1.9923500228397387e-25+0j), (5.349931239701849e-36+0j), (-1.5483121894632944e-41+0j), (2.298125366307956e-25+0j), (-1.3994732425247365e-26+0j), (3.134503031017805e-35+0j), (-2.8092191679137645e-26+0j), (3.8022013976504636e-36+0j), (-2.6872525488048976e-42+0j), (1.0979082949008691e-35+0j), (-1.4922077021978896e-42+0j), (-6.506129680573343e-68+0j), (-1.263344962493346e-25+0j), (3.6234048206413366e-36+0j), (-2.1033183415476433e-41+0j), (7.674133838839215e-36+0j), (-1.2718754139713172e-42+0j), (-2.780996028278103e-68+0j), (-1.315574578151959e-41+0j), (-8.079490578136396e-69+0j), (7.383374169201289e-95+0j), (1.5600923469321444e-24+0j), (-1.2178714600259434e-26+0j), (2.3075011319976244e-34+0j), (-1.9404006116173838e-26+0j), (7.675061436590303e-36+0j), (-2.930312146499738e-42+0j), (5.544525660501672e-35+0j), (-1.3086731832151593e-42+0j), (-2.3286227829220384e-67+0j), (-1.9178628555552195e-25+0j), (9.960366707300572e-36+0j), (-2.3535158032951384e-41+0j), (5.403455121026307e-36+0j), (-1.2741826299975395e-42+0j), (-2.846340511598053e-68+0j), (-1.6231595910712564e-41+0j), (-1.7575901494516182e-68+0j), (-3.1775785443210176e-95+0j), (2.3563618651565513e-35+0j), (-2.2094864059799028e-42+0j), (-7.562878360030044e-69+0j), (-2.8496121000629813e-42+0j), (-3.719481702569212e-69+0j), (2.504908324751686e-96+0j), (-5.182178867558369e-69+0j), (-4.1185155352371447e-97+0j), (9.383150460360959e-270+0j), (2.9346581423258307e-18+0j), (3.7474048441618945e-25+0j), (-5.8145726483270526e-24+0j), (5.1599865127049515e-25+0j), (-9.659689016107576e-26+0j), (7.584639673213463e-35+0j), (-3.1297484562700896e-24+0j), (3.894902567234528e-35+0j), (-2.9196403828823645e-40+0j), (1.3000964143457825e-24+0j), (-1.8705154240238544e-25+0j), (1.2688988477440875e-34+0j), (-7.266513342352828e-26+0j), (9.299568310624305e-36+0j), (-1.4443161376507384e-41+0j), (6.584538094493258e-35+0j), (-1.778878335537139e-41+0j), (-1.454654692123489e-67+0j), (-3.9133811786203954e-26+0j), (6.174682810726116e-35+0j), (-3.076336423550671e-40+0j), (3.4627489893975167e-35+0j), (-1.1481473281579873e-41+0j), (-9.88199192435847e-68+0j), (-2.4957501701205067e-40+0j), (-4.995807420465759e-68+0j), (6.748714832478017e-96+0j), (8.660856546767175e-25+0j), (-8.620671972255217e-27+0j), (1.418551593354799e-34+0j), (-8.129842717026703e-26+0j), (5.1003282444084505e-36+0j), (-1.5057718334267837e-41+0j), (1.6964928457534294e-35+0j), (-1.6352961494896196e-42+0j), (-1.496609973453035e-67+0j), (-3.609334464225587e-26+0j), (7.349287705836555e-36+0j), (-1.6010168858060737e-41+0j), (4.902585132172329e-36+0j), (-1.2955100094570144e-42+0j), (-1.5946848108525642e-68+0j), (-8.17634971783095e-42+0j), (-7.383912670631358e-69+0j), (3.96583992708168e-96+0j), (1.955840029915209e-35+0j), (-1.5150503040446922e-42+0j), (-2.8370336306864253e-68+0j), (-6.324530033972993e-42+0j), (-5.637515601911159e-69+0j), (4.0264520220839656e-97+0j), (-1.729766522140354e-68+0j), (2.2569112790876904e-97+0j), (-2.031466036616041e-271+0j), (-8.112606106247624e-26+0j), (1.1160050315676182e-35+0j), (-3.899993933916805e-40+0j), (1.5304503394380868e-35+0j), (-1.0779959527635397e-41+0j), (-5.834710530809881e-68+0j), (-8.76779282875255e-41+0j), (-2.093972625657862e-68+0j), (-7.005956155183291e-95+0j), (1.2957800285373355e-35+0j), (-1.0219683384876207e-41+0j), (-4.3313814944137894e-68+0j), (-3.660002186018006e-42+0j), (-4.434803513877629e-69+0j), (-1.9757572502584045e-96+0j), (-8.152046973554295e-69+0j), (-6.677720528463707e-97+0j), (-1.368322787972925e-271+0j), (-5.510434529658061e-41+0j), (-1.6600522179147908e-68+0j), (5.4099987896466265e-96+0j), (-1.1981842250511698e-68+0j), (5.751319240738143e-98+0j), (4.493950078752478e-273+0j), (3.2152678128856337e-96+0j), (1.4981942266416975e-273+0j), 0j, (4.9604366973579964e-20+0j), (1.3416869535610109e-21+0j), (4.240714278076423e-24+0j), (7.792868550862074e-21+0j), (1.367758591908399e-25+0j), (-6.680333637982871e-28+0j), (3.0272063921313195e-24+0j), (-2.1760362156240186e-27+0j), (3.1080279633241523e-34+0j), (1.1330720030751373e-20+0j), (1.334752651633952e-25+0j), (-3.8063369899507395e-26+0j), (1.4035570997873286e-25+0j), (-1.9503160100840033e-27+0j), (1.4188904902417738e-35+0j), (-2.144039401328194e-26+0j), (1.3819746865804575e-35+0j), (-1.3281682007178657e-42+0j), (2.5189149119048337e-24+0j), (-1.4970101247586761e-27+0j), (2.2366373316657805e-34+0j), (-2.0282131566445773e-26+0j), (9.41019679459869e-36+0j), (-1.5299551795806393e-42+0j), (1.1652790071405208e-34+0j), (-2.5723679700187677e-43+0j), (-3.1617093132301566e-67+0j), (2.64910381434395e-19+0j), (3.6530176368522004e-26+0j), (-4.015897305797011e-25+0j), (1.453807539449907e-25+0j), (-1.429340001440334e-26+0j), (2.289595979660997e-35+0j), (-1.711733501020969e-25+0j), (3.759606655468929e-36+0j), (-1.3632882434800064e-41+0j), (1.2607653873332733e-25+0j), (-1.5355244451379296e-26+0j), (1.2832118977384227e-35+0j), (-9.98737349055062e-27+0j), (1.2926102262616019e-36+0j), (-1.2795716078808511e-42+0j), (5.866252073335748e-36+0j), (-1.1941307183119212e-42+0j), (-2.43948690035677e-68+0j), (-1.6455322938548218e-25+0j), (5.000462884945323e-36+0j), (-1.8625599386746862e-41+0j), (7.065362884126589e-36+0j), (-8.060695724922197e-43+0j), (-1.8140843397297065e-68+0j), (-1.9564052663135127e-41+0j), (-1.17182027141341e-68+0j), (1.2341278831677538e-95+0j), (9.460359185599619e-25+0j), (-2.0610963301161986e-27+0j), (1.0673965999213967e-34+0j), (-7.224092347174568e-27+0j), (3.626462136583972e-36+0j), (-1.2500485482428203e-42+0j), (2.505321624924175e-35+0j), (-1.3542543842883655e-43+0j), (-9.658408288808637e-68+0j), (-1.1494590857588516e-26+0j), (4.228358740106443e-36+0j), (-1.7318543717970015e-42+0j), (1.4631752646198583e-36+0j), (-1.3850925296922325e-43+0j), (-5.817241726621736e-69+0j), (-8.430288194887867e-43+0j), (-3.335213890894923e-69+0j), (1.7971404749336212e-96+0j), (8.433991250795409e-36+0j), (-2.2191097273917663e-43+0j), (-8.768029342417485e-69+0j), (-9.217359670292281e-43+0j), (-1.8026521597809047e-69+0j), (4.0203112326795996e-96+0j), (-7.847061179917469e-69+0j), (1.4574257584877784e-97+0j), (-2.2911786767549745e-271+0j), (1.1030122596149603e-19+0j), (1.7602839454327963e-26+0j), (-1.8695441812444102e-25+0j), (3.0392246880223617e-26+0j), (-7.394882832941537e-27+0j), (4.020897571128395e-36+0j), (-7.93862283360113e-26+0j), (1.9904073147345838e-36+0j), (-1.4015217962675688e-41+0j), (1.1450355482328247e-25+0j), (-6.360191048344408e-27+0j), (1.109017490088944e-35+0j), (-4.2733581311953736e-27+0j), (6.343831907661974e-37+0j), (-9.21797119885795e-43+0j), (6.881796691069182e-36+0j), (-8.434475668814463e-43+0j), (-1.263363980475294e-68+0j), (-2.6504937554520776e-26+0j), (2.7862856955677908e-36+0j), (-1.6020812705185267e-41+0j), (1.6506242996413446e-36+0j), (-4.584361952707838e-43+0j), (-3.5829107251425123e-69+0j), (-9.69550128310375e-42+0j), (-4.27466202607351e-69+0j), (-6.735023030886742e-97+0j), (1.1483960956890663e-25+0j), (-8.202674300101233e-28+0j), (1.3065010662337527e-35+0j), (-2.6425361210706606e-27+0j), (6.084738492117435e-37+0j), (-9.002193130640433e-43+0j), (2.931617273401603e-36+0j), (-1.1113997128093772e-43+0j), (-1.806158740327983e-68+0j), (-5.154417154714091e-27+0j), (7.533192916586e-37+0j), (-1.716508511466007e-42+0j), (7.514911254521772e-37+0j), (-1.104061370021348e-43+0j), (-1.1817373981758202e-69+0j), (-7.162032260269075e-43+0j), (-6.274478939638875e-70+0j), (1.5561859628414315e-96+0j), (1.6649575002104912e-36+0j), (-7.5773264409232e-44+0j), (-3.863349488564252e-69+0j), (-3.641023012104348e-43+0j), (-3.1949799057470676e-70+0j), (3.0251099702560426e-97+0j), (-1.6768839585573467e-69+0j), (1.2793829695568922e-97+0j), (-2.455870725742297e-271+0j), (-1.4067214180933964e-26+0j), (4.5433019917969535e-37+0j), (-1.152579926681675e-41+0j), (8.884294109565996e-37+0j), (-3.844838320032434e-43+0j), (-1.5694288326881503e-69+0j), (-3.0383003829434763e-42+0j), (-9.057151403299754e-70+0j), (-2.3618437263201076e-96+0j), (7.729500175039685e-37+0j), (-3.99289280691975e-43+0j), (-3.1047815853438685e-69+0j), (-2.024651020339594e-43+0j), (-9.535853845896391e-71+0j), (-1.0502239572794349e-97+0j), (-1.9713788869648933e-70+0j), (-1.9057830159135014e-98+0j), (1.1378748343218857e-272+0j), (-1.4464421204511576e-42+0j), (-4.193775965665889e-70+0j), (-1.8000094386226006e-99+0j), (-2.0895714171294626e-70+0j), (1.3438952299822721e-98+0j), (3.995276447260897e-273+0j), (5.6307875480721045e-98+0j), (-2.36153650922152e-274+0j), 0j, (4.299471399310452e-25+0j), (-6.526837914572346e-28+0j), (5.372989329587499e-35+0j), (-1.947648271184102e-27+0j), (1.743098350787002e-36+0j), (-6.30463200631624e-43+0j), (8.644365058716355e-36+0j), (-1.1065921574700401e-43+0j), (-5.038343044106251e-68+0j), (-2.6131264004478898e-27+0j), (1.7060117828027282e-36+0j), (-1.3405960006847347e-42+0j), (8.427143219841247e-37+0j), (-8.799596710330738e-44+0j), (-7.572383052773305e-70+0j), (-6.960962309340607e-43+0j), (-2.4555974509727725e-69+0j), (8.951209914350727e-98+0j), (3.0679047391534474e-36+0j), (-1.1249320274471624e-43+0j), (-7.276357106913752e-69+0j), (-2.4753814104389464e-43+0j), (-4.5776036708209676e-70+0j), (5.997625926868903e-98+0j), (-1.2285446149105464e-69+0j), (-1.7340988336647058e-98+0j), (3.0677852043187825e-273+0j), (-9.979768378386224e-27+0j), (2.960117029489505e-37+0j), (-1.1236119126072478e-41+0j), (9.350712483961331e-37+0j), (-3.2313026149766485e-43+0j), (-3.4131230925323155e-69+0j), (-2.372453952408709e-42+0j), (-6.175758294868414e-70+0j), (-4.160787751720758e-98+0j), (4.161472001475053e-37+0j), (-3.865984165312799e-43+0j), (-1.0618693308263554e-69+0j), (-1.3684376507712937e-43+0j), (-2.4230428239312893e-71+0j), (8.630069874254366e-99+0j), (-4.128823680002554e-70+0j), (1.1510306637945789e-98+0j), (4.764924202868176e-273+0j), (-1.0804037099125851e-42+0j), (-3.248548000797075e-70+0j), (1.2974702660409222e-97+0j), (-2.4913287418863938e-70+0j), (4.72381396929875e-99+0j), (-1.6520683400397958e-274+0j), (5.731291760144435e-99+0j), (-6.543227874900368e-275+0j), 0j, (-1.315767106529323e-37+0j), (-4.134564010032895e-44+0j), (2.4155501286175123e-71+0j), (-1.1565495410176245e-43+0j), (-2.3912210395696767e-70+0j), (-2.2134085939631436e-99+0j), (-9.167835963511497e-70+0j), (3.055410106945901e-99+0j), (-9.554198470105873e-275+0j), (-1.2377224977556998e-43+0j), (-2.3905356732331444e-70+0j), (6.705601262464562e-99+0j), (-1.8466201051245145e-72+0j), (1.12312472822385e-99+0j), (-9.623847635094872e-275+0j), (1.9255792931892528e-99+0j), (4.490040131308806e-276+0j), 0j, (-3.6898962857844965e-70+0j), (-1.0003072721755015e-99+0j), (1.4975694272902556e-276+0j), (3.401841505287711e-99+0j), (1.5913192119801335e-276+0j), 0j, (-3.128610986853152e-278+0j), 0j, 0j, (4.368200194188405e-19+0j), (6.261977865644392e-26+0j), (-1.9029857641846745e-24+0j), (1.3323347929594496e-25+0j), (-4.995895555809929e-26+0j), (1.6907095776241948e-35+0j), (-3.0489079556339496e-25+0j), (8.49264714672212e-36+0j), (-2.913814265564049e-40+0j), (2.14002778014035e-25+0j), (-6.416436830886666e-26+0j), (2.5891131096393393e-35+0j), (-1.231150562875145e-26+0j), (1.9854855058277277e-36+0j), (-1.6453638768353595e-41+0j), (7.443393493094636e-36+0j), (-1.170564135066143e-41+0j), (-2.9216525800872795e-68+0j), (-2.166977240814963e-25+0j), (1.0782952114208196e-35+0j), (-1.8181620958378474e-40+0j), (6.117749148078182e-36+0j), (-8.095559182272169e-42+0j), (-1.1782758355007705e-68+0j), (-8.125815220904295e-41+0j), (-7.424932236842877e-69+0j), (-2.3737155797017352e-95+0j), (3.9834339991562843e-25+0j), (-2.6593980229197597e-27+0j), (4.8036751423041e-35+0j), (-6.894619659728355e-27+0j), (2.0010829795176177e-36+0j), (-1.1624428988685041e-41+0j), (1.2582301885320762e-35+0j), (-6.527792892126562e-43+0j), (-1.654602161783924e-68+0j), (-1.2193287426533095e-26+0j), (1.8770021096944516e-36+0j), (-7.202659224675654e-42+0j), (5.637687677557129e-37+0j), (-6.65883533854473e-43+0j), (-2.9428212355518213e-69+0j), (-2.8708487872703134e-42+0j), (-1.9202424757224256e-69+0j), (-2.5775781218198003e-97+0j), (3.673081873143031e-36+0j), (-5.441760282270081e-43+0j), (-1.5912834245164423e-68+0j), (-3.187904774012999e-42+0j), (-6.553331881158275e-70+0j), (6.848604775904515e-98+0j), (-5.649541037810212e-70+0j), (-8.991410728886732e-99+0j), (4.442531098669548e-273+0j), (-1.5791315295315568e-25+0j), (1.2706874322645906e-36+0j), (-6.95249433216981e-41+0j), (2.5056346795924787e-36+0j), (-2.0570765207047848e-42+0j), (-9.692571561644419e-69+0j), (-9.632267775429563e-42+0j), (-1.3257726413880717e-69+0j), (5.095175320549152e-96+0j), (1.1582541340135737e-36+0j), (-2.079587469869771e-42+0j), (-4.922183191419894e-69+0j), (-5.534373330135661e-43+0j), (-3.967342527678945e-70+0j), (-1.9393295680679646e-97+0j), (-1.1859402218622827e-69+0j), (1.6481218903803672e-97+0j), (-1.211221235839034e-274+0j), (-7.35970584488834e-42+0j), (-3.443793394061075e-69+0j), (-6.076399501015256e-97+0j), (-2.9723851626392617e-69+0j), (-1.3308428233430045e-98+0j), (-6.962374620607539e-275+0j), (-5.219542706876581e-97+0j), (3.152281640696646e-276+0j), 0j, (3.2943225832482063e-25+0j), (-1.6153652667630388e-27+0j), (3.3928905397152714e-35+0j), (-1.012564601609389e-27+0j), (9.794202520102323e-37+0j), (-7.377046131600682e-43+0j), (8.872793283208595e-36+0j), (-3.5368860050956167e-43+0j), (-2.3005422577099464e-68+0j), (-1.1874513665113941e-26+0j), (1.359953403398061e-36+0j), (-6.905253810341802e-42+0j), (5.133949954786237e-37+0j), (-2.9037978501690346e-43+0j), (-1.8267605354363223e-69+0j), (-3.0605598089126546e-42+0j), (-1.4235449274372128e-69+0j), (-5.953314693458314e-97+0j), (1.8037585163051248e-36+0j), (-3.4415729704113725e-43+0j), (-6.119958825846895e-69+0j), (-2.184474131251624e-43+0j), (-3.580040122961577e-70+0j), (-4.672957810898193e-98+0j), (-1.239619882026812e-69+0j), (2.771341582707332e-98+0j), (1.3609179704564438e-273+0j), (-6.071227600976458e-27+0j), (1.502170008618285e-37+0j), (-3.628497987648384e-42+0j), (3.5555778890035543e-37+0j), (-1.1509736159384743e-43+0j), (-1.3882162426664797e-69+0j), (-5.829030551666537e-43+0j), (-2.709452105635672e-70+0j), (4.5972881527436004e-98+0j), (2.643830735656763e-37+0j), (-1.465533994161582e-43+0j), (-8.24436152759661e-70+0j), (-1.2169098627107856e-43+0j), (-6.5156230012720225e-71+0j), (1.3410021036451823e-99+0j), (-2.263278620553148e-70+0j), (-1.1961310629855099e-99+0j), (-2.305706676805569e-274+0j), (-3.1245979564119615e-43+0j), (-4.037139351047734e-71+0j), (9.37342086107704e-98+0j), (-2.150358989461577e-70+0j), (1.142419854627936e-99+0j), (-4.3301529641592524e-275+0j), (-7.76084351157909e-99+0j), (5.6040456719019915e-276+0j), 0j, (-2.939585231497932e-37+0j), (-5.86964879083272e-44+0j), (-1.7769294029949306e-69+0j), (-6.158950864331411e-44+0j), (-5.228018632251349e-71+0j), (9.685467868815105e-99+0j), (-7.193571905355718e-70+0j), (4.5155750940119416e-99+0j), (2.565472173349532e-276+0j), (-1.8202830489828618e-43+0j), (-1.8478360776570718e-70+0j), (2.9086308458294774e-99+0j), (-5.3598963805550635e-71+0j), (2.0623064058272284e-99+0j), (4.329645529445028e-276+0j), (-1.5503851174195377e-98+0j), (-1.6936801193324349e-277+0j), 0j, (-5.139531758342367e-70+0j), (2.270144053664762e-99+0j), (1.981147916479132e-277+0j), (-1.058435531320795e-99+0j), (-1.0897582814198451e-277+0j), 0j, (1.6219300063096991e-279+0j), 0j, 0j, (-7.639555803670529e-26+0j), (2.5297357776670783e-37+0j), (-3.2122715210048376e-41+0j), (1.4581999698232137e-37+0j), (-9.19602353752838e-43+0j), (-1.6211755094770934e-69+0j), (-5.750002576141355e-42+0j), (-5.2520285626214807e-70+0j), (1.7760030136279938e-96+0j), (2.808547153712827e-37+0j), (-1.0161543752460926e-42+0j), (-1.0674735451631004e-69+0j), (-2.2805903634703317e-43+0j), (-1.886780361586157e-70+0j), (5.770507436911608e-98+0j), (-6.492806934842947e-70+0j), (8.019562944566698e-98+0j), (-3.10418149093223e-275+0j), (-9.557342213762269e-43+0j), (-1.2280217149253656e-70+0j), (-3.94413123807016e-97+0j), (-4.685838588575551e-70+0j), (-1.3245832121450517e-98+0j), (1.1227417257205612e-276+0j), (-4.243903011150242e-98+0j), (4.775741526214194e-277+0j), 0j, (-2.051286271874312e-37+0j), (-4.2661171716728237e-44+0j), (-2.5846093111475893e-69+0j), (-1.7696693442564035e-43+0j), (-4.877210556608569e-71+0j), (1.0703497139728369e-97+0j), (-5.6722189255424054e-70+0j), (2.4854607916213494e-99+0j), (1.845465303289119e-276+0j), (-3.159390900716304e-44+0j), (-5.721924566384623e-71+0j), (-3.196190707748122e-99+0j), (-8.639705929734611e-71+0j), (2.0668481318122016e-99+0j), (1.259684692770248e-276+0j), (-5.0437190754529835e-99+0j), (-1.0204316886707888e-277+0j), 0j, (-3.198025723856288e-70+0j), (-4.16142877703489e-100+0j), (-1.9821531925398664e-278+0j), (-6.987786714135838e-99+0j), (-2.3424005883131765e-278+0j), 0j, (7.522339544343603e-280+0j), 0j, 0j, (3.1030320907512195e-43+0j), (-1.3628468148335624e-70+0j), (-2.0624663300387526e-97+0j), (-2.4071488979592084e-70+0j), (-1.9745351366341113e-99+0j), (-3.8595608545400394e-278+0j), (2.937043137100999e-98+0j), (1.4457522012649799e-279+0j), 0j, (-2.8273544605367902e-70+0j), (-4.813471226214856e-99+0j), (-7.067208224120211e-279+0j), (2.9497277475839823e-100+0j), (2.1102951239890924e-279+0j), 0j, (-2.839551395432627e-281+0j), 0j, 0j, (2.1890739117731095e-98+0j), (1.4604138956786085e-280+0j), 0j, (-1.4744881881887886e-281+0j), 0j, 0j, 0j, 0j, 0j]
eigenstate_data = np.array(eigenstate_data)

basis_8_3 = list(itertools.product(range(3), repeat=8))

eigenstate_qobj = qt.Qobj(eigenstate_data, dims=[[3]*8, [1]*8])
eigenstate_density_matrix = eigenstate_qobj * eigenstate_qobj.dag()

In [16]:
# diagonalize full Hamiltonian to get eigenstate

num_levels = 3
num_qubits = 4
num_particles = 2
current_measurement_simulation = CurrentMeasurementSimulation(num_levels, num_qubits, num_particles, J, J, U, [0,1], [0,1], [2,3], psi0=-1, print_logs=True)

eigenstate_qobj = current_measurement_simulation.psi0
eigenstate_density_matrix = eigenstate_qobj * eigenstate_qobj.dag()

__init__
processing psi0, given type: <class 'int'>
psi0 dims: [[3, 3, 3, 3], [1, 1, 1, 1]]
psi0 is a state vector


In [25]:
four_qubit_basis = list(itertools.product(range(nmax+1), repeat=4))

ramp_states = [T2_to_states[5][ramp_time_index], T2_to_states[5][0], eigenstate_density_matrix]
ramp_states = [T2_to_states[5][ramp_time_index], T2_to_states[5][0]]
ramp_state_qobjs = []

ramp_state_labels = ['ramp 5mus', 'ramp 5mus initial', 'eigenstate']

for ramp_state in ramp_states:

    rho_reduced, local_basis = partial_trace_from_restricted_basis(
        rho_restricted=ramp_state,
        basis_restricted=reduced_basis,
        keep=(0,1,2,3),
        total_sites=8
    )

    rho = convert_density_matrix_to_basis(
        rho=rho_reduced,
        local_basis=local_basis,
        basis=four_qubit_basis
    )

    ramp_state_qobjs.append(rho)


basis_4_3 = list(itertools.product(range(3), repeat=4))
eigenstate_density_matrix = convert_density_matrix_to_basis(
        rho=eigenstate_density_matrix,
        local_basis=basis_4_3,
        basis=four_qubit_basis
    )


ramp_state_qobjs.append(eigenstate_density_matrix)

In [26]:

current_no_correlation_state = qt.basis([nmax+1]*4, [1,0,1,0]) + 1j*qt.basis([nmax+1]*4, [1,0,0,1]) + 1j* qt.basis([nmax+1]*4, [0,1,1,0]) - qt.basis([nmax+1]*4, [0,1,0,1])
current_no_correlation_state = current_no_correlation_state.unit()


psi12_plus = (qt.basis([nmax+1, nmax+1], [1,0]) + 1j * qt.basis([nmax+1, nmax+1], [0,1])).unit()
psi12_minus = (qt.basis([nmax+1, nmax+1], [1,0]) - 1j * qt.basis([nmax+1, nmax+1], [0,1])).unit()
psi34_plus = (qt.basis([nmax+1, nmax+1], [1,0]) + 1j * qt.basis([nmax+1, nmax+1], [0,1])).unit()
psi34_minus = (qt.basis([nmax+1, nmax+1], [1,0]) - 1j * qt.basis([nmax+1, nmax+1], [0,1])).unit()

current_correlation_state = (qt.tensor(psi12_plus, psi34_plus) + qt.tensor(psi12_minus, psi34_minus)).unit()
current_correlation_state = current_correlation_state.unit()


init_states = [
    current_no_correlation_state,
    current_correlation_state
]
init_states.extend(ramp_state_qobjs)

state_labels = ['current no correlation', 'current_correlation']
state_labels.extend(ramp_state_labels)




In [27]:
# --------------------------------------------------------------------------
# Two-point connected correlator analysis
# --------------------------------------------------------------------------

pairs = [(0,1), (2,3)]  # Example: choose which bonds you want to compare
print("===== Strongly interacting case (U/J = 20, nmax=3): two-point correlations =====")


nmax = 3
ops = build_correlation_operators(n_sites=4, nmax=nmax, J=J)
H = ops['H']

two_point_results = analyze_two_point_correlators(
    H=H,
    a_list=ops['a_list'],
    n_list=ops['n_list'],
    pairs=pairs,
    J=J,
    t=t_star,
    init_states=init_states,
    state_labels=state_labels
)

summarize_two_point_results(two_point_results)

# --------------------------------------------------------------------------
# Output interpretation
# --------------------------------------------------------------------------
print("\nInterpretation:")
print("- Each line compares <j_ij(0) j_kl(0)>_c and <D_ij(t*) D_kl(t*)>_c for a given state.")
print("- If both connected correlators are close, the mapping preserves not just local currents,")
print("  but also their correlations — indicating a faithful mapping at the two-body level.")
print("- Significant differences ΔC indicate correlation errors (e.g. higher-order processes).")


===== Strongly interacting case (U/J = 20, nmax=3): two-point correlations =====

===== Connected two-point correlator results =====

State: current no correlation
  Pairs (0,1) & (2,3):
    <j_ij j_kl>_c = 0.0000e+00
    <D_ij D_kl>_c = 4.1184e-02
    Difference ΔC = 4.1184e-02

State: current_correlation
  Pairs (0,1) & (2,3):
    <j_ij j_kl>_c = 1.0000e+00
    <D_ij D_kl>_c = 5.8237e-01
    Difference ΔC = -4.1763e-01

State: ramp 5mus
  Pairs (0,1) & (2,3):
    <j_ij j_kl>_c = 1.9128e-01
    <D_ij D_kl>_c = 8.7672e-02
    Difference ΔC = -1.0361e-01

State: ramp 5mus initial
  Pairs (0,1) & (2,3):
    <j_ij j_kl>_c = 1.5559e-02
    <D_ij D_kl>_c = 1.1603e-01
    Difference ΔC = 1.0048e-01

State: eigenstate
  Pairs (0,1) & (2,3):
    <j_ij j_kl>_c = -2.2829e-03
    <D_ij D_kl>_c = 2.9498e-01
    Difference ΔC = 2.9726e-01

Interpretation:
- Each line compares <j_ij(0) j_kl(0)>_c and <D_ij(t*) D_kl(t*)>_c for a given state.
- If both connected correlators are close, the mapping pres